################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

### IBM AutoAI Auto-Generated Notebook v1.13.0

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered,   
there is no guarantee it will successfully execute. This pipeline is optimized for the original dataset.  
The pipeline may fail or produce sub-optimium results if used with different data. For different data,  
please consider returning to AutoAI Experiments to generate a new pipeline. Please read our documentation   
for more information:   
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .


Before modifying the pipeline or trying to re-fit the pipeline, consider:   
The notebook converts dataframes to numpy arrays before fitting the pipeline   
(a current restriction of the preprocessor pipeline). The known_values_list is passed by reference   
and populated with categorical values during fit of the preprocessing pipeline. Delete its members before re-fitting.

### Representing Pipeline from run: Pipeline_8 from run 7a23b2c9-168e-4834-9142-613bba951b38


### 1. Set Up
If lightgbm or xgboost installation fails, please follow:
 - [lightgbm docs](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html)
 - [xgboost docs](https://xgboost.readthedocs.io/en/latest/build.html)

In [ ]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

In [ ]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        if module_name == 'lightgbm':
                            try:
                                print('attempting pip install of ' + module_name)
                                process = subprocess.Popen('pip install ' + module_name, shell=True)
                                process.wait()
                            except Exception as E:
                                print(E)
                                try:
                                    import sys
                                    print('attempting conda install of ' + module_name)
                                    process = subprocess.Popen('conda install --yes --prefix {sys.prefix} -c powerai ' + module_name, shell = True)
                                    process.wait()
                                except Exception as lightgbm_installation_error:
                                    print('lightgbm installation failed!' + lightgbm_installation_error)
                        else:
                            print('attempting pip install of ' + module_name)
                            process = subprocess.Popen('pip install ' + module_name, shell=True)
                            process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

In [ ]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'PERFIL', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 'DIFICULDADE', 'run_uid': '7a23b2c9-168e-4834-9142-613bba951b38', 'pn': 'P8', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'PERFIL', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 'DIFICULDADE', 'run_uid': '7a23b2c9-168e-4834-9142-613bba951b38', 'pn': 'P8', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}
    steps = []
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[1, 2, 3, 4, 5, 10, 11, 12, 13])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['char_str', 'int_num', 'int_num', 'int_num', 'int_num', 'float_int_num', 'int_num', 'int_num', 'int_num'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[], [], [], [], [], [nan], [], [], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[nan])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[nan, nan, nan, nan, nan, 100001, nan, nan, nan], known_values_list=[[276655856341666674415060334604710739754, 286037535764406468748638292537568044951, 55055176479664422507706115565941203295, 323795183285153335337371356156502211007, 21682258711971068003561191247532255089, 93642205633704043287416883464027947145, 284841813039534400543901353776492031536, 60094608845497687701115831027880991483, 182760357324802025061755252301453848809, 315872763390995179893593130680303885500, 19384803928597598683822904190569511663, 132964287219904235714870991543605531587, 155620114265525107592860053162333986087, 281468034042562232323899610241678019271, 177646406591314213139656606431962626795, 204179854124018402547261198432700771288, 110185862489967183940022785584771827956, 24360950825078544630229389062918668988, 229293065145933210911544881430629054193, 260048067036504380534764537308232980373, 316063544917057716139413562935931893497, 113689736544168872565360481378130087356, 337552540131147353878972975437217894379, 286065033556226321564598172144744377505, 334687445817322280654225799997354431669, 80901918074508240867109675396552016263, 298932781493614805174378942419399662143, 77190680767355725578008243130506907054, 266996913742599514653765874844643943364, 42557981679531946839826406247831804512, 114882572805642797112819416403744730819, 84889540157730427123896263153338839161, 180395960690636642462141707779811221648, 207953874464239334758179480785264641099, 120412573971630715134228451191735364433, 298281601415958428775459228309391067301, 193972317261084544435621096264187741467, 189001990418322878790341280023097660116, 238723821751655340267318445400191566850, 83263604147651982249946685945297284452, 55736694993315149286499696147512014946, 98904316639543803314711620939452425145, 86156179392678141363757304015738969328, 165983338733644755377526177360258525307, 114117293793497087363618201968284889580, 213750876743146902446821370981791388570, 314261085570218840172985567861125786876, 96944939114597604080031537941987144724, 53514813480612858786509439108592449289, 277564694422822954409354853349864702955, 100036796751006774599865205861622852807, 221070072030795567333430253853645630092, 286476291498413392306087181506267641462, 287466454783091833247558787565045724662, 179805647575931520750579478623711418276, 229988753008922573151187514050439451320, 135228918363472839812408375653222472532, 23325822024808017267693411446620883124, 121141904420787925133456890922973972315, 306539256647163162586614972949571376628, 268349371100844500060043638787506514510, 58019498247227562015148250106310385822, 197606435695795766135518596736795260043, 10066873131709046069915221146035152527, 175007981932519077027993566332926620672, 97555890230604337369976283351187400832, 162675885568202846185166005825593426110, 212348596419846660212204463159074819255, 184488008838398648114571851647836224630, 177458802700453170294430276769121805291, 28220749845756550919751110046334532702, 195904846381761411418732555561699494704, 213007776481049331787604744557991442225, 254230169589891477089647566331165073609, 11408745792709880087917243625513032847, 185179548560488173952413233563853338912, 66391559057989481692442600727378293301, 299152857363316548932158680826445451813, 234317946601968130784395810382940157423, 207326679018281607779157606688724377744, 291542559019179310835294100535813538977, 176882618132153654618472187016758990809, 220135786575536859824814073244986741244, 129435376289309352114416419994866759304, 59250997533546613141788814933284798660, 177109412882639455998780390981443559420, 300196997926779436112052342992630071338, 192466538924439720304669143930437729365, 245336856393924069764092596965326268601, 144442963993427387695740881877084129231, 265442371398199156990099032853004032911, 1233194231500500677761460277299506908, 279438848659743535172119214793153728920, 39724487203125159379413252239218774767, 114102569608967077697926612233421421626, 227390480696486887307152098851385853886, 314915218375826357837021769357091675786, 196218677299092084104668224108220243172, 338654141248100447244543156943534610774, 107558036017089537388000770627950170582, 186099250809906163920338206903314505832, 157274046234560470724285629072299925056, 125493668264681378454782866209278014874, 298696915583325501142824828263094297967, 80538120461558241580338885200375086427, 304177143493098986342633245952734711495, 231725272791657389918165995386748208659, 137672467328955880073731392515990897250, 101748669903140161418271882669007003567, 118567667216150021513567535913064427049, 268208874805068934173510536955682569824, 312650547570220200821168593853328629966, 88058424619486414346085282775431503115, 276355401642574283937893903186877169563, 107465290553136908771712744094801479191, 133682137237526757329617975468662329062, 193666190446650452386384054787321443700, 233168157248398830416010962069774453564, 52331205643154048442077556215973848554, 218969122659257905545987230084867588198, 30296601709003077911427560263914962627, 163998831231619002445811604183871677573, 12596417693057138705248411625745745137, 143168244597350934950046362268110867129, 20008585146902643912850837551646931370, 26261757233764267676226543780289608284, 53292607681934278766220669069306923424, 47562844912311375772107905512515027238, 129237256148316357836680353076983488917, 175205197268070167042140805009789586409, 262740894659569795105615223253459980510, 141083201062305876618626177485381020517, 286980579171918541728657184974816808093, 199374137298663010844542944541830715093, 36121058478417420393362702092867587770, 59190305158892475092859560145680788672, 306477210312479610547061639001898080302, 2151312306875817121716911594319439437, 192403701730542840079439013859547065551, 202596730084129613223888760049900240203, 153485759075318488645065286520936758564, 40166268741693742229347734696655109575, 235935215516968916503550659317854419373, 31763479448524428230822314624357834731, 329161210660065351986208443580065557944, 118173471871062791325747928846511316805, 73163198327129275672999863641437111066, 17402485694650833237699403444261207861, 186270550843686754197540245318530125844, 209738365787761185199399310405226248195, 50882937803240793063227318861936082127, 60057148159311688466203869240333585567, 149911336795201451947777250052466338284, 209339810082408195241394572700884336468, 69280739081273121021839183419852086629, 222344551752758862370341316547971044233, 122505859217033655001557374930888619480, 124713978762747590976368836842412647774, 26230073805210623969108345212791790493, 288471124015980195135657888931338566785, 336341747900723548414275194105758928163, 196059176155447804817092109022864990103, 56674926152620375032713533151551846976, 135866525895661442602360613178953127481, 75369772520059700997992020925986803337, 39906475418360944638761911394089283939, 95549314204505817603979558397885246423, 183401192416284969792607455956090587802, 120504299221086699426035465287087141775, 210092543832951314725363778365606149960, 155155482032309505102479733122471967212, 30202021701902261941126683179156063282, 46291258742964028921080214601734597906, 112575468462246004085304678192499900099, 70153947846211572634330544527930001032, 261439304272541070518683936008150425101, 250925019865964343342507315077455821425, 41039024914602684897266681669453723663, 219068164749924010254939658995762541753, 310842939343541111962887860230387767404, 276291945680479482265920776670812916246, 324974388949172545761711902939126430596, 38072682625284952672663865913634226097, 160176618621819475596288583548108785897, 278766325565698048309308595341805415348, 32732746820766031387701286147714754264, 185363554016630622988105536307951597351, 81821466524613810419655983245660835081, 249807126000764534980760221447903479638, 141818266226141532099839492813232486006, 56519360867237331799217333465808276631, 222315104163538081281010725635293337085, 274031903268914155840191402569559252576, 304371388674455513320431916026853739369, 162205742717678210195011443756644654934, 54896994377588912591424848545710457434, 23246491120612054993217968098794856499, 291991910766448259466499345876368062436, 200746067917795750233956374029811315773, 158927780209003552483072921299831388367, 185603033293602844568174940393063595139, 290705671024957948774851263248459169114, 23204596893352987135807687137001107997, 76379983221107217228159049339170095405, 259225386524867438593056345882229875392, 102886430850795769449988933918878977425, 140348556279141376167726175320899322997, 28722005041715488295275752436299275651, 176207844968781539690700901945100122384, 96896669168011207972112137647870197526, 142066206406413499977934637462585134151, 333057958290289695251340643926840323876, 173880883088455277215981517106510420320, 149868439571321990240248024055837383764, 149197146647253105030531857554308102040, 190447172693295673953545497198436008881, 50846713776927380481545132776988965775, 154443193979401832670630513700801971661, 63822832754124313954195558835957662707, 218319526911843143621953610726905443912, 58227240122439079155318519315093390215, 261928962465755878869510597611875044129, 161261303187061920628768521265916174911, 118239238432765637483569836020698220574, 46727959913613315386901371225921671743, 73692959284796460961685951603483240241, 243888053076493787899931787265997016122, 243757195240863314530930597699833114178, 51956823671470713540340032326031971480, 16571604265596472360254457646712911892, 125390304919978965172225016232003582745, 114380744364674226656501988323010512157, 188975512224566113038090661926514361923, 315790732748475869826518483584291795967, 211660731294522084082548494260121866531, 23718406325440069923422804471658449889, 173025185114827775969696026969613081954, 249783600766283910111315687781312862278, 275306070274153323890068130441995334425, 210911233098184079147003996617819818566, 265136067878013980216123993942818622173, 267733315757437721352062516655737736141, 312846112552862416769315338879657331549, 241731435344822976395151398729631133042, 6417063021344650671078436841600597698, 55072378081619962259308954058441867489, 14081828207203986083817844153160301964, 169089151076608047578064480103814839524, 55718554049733403651973260069102659206, 96135077790284535909990354413435631660, 143933908653523634078389851301554343535, 286623791002952049320084381350655694888, 47494715988295970828082342092612396587, 275636486139440060940359076901845846218, 302588697157601893718262818135247193705, 9205287593861339734924859704193691965, 118570939108390308357458739743144010506, 175565360429497963098111682643218828118, 5566489034286865546437974222368752087, 125157471091651206911901821768202778711, 77301529815631844162433104437899179940, 209439361595189285158682500162144047107, 319584974548260914773414881731532728947, 53217419013946598433567002019930161489, 20613544693243547634607956197034380388, 178045604604739846443737364281438482864, 256850168854857512124376029189079408464, 323346423028808322207611311938639272612, 107004095207352657899049949736137330047, 205743152818319782682259595560232420675, 79723274283866183170188956726918172346, 195533624879928333903868025120052704341, 61687491046030138366621823227654969458, 114181247625339535911067456332738567008, 29028034688474473137414074635664822019, 212478402149749192733580075864835990685, 244518829521868531270508471794264055929, 303567497022100294788655876631137745993, 283376562027443086810026280400081534423, 234802975063661472939159970164329337671, 246233035755689901604393864489560815088, 227322149900559251842775259745926809227, 315877684245852089632444144972235436035, 246308896994685068495408877113906524556, 112940968800814850248838119427016597762, 258147243785705102461224530698465642407, 157291619802802032704006449497105453156, 333404161048475861390422930768983932974, 91053969267321352194088151764893994692, 289840623284466577162406488779259375001, 339853468608491160862820188531778476085, 102341895527187308864255084222151000128, 328157243696649726266854008802564527837, 42466466869596658504839078015063814901, 27591186736055076441888621752278458396, 270245419227157910873989396525381509272, 71651786013757185796164188291630324324, 308676666177243229858960366248872499588, 274819959872343399099074432570407523872, 256104367281038957355909980033766462199, 123844631217581433566879346462786672733, 159941380957390361716894815226246361812, 153852262802029268808219353357606016476, 297529160903878706446560990211578679398, 90435487512905725449094492555139888737, 74077941061068572713744552970657040215, 200238977981710252279490324384728958073, 151890110766660338769851710506663996599, 59445832685184075937946372568113739197, 338302117493737329335775721925387243381, 113592567923949530485565577772351235120, 32173932999277796345931130555864905781, 122810893804344090080840953843576862926, 204188364919770087609724014990142627348, 149227054893087808010297692006950766906, 134178291129224904877478531677825366658, 318079449121449704658552603826477504559, 27331798967780291627909821321163041856, 147991235800573375579778421107315096120, 38706823825579703062587702528650736985, 273067573770395129782824983772576139777, 91607521887050126891806524578419489275, 12268675371238228035531822803750002870, 236192843774477968432979533790555603727, 42318960127319672174974959305800154823, 10248114950506179825642102996749601118, 79540440645623737985905823313600426797, 40005450745649793937877326623108341772, 298935800989759310549742459312819590793, 229275500098373564218435952525438568021, 299169198657605496840794118672675170711, 81329808044192528925057025494369543279, 246271679272701547303433327266454891144, 286795936177579034117760738208037707691, 30703459329405181857583971558917073697, 63083513271511327048836709014725542795, 204057877780196465994386101833762901511, 222148469373509084797585457725948705163, 193997360216626362906005867852153529061, 331546089604335061904381045522765863353, 243794003591157206425370038911226355605, 34312776017743053026678847862003610644, 310845755318085181321608722432604194456, 153842409860698525303502896061113808725, 147272933717546887259212048714044305886, 304900141988345049771759039694777283789, 153008159619309968102782708755562274993, 118075381715326509952259878462606744312, 22428587394378428521492903353277547821, 257557575106745292621243386451026317276, 222070820952488490502986322940441502680, 171297518840552049404428928308813534225, 2831825618394941326706875023104586336, 175476529823634181063367449509323488446, 263091378547186595157694186188113349780, 47525752682021325603548036947041263973, 103835102801249488349420255851993772445, 236439042226454475332031150377882020927, 323825463718776581224456670429718498722, 306389244277251050086922572312362277632, 320758170837364469567471809705794012208, 175839716429789865806535615679083757287, 32574443257554004523656273387650063645, 114357081961513216636600421194899598121, 151963014977546464739304675925934893458, 25753543560430150537452743127852219820, 214307011861526350718007195442520593304, 48584075123888060183627508453915267254, 3078356908686735073122699380189635264, 89143067933398317047340773638153056538, 57732038235349151710268003430738706388, 94552555223564332808541223255621750618, 72022104789159819834017517900624148112, 121255643431656146894619055159812842464, 282315990302613476195991606878100619047, 332568261925420150431424578757907320679, 303425606765309481546590002678801414350, 96853382170774060909642557534025825207, 273147707078836934737907668282719674312, 270788946027261545696049708618139431353, 70952307946836605578575263261576876567, 198512114709341008784134348643924298469, 4711169019744392025061701335238906471, 91371702515330306192223440445808468000, 307973250917925457183808131351448517655, 53332851560010719306102757089258765848, 59717898002569149806658412657706408587, 81263490504368041467331451165664782031, 216754860818426137413268327712674525676, 8638292374433624898718962578625559398, 167412778944949288836570911782356735190, 172522880695517932143989921978645089885, 177652007386866469900051685620710603761, 111086702079151534320033312317275754809, 272512467765252028758004324859254133066, 289830107314945974824694538363791716915, 52283463583345354165785903850274743183, 79534641298572509068795008721962958152, 84991520316643132872584237790691138804, 109690020646208544972447608351746072941, 247696686935814090751191689732463920314, 273986362888170305048454817634829207857, 243294314541400515909091375777077184133, 214095183610711423263370819945791309001, 323870318830980138257996234255201811816, 126600652630839700919166011578392968922, 86307501484242828176131692426135993376, 278921646366201311287841233830463845751, 245547370416615572191749316860984081690, 116466225022726512441042020490366788654, 119205805882045116857636870126039891276, 293214794045104455633724440459028271801, 427440025852215538368297824569960641, 170315143507007352705389404009500026085, 101569286046107856934750801108167693657, 158747008707538474315942387962319126867, 131944502274407254481779564497809245335, 268223788839525598638503397921321707199, 95790178383508734873512810659991716490, 265865503850731904641305502542572429634, 307973833263477994600771630818471842150, 327278810084487878721927461145762007949, 232641683273696180806134522897697757101, 114496518037565809127274435779586001465, 275432235610938047335429540053880882431, 306115939227659686669515359849179482436, 222615948131716329003357414983876899632, 191969161632545282337619992775782525063, 203211139585793357494635560256095785830, 229025977268227866199101312745947289352, 199927183201579193214751421119061709837, 202525653176170718669540379271658322066, 253288700197276381239859353160438783360, 330063633387655701692764105010537648364, 285295679310411042763593106411149854014, 295870022159988488720136819241822093882, 199811100164789384957534402128880632869, 249119045037946937913607390417486133010, 310306989776988787619560251438894407896, 271818047608413697267772602494103701915, 322637209131291040027231008369741671724, 168557717828760430609430462188836414788, 293404542765521241323095260872575714858, 262314642436052542231758942595042652329, 301604824940195799132209022644324904419, 205381616135192146739616009606801727302, 269719692713170993015051569189885673476, 37063605417589831788675473890811738861, 195565947168260823393820836322344881432, 10991035284121950069708730045821291691, 106941774684755391278169066532917819686, 48107862222194185385984195811574459179, 183806849685540979145687437260610176030, 335533031582405465477678238463366413328, 187467321221796021528124426250765156218, 230385875224970159572446654903671776558, 134722892011575947210864334620804470263, 76153534194021718479233376629527778324, 128206606320113298614520089281822163963, 152051461325897370396560323938323003946, 297007507587988789499337030292267855401, 271619295547299271494985789042164306047, 69489785005127977524441770003054231384, 294742471238577190663119722861383852814, 181260036739525543771793628593877543928, 283636684053125164464769701862088632872, 283883000809854820662791391493049838227, 116537803109960412664003494613658298928, 90738638550276189916635833016311062499, 69349735257682854821678427962245381982, 118452176325270439098262483618891843991, 7584193573385596872206552579126175149, 52566315011633316103433696584280430054, 288179617300307440791880685394885046548, 227296569325194396962950049894540130313, 92714144187364897841323531697213704753, 203286823216083370718512113262760628467, 270035289663541884490168279968577412088, 189948315474120700566596265197775713658, 23129315276293250595481918883822173967, 322580280966367196648389517081482997022, 137600739315361810245499510118604770359, 272388079371646230195299089078277212551, 72684262210180707343428384434503322575, 150495455935495340763095135558249836785, 4186605045137668885520050487504830877, 78980890487255102457013914048735787969, 328926608979179042046134310436976496341, 163198025359042910254972341154702046574, 319320090093510573819872292428929413344, 227826612392851649952600034004377743298, 61174563695255151668762057385023048891, 197531705970083417177078357995350696720, 116135927574497182123789714282352947353, 309146433808897751684083879433197178070, 45115642142174788363080264424773786007, 134882865109574252482780184060116547709, 17713962907643173711908676978921865259, 210986841304637759339223406712209816852, 60634332434175038735865748311681531393, 182721070963245606680821908793650666203, 14913131809724315788499477298090331788, 255755446906182791812836776721846305084, 59539244802301447574747272788377621034, 243689126231243283218605933899682647686, 239430150716608735387946605911671107326, 310794591087975901872440089464490661846, 90225412990717118657954900827014062926, 275237116612812894697304859153478623373, 146977220724979796341628134207788884939, 80128762876823076758637001570093940750, 52307928169577809167918099067623708250, 332617435548200629258088076940282304748, 319423981965023485376444738726012000283, 339152623729002696296067748172698203487, 16068589379887226896075687362690546309, 251139664875877460694526006381007735359, 337427451965172178983244975808073017364, 292095186867285193491458062672658822465, 236908141686698808117876432064929926632, 219324913166038444153950014541476194081, 38670268687661864221972011573019744952, 75187653829908231912946897860315761784, 328066754774670113017326019664951718568, 5523536857038111255575937228092510690, 226562713644111299061810740427334144820, 85103642138281457032935517736775785462, 308257530544958622994149655598242978820, 63401846432553635656253199170798466607, 39887205046230082168349998814581025498, 202886661429272019821476518603400808857, 135499706888059140598801347557462489062, 288484552637021164791627976591598541072, 124034064003752788692514286191968046702, 273180754857080439507696477204455188187, 278705181126769096158432978329981691809, 326151953404753848892683800431252086161, 178945491605958094221919145461673284610, 95491025296594937985430302570194372970, 261296597048250673227687624703984641265, 207336629536159219992788976931482426459, 45359327163608159852402165717868519920, 12407981747696139715043976697803790909, 68321286024342932488913696503754595485, 233946450770647657374464266225540673592, 190104748073021546578980197702027338495, 310277648635364961705011366153002818360, 207518233592616125113978027589904921563, 301684979260362089852065802083622535768, 74398102372907456640187045921784009084, 105830288206450664258518801763002371276, 214655775369940914384738431123306195050, 108141570473948005255374102999082428028, 109685688902965819152883280408782674999, 324218329231870632611014822751238263942, 287786119610021036403062392280365187880, 68608136066889815794797104502891039898, 202912574968009545300530064505673972098, 139748560362913246579344128975315126611, 60315765990268049126294840632756243092, 172118199980739592503905663254751414323, 61248395436661277877823694395138241816, 41975357318949769371822446741548226901, 301062945361046015397315391891648541768, 4478888434869516352724086299394091077, 55758969329999902110609169075541523693, 58649712995259901173436690565273327228, 288837414179395447074779770959075201218, 54463863604581846476210398732538384598, 308680120526490335212696663633353290375, 251824341292978538862597729294645336650, 300766726782748075543260788349292821225, 131706394719438233070240103689808787238, 263865463533478861070247112597650224102, 114520764460690909540263734921758346697, 281473762643742571464462103054667432128, 183780157393518346786103406271279582775, 318168766102432276545655446848837318448, 152164751212416241237712703187092604692, 1196034887239914444478454975518692138, 161255033156499081945870335442751729602, 33129637895377753065180355883517022753, 229717774840898663002852204819023851711, 260447552796213494669432747156164554573, 306588286269938227119501016758076989725, 285900715300350737858445774918202624692, 157085824909528296220039296176820289535, 225495439028582079702385336865906055159, 24450626583819957743677992804703612049, 154931337962047417581210736467758664522, 6153549367975147300041772733920333356, 305115205696797643622281279639599249455, 217621769396295021631640387251454625286, 191731964734411376790420738717752728748, 62353484837469382684404755747188691678, 305339853440470039388375144534220790445, 90481442820732175848188000257108812042, 89768241057019344269759337738408726356, 16486916265545701022253407096834824371, 320383970694962718225040324194709647665, 291920108886165891155924684556114804137, 268068026418081018339156605584413352506, 38120723754521238586094999985819802469, 183179558907549045065034910169032457159, 271406096689518440893935696482708080606, 214424790198223689746702953858950758530, 8809701991299164744272489842530715905, 128486696024502882719096223596947029659, 151801527894598554657163611790260664193, 124942781382758335909513666142712769185, 9659248472489964049395569876349635597, 315237219600101959959010312067648583195, 182529720372491046146259270406300114736, 307487120526815616941616871829518114060, 321868175056322174104402303652688106761, 151588905679941232609727250943576439143, 59553393001270694490942413808611582236, 125867695285307166377644196761899781249, 107996507271143897562693403092620201388, 282449464782291607642658678646803129266, 305522786038597330482169736966367593384, 170499384277813532890784335035196601869, 177202125417383857987109687744615644030, 214372266923687372139310531267227358758, 248686432216237786007567950656501748161, 72557013910479142050463320997139314740, 156344412669796567196006043380943098112, 331477099279156212781783090138698645163, 72546488305534821790277700478424684052, 291007270017389806013770530355650241500, 110436984572064183516857224196855172674, 22662713435096322671167324386721072273, 132197900096540661056399892499398626849, 163943555275699165833616581323601785330, 3572499385155909963749717319570617915, 284084337020057061917230910283728741982, 23199301203000055470618293643779807320, 259438519671490646459583291018149287994, 332202811245886997585273569886007045504, 237520160257587164730644180350271141609, 149180557007782098224982372043189481986, 217278486532527498888959196077916082305, 336055016606278178180042195625189930765, 284108403848121099991985929094021701089, 79069509099837329347509423560213649908, 315039967568751287371882817215701529197, 230865814170787931461281252013627167351, 322053290560722485902964772807039038913, 239159149484781681453978189786774984150, 18243467210794742049187760272411497998, 109353340972620124150813662234217704818, 278324607345217693434011093376845136624, 201667231768671302892830336371464966864, 89536030390686414496800669539312871132, 56911521941599383257395415571781822355, 195318615389865060298654872066037816591, 315299448276531872772382192941393847857, 149741899977791754624199630422892875839, 304896417976235422055647099972032520181, 301600222137555373474853991183823308484, 168564887805085992525440751961156794890, 75077491312445509712282576988162600735, 292719830550526517847082609990429182104, 16024788605721647361782693050539463999, 221501642902269063861674724926140553557, 300529544359734643022197624095333074740, 159409380860739776227695659159489565181, 51073869984958462247352886771503836932, 136507629418215016204268922845031531258, 23464843872296866023911427585238208411, 119353368772356593005891291185186969242, 57766865315050338494451704351211911821, 236093569371599539520893281860151342455, 303106990408786492212243244007095433348, 327123449733712426317907015684330217086, 284661852012599180607752509017315899893, 180934531115483321108750848310118768258, 283119546372422076045145168429012464529, 338734240171872493798629123234780849059, 202990747859316591843338152341639795158, 16247130726657744293933277956864713338, 171106726430103212840740100134705832008, 273920381905453904972627827750916432281, 234004248066710106515244989330993524542, 191209810868175763513194184250207790337, 210620345969750584818568003765681514537, 234749823240213424142071072548698012144, 102107136406710819125525277744570919451, 186708710269933494249851789201644422303, 128333748664903185612929821796982293048, 310279640593183672035606578839967161232, 90078242897322061510795733972618497520, 308244090637216790635547619182901559068, 187119886313463073500432494390678599924, 314252294887859505393011608124959459278, 63878889281886276782365369912356601435, 16617657867581898458086176136921953782, 289789544925167804967452117754582724492, 113309032125235728805395597771890619643, 14540671747731978117244710896425446771, 100523131989182860423540024072860686385, 108826532322375057199650569402183409965, 118949616031723976638384646734168501321, 326193470107012595689687553108459969715, 39277115858425664854571901221732567246, 170535337927905665880302672932869617774, 9396649092491046662658092167517412458, 145392499526033210205420608508366242860, 133080278384117662113161318272780637649, 43368323481501196137787871691506404551, 186876415940297494901018721636305987669, 300979368082097685242609102431733991091, 265550239591577334708813760927019113061, 85947074905653084031435255695424404784, 167418858195441624933951899671173038024, 277093660831260071308874628376064709037, 273385963419979700473946858616021798786, 331796361988712718353190441079300589935, 104385965576234209244904322346804550149, 268584246210547249703644156592775471726, 97900988056706523522652961558055953650, 271316117756104249347610121855048203877, 327956716121415402281385451056409763195, 188567170540000416604294292822560570511, 149735454968526930486135595467105281234, 298566685970136869079338595412361367873, 275558800711815704649925432878165927382, 50166035533149564114053344047236597211, 64200168883581829442906189347682842207, 10657668226693712507426949404092384349, 298485645434495280023820704440783752482, 28599722706604159963408369756612882879, 166169496691925159404482337403680260552, 58901056751369812442454212048145182196, 267772453378218269768219256772732543723, 66072608489578697885218556274677592136, 279163601517476881466627230696898216267, 110375522502685874005371778828674930357, 7052942791135667482110367510950774785, 276343691413635662311023883523191098235, 332492380423598194566921986494460829330, 278188863189590858797397356595912671772, 140285697111813344881750279407752073341, 55763813754587836452038573422846713611, 335178927535771651683948354047014196763, 318869835359359419779216762923086123318, 189316424121727914951096306199694690029, 39558883762651407509266658584326287486, 179597811430219890233402620330988150806, 1561885311758750480019191845439671964, 52852548857930683267213990920854060546, 117543746667434816158892953971185218387, 295232313593402635291895261081551912769, 316696596866924519669173046784299084074, 117812957741486309942132020963080103373, 294444417971101769725180382174437334804, 298373405813215137463605294216464913849, 125511420590534805156785364485024085081, 291581472249557396839442356556343544556, 228987855928491849388377954599185248839, 298959535855917008930487522279648436484, 186681208910785875059710352014702724985, 142699656374205844109970899623916069716, 303125511147417203881779182176095548419, 160792750387594936045179433395084394389, 30766867063750899445737069276545517750, 131477614393814210149661287508379805421, 303883873506825562248729105906057616001, 299270423062425626412148327443808962155, 214994795260402729989364204111282698311, 199434943327909332419310428506152317520, 42785895796394256645309533053892047173, 99999109556398694976764792671156330576, 320248880047663487965333698314995075576, 108757122148687467492665266147813750327, 93824772267192638074903950493168664656, 60238531117680615555298847599492292635, 227936257248424305496076769381881055786, 284883817651196561046591879118217971170, 264307895995513903922091840893964149248, 78954516184480458208485042288528114239, 319336710724135150327947113275342319297, 210174050627430857628521724433628262701, 93740687669333925872604357886089480313, 144291427397753886112528889925961506768, 22200181239881927086300371687821573692, 173455785466039327926857401778935674909, 95729025578104346001780491228685114846, 102183742860613545731862271026124624299, 56549609786382633272581675325218214855, 86062835143754190605039258933689712674, 283685988501241936408867455722018055445, 191538742693648896276896229791809912385, 95387846356903301675495831597750420421, 20897028543280297499382655855903850699, 37141056976168987488791566160483914220, 245325990107726110177291451585873458898, 68397282562494705298340186285074712386, 294630718441431879281811259458936058202, 271630589163413246929989872250995939779, 255274160524411643715199463086181231887, 229560226256066523177329754274456218677, 126624328052206856921551960587017927141, 328276966199402043412699232562482645859, 10154347796754232003516749719953681213, 99422185659391086261593740577300782749, 321431778519831561227204080279679130889, 329705174486323618388480001006164993122, 7754169649631139441134796857695248861, 301951418896292626583062417781209019049, 166417388965355996111150255840036350177, 150086003012209286003705595696391853654, 296155555446369230796948774437619640105, 229665634183459781009279700800093284361, 59657070856234502309038725216334418735, 167075747914666029978803478751149819976, 148932220056041893139054286665710081765, 250675172353677264234696019808554185318, 19846508745107421956370532254427400852, 94586186102000124730305244203145730173, 60195394437286186273659167424214927625, 284042021556433899600210180756307126945, 181559027209647835686893392294391662094, 228437121869883460424676275387600584827, 198935820685640019032509801869183556951, 308191037310110343867220592888055191967, 1306780551833758169317529098679081852, 154465435852950199697473652229227140684, 113496035120308869497297233148919875297, 172219228904186986494324059515873772582, 116675683051853307095392051989256934256, 278650037546694443769051185407923499963, 268678623632483127177939280316706840815, 242957486344609568513361178992539878781, 230419161493557914263983538560219819937, 155022701260712015861013951410073907548, 291023595768593831926357721119472237490, 35346926345256985143858407619649907186, 90152406630684680033321032733098978385, 296029787515052018006016742448423485760, 207207001255255549447693442681318603782, 294854410612059062936520252438812997162, 67797586126823165160374224653952653994, 4627984707382938099102489875007082715, 108001584241287587963090747790887820936, 213523432005849072694672789998736103846, 337015431916521496449120327350973386095, 29741774210147008810878714898071004712, 1207254333820978472549124539964478680, 77033666321547657868286130692247934786, 83476020557964120770790420280540543538, 284830172579730564858528512485919215873, 316508868921012685029060407887294088398, 301394901347494102230116450483463737892, 308504993188194102819820220783221348015, 56647044270536414255463862906799007236, 15219306628374926073075544507691109752, 110772789305952747319465338074298408089, 119767658118696751436676989192150222890, 190557220329572331935049330716748914078, 242951815187945538125068507200863648322, 167659177902390644115673731342287331238, 44526852518148999856902922960893928068, 19065294633277400920524698949234292703, 46832667692176216517641863395515316012, 109926203350570653447076018740456669451, 331785107218625005935479067658917385484, 69384158436460347394752575655801629795, 128180207914285720788836837416880871199, 23525842646711986479939615424073753055, 56244030879891045130628141396592593636, 37947558921730137302924247732267600413, 31032156532352044948146221079338266871, 110020993026243502560492155192889956305, 20870585522512965449144003175011907921, 116223092172643423676931628131542426388, 213251776262546331528531037387749597854, 203514478121467881696581604250100891541, 42903728778805682078017158975866809611, 269527220514563812720877311113698359661, 64956854400953214940576352862461417967, 240150535691186816609861219272870488292, 229031917950685426547293998263439734898, 339980656656975283800943803320929863542, 173587045627576146977404579504100976407, 27978060042959518782591504652629749228, 169828892188558887480268878111428971499, 321174923895682114697729046825743006448, 136279871464647760990788394825049498596, 329528469216754706628036510732628384480, 247479735207789690464073536011984422121, 71552062504173367876544555827633528858, 67683187251990571991968269551652840902, 161849970307497487842298085374671929361, 333730174257197308407899520715416660877, 303829056429745921007739835213345089254, 20978989238514587260809047753585765977, 16301439261220169092789307910318188706, 267545285035180735390750956420882870484, 255558732368366135925416015186687580571, 305940764967522367133649768196373123850, 253769102679771016071973260989454897538, 313247100768847406945331164173728183671, 30078580089026310978330803263763858772, 148176805462091882088464792534976658660, 141648524335638032843398397009443885867, 119398489918513583110267566743639686718, 163120723144270161956243445946886975088, 288310846102366505431656937051660270824, 238412102830437159560484357499845818892, 314402704069357896945531251928386911149, 246411596225643076391857992802200326831, 282400489301917215111281872938245071091, 64930786431375494789540282710429691008, 224768195560005854514170551931994425592, 89350488059804474732535112556635274210, 270179136325120465479706138236091560638, 314622722743202268864660811177600497520, 22577700607803941862094907097549445174, 11801439913707336107175631245282530675, 13390446187815931713888344014813875422, 75735482216861277942009993988687316882, 66689382887880396911270187409382326261, 799976078415945432745685877211665661, 133567318760113652507935840212020915235, 84008091890556860694991677782179087783, 207957628813966358103706556995774957797, 247241596869245289836493805317290331932, 86971408766002049832585567950697045761, 54006092864831848644870397858014108028, 209784866205756863397553962033807868485, 134240007598104128916118046051156382924, 105064156396206381761194317745248730603, 336221854811991174578767199839650140089, 53352657573422743698641487932295779373, 330743078950854454985675070401074376162, 262092848394008712191315339776813773245, 217738417756960440029973212113442525115, 181607369343359301734705885251891415657, 220794000504280108291422891599919614835, 278440836014876482349398103082329507987, 57447500331375281064275170144002118697, 40614555213387958661973981476575822655, 248618059078740436304039470437789311745, 8155105365698439040639083309756640008, 193604250358365324712767814059947965991, 269765682625081675081767978698585835478, 279256944366525172251043507166455500979, 101842567488034286082170108311329712316, 307343222440376997589300665820133507702, 242096732205089274901446713794097609446, 61428195262812392716373611684227775397, 147912330274007160612269707320649884188, 184953515344774881687000495370137224683, 151625752747668380435039907931924361070, 271426521174843883042235372202960476866, 140575699296825899506010440621300039779, 23389429640866306325273714504503165720, 152930726487995348694893191366879604270, 334366582742935828388019585758134023316, 285448184496540647512637128031310363667, 105385207250364111867444934321531518387, 292258998823068118350454630597058099033, 262786129696939800258087668844549296496, 686546487212991314925330084463063866, 10356649422486025405943066832641895225, 292087654531572857512593697564685184764, 88934745817571272549174384055050424577, 110174944904732603448889038372504343741, 256522454346976852242383544817506986637, 11404745742445036421448146447160818863, 268875657230530937947293537956089501205, 298459867421421353309079472623655008307, 143449384380660551096167875543643971066, 224255184863885448779672735035883212198, 170837458700096076137109890039199573438, 93821219686744613764971919142830732013, 56003350862506384062356476337790835532, 26947285175772017522509077369431271539, 246070269654732591630078630516163352374, 77268316926295118702314657064246962034, 131240595055727616172107185845846335803, 201382040401548466417631380761047294511, 292697910115241380932641901762856255340, 136975220276334587631536081485895272215, 251595252508063996164614392630215453027, 26081825833583148809252467440880913343, 63323382451655596082634253532413149429, 40627260231359697048694857897337772064, 80161120313413109080098211977959813532, 137586479950059234854739467824956898886, 180242571657860160146100060616829010283, 213141273098204161999789136752399678735, 296526648672580548272529308113902977515, 194215292492406929234651871745151621594, 192214664304461636990731439879301225692, 281195930462064725188185474232534527831, 177599763995998771479439353329178178611, 332259362521133990619775854164107383761, 94554702264926913943094468321651816987, 303393969732059628484904025408940379225, 291175205714307751074931018743865408297, 237947572412883364598722471093327145385, 190619352224948410905426940994069538892, 332064179576057926905661102156343497883, 137521991759085650386163396197091558345, 52722868565330588816285030034648003032, 11203802764740273378144936795522052880, 154209940890240829936594015163272186045, 128192697583721861352508405196974230942, 296509387621477018169567942166905492657, 168914609521604808924252758860489256669, 144503807308917018007812263081470397020, 73087519163284752775840757567595645662, 241984435203059489060413492333130811005, 329788545276093300669813203148454146462, 164204800450728259286422112470091025524, 324811708991866281786074585494932255956, 227892481494554902604395517223111901826, 167587690700152376641476264144739516197, 306776590015981434425777113877801140429, 42716428132444651062176527393773980751, 17466781158336814757495070322590208981, 203691637088412862781990579639541407443, 5063766775819481149946831945722880678, 328560547744983249488924277238958550498, 302130677296203122208881409051663453089, 263082578986724092901730201177975333830, 32225355811412543701440318083739395923, 260529352836879184999684397147053577151, 182626764303542726417131975737477740681, 234471556727443610027509819217714722177, 49586163164987228991174259565146509168, 176477682817753411665057828640961182271, 93465137972483479850578430172789504893, 186528593836283291954753450191872184846, 236545313162264750002659435885804762963, 162840887123573761930747291811825585927, 103792674902497910389777046257733388500, 118316065816188163492183418919689182046, 191783713962667995007596876985599680116, 26645289264871700389054342006979916475, 170211272424015452866126939737401003735, 15823773738675946870727448442228565261, 168075660686622425743514948730904452613, 51505089280211040153037755396488826480, 300813130298809664692951432109193283038, 139948835399151479279041819150196181027, 235737667730353055460589239993805773990, 155724335833521035193607543818682008534, 304588524645199527615220227028185956486, 104588697729140278946974712720015521622, 110037991079162154020053091130839212301, 313846636197712705903162839294326361235, 331542386935724459223301886797969480107, 205961324310844251819020817315076557062, 68822791936900255486954533696798195980, 131590086941485417240373568736255479537, 290655099914625570467184991907763165881, 294176642116608624278372332757772137330, 199260150569398317493444449282473772016, 84715812268483228352525167213730427771, 307515377933553571389506589855724192873, 12628331279980901517712392235460107835, 84690977074709227290443690499167664079, 335942259985222227754958985959351107984, 248233289775563915399045532401047630564, 274632484285598399762831882962040819867, 201662919637157832490754067714148617460, 32545217500442494679780275787811989185, 112871976325738618896579639257980989781, 262817265302936038310731344937411275891, 255618888257426983486578328060254713544, 118179643689145653477053171780505978276, 127811243664506784884443720297411870619, 269535903584121823766765704740572943812, 69272514482857835814828511009692072772, 273700702208265107696621560468236783673, 261292107972870913801459519640119912459, 190066153123423206698719955952834498176, 35334737232651503708835425714673307751, 127776774339209772017192104989891258845, 253460179878124059546722988140856156569, 300250831815085105631449380284765231253, 7429172776597242945613823172577151171, 266860332819728452164252868642300035384, 60267650142203317831469178400125191989, 108191585622126867630197943438317416648, 71620720745688213435967144583712843715, 235081684193286219237584968398082156789, 203554496465629345113709204421393712492, 10972860004060590067464190801080373379, 250082791539692851127243393759559052171, 236315290911881436031530790021604511583, 105315504758988373201382538293032744085, 259323981206255389375832328821310829541, 146883526672983548535052526305550441148, 39666190704195505374973491570188143150, 132256024569841171188332213798647018678, 263554738150809176504284822243866776012, 264960706081208447920170670893198282679, 215055213434455329570118410450246348805, 9296547764554463471759382644348184210, 22749132705521950152216938829821594938, 333660101579346193071673656110677296786, 3221303059650438051294015320880482049, 186027779943542150869915897538088024100, 259319791276021675046538286169361300298, 165175471752754428370747572225423108871, 322232755869106102271590785386880162462, 166200271687065112799043589064321609427, 285398796281956243296894939945420111795, 4884765715551058781843346222596342861, 186023829810106254855746024357653569188, 312335109245036456821456411597652347512, 11165405830904795353875370161424418815, 38473443435801468744901617542927007477, 127482993025932903080204350258452317362, 14331322308961398496775838828883374551, 172944696539431414605996153746243921662, 167273309856191399984737444992782185257, 58673358403223134563401869719773124357, 55599494381109497952117803994183396869, 138814953108978393622653510975961586390, 135535135827875416370935757065786258542, 157256127896207313791368820114411196028, 338710538570230262033304960272531733628, 217868910653947524329728631397741378879, 229806460001858054212699906947604071517, 48068920936729026872230686233289254708, 278794486673809474342606474091249520437, 174724711393738089952824566422481505318, 46046802270209147846702071597392433788, 70810270676043567273545591169443739684, 314363301622739446679022021407961926481, 304086467889250993404158937448089766268, 89875039611618249745061932064252201723, 211543252938553763527680235679155405535, 274583734374647463355363085623608769378, 144849876400170568604366294810349825083, 36964851052302236950391112013811412316, 188880708295758112860205578335936735204, 320698595735529517296503646176126292310, 219150386001408558438026940203476955607, 14981987308464598611777135093182208134, 48098907594077435850620167244263543959, 219742842791031083714246514666492791034, 30986847418420582011052989620016347459, 3719224270718387371464163097294703221, 275371579952244462059409285919789538791, 43084292631885707212686116934489023926, 326881350027465776459769578239576352660, 140620936559275224533805856079095395703, 280808195222599487280239439313074068460, 32459792860793597343861052465279896412, 290652187752859237406442064789006847087, 64145073906013936447777539230915502454, 323172865355070762887494636843516060435, 321235257559308514537497108047582515703, 288534166775019364299090881093714849484, 272714994590615995757202678416329606721, 203969374744011289672776277674507269810, 105509574360427334355176520493550355249, 283326922410678406357457276050445002790, 14993722624361306658413009605623745461, 203597415412952047829609758460182072629, 223175805960839690381604884425078209592, 290091670164802298302986475522702033400, 98272277488890565307519451176819201692, 251495165034060691756892444038926957141, 3873359505625270891053238773228227023, 152068550130123763867961662044647487092, 3296078353514833953814304824865189276, 142241208361799007365554066502582863004, 205301966353204470274417962827753113896, 176958545139994353481709110626293838059, 126547001393556951749292559132727753651, 170670789198102847453833927389357696674, 266994042840520770847489901349521162595, 173515023136669958185202610856372084080, 277678525658343936186715769288746422256, 14300734267068098059505738746062408201, 291096031931262880177635239743961113996, 131589864922833456404086547027103499272, 203333254623651806541712327023302771822, 69526564305599503438215583857865480717, 267499547742282385872507709742472187575, 210266796369822337956289757607294783175, 95470832638479934453158509562098110760, 165821989355738471023006954215931632887, 271911706853314223355617419967043678370, 124280071178324694059012526688743242707, 304518557939254460201259476397707582776, 110154168382909476210450137571899256060, 253831327942092526865246456838025999734, 260365681518029115393697656513683548282, 64338349041917780127241761363667737194, 121109493589058524273169382107533001383, 203720826693357546168359690187109447484, 9391581022813758400231684520981655744, 153148724919693121432990191470851906507, 151065032290092891050835820378025201404, 198434747806141102740559860499863835569, 325731579121293838820880219934309146052, 49247767521426505501710804451074132833, 187077082323398509940424784070838345829, 264418489090654417196701686020415260611, 207665743073607282384346344846476843595, 213165734738664780231532863598078176184, 93831619188280671740907901822738882556, 192435992038373421024148720004251640522, 305612356440389006952932276747772606191, 91067141326274600848462601579594838193, 21011694071185629523806125850856751319, 165023859090650964351094378334535234907, 78877415713481200435732674332955617496, 221685805362512943210724762201175809221, 178617925744629711372585399490413877115, 130205865338207121553391111037323402951, 54671581974300382084875447505314675438, 305955061402774687284770402695576757801, 228843179326985132036240099629415347853, 98378528507306362526332854252979435976, 131834204434931887673092870450472507091, 310288751017267842928960599991589217399, 46324297015317880126843790166092654624, 286735521439642075001105740459858178636, 123028931562264475251338779881191738603, 222891579481090513316509429744641152266, 193185146100580370771670619187242693437, 314612932976426845807949668781095131845, 85568830966309180320878352878648742795, 215523104977627450565717622105287003782, 170893081935132370719749356003403753414, 130232041594668136355730179500942344359, 216789878257782795609466022898104069460, 289673900342706718755081263688696131168, 240339547050062736134742558006600642317, 19395722174277929439137126250010247109, 206207155731579398250725017945931970961, 85167723996181610801858726314360513558, 300820263110646554354221434202595632566, 310900406338547893682725327825400086261, 309586977786001107882841969484930928135, 140920769289514375501956367160980408114, 138206185661359873131748697271151422177, 101397595223167914948003192221001178840, 112947967547706760278044870879461093251, 75225787617390987927150945509097067544, 237624191301568743762495144212710896193, 319149264300967110213910472667029347496, 37895362799098346802830201478494138843, 287045808926257031443853030191464381444, 77085846867452055420274035291887831659, 120345878126382835431440313753633719717, 145882692096493965215065169512630350242, 180977323043010773821301962081672875670, 91910587059843532261559602429996121908, 270662118365274952933202829460242553079, 308742749215622752347616580709108688731, 12689400957431308454410868874259289490, 234406624073464355904775182390040330047, 83080137496245283761085305779102561242, 212657223509515802325387778589250909934, 298958492944469787411815207075667158827, 234982452271766349407676720954298203305, 252897406872487431166754314673933963759, 169783910830881079423631862051720807937, 31753169671522045119621970969105612880, 236831667079123629623456435620528551101, 287825894882215141618269195648134742489, 19239726434358985919358044000351913519, 241904970158176780953365558830717543463, 95408410253704098816128647429019850276, 147623513886692555108827769887721219684, 314624106411204672993566861767425421922, 145790293021717314926494502097863489175, 274192220229961651346720646475600055768, 99855922100596201862657852409363869457, 101202755527421632973643070805987576992, 259582032810764038410236983470796767589, 46512272781245026696754676959365178124, 256654230764594059894289119090651887919, 235056786575907432081761442451630822235, 258283060737505687361725620006656451717, 36031790686540851142787530219812479008, 296299288258620816902052895184705412040, 28607812639016638836061395532234454101, 159247974039021537883037534817954565360, 266158483183107025729159312594968704223, 103451357274080691914396406580866977252, 129354277236166007022578380965580066662, 326404566310945356548649215780354947048, 40390085392307908485232240324395517580, 249388775815345321142259399642844771293, 33849793442703004548994193502590257845, 164661269119920379960963797040482146206, 336817542118648626935202365306923236096, 258812338075460164876586025852961220906, 87843882678738705725617369551095528058, 169152814276646444421505677556679324805, 236250249297565187658281259670726236509, 3498859740954616549782764047783181303, 7231847329587263833830650740146618094, 1859028093212301051392884274010630643, 241654850221082935197142824079855009508, 36412184913624865704815499582363093184, 309333547360691362978596140379774511075, 196651503670564716048390911884917026889, 7739773447118426619857302601367873482, 251468036902482294459633472612941864226, 26021208516640435814409922127094742732, 133961943244070362532817443709991502492, 241940586592473802059973079918850962967, 151650400521379169857939909215025867605, 181728364636440041136988572018988421560, 309231662762743888206570619063261486755, 52548684165507091114441546989988826406, 156309662410073212065013741423029297419, 93123210335978398027565504938864414532, 219688160559200184475232321248799100499, 168921228430864627818347940590060143442, 238759391508839500144252478621273558871, 57716974221641138107195251887239465940, 55859118376114614371939746926452099539, 46041038392925508386755054828666278770, 16207238029892337639782981717458102156, 280734160624999253944004476726432391552, 216940728805041063446374593743022530552, 316975282738543254691292528143508019665, 319738380096454115776953576387588855800, 275852308658686589954227576526895357768, 108059261208637680329538661099793779319, 113812769743441108441055666338057969219, 314040610216292396641753859589313853738, 296589699903086254550191273495326006429, 53221448960033393845744325342270063724, 93693497550401868774455392213612666849, 32349621802305842907252880808015436183, 266081378557734675740697803820984048402, 265886785183411637156433547967167737218, 1400136721776282996206360498434117275, 305226298252287508563568863565414397096, 147988577099126577606003391457753460577, 214296617156907122039514528721232016167, 244182877364168937673506057840783928665, 197187467452107220504305719075766995344, 186725255944816870488627912147244818879, 269778931150341065816047166688302976237, 257875075624548405068102168740073266418, 108556834048046103095700843713941503498, 316452958081365505056050945494501482805, 322241157766651113704761955929745780494, 33657768990489776672244669882011501968, 221127591391362788249481671246970172637, 57471971638212652754198822195472521688, 283696594114429082653487521072292463377, 169031280530435821270638278847694400075, 250651209592391628802607204557395448504, 112000460511437742192218460247496074320, 265144407027462943004805662796628374792, 335420058896097745879493915064757134162, 287458008316306107620024098049597049935, 180644960886406907146530821963393820270, 147727943696694087116081772222327257840, 125160115405568917615268166026031654565, 111836483613932145601998091330692014087, 227433810828795908470221071039343334073, 138268649468541059263231899668590909724, 220327154554274105097921654512411288941, 32705670276830047036864346816090530381, 192814913552428713816694923754748136548, 30900775819330844507049217178885641583, 72216284643640734722405131198621049869, 227310415369879033296526016995790666171, 167103102081739582628221258325969871799, 315839484360394826154054813975551287382, 55379413908025918991211750315136619332, 88186645928752132078999353592862604353, 179155050805229005291824205265884817752, 112349098325282346388149812760193189830, 332466020086189008824918816840663208437, 199401255155508173456425997926510976948, 29717595755088619006438157850531154118, 219471616938087410929403655003923353203, 130348641728623021958142546913702127481, 326468724318379013519344008783641405758, 42216548879466253753301010117572109553, 105099056315424347477702812531667656449, 189855186574835836236597567930712896851, 175693861850987397238235390248791065903, 169267718033326635701716138151236148744, 12616370767967569999999736873720916557, 183926507899814196486965427416107605400, 145931332464432315216410806288218465051, 101257271695971687243852668990973896996, 320772332363096496593065997538138297918, 318369918631796197483840570408448293338, 56869843066209968414536078773421067553, 329682341661642626566642294890737086055, 339528487001320556665937014146163966068, 58683390726482467947791969958832829100, 325586540180395870150539953549072246760, 229962465554264747933385386385147191521, 305081017574746088402751775202751186512, 64672900066028382314480597163903545212, 24929148249564089900137674464144683048, 318982401877933616074657295701519525025, 168420397200900748908880136245020002843, 180572624775785792662375818673163296031, 340129527429725637234984576652596257032, 230005214571814010519566785888759498382, 198111985574032036098558439219256493820, 311089453672518415005442859676677863023, 13147986294654231781243779242798133368, 78821232753017112035369138200204503316, 40842123779627588559870429443192865496, 241872861021667787376783053656980203223, 49549664618591127402717758139496593463, 213747987165511387870241598928800001689, 191903743348108742360986341280932187033, 280388907198592493585033429718879318869, 277945373433244933751321086828219514685, 331150908281010499071400382531379294714, 135990464935391093438678449120450038246, 245423957899758674479203484961662324953, 75337555158349465404636657751178071107, 188098200358852560466179132051017553183, 93524170542944005274959206936380026718, 65894183531168702970776217796189038563, 192278842814464124761207831273259933550, 277488029631140959502367946961762966356, 57982404770202585523799725473621580773, 266232912341614614326874743936762734742, 152511732575492395889931363085763959400, 199116757135069585957895857120716335759, 174945768513989812885564472065989041407, 33385203400459121998123211775585181024, 73501414897039124025555929418202185568, 24214311439534253242898202278673100705, 122910917409886852313315718603035757246, 8581794404412397969964636676377599493, 74230381041476987146178227462515115064, 31999544047145280560581554864493297225, 33214728357184216865214493254527325326, 246062327373246913973818750746199378675, 332742596792626483872098231473425046794, 200712540745223537934083330150544008515, 229706620505752028044536405496502763640, 6574003845420470610343845110697692039, 233957592138379051895126977324599233772, 11646726373145610051736056182882535963, 147514072658412545476237437241649636466, 320238960858264668992516647217315163876, 276291426636084084199099365708181276120, 290557159843733191727385896490031271574, 58121147729909059706915435070195035087, 315007106134445861233482140011293016993, 202656532921922084657050405150472904486, 26166664009989419533653890182859590224, 295166734252448235921315227069542379746, 158294765428173706454201942493029493202, 39760431886545529940006530299883193690, 158460500219052786277846749660432518003, 15020156935824168469167189170954972916, 243981958308209338639818988850169713665, 299251799079052150882334039319144325052, 214369829637205557044837758803986176749, 171583371097552577081409858081096313753, 199855676162650056509226978162159732608, 187099559130291803651125277630673518922, 37005469072935837683400221557448797918, 126825945966084271787469493756812509458, 188537158476507108906133539615795030569, 103455471794823567078438608446121089030, 309545095975243430516574339113389465540, 199350193673518863549973400825719369936, 253645714045643970480068246912771842943, 90814022854740836372315334328819835836, 267480715258152120507918151731417260853, 134297271202896350840975930238225005844, 177584560627717963144250671193671342552, 97426050321632368561082527841758594916, 114073507391940687280940079258560223014, 47277729783686834988611842065742119228, 240612132431749097588725790120256498803, 30534815278789320690183465248671490045, 312728496674560676913983331938947281479, 270833266702161232026901685994692265588, 44057511145218347912231926147606408973, 97164679475593692881689519447745398933, 238834642220583550454000125738304086220, 258870395236248187493093974939235780689, 80133623787246825651225077366079621549, 113530488295101634059035276411840551890, 34966478943019758668973284210153724207, 102713723560435494787118019702459899684, 214508531686958182749317770333954054566, 246253853524424493274402444828218146024, 327953677607063289309198246280006961355, 236013984241140797784698129839232735702, 328376475893231895670356034891875454493, 38000890740025503699529694334707136386, 80752238495043340866180861635711197801, 39654357236898006647442427431674546293, 306828122992308253676262482601992586842, 327830309465137591467711056582001926091, 277876891849877361225634094548515101933, 112885259742296519628117424771234043428, 218386083174597902581861742238907505459, 325257788650447497414405387648111540909, 318964343909344718425245584794579438021, 184953838247374772858294451211161988672, 71124460289109586197484373645493799532, 301490604963839765754659305277462858030, 324094512479373030968908989039308037247, 201991197190856330841873117151719096873, 263284034866021879297453034416237728507, 278124312042671152000878878309680283186, 97844216802199743164951243392832578529, 281111287594453383154857616555737276122, 327853884213917969644948365946012192052, 42126315945914654619697374451108653478, 226780105426037126990172827796276760036, 339786241093084529357290179145743511494, 138270180926524509423225203316305649223, 121251281225330819663303552069320785956, 38805673278374785901145778790766344710, 262071759223554713041950880225458565064, 265010272138248701832524523157838079564, 213884115559533706369754020437439049698, 2605282696001787997810505780313418390, 211098875359626672241220808712399806502, 20238473084866544329458777318311488117, 159320916024246781662223884463630003376, 9469801786389177273609885145544990709, 99340636770694097920453361939989978033, 141561517878275472166013718567775655454, 137149722349080577921214134871080059096, 249542384672941281852876482999532692691, 32370639290340928840910588118410426185, 166659666468749755344375452749727561838, 36228577792375422823577716961755370143, 257613200934533879566997362807523818493, 118671395683967982298293197737262186375, 294851334825884520114540822961407866014, 195426653175735853025787133892127065830, 243398913883511194536035807617157979395, 24673394894236661589001276488511695952, 71851831927182280611725636459082495526, 9889397120927056701329485861005265193, 282800322774660595868421509227357480726, 147489722254579114312504846734918171348, 39999826770879894726380273510178641887, 111786566712896618021682238058304455227, 134206047607643404839128400378831128318, 306370001187764650008058520402593297782, 170181449479737746101826663671647554471, 204790925473160038332112986758108491846, 103621389986572051714625980313218268374, 104505732605586988916193797318479709233, 23816739732109857215406431351221698582, 10006807723285829244944701432239430605, 96695429079576238147499516474948526875, 8653024706297447843728849109555881862, 339639136346167629141462144376546495156, 102999166412718108341751515001216908564, 77119141776921632060882415168972129284, 272331061566163334272259113165275239823, 194081387160946536864033838654758453807, 205269502598393050496657070373232930030, 29955224845733474512787328527703668488, 146596486118265534182775815436392048787, 45178702219678370825343355028327775912, 332647996757519301523399804853396945886, 249304074325857885274325614251241847493, 235280530278800138344488713481658869473, 188977403518441670685118785295041652292, 35225881738584237395934869522813444117, 193567852136504875081507809051890312865, 226138519889196326172711716719564310202, 108720710451653334578168429929074084108, 270876904896518695886252182837272563504, 98464017120200468095937556341662098876, 242423178467841682167001988588243680739, 131074689913099580005517520798823281904, 110503158864478151758604177012598521105, 227382069219218689544373298234163915211, 261600327674210887851597974393698726594, 103602723858133620615961306821494348058, 165693427224766565770829055206024272864, 211327028246716682959622951512676170919, 252730574133319721968627779796748153169, 182927858404443190981061830001828644414, 152374104948056559663788462808275358999, 148012350494115121188212934062119075381, 65202846575739384802555508393438426909, 9109974556977938507069210173076365809, 115736704035908618710333085399977276318, 160825000125292547136620385125998535924, 205573176490432437201288186235129316327, 184621758263631916296442707300966999231, 50020790060148574208646749042214671070, 337630900400038335746866832125252341194, 111734547967197622543641277935441438040, 285487738544201354769096398919887153511, 334801674411573058848665369795818676452, 296800135849825335977316991170540036316, 51816404773847574022217854069279942200, 312953632245870429519497943874661231137, 329311690321795440732304160357801983833, 44905281916025747649400999352294283990, 91018707696088785543143690967362363345, 260684038947046182022649800666438011004, 55410340707020610200617150974322621438, 199584585356833770829482366538875339173, 185550933303755021519413034971133817459, 90863869219225217156573909348232930184, 92760168827011693441149428565577784116, 129584902243801973748507024515704185386, 88267626751666705674124231208475282921, 244552253017011476373264453703564384864, 48147076350177256354582384085641584864, 224601825718549404900961420105935657660, 29658371831676199319288447508146793947, 107587884161425051037303901265404886324, 73541995042044905734401558966465175905, 24222714813058142180483188124580621020, 126343885489560995365218946336845597789, 91056050483949656713838299019739044572, 92754812623257643615819651529203961296, 838160930105606676802041931328475060, 195498151426031938804798659494666128176, 290648324533373635935861518607880693665, 59775888759162134628071624847862178149, 64381447517441685063807408044691797518, 144510506968385404706013593714443299265, 338947073770674728614413295094385281520, 263688349127132744273447914293225761317, 160459992121843589761880805499671650003, 158169119803040422985763479913697202615, 272474948441201797240842020043332542767, 287801028952698564334822140108230348185, 100394569964375251339347745223531372450, 237719062508916502424517452092552366683, 180901813953195188575950792519047093198, 264001076292730437003822866507829558728, 217623206020158365459902663969740164283, 28765559223602400955112263874212519876, 99625760530626449777886388763348232939, 131271318691934599037969336108157054549, 270637769419857073694488221477881485755, 178727552721597151320643108147721107599, 320925345652005445137741124780643504129, 337601021162134556568092321012136967735, 261928655588508933754284696511982549181, 238864742717760568239337406148558768387, 63991911933595241820654128561618996316, 89241786124370288696286438189346467226, 273062997079788630002345457365120511994, 309039896494910123460558088873861353929, 241086294358265975812589072728914618331, 170834158496953719587382192995070728571, 170576034265475325066913248929106239317, 251649277336885345179949694386103230069, 2161727375907684743886824359349468708, 123991501652346189241487068840390534828, 240343395062381051549585680112497213170, 213184516427795510691896806118832435189, 276568909389860729947632542129838709606, 32057587685821931448810270222593437937, 155781250205081403662852395886989354501, 272183539648631946820823780338408475144, 184589093158056744926419610930350371356, 331178113995807339532427109672177557210, 267380332243401389795156441959103755361, 140853452247523816213222038650877534413, 11192571077880764238137361579513778335, 84623735984126327283338206896854867610, 321926478660843701014241800993688646288, 30006628040356565506563006439989049745, 100252584723836522451782255283720301964, 31671591982780207526428892841582262654, 38636370672522390605496934435585844449, 103757642309226530066708732678816642626, 281680159835457387843180066183526330085, 158878990986314466703640694733639155259, 191947150254968146319467252284951370400, 267947102151472870762995183175762324385, 339040251275459357918391039206077306308, 100341787313754670570517667263337511763, 286514277443497023457898428558565061360, 258731574482919629883355421401467573949, 208164991228300919945788474708228744534, 184260410803721298143375309225702626387, 271530455588049361636841543182244129465, 96418576231303757985800006399362122081, 41747846698956054816211783832260202660, 314491156253137808956224082285732868485, 138341063959816696860152064206339761065, 27442429371704614784935006168397670376, 80513432759114850005777695950409384168, 260791210273666143503742475565698020580, 275720224695582781393502963508726208595, 230745497677167535882056102495021237967, 23194462987439936864229337500052048328, 189282882588640776158291070232575694172, 200129925841245938793527759239054622872, 241857194200947814660684106770620142505, 14681450090855113774015379939362192604, 120474346008214935297123779548907961999, 237174364780278606305709859719093009976, 24765395419324235645579664251207917722, 53067826925624491946248900095727163778, 112290834093817851737310151770316442851, 302490968587750718022063307880441608031, 311515530703825143638504870156420309877, 265378434517086515605932505797062987680, 275263612650926264929353763721262190194, 154785370318461274935022053320864682375, 200785656866715976514937204444391768629, 276433303655031708638334345690248663323, 54108311906869013088833930336158262809, 235287792086810216259855961420508054343, 308874645268785933959333334453056130776, 57078371948791557551104388763045021314, 73611405894712853438486283639423028944, 314420227212863464237084707620994301753, 301357044952451812332745101441473638562, 35688044615407869448613493152056175038, 26923310851940641828266001037082454421, 339914688391322993810647803598779258279, 38464990224332573428311587525132216581, 125065532622086623385575685700953632279, 176486101278984644363376752122681304006, 198822241462051606331194047360327678983, 306443903086516079670412524006220360056, 101090262471616245320219113339142210449, 234062754492398780270487669023524122467, 75907908117212453474526330669392135483, 325932896174570955042293066820230356947, 249918639093350882607889331404281210056, 92175738662869927742629909710283708595, 279716062479273887733935461172294394282, 93028677912411394827165377651388290287, 333102578596755949317719921895917822824, 86798136156521785619935115760148258747, 231223656398445240339107654573900089264, 269389470589892485858871863905633505304, 131044540967944948628800633085790021645, 187326269912841434747040234288739071372, 93615117874262563029742494536929992903, 191083085145093011876266567752303051369, 124733248082140630506407488057743729633, 12396108043048988869484982190678971975, 113366260253428927284972623959624822453, 241981732989912456587815256700898148696, 312483168309230507184864677278631122762, 249672661632140627113952664585577103929, 251311204858616374507974315552432501062, 113721084162385405985673957412709450917, 36007131578155381131642021851782531527, 331308946219137220075961530082304915752, 165467599639056722180684481517283697218, 145812393167339423324781247343241696731, 38204975368703985979889842144449526525, 7929462736267900943917980014739074857, 56247077105674693658323615152551408305, 140133746065762377134187237630900130061, 169366700706137560520878303459541524971, 287643410619107928738597191394686476027, 171813308006039518344375511768584666838, 119872324141408280585377996883554407875, 162595991221199016500130725021817159892, 116842564094376248914355632645806671379, 58076061868070874016893821840966888446, 107109084030763754506335413068572972593, 229470913660045470463245950727300818249, 110760295565390458790828104825556711113, 58674048157592020184577576099353231785, 48000527540406821360360039029487911041, 8025273323475259976102544373281485851, 186703382577537619501605168896714740821, 3133351389197614751323368558233929540, 152756007303038404354210974139452775312, 191401286600417710017818061654126005257, 314425072706116657323811458933325990519, 46568333441141467063657804414957069510, 96266661461909967909209090567409457873, 156927690298076433668949970346251829047, 166812116472880746787346930135883207398, 24093305685078260834686223090750762724, 108894094411562866637954186899701526905, 238315180660552301600830680385500143330, 257994255539897051322468644053624575807, 69685736381095818563082775391282963360, 17813065816314768983542326469442224988, 306166537113468991549313552793532871257, 250870760633363403446454171996398132341, 97507991894141381648961287791146612347, 135968796734695380216350962056929639893, 332270083526175311840388276075946175181, 69661785435413042337146346366737091717, 140202930057218026571746721645237279671, 86080770933755648814172713858411579982, 292612728229654984223101471119357346877, 219655593399629661532835785935458289750, 42009852907879288506038763811532783915, 107176711281466448910831775329029235943, 204650438022076115719628585911717318612, 143717582666175971161398348101990440301, 197964403815735722974560970151767363162, 54648723329172568601671076101565066033, 73033994992876275338499333839082764327, 309296968756804410488942999601610567827, 233846863476586729882305143136934463968, 323106945800130035071427981175486121408, 45727753508746376849578027058550381197, 301684602412272285194403507144535382636, 36694202020127908119965678097364344820, 38962926983430929344611751273140019976, 81548006509352420725544043489720389733, 151262912701002553995123504375319241897, 158872759533685449864477332013250252473, 100039526760646861245604613340513811329, 150947160532273748105724126737300220459, 176449265123557870691304207243524542765, 285454655257574021549628804929556400258, 99374321132879332629608603491484023883, 232151438970533555538862093304616154891, 205024310785876475663175946394666729311, 248063449192174226842604031416296907083, 8307404024761604585288437653277288421, 35307925840179932124620924600055304880, 214984926714289547298915473872335398098, 319419044785889907941871370705830620140, 196605574291307882238215916579407042300, 164607233384794106254163539458889071902, 257021416118209436885273549009087816480, 82079210117439791069723211510519007299, 152807961196477638344117946465125483919, 217692989930813020980378102915813627114, 114549973449097941015806820950458602452, 339093155094301994003466587705987517934, 18491517575902729904484696256862665395, 51078783731116898803661232021538876452, 49334261781512308916340760675810289900, 192682206681101459726593769692717243727, 271182951613482879368192959187673731477, 254654520886472840559965730980863109030, 67532042355913926704230313608328867904, 180819006770214696069971507606517092913, 315146882930840829472687805901725034923, 4760887348376857970967761507469615808, 251929967204322866541956280200134014627, 182087850888891825468874333144446452653, 136253176807411438563875121564004761222, 198319570898805134417196331337545569383, 299064785888350403093059243658361314402, 272816014749643619773785990240290319193, 211271184999868657003288805562519267074, 21257028766976083534668359914951433649, 218622257946314924713250433119879073866, 173780973230656882378696133973945545414, 6762630447880705199404812852443827218, 25128837648191627712062037168367876675, 209681549096837638677632428894682717834, 127710618009397844838034988031027570897, 722300143378777785988079858698113967, 336927924032356479071372723519872029335, 168823613755830565792614657449703736143, 335041414048542487059042987352539175743, 83459853676571193852852688331063324867, 236838189083622866524167102076259111184, 221238970203387550642994763183144805835, 116309727877207552780124020958841245892, 199710425921663149998002224766249621033, 54456241555217659241332741968731875848, 156001822699590733007668545181512231900, 319982710573566667012497359086567683214, 207711010682119335921414651135777110834, 297105373553946388281330530443661900922, 86089382337085980472330967264518785832, 274249068843740482157677785986759258937, 112575410827687623393480770955460820209, 181252684129976598300806845023104481249, 249497861882143143156753768292103360999, 35414089395471688698562343084499847381, 222962884166213820414215789084786422033, 37012076237235533534937987500146287483, 305511239060034755017438994281884856626, 271098216387157283275376831469788832499, 25372186286478397984074398610129258457, 132465644751524371243039128225826954618, 138407418218061764512451550689572788577, 14385758444740864686910988864669416535, 153679077221861679294909429296370721751, 1827831250811802973477161459240382329, 31864804082626973550594174065133835542, 243452726389663213315930866346378320025, 204353830607509272638411824853453256884, 300630485680596528879083822861394831927, 24461871417875084331832363522323511861, 64289943743509983648049985417652411889, 3279141824352266184610315061101878199, 241634936312919650227081355471473282262, 162864714583262373137515774076346128937, 185579050165261997685167792680917943203, 234289054202091028737965113732456148598, 19795617089822318370984737092435400711, 258550753357325235173343687411034176989, 90752026949492325450327245756405500698, 113392741167062931380175106624390647387, 47321193418809837207404537854260477796, 81450646666776755010184422134330162709, 337682049411238294816428465695435381661, 302500914757000776047809546217906067617, 19999293935090562215060822448935781170, 248703768525204862912090944578138258031, 222501827135428229919296350588546929917, 150058159655488372770273155469610704472, 169062270879917215117658444466849672456, 130653800589712995062534465366021767503, 161871744677362190692350818803485906723, 56955674679448372070122272148444975032, 30244531745449130792709372191545832400, 63174255036103228382126920510381838034, 123937535431294306634795144418254565345, 221673320581199394950523885293533311536, 120518470447650260783881116935437986985, 323562176101570030965409650255587722894, 44243443048027260986447176971900756648, 19391651701888194161071011709318310691, 211178255491489927553754033898401366169, 179936777394567548847888092014221104223, 220569525128615834311145644701041696724, 263007365951305844359582839820460840301, 272869191061073622133421977842400534976, 253745197333681995206784449732463340021, 14134128123291157523148697018466888273, 44024702668840364942374157996173742340, 85369239576335685330534462078567930345, 132936410039695631566370304693743809914, 195607699590291314584826161835336294468, 271798857242926952775086505940076927458, 200685771114465592248216669960027107302, 239984081053779603989720768692138907586, 91614472607079305581332365661290602060, 260778253391843529939647808361637702264, 104580169062172248275211745407589612478, 189018812347861135488506109686423030733, 231814222265175224459097142982091275160, 124057079850254293645028418098727610087, 178723999005059067368594351356792223521, 8082667692755792347512120163749782928, 332819578623833754168944539829833420036, 167110626074949716759012270746132588781, 93700345164531130305190031951595391049, 45586132829415460256314212139546163950, 274185225404459299283118703572552391227, 305344969517624166585397324400998208284, 43062356469762881263790725671128251486, 292372505987137830312072311050759852802, 25837138859111754629946769173062976539, 89900124904549201757518958287565404033, 77928098823481071593856516693366526259, 28177060834585638632478023093235846307, 199555806474623246738681588075571996596, 16036326381394545661024596544885096902, 322404688101668358692643484941702232356, 123726007588013226094668305531789051800, 328695168945487123712382697751552801280, 219702317406906309091496703446035482643, 336828218448976491395697386480932770101, 302882784020003026885660866084635650001, 210185118604142409936682231717202502627, 299356098918378434436518314142933680497, 319826004564760178992080348201123446033, 108343218530236651366029379839880158085, 8220029195391797922068994858418619181, 287693829790724280214228117183178990498, 33535602906955359919636246789299231165, 290803029012172940473430049541823102637, 9827918838037222012271185631144457761, 46172813306765845455463935162963067029, 259076907382282282998584505274002209845, 78540124051145648653033955906762478146, 208646348045519326283172173738805643228, 201254799801671097596256725867199442899, 191052649924389215349349493669616286483, 229413454003242014767715665973630975382, 26732729024649878974701199726437525780, 79234736825671304632194535707801786093, 81658141176062563168518932476770831003, 108299581265402580489918421670766678546, 127070335247558017757027069143279671603, 157068845662275524781347984075991014683, 237854123362784367662553499205402444743, 269284297110902304721575066352507292264, 93081395354634514943290483308678569486, 28771343596300953247012434827634397885, 32108966147050044196821288366637847460, 312620235495805933272058321051415635779, 156753658057411706381522779088113990456, 330807625769633877634225105423391547655, 221187912947335834728464706200324706333, 245496802421914470001119551867338302374, 106986924271318204502206841328726578255, 245828611456915157434093572007761224374, 86744167247600517245184176712655001269, 268300501320820641569751865619930953397, 128351185617227890798582937328277783096, 23669332356476537386817168411875363125, 67148925097567607150863899783359999789, 51638245047440080847968177613625061987, 85636061342294226643687113409303863886, 301819970391420169560652115536534898695, 198376298217818131885674566689192056361, 174224526493077383437915339582763378561, 242941119674775970528775273311530022494, 251471566942635509772506701113743066446, 32681445559826497174430142712031517536, 240410767721609239014945694566795748574, 138256814671104389766347462169936336435, 115336253256623873781952034407730046712, 125973409651336868731865652828313180012, 291342854473305056251521704500170724874, 306522060014157799239476461004236497212, 111749777466290366594066702731526546356, 105115199609888799853220070517654057635, 48247781843496763123872930606577534514, 238213567571308102815864385056067679307, 184691225636579318900839863350298509824, 194743369927228776936680111251283671382, 186947587720180678232253830740870008875, 311025630689462763561003151214140880421, 251508802447235513540740703590953950199, 11430613005340250890566247076934458456, 44796933235993445269950892082162768488, 10085076748359464249193202705450876785, 260998473634957905736434443630171143039, 190807554291844684423924472878719746494, 247737264184482905658161652057494129084, 48265698520372860297171687479299915451, 167185339918218846433496981220922875200, 63759540241379884613863197348787276638, 93706822872172362602981364539759182642, 22354407760092491464272216211288285168, 257774255708053545244664169071610047283, 187133941395405317696300599333227490643, 147862223208260166026958005133480315120, 76467148001000181602439181112571701281, 202542877908664060063986544161978104626, 25900741943442576445045972858426512852, 127677953580593183322573587594932628421, 72857108374296964053240129975424932183, 275250146668599791460901706262035564109, 311618002619287452151455473374214534828, 313429209298077864416610042831944360206, 323628832167004286483433495466498846199, 195354840124952048486401583975998622203, 328411335326106811908956066099027699138, 225307736927931014415682187526810281691, 106717356459635243540309727954473551961, 77726579788957630720489089962446516885, 40585125076061089630985248901565214358, 149619414953916757443935754160136397789, 129502840841010581502852618376362845341, 154854910018443012079196735961343469963, 197430206500628006546918180542993580961, 133768175669988393198189287098192404065, 48398678376676629366367664795635239877, 295930113737864066576565275482930396007, 40360817081793845436077779248266882902, 152506352862762449395462761615251560208, 162332713681848734331583327798061263584, 48709055781423393953733765759317771337, 6091903728578111410448197495638701864, 197066016425523107850289428820204978038, 90022595606157362367887823414277212378, 178962288785409778432598758277752042731, 198863997891954698751672523555774599777, 28293229493794011929489198084549936219, 251966422811496028370436727324989508563, 236082692524528024544605817113016661726, 117798684303373266201091048249645753257, 213393677491398596587201779161767437693, 218742902650134895407243704774847119861, 248919058090988498791932737585616804611, 326915088504933418635823373288968272408, 220247277094119349398003233981068332460, 151857421667245014366215210287469673778, 200983509849936130402742082590777507370, 287590135601739055465694975468382009259, 104285057966527933694996898794336292430, 53452784563845337887959361768992144634, 308608869261255558614827947269175140419, 61475059706843374826144999845773680961, 90967326002267747999505396757348189725, 107083442170655021969855474720123895387, 136158074950640517439928556684943039165, 196912109015649092674857459907049762883, 164022603348142935703900789060828116185, 64574890738396220052351930284662852322, 91831063510245504003058189468725633645, 287531380721171138741321060838979391518, 132381613633836703988331109642481442723, 314020857862020139586251769182857430001, 51148781378443835762093065938664191412, 147258990397817603858349843428311899806, 210269883948958056970294463232738507728, 27024078190517087594757985473213014038, 328348291530771964407313041283056229043, 9543546739405084848303557728126187731, 307420058925245209693720292998485922915, 190162214928251636077946375301701030546, 162400779348684606618666866504806129578, 282311636343498370877507997450464742349, 162712681679194250640649845675633820282, 295306424007459183176410335558632332777, 42884697259031687892859621612284701569, 256459970205511130754404896433305767490, 71528048343768971694788330230873409826, 149063641134572348987233493263056214096, 123532759145361589676559165914203343219, 208791073222752918576707995353655357230, 137246525866631462413678677663018092996, 216621692437889915676304834728986313902, 213011848123468937966302505433680341457, 287894244902393323805109038621296509086, 298098135242629189079996706857883972326, 100221523442557397391115658347990894103, 60143286765233685157783191850842043804, 315508686181597953798756760107631947198, 235030299513002583027792323135473777986, 313936850617963402820004204925798635792, 127147656960822454149937074300802788357, 254971119510879969083383146516462705201, 128176353640970307711261931477944314572, 260038125617504612704474291368423051240, 144572254901004200121019039843625966426, 102109145042025535766274917342597810279, 136068305176256300279440941883036879584, 222060307301452879593729612572427996380, 5061756399522597561144149479630130477, 47701911216905896677988174635546362669, 43730086721748598086084808129041438178, 6936890073399542655557655618062798676, 27887663126366616062682242911225357194, 129780764226156990352592408320542975043, 219224111238553545546451399975937815686, 209190515001764599063589316437472142072, 136327881369133497723826547206494297431, 104886020181220448599114948629419695258, 72495111959434518376405483001184946672, 122197825030918262711923762490506633656, 261133502839034275964051988336374299622, 33687367455218297588222983967784380685, 224753723230624822401978404871869639723, 265636931496676383288622945805332371474, 10130608875110480063259481202011058539, 198076558269788595821280051405438265745, 232294210080440069341203933649557718444, 224355313147153033449310195637427561285, 237978714456269330340147106348991819103, 118154783227370374833848947222359617609, 225126090276169652530496861150137490505, 328458284775434505260347765495225494691, 155593394741308278672338606212368221764, 224049972873386366386276322087962275908, 275572388135828731201626168007865795257, 88191786004696664873834704322786012469, 101072449879121396546326673184800417238, 238782195764935288882739017874920978634, 270125121276959076946930486047973441284, 54584398001841414638416741342796911031, 252613546045851206570025531608675101372, 338757544857548577141564497102004107250, 94863998339847280344135766756348942803, 29670461355187337663164120855417621422, 187696368580823886177094595676820835207, 38166504199249072902508178459007928898, 282685948607273068321542862394344907569, 61689764921918805947230028678138058595, 52591351478363476940220983071417091565, 49745714425034898230839851146291860702, 226060541246715144374896597026580981577, 336575378016476324148898374107410484113, 39040840314188609125397994542081358321, 295784193282144304946616049785937345269, 59273598667506302701106851838326630026, 315724245227391160300716844851034561588, 182521937523939645358817107424133387879, 56100043941694014942653129134815467348, 39384659808754440819689578047898603021, 237019043522482469100111033918855655237, 284463901138026222663785386418818115445, 133412042425271467913432156530031948424, 71978715322057834420886914594208742821, 122608455362680243266872667119369433598, 143054151966172828263241774981145629051, 116524358098283234175614373964980792810, 327352041525785928331500690849977886332, 232442329664923494158465127221439311084, 144301174890951110817318947317294061787, 293562165208030946770947365038411918318, 42931529605953693309438127413944719568, 70219910183064684766474095431823502123, 267862426896014281937508768829781548588, 187620095662893863820714959129425986526, 114438400199128699616946810459303844559, 78618941881030335254197806009617723007, 71859850935066067966066847492834659326, 182199073096743227216083246593828430128, 278305232848511254967454342740435129542, 76899010792055502366994918667484848130, 169204993395796687042348071299858734087, 313382437879606550850899326952375390879, 13086378714673490653088243662857483460, 166163509059836544129176756378828448964, 58491338380492895103662597909544071038, 168660200539685659451872381723979539946, 305195771160346161440520820465471084610, 256905773712667071615552227862364741652, 189416828049887888059729694441905879253, 267819223362172009491149403649050229599, 8843210865827733904703129760027956190, 16695600108091529633072924908417964731, 60531602541540219438765168008159020109, 8914219262141306607617509468647423946, 28563191474979115300344323203988151139, 198941037148061816740029984504956444087, 311557693448039474690953009484144004479, 135961116393623196139406631497501009226, 228972216233712084016345843570136290628, 162831911881320891389159501818635703294, 67380515635223497525995990095343569061, 32632259943272844969308963680615969544, 127278337465631931072831920757946494, 8694393784269467098256392614320464420, 255252810237797650765377570698523537429, 88189745643590110368128864997814549983, 328387960795347405528286612374674068844, 102305238294823461394368612082022578668, 138458952508198394143658683836570799193, 191200767359452070408500232616507748487, 21016220335149703843419531115081169839, 215180766749890254851656268413244364375, 209913380940159189444475967590805314706, 166574519473574760320208358084261252799, 278123047297088788579727106170319888618, 254452766459789775574012118116474494222, 19388651487837550717278660195431502840, 293392603028106784565687825823551746985, 288894820162245028263021842672522158865, 264030867961000706402925436025215658517, 320470545489340301727060389982349146363, 21917459618668903140706965987208262352, 274846571955309807941446598183971006262, 128059770474099688167595458474290882465, 27296541148960321186130338763574823220, 330379272725926787463299557855657663913, 98962243857632661853430025112539371267, 89548090944932212547749126897829041347, 301362553024654120034233915488523616314, 25436460313914178089408983972573964227, 4422666179850920011055161288045485700, 184097063437154580351181764168964512385, 170570703360027874717768632113219040063, 204422391467554575610977579745567226885, 71125939534653639562027440638034503303, 49411046748920594955798232928637596706, 232423099637922819287231175342870899744, 301495958548828815166102082611232558574, 172091340378413089783836435567063032755, 133236515965984399692255809733696102655, 239211440787854106264048604774770036556, 195888395559174364333763893789547453382, 180744973942208773077925427270761853957, 117722564786077275926794073254059637586, 27661310099953180629721913768511778690, 207056109951684580689912210274025000295, 312918431038594360698711047786109914534, 1146337695903429801661692818908598831, 282457726834115550679354223342018072227, 94017151961716814798623744083187525850, 155529748556567182744251414507291897848, 91078826328268873789960446890096688066, 251654286573598927748520243109722018027, 149372329969481112047273268049022677632, 338315349068912412702250266661096187358, 339414231576107848088474179223132455403, 255777577780508671879164538325739598881, 230647391508256011247524790506122906259, 328930363169577713006112917277592847490, 144368004287586794639849765390791474859, 81999642033450855822594174680074840973, 13083071217571187930242150843316947584, 43196426682753629058125672822966225111, 327686652647019679187446673329257730950, 230150956223686980485382510434247808894, 200863441412618221953441043799127949908, 115518897450015480521073386294986875886, 183860054294981119300563535196887909979, 39171349862614602015306953765198798060, 103594548593064391746431191205134199842, 274556647692122725999821766074458708732, 207707310679647665099659490224901006203, 71476101878215965692444787074509187348, 22338142874216674095954199586534132177, 57940596162419016915039348706977493137, 101951622400934212803785354544251461720, 239546144708111120630681625674441245725, 163184857614870464436804684785703023118, 5664122903420489317596638113398914555, 34632795864162913799979770004982319112, 186743793111266227482832080269012395434, 36275606032381013662090364974684761087, 240276172296858674724026980581848895803, 140996574899370459063111775817315210676, 217383228110917834819208532120467685008, 269650229439424535738313698881777476628, 313324194099759379602512647906485175866, 331543179400424222369177764849108811168, 338031445139540435547512496769506589039, 88012554719877863291898849233234230629, 63245789541832604451644047914079765608, 142984503559856856292530408050863184863, 16633594652860910464366306827510459646, 161784362196541498402237224984175296188, 308424342217046099495932205079152046151, 248002725527363530023002565181756233357, 238260333579673098340056011434231148938, 159307004856939837584965261304574844226, 129369867068536189748545310690128662801, 65453562233959157328289559223635924783, 260289755493444729982918201096648385225, 240314481892072140153482513315729332045, 69985236580536493482965804148292736784, 63135878083269177119449541177816493912, 101297375691836620874552916051535446629, 336604544974732619356679824747353801609, 325151574135821056926675273268443090520, 22847536631207891267242141854161107761, 174241563870776416849587604419414849101, 210268751461017180877034893581132238676, 243576098884384938156708103591327082428, 128802095032522741417678195228896215240, 163755002906045797018506447705903320628, 99513343698067300902638422574901927237, 194414004005381811667938036498022391309, 47873870702673638878071782282196700290, 115713270982688400965787572249370496759, 283042812401386033811529810391924738283, 143584752700899207254155167735896107433, 133607827846512911857635610234175779347, 332915887761890399011699064355737435469, 265383946024193688686766882208495360020, 202178381878601398353099533832658437493, 138023659310516345011705531574265974082, 259469018014325424817777293205039442650, 300636145554446855315321772927755736121, 278142741074736281645656620093939279988, 285148896024433671189557138544479484048, 252085038087667443040516166283860141000, 166873098602523497393714973994505181997, 59411261108529427121444406166004328907, 151784096879533779551390507621946790879, 293349607426966842466656189954760950227, 88932423355237404085802164653418432846, 82067897120649025554588072371202369658, 312499835972935567434920842458956818512, 103438706917604652402334060424938109249, 214218709676072112173509672012607078607, 250632053566004107449036202028934298562, 339184027303952516420028898027555178391, 206834619658718663506786325845688851443, 248017108260912255083631931340046509682, 257474870995730146494773367598523552057, 102747567308554405523769576006062373415, 292407865213698187646769725333588955726, 1696364285016402841398349408727808025, 255318696621867876623583793796836877353, 26629524421882321939983665331859658404, 65843315429872546836601328510508471375, 84807361120415655858040987661743323041, 124504093450203997938992658234982487937, 120555174907305844511336286262393393272, 229151028161834828201785390546463221882, 228782531249906365248886234201764094583, 294020414354947228133054660017222036271, 215349476193637436954903133688923924163, 318449422338521609047232791749667198456, 37267861094244042979392811549489776236, 29719371807653101917644725808013441399, 184878776095447813139144492483577348400, 183768759115506986703719909357433590344, 29756925403709899864458371290293948284, 30703966857288954452046122375054844092, 275278103046164870146518701624531988621, 212464255506053269491487599843477450369, 179541827133012279393121346575888276072, 227523619696162541496700525996299881303, 94284080990267232041551574542139595499, 225659994049264308936656085368217059273, 115180251866208271252685746367483445905, 114140295050324827262311869305135389044, 188097514851637398671853723350898703318, 206321134700268494231116330903209688223, 328217807987860903627639044469709452935, 90972514796255610057928769342883451657, 316571950748114644779187000681628434327, 23051092074867485970919726225147519559, 207508214233271322371436023771620962486, 19502464811941083934434676262923453756, 111117621641882507204055843529895405449, 236123932958862715855007295697886678273, 248713337500919497572323498495703353172, 305317879295974290364894752216011033011, 10708422599591353522267810449380263000, 228645326233837975486116272264438498480, 150884468839672839149447235151473375288, 39359782491464033161589499553933488531, 36969903280185609659556161977308983020, 212149074908447308144349489676476991210, 268266739468387711412780064144531582274, 11869772142932175016336876485751113059, 84436281167875310016178407833593570774, 63084642856068233689174260782966212023, 18344467296974511146199343104883080720, 279142187278947913940362850800564400481, 136191406094254112934732441534617116002, 144313074188541025086896141930082176783, 48916353102556506293639075345727869570, 297094468031839687175531386225547971298, 160133693282806935363195984821013305198, 99325283477438442512505607774901144815, 333711122890372974022144588960614073941, 309224714408634374984263163849237415172, 280411243465624080567458000251497202723, 158687685829122268294014651177171237705, 113243181121621102928979928417331703928, 21370315036766263054486198239741160109, 15898298504978900032444871943755482149, 144311025864352316681826206926965338782, 175675794207539223388736807691508536677, 83295232083213622931233021838123389151, 163251382479616094125606617785259325047, 201831043536640335742363570577996255329, 148640026988304414638736964356456641909, 95121126155591305885302212918345321982, 151057923683379310503640478896717962908, 51194201573067879129511538026176126465, 10559912299481618292682666779783022102, 258166197711038978146958509738348202335, 165959091901770737370709938888076763120, 273137679456988235704573375739000526396, 221641375432982688253754919234358407274, 164234808055089999384908658424345302458, 219481812332481392139990281993919196994, 248875567277346869080229935696362487315, 21274147297077203370602419377061162947, 234301631535716969203272721414312130693, 121449555730278013549411026065441038281, 288561890965991306692970982084248522604, 199358772689674316463604204120046724472, 31528245505747047227292066754050082379, 57330729116377922197528797245421158867, 285049010764885259977679881204650795764, 97045946856644823407109448148113699955, 325408639765972000823566425765640712355, 28168147431841085893516403051667444090, 5307602252615931516475738699306222390, 236475623237103988264943303322203584164, 259259391693315723582403783289408663927, 13951194840096268301233484801260236543, 272395711982831463969941202600763259400, 177462758673391111662519293411667011878, 262546939405961135235643360935811649924, 297230351842345728804346280287402115188, 338321459780804151670337583893609518537, 156014544629148774114248497427468976029, 159787082159190455674947203988251769359, 104968291818345943349506710898486796548, 270040863765567027871071902259621368122, 181350370832746487870416596811696504904, 54931181493048125734937712689734237881, 152389120831830416188020143195686809958, 216159641098211506017156893041791009760, 283307355330386824771802858455482789315, 138517523189502794718800346783805988121, 247880182466709789531327262640017509721, 334513180712280840338379126731613150126, 203884938449531293232329122625376206587, 257160639716239640806488803839159944777, 228733205317410828299705324404894929818, 178355409315543351190142426805472045240, 306947267157489649151382260854971530160, 101578590353561296058819292438554637765, 13144007012168785538210860887497953492, 240881177750777850737903193703771846167, 215924278411153983550556670983205446533, 318994016874140978957934592523026459813, 176544076293010487014467470332290741958, 242852877214385902477811109650901158295, 82867704047053884928595780668979245907, 121510644017992521621710200042725844334, 196665068266232249069685249679931498695, 240688989326865400939268674904559845107, 217058929418308468934872903729156664805, 282169379563253301134525855648783406700, 130779697142343566873824524605528383562, 25134984178732782320806945469699926026, 58405075922126182130776046570305556024, 242843110684528537523791345800382994910, 273320242180628208515969076541181807516, 254970006026745782625661343504764423222, 49487268951210210597088779850909922299, 199897364906301867725845829493281180040, 10889485938143453144610268141396431162, 176572113841051894925520220299423836400, 223191431224431840543609765008361677205, 39973667824984637540184863541275832104, 264034256602898630382686457888015874364, 211567752387262407644311537121367787907, 338313966731234753598801311836594119434, 163476996380793184092559547279061811881, 58372612895575356681343205045164976847, 340008819544818679148547917984670795423, 280868656161478072008905535823791589372, 201090769715158559828196743012766840705, 133204020778179143604150449218075590985, 234083272183208477978650097839699859056, 236691748004613187713566478700120374946, 63565787871438781301036331752725203368, 259685629322333244682274291632358963294, 55215814043075014751790007976616797458, 171120444316144956379390514502248170528, 82454502457122019574999877791758087373, 191388530033993961996647412847615268258, 100506706880835247343297822969846984742, 293320282060116102789701168913571227558, 180616670000426095041899684563404925001, 150172614165185975164260522870912256604, 241977188566005955772691364730918267870, 24831261047270502129726826875402001958, 319317683984261380145596283953312000364, 20164353966492925285350389082489325858, 153085405958347248698666037102058227579, 41006863962930086542333223066176966453, 152692467179429672604825742288052327497, 269709066206456292768509804709385008986, 144451559576219414171850110182117897992, 30372549312284596936909991318800554280, 343895194916721971584956217185564708, 146661310761785437210708523259734822824, 228195071216241917455148466464717246042, 10959724481056236810616178932238753100, 231250615403107064072737662182573503823, 111540353886992722568634129496794431405, 115513922865572867742976701038981628520, 94942389062929926851168250231830912080, 287619716381664387605823993839418477915, 312977979684400447506512449494645678418, 105255768521377217914117486008733423217, 145322306580729180848852744474511256676, 228148505516282223863040535598707020762, 307378572999507003378027338502538793168, 171423353759980277904197464651206185165, 310722947583374948180439507639858586075, 72079290951996447731790230954494783099, 161859707716066603353035479939976023699, 184074639981994793149812485296309870990, 68087917740656801118173143839087956923, 140966299873321569335423911686537349341, 255433389446334682527060382676426950020, 301324096573013758972315319028141857328, 102111122163871548814889720566718117166, 236370569137816202050803596428586359784, 110626047589832553022050087886347144842, 339838396659458886762405876069101048511, 308807400791497160923234015384411925023, 276877621033900737715131339512965410532, 208486036239857911888052341684500726834, 300669331869705462843020133147761061048, 48436537135751334465709497365504946222, 26405890424196196147128196879962453728, 217804101377026300009862392672542647703, 16308997500548034479878778116219769903, 140038186222178773364465479132272182147, 268980748560952701409730038899853856786, 211385613978074632128243891203923808547, 179544162591699645020370904024544348794, 249594270403677038577298319362401618113, 192262405107406093546694670374502333699, 332110116055877243700544460812565151167, 95586455529696588132528612462635437604, 153830180519729032664830713804093618074, 320467095050583471840406408875090799002, 87774859188597578549459866880030622696, 212946673903499520151207380456896384450, 198801122307276151188640358374962076209, 191636811386134875325751268783772748727, 48372293862597842864026273845401733508, 337893717501038565091532415174842653578, 35654834722750700677453050606068362536, 10520750581177709459243060106385908672, 169296128514646742944701891066341927301, 304547649051279934049510875930385477416, 222528188538330416288708679270135661329, 108082505796803938150344028839304797667, 152228835189498456245016643369389673300, 318291178382735023023596206117780214710, 20066783359926685211376442863280699609, 22595965580802695072928718351665895416, 60920619113839990329261044911910920537, 32491721102454628179411215100496925646, 81004713441042360047773793014238833136, 39945835302906459763201609735150133806, 324431219107337615760065274656916470549, 148645253174939286513865539053696738675, 222183262611650218503692997319745423625, 153139063294008888207480788460728939987, 248480059009095820818723235852421111472, 91713372239384184878684162022284770145, 16064699947923374774161747958048650428, 211721501113612826430680828488606776212, 216357251054015052584567839304836112508, 53360254841266301320385879669668656415, 128768163936147439834079768026981526445, 251535440359704404333511263526982818540, 303047374997649144820751045232126230443, 276123127942682642782638227100704007381, 318113431855710429516629385812256840619, 201568471082758737813804880635348325415, 83452792374790661349136989709657645266, 241842679649273960964751347776656235069, 83502991027979263969221142096089823821, 78170489339941735470577038236205171979, 79262599306160231960635585001161242079, 285208970529010816271210746613019025624, 224828363284170205559465520209262080985, 133688890471696611360038869223393968737, 22476063857872921063273069811015204592, 38247782833936898746456988796838660397, 98418732430788129275529999604837065672, 289015448003385470954718443730650654810, 109445539459165331182265374397703246474, 334176663145784577910772858762093793459, 8202323609572829420730297280494331223, 67264864511847537787939615566146601843, 85002327856679207917780130026076491590, 192427067708364207951464115056630664333, 143747190529598199662306437858740844371, 303476528784120420577273785530004138640, 240003028255971420831295421720862753782, 10131671088221140251121213463945374050, 307129162770203165745677488956557653544, 83106505124386353779560165950775706665, 286518593910224306052984382417857842540, 256885818211538725991294437692338289105, 76133722679957553179196905268167578415, 197135941701612929148345771968067216576, 238124671289737645805749227128298964842, 252248980683274949196533009025741793205, 70890389788152553138693458045121015100, 38010899131189922776430360548195153312, 24506437993920215481595919104180161389, 140412278700280070741632480539647555684, 239621693340948078904970399059511644707, 259334301045814407172579823700353045053, 36847262311645055207709955438595100392, 87354804922110044060427877231073104878, 44452895595804921371952165200047887256, 197491898130407648905375366873764742901, 259266475920921618258832295175728588469, 207916859081249095083064888358461048636, 330943222617029127411921768833676860221, 119275395951089117758489720569263990096, 46053340197616768448710074190820846247, 266535143617167904063986135449559699565, 258162790565173443144348138230838474483, 276887573408642643734928180138346927827, 297588620879787541283838216472156523452, 43092483868686973416911688210064406924, 7269222273477176698855959074600881223, 153903766645159387469752228163848003847, 335446837726353148224116183236069182214, 104928679747686237472362387168148835395, 52904075209518814539664853876306399572, 169284923025318586162632421132720085486, 63330693796715154516596624695764689922, 73016141144562419476578920576086758689, 46864429799173821787558044787635375089, 42580887244520232814747897825237253694, 234133212634104584943380170881594836279, 65260623194045020857840324390113943346, 83699719498826783244292118147821934017, 160141599351406922233802937161551324678, 152174280053687413919374891216635168088, 234756802287430485187295099328371611598, 249947355340826964539781204231146101625, 305896859099279579627238952575645984256, 154058251418428110836014312428828129672, 105725694569126707895841105356305685719, 203968561207173385552076247572208659179, 18956866314896519720800144098377243570, 135149437149652021460234120477175220932, 142173787920327423676315891824320862146, 13239256580496704885755297381266871133, 232114759899315950609183890859798347902, 216246703590285210557244925812094170838, 122531190751216245517928586221363025784, 8757987575090917937997904258890256507, 138726714525654447169048668516160434379, 125761346377122613814103425365607203131, 76624892124104804678461701386372936055, 62815128549195068645870559379494046667, 27951297345355029705719271862853557485, 68717550771601721089488790245217428430, 334338740395260539887239292316306827541, 312191179653138183662920550754058843716, 209063986177593329010050298955710242955, 64854950711244611391511074617083840386, 164521508612601726383502565088319432035, 243519467777797506862352210723158673725, 289102001653032928842401606215130965105, 155128473004495878941812574391423158265, 74398773029129571498097251839829377463, 178890030129785137997090695288454543821, 259213982909038766640774552138828948200, 61191332356332048931696722235977234489, 122732776021233339789690161178041545780, 139573139683039095620430527918665316715, 4226485733757206137436633252933540632, 138085836853790588169545365126976626115, 61790352951283211191352190320793646225, 92643058138894120203019292942251002120, 207966950468048564002308898025957053439, 112982416575683923521949788480768043129, 201100764135265761559028965462045091847, 218676982512976028877007052360151177840, 126070722320620259131494073585557247022, 312087012573322918565122048443784505848, 108100991883274446250643187551465497988, 193534688243661030121855476509756066719, 251565607077943510946086661418589498007, 155686849328752057430479613881247095616, 122113445996379616621398870728011211656, 6599584297676777565242806423118675970, 64810083727381087743247217705589561895, 242772796388656286022942080174121615191, 229281754765915775998642444005969065223, 64049378271083211296701189315971004414, 258011241118951957757349897492214913381, 195414294821503717691414695886762730236, 99818839427050992800558387747820909273, 229855946311695872242220978581547960326, 135349220827408894096900682019692424835, 67672225488567810260245527920207225132, 1908986455815900571880999318655633275, 311982748153790792819082232244656162846, 269748023020671450486956433787275105657, 133285322970651886925853825009105429305, 225364997230366907473655613028948733379, 234717972652448090332456549571203357929, 318291021510912139550243234304575861017, 35332599033225736124965364277251489830, 294344371649323150492964765341999418761, 235901274561574374861286808090078553169, 318734790097731226440373142135839566196, 222346926359071150767288211508855269352, 146366971435988415012006888348411364477, 51218042016865868571765620062343513755, 286757984017125436550030440982433682603, 244351088098163989282612146463344683395, 92761492382056712137757509755799147741, 125623303932320562421847092469398630400, 40604817893448639641107333312469845973, 164966526087423643002156923002885578575, 335655217768020273663569552500472797414, 206255915708481594673061014095961212142, 292465590011210174415877701314966958182, 258615573096405552124926844054017616756, 286269966117304647631337392554474767252, 250710373596909748972606087857268638921, 298687706635474306973304294536817406404, 223178568070623827847950314157657904429, 38627326558668014512633343027053561248, 87419593524800268165946119914592838906, 270718345576423435046902389356670339633, 27078415535387755202938747676257461576, 98566535826255593772913633327073085740, 49796553401316076662215788804316753202, 90526274783661956879369480906352515745, 7050692588228903748717788710974676011, 253256365896863986301569026221285366099, 7929456216016057430399148381199799986, 319935233604462472864216335631520404374, 177943558768661612911794613943437311985, 324522533211706744192418895751483125551, 173013921501168379626181868565983819942, 332200251985359339141403630761284838228, 201138179012131116415816150609696345856, 106764044341853737807405899510387796594, 179375750446942430935339935376057268737, 254791948910801084175647124714339300707, 117886504249356534004702745937969883733, 160897017329087070311428996321071904074, 242711030349607539071326220605862640419, 79269823133748651176468542909267646816, 178000422477017089463047118048785904557, 122940202543038598361939686079875380485, 103063221010678018621284482274920350121, 97692014557510729935152895907935864348, 47467576640925621981413367616399318139, 244189454423420167622072102282224785141, 6309716769992256597501215211052888417, 13790634783668233985813947455813439392, 161651100528696988224582647520148583847, 237630883604783281427626198189465450583, 39194187739232291375167003320552084986, 140084233073171436518585329383555740652, 216517293789286247518236372555659528474, 287530354598226957980430525631305240474, 114993207134470684330694107213358634012, 45512959620006682130430023695547726646, 46503076345136820817511187580239549684, 333146353518350734151067889280742981647, 210736675620066113731785974333291881763, 295215054026738744207363347889860736189, 235812633669369018171353101228613522597, 57537067580129420945577832258396010609, 170783173776837349760112570694216271765, 273664639975345814997541164910506208503, 171908048682623300578854015606013007094, 53823911074750176872883623231516662559, 144286689615614333541716959767934859921, 129044563623926943309767253236377280569, 134326154476189277307311770681136180321, 67047146174646993300862682425906407563, 225970512008642165198048790912581146840, 217475408106874841154763878909504462833, 316848188605397889234845621855310638864, 55818056368682998410598272865190938269, 327359994931981031546103694072959106484, 144201516382979516382934999175111640699, 314493161116342355306442753318810054293, 101498816051364685604697131903007456804, 122239554015401655383561944510282930706, 335104508943540902561814294510952734615, 220584037046985317976923146788456367537, 138390487729986463609464647889210261092, 261870858190974428613732165892869750155, 152133630727170933094223557219501593288, 223396018207951036846485945412223706590, 296674556777731401313653564478941128210, 16139995526488038597207940506126857017, 175403453749838854125864446094234462664, 166520791769030465443128837188377227602, 178499410660171019417940229870869116881, 248058093506445330346266841770423003686, 49588620336163984436027431767588494087, 13085409789735923579939003148997527112, 326902627013577026925690757560071370083, 95084764570939140770724951785743241611, 46943616883345509237795239450468541759, 296778545809861684901507058513043866625, 52293262380840871239350672218165657758, 14302841367636196111002834043830342245, 218719853090498370222563255405640142609, 173762876621409622220830013151026284491, 182524844152483995254673259199032960206, 49242788080637112520302884037937392987, 157588934013178615999131211722178046682, 155285956031673063804620866217582989266, 22223939884027041197036760099693992600, 238243419260439508435082952006508177746, 100511136848023661002218263264636120538, 247236979531630841177846761488387801325, 224259691988573376797748459019573590554, 155609309535210270694998090585032487008, 71120890128806028711648834337883842718, 264866163665501285919193325256374180902, 111102301997673852639675891052254617570, 180180838635952918818934699178009622206, 205109936882836726363631425357292236931, 286250727218512807884376170749890126361, 39853277815440651798383601628673061922, 281865165779334564738653294618934081817, 49379844137168057054191065386198122632, 248105708923655934907227260316561434551, 19746770765154494679929125915079522003, 54654313610720381467887732740376637359, 194768004439437729314055042047534112745, 112568748198408364757792843018246944034, 176731318154502833021554928122754142008, 30727373173910228069702205196160952003, 238468495534484444128015912319093303213, 227913742875728899706916635645415782926, 332415937031453251281232636934526794714, 338225532063597533115902620571976689873, 235643608363010749942084920060098722494, 85580678631928006440591117692701269354, 205788280452408750672081507852885391645, 119585372171918408843582152285746322263, 294689438516746555856856044194364735137, 268749634926705971864132362635151607065, 54732758099453129505410126788740589323, 176935804855938592234414201419029609639, 205803963145811765316320629658954403888, 191865004466741052845714933823180503398, 203323451884597555304774015312467315294, 160602718011610792443617954663115129009, 168519945508834392259872619209837726145, 220774887233507339942259278007068138011, 228677586224698638349658685840661292386, 255136983535333958089086667509803266957, 225268651735459606213988037689709162701, 34236293079308792537716378635416111116, 194495125408327763664904147545711651667, 101382987928695730681219802812633853255, 189201713698710437090170442514429950861, 318453982235911050768734080335776918690, 41529673296105234769645132668347004388, 218793594269893886741193632439570786674, 163312767648192190342557820751489835884, 208524047656762634415279027530125948904, 17925206647828310650368973932675311673, 145651163722076463545195106900088987613, 64405099259814668910874972032319004506, 22758083130490865634315432479597896356, 97570312020007996590614702389850780386, 266983615655372334749707721762087567381, 8010362864116034806202687689940049789, 164971641090811392750156295234041193923, 328342734946307945688795086357372580525, 105521191195833897976916773101206770769, 182310702617035385046601447893844672268, 210241019403053222220932246333976944743, 194781833110190084323787972690969584012, 254503681914887759668774512312643050284, 45417540188590899139583965103831904061, 295990213813806866962309678679659235736, 43364404354850872975642799602786293908, 59594564419612246236612563249114155368, 132743501179846576724479605890244605857, 25779256053551919524807701078340677717, 196377422059387084959021440957555723082, 265768613234459228371506637522138501247, 164400892619797731205821664660754939893, 26474505533637510468858670452074133003, 5104322102829801284702973682439758949, 267706301828191678578105927907421890863, 54170349293634499018666418140302985347, 114411087881325634726239435139644392789, 5286393265693611533358291593973159166, 93195207353782987620914450622956812981, 66782354979666367065505686284402207186, 87611847419487958792452760791846659756, 290672451691911364315562150187636522254, 34932541630696695794313389291072583859, 221295823209735392346300670492655263534, 97975125665884906310071786306513986423, 94227248206669528307086948068026661860, 37182119922789159888654872334571448101, 11299984623616324005646999028570950948, 309949539378092965814077207558759706509, 36991999358645709126385620969206158050, 139344586075922691268662185766868849154, 201321844245976090311025840941885651218, 134740977044189397151287401850260727417, 36474047131648280169568690386161804549, 201022860488789090429359196832508087332, 126515502109683849332601027293172160378, 300544255680815006354967421475433029348, 166932316462994512811168631338171198388, 321756461872606759359306419061166289987, 247723595158322332794209398167199104708, 232250587255671160386659821054754503307, 238860662267596657260876860648894096071, 17758059832968119152698060318076563075, 225383110234289297602493187318154274276, 170912949347073236906975287233269540861, 55865371914245088611288181109392978898, 231090669533919190395639648201748469572, 28244949352854093875373396175239564059, 215807411930553038643885497667136950608, 268977224109669553149571915462557983245, 106109367645988206457585205572240930783, 24252810083999930109707085314614348339, 293745845460733220479396206275800587682, 221906911166439361163385311949019406965, 31184984251572905443433805791819597005, 217253354249266105132364841780883860686, 270924587519615686412992064713946192220, 94342111484477936377774238612802049624, 208896271224276848056522200792982800322, 83169002064020043125212767991955197198, 54453503273685501659557669360980835083, 273852936436555289891698142217144603111, 134425488764835126428806544515367429837, 29571489121434129575423578145490678212, 262844935785406855351107655677746170440, 107666619304934385340265936818889632733, 3409712499960451807963421856296966957, 80364357169412255157440684397860051136, 197702298927919458612728807731316138564, 280056770446768745654662830177305293099, 51756076127101826621156407979566635189, 245523844322975053973033634182018090973, 272312750267008220750106985360405504536, 155871156657625858919304765477396650601, 84052696628882864110082878874303978957, 75757226836948594214978643457531256042, 280481021140795997495737088689336130893, 153027654772389448618178600750106325868, 26544921434241212072447819900808817290, 290181780477025239106232539574520089952, 264805591913327952707963065332832690320, 67010036663621056734139244500544879727, 237927625857635005664062878327818029693, 270549530484053230728734797389994462274, 169236751810153990770865501751163206195, 202398577368058654800945722203158521113, 90915859688753627234492182354969269567, 16315824351045971950328687944294223075, 202906351917434610573027880433349561725, 109940850445275289311245023646350292438, 15917866784872576694198843757412148609, 281983239029894305843600383422616716967, 222644013659634050768469355633409230516, 16177427115361422167259525195083119088, 7539865176319431228804368367519397758, 23575819336755195533191920367169292759, 178501674565376675350421591995297653411, 319165699264853395395134927006251956454, 84612596050397058355304234018994838941, 238938980313645505552695890617656556137, 48962744087641569138702587183493361245, 236340022528801318624273015965163219206, 6272872870705084723055848875274662912, 45526479720407434899344740419825413948, 130477390429243511369451499681287435944, 292706951648363225674320342463816769031, 279882584552432331820238682329908085628, 10482060399491596546508122594939482688, 159131732253512867579894828406782567010, 323568723607821222297551419929787073900, 65084487315868360701340680299464231844, 162713472726259291418705100956857971863, 102519868020136285899449221349239003666, 316032621606813638643885874955117032419, 163987923486974742191025205314095783216, 289082574236937660243863486402568538459, 273007978308116774408601221661839620212, 278167408506760379281135189459099132525, 8145879250777338257941606567152652915, 318744522126147745990442034790359933041, 112950949059802389075965469485601037523, 303471043340190070682473161825290050526, 303476297949101040102809318248688272623, 166485609754838476248052713300308602697, 158473200648849520830796280486823149324, 281224236799358462184146269833712310238, 41596037330329668064625640581926971366, 1024106851735423042928842711635473579, 117535901697549499470531813049362401199, 17646954312857827390126490308257492777, 109086989363288977173871530225339566343, 203101390832794819565494140233876768310, 213539742291376921338976090049899674786, 28795686223228826221830876096844481123, 10934224101443210108344441280989225488, 50041291848103188962287232750599100719, 130161120998631449360876315725156703983, 283548985231807794417313834904444420145, 281335715598288632977291216258721825166, 76432467185572245782559192313070380096, 80091555768193636734246722613712240382, 282088357386513333453780738553862920959, 164773246363161406927865256852095056548, 20259523942680591880335308656591103883, 197347776057053521644952905482073445449, 89398649020494660299671065921665783039, 9968651427638012162224827759213186449, 117257034981914704769220909608169492340, 129414069209177103801500508123860670367, 182072069231616312467594103029507899247, 92646395827186279803912737129727197206, 231449533782348132465183903545682162825, 194696768825120051078839266949620661858, 292449094109163314642022648714835490594, 90333227800479869228754949040953680004, 234197524835372396794541020871738002272, 36079828487161876998161607805827616485, 261785792980176665320526083052048647139, 189732707338215223103245946889797377964, 21534565422887499150376261055795660934, 82858728583833008152243010498586423924, 260821509704313676545969669971935710794, 181712747232523553127372538101834493505, 10281467745967617974292158103739461366, 264682233597028150019823898278512000895, 201469557276323754054186650889409124628, 264119298492496278031317470380864019803, 172565480663022135065920937278654462391, 14524945042604405963572060530969118093, 247338131203723803758509709891414285298, 76264130891465358766571910779164049826, 147065842302535383714435172288416513299, 141492427745954166882369793106885122905, 182041746946975637046624053612930494383, 8985859122267711405389297652714651218, 282725497344887319705371066272113499260, 154449408075072033502451248615749042029, 125233107506374061091696068830718232612, 300276332078651565165008567321017327434, 218258571618506163913447424497804262050, 202033977847003134060218185665570809669, 284017152826083754476747163525750846393, 195676533549370701848915569095218712212, 7157800818998281201539728149135857613, 265419358057846149231527095292465252563, 87442839128859228817870805225934877908, 45422582608323543851001031323225650674, 263242798738934247907003376932898665227, 287170640445035920735576710611473206959, 29553451739878063975242924956427959394, 155289227102199485495677321578961040227, 260666676690589335356894303116668611711, 92207211722973767983597832399178573078, 165723828582145173968162432632700985775, 146084256621448841199081754664931268726, 319523502019309651676852063848672888326, 212271266064354528172493963083100669470, 273492911838310979043589933258217015052, 330805694246901826784082688533606707354, 16874686743167848464953325264250525591, 288482635462960395654570407650121470502, 261792377279311830995481431653303197648, 270214764066466920163146820637519016663, 284667294679851324331457546263399200268, 86620490611731183934950290838966963190, 71078861739340677472205248518165456508, 34272692047526205759043220045489903971, 196978615344056459964320259143442312726, 282577143493646474190928476352246959713, 180287598706457735113638897315750766780, 12553728919787915617384111473532988578, 327743920973310939177503362252673226674, 41635642397992295286389965832130000888, 310656718417828552416773155752707870109, 169565832929065694123606493013965526608, 83302496466613055357183352879864352248, 102556689674260996957907264299707470130, 168897995857145327970912388461556080078, 111643864284893961886941348152938349671, 201205410187604158329314783771583880590, 201922286717822176720796154885497274057, 85000644875013551312251251398195976108, 198026200920302904246632163691922812789, 91936537837203342788380780016994535585, 320865786489453544665743206147192768301, 96683404793685015090980024439313222133, 207792947018560436855518567060210420474, 22732060241347721790557904377967973270, 141224173080867710159219162058020265742, 223581044010437386611657004099286303407, 331074754143130948279897238773461804498, 322276297150724612941417155652883552005, 338576209976979278844723965757035791527, 286917812354103603170539672031304764735, 35569340262132512555363119778095234066, 201767585527389093912972494161573981435, 98782367390258307214648698438075147342, 96190847232866706127379688427422418403, 253960505097015644030848908079576926074, 79485108114829048952759264391500523798, 231164806067111465518058513757403576560, 173824930789371240218930998398882152510, 249498939772911587359555790620561936505, 39983308861553997409855788708320804238, 40616572701104494636619744769406436289, 225922349194673112769194063680762488586, 312894986637894597187751769817802528398, 115407594628099834830700966328746249292, 140714359268910088189408813924401319233, 170848198482830568147822249733870874100, 268220264987685183613242252075928755262, 141879168139039251619626902412263217463, 270579912035022621412356494752789806098, 92082385076234054926587777848623718694, 71907279818709897035518600879316786020, 128223847691978946943365717003955858639, 275007808817536766712594556972638619615, 47370833837118611820200213499483995599, 142904482789294335732294748860870910429, 31789309352832231784698974421227024071, 286776295071207625884165759166836344103, 309163501224541983661298825913074419948, 13180727653285105700637726789349519874, 308967759915200852841142902769161506537, 17017679905343748594069268509121506651, 125779155834338526352933489698453003337, 120109984343946038602507336305944449073, 92365808978939912796569929981290072906, 8815654738618774675919648807872194959, 283153410066474845198286046119221414358, 307706305354084534258390868552464014921, 6650066598679819131739017809424044928, 29777146277864220651899502206404555113, 103777653178055284941586750076246700283, 33743394635395390829307881105653604802, 118884301067675576096537188580937146063, 106796609325379803377626866167265059315, 299558831170208468750903532169211889417, 259158993683865912424106969584006895814, 231484435930286114061849021496468381527, 155476369776817159989298508846807735989, 169585273675993350930403730313416655307, 24478221552249133035510695535171186171, 113920082312177682567698061070612685188, 321017748672402652567441127451958856309, 185186696214619937574887615949608263265, 191833229315025470538867150585016737680, 110999014967286300803085207208938012268, 26474711991363025261390091146810718420, 230736306908808702371950069323909009621, 79349419925338264323886351129677987508, 67946206640920671279888271060533434139, 109435957709658436526249293482030728785, 338811912832829051932263397431930938397, 93703723378539629665229170464529873593, 305253638988961296948439240355379422073, 332656781095707210467132766590317443737, 64187989228410783001254870056284380659, 105806125829051355108743440428254381528, 219033165290959355998275237836017536442, 239574111665430277212900698863798993400, 142208962129284400864914774781936387470, 293807646680968178320803020555156641716, 269963493472164088886734062032571249133, 116893340287634408317302220206479267218, 90001054055978731698931891429597927721, 10287572111084033108457218221007047796, 75416895935052621545954631221119704023, 77756321107303262976693414309184568384, 133966861767466900385635351764508283625, 214808818094030483261265782595734411773, 313113001502757803907148339541594287547, 16741239173793607394288244117932886321, 89693243438394939888602704880436564365, 264851097308659812154677841202227026898, 222945421104470195801913746803905931403, 24360634052502062914850736488361625049, 126238709023376350270897608917836577099, 63786868606251691883734621124387559182, 228528007233369670419875389256488639533, 280067144230675199312280485198373852837, 153555904844746898946021783999609836743, 141708697363180076663742930398263921199, 4389544363570019494377558808484924, 3720701213651890096899888789207468881, 231409451200726062281796281234683787984, 122118635364678769027455133761525753698, 129709730665574632894176226796856918736, 2718298381324790070910815104790092005, 215000930476368000667313052392982162220, 246334762678417721398872407908936679849, 299300487787014035203096383268751517596, 190873142388106018528760732681172358110, 273182753506369452439249400245047997447, 209027654213254601670101021404453120976, 279117540637510841917103317811121919083, 335252008205432916739359988087983806250, 72588315237161587168172438915074777455, 155232243146460831429312184932291282269, 113315679313552355777141000833778588198, 232820350487143396455651626022675492752, 28089908442910413332584359524466019219, 224946311780271349269551889803060449258, 105468805826569204533159362935918956543, 236769052262492156218074213582555780511, 214025312975397623429287240045352984109, 99410563719841515875341118127951221159, 271277382848197575533158812768326010212, 198494603959709287809769450229427326604, 132427087797976075752013957025674232953, 257790269677978793990377777244545613556, 131627243241045362040541011476049417398, 213451189641913678894278648957158152797, 74014030602503180890121655451516379127, 167286419433233676468744677488938322141, 238282581458481170071079978493776228408, 36915477856089287498416992823328230694, 113223631140143589422139740482834508518, 283149724151969306920052206317838952293, 241243203172958039425954059677285408746, 134017623388168637463637630295383204519, 51408747739264172259260643472609425222, 211370083375526107713008514629683996780, 339359145554096732311409357145134393296, 198763932734947765933670251868218955833, 69894002507562725536956441764581390927, 80785018057895940705823818291754813355, 208750707294604258513175892894433269564, 98660286730971439038657526991052431100, 253761622376820387110117884340600336699, 292160150698945665308829367087231021098, 310477453072030396340849044382892777826, 189440099998192328208108926004187454998, 325519379175020804972522937216980524914, 113128637510994088781956036067934903174, 216487314756772064000398495622828117453, 89658777943890413238523758145493954146, 170172076918848839628825167846575345955, 79435386092744486368212467014280557767, 331526419851901455105956337675954341985, 18025179102014249522346312161859944638, 107760896742488969509490924006752400468, 43247710866166681828507224721794301022, 123717589227883163405024257122270020882, 124530730119293834677376021599118316797, 236269362343286994275742782358486395553, 265629378551760497037440097925519090672, 39936697411866683352498476605678324142, 292907496953524856638863556027814674211, 199314578117044306626073787375209213605, 4454065907884153350697948566290614378, 201678118583701379195080752887567059295, 71328929578921166361073464456005025375, 108247029614124372950691852766453115865, 112289446240842334162178412037829876476, 259144507651815984167470799367143189806, 128245402836222054948559944773809005328, 245424004956234361459932299250724266673, 338759007528970520362146799535236749665, 66665264314031232264334123774654686827, 153564205700093367218180589280912367088, 189678118980627481638729712177188591452, 104989272820407510238146568096461850019, 173705905596702889430264978987715458599, 301253661659841261853649948367270182132, 129359068579322735527590699200040099859, 169344340080670619702304571375696325828, 157108551289820069934471008523886516801, 125199515875524206838762784260308402946, 257687833586635832974221114598598463970, 242884694223475814757385803707633613083, 259670127088636927365174244033202109480, 213508774322696719720124423303303218849, 46393914563561174944604960896307613076, 275996047706466664319878248779666054718, 182777531144049633879361429579425963820, 331209056394631630384489247926969584750, 205499721683256469677506973035276000641, 52186045385192591610252783852877434918, 45749238288492181215797387473655160986, 32911058349897935577168237560956855334, 35074854220254577374086395339942314383, 72409534776466351437234264204885499405, 194560583872346332040486082803292994191, 161052150322263221359684537059721891016, 50259512459361381665964693967582598795, 323295846687166491257518436588576075682, 159147545316390336182820626140959148488, 338307528682450678595262223652044112457, 206234710499470033383102147580534032995, 19631105542792947323357723585564445573, 75546536735023709332838817474410134085, 85240746761506960065247509039611319823, 196030693613736268171588370389594879078, 326080771956820463564522146813934073324, 148959119780399128634247661091782625908, 189924723114237914833394068822835639391, 331066974817411507356833272666364594382, 253384575530879592262870640059710351794, 291162033905281993868794301492776079956, 36958184847137673356121470849039351286, 272627175694348268983721811992974992124, 46004802404885808770842364363568539106, 330012698559217974230486873165713477914, 111379513885115719625111464413611007139, 201282004258544516659171410723697358542, 337203980933865230859056297676157782667, 7836281382718452753643869383456470464, 211167350976232775523012693562627004646, 39691818744042943636200213529646468909, 169944672658821818975388920203673995424, 73441070027169049596364901947142587464, 224325252804947798748030590290118132126, 19136459038569193815635220160270348370, 304520871115746102418809666919253212636, 58884390597345101833727821755158410005, 69357938288495467938612196205265532984, 85179576886918681728289411657763795189, 63790607457106415605389491814958924796, 198950131607351760745512698225992136414, 93389549593213473649017026821281096741, 184786193285819827960502048510740877428, 181342957688819586781847260240488000504, 221515315549380413714528749701924610038, 63659465759642413105044745044692786170, 236987302624594311417695561463025849618, 290274114735428165035050220036342821590, 302502158151848426104046786890507459061, 216297080665834435731559626381648426407, 13119772568976141998713058215440520573, 73726526331467906791985154619645628079, 147610038348844220930386157369307431482, 162950989999113237955923627341802841973, 34526922705388077017317033126391558150, 16054139444843972070410837051383977500, 260504226848480800597146642302687516972, 96496939688813738463828641104178770790, 254186957345896566848793114297333381420, 96972590281285559472270554654747086726, 313815893625352430949154775441486825413, 165018831918194038656783151962689779306, 189579626546487516393189564858540161158, 54596653699034363923947835079217270415, 208936352465262669375650431253352012593, 321621481668538421774205029168620594355, 309050455351509014023105765873047315926, 144636724342176456740126805920772291834, 100611235231829914190857089363858338058, 165236342525684246457404624418148716348, 196665520317434758641787387764145278984, 19260673513857158607593574980222998674, 115076614614312729270205951378213049036, 142240171738055705102048967371072195757, 130246635456032619652290567345119636194, 102012349394427667781121299530448324155, 51336537449521908158557052277475695092, 253738630918252169382174541836515686966, 1480446464498050304817543370906966167, 283858726901758940079204556802092290076, 270411470772614526563946071324667307921, 2240043238104646976872545848443368011, 148420509063015548649343594857957656371, 272598258467074174668263315250448403243, 142549321154567164106894679940376303947, 49890086012528461162902482751378717619, 236739319783404814681896965522356927447, 27940158434844875958560144711203482739, 230141132743933300333563966917061916694, 45214739310106777651241976086772385054, 219959452565812433041830342298479050257, 984688045675816068731200936445573935, 297896278785414473298199002529168448102, 317572931709240127492911311557413462242, 57388354121423872448272375418674451049, 52723894567239066121748505139962997798, 28207179348886648364335953298356322121, 25613738580406913987953463587653510651, 158989220638989280928884278830315416226, 139077872603144493909608566841895075427, 40397988811966081386175180110339523371, 197410349660305376582190735674267064815, 327818650437517055542714852663726739636, 132123344283143269017082779669736672662, 247393215075517620797815646905728777965, 218472134102513410687662029601967549705, 287711665797554796630611559626499573677, 24079355207468273127912519509162362798, 258363233945185578896541588513470473363, 242460692566609927134863907787539288310, 20356847095600320973572547492505442706, 339287989689274628520642527101614360953, 187414350617357974074913183379085331592, 322813967503105376252957999184460984403, 296952607530907076059074278825174278579, 114544968452211116193580487408294519652, 285996543070897841307677192838389267403, 216388502154113009196903912083367556833, 183706394948101778968862367852818464531, 57779638638615586546863368208294846844, 223725422736897755911541608515029025907, 77712639784857133247103905277158461737, 178439646060520929703164611244698149914, 80027624564787476090629823771123143529, 57740284452549660494973543612118409610, 249323380867515544187036449197628409632, 338639417092901933553238514304901084, 165764313560638292225559231249449625503, 285775040342523292081113186410393193271, 123012866405337664428313910494995875631, 76635541103385447893688697315578177876, 172275365084977814691924047609904438700, 95102878704365318915616180816642434104, 118416354733970290407487740890030429573, 308650632171967303531299234934346719192, 311122107914881199532296005776641555429, 118101146096530284046672076801131135327, 209589914019320793553235580637986893630, 165924320954451106859477322281102274809, 261684192720311252776785525384980578291, 38921328451533399439000195924167530756, 200149031451444121817637966081734664687, 226702886324914522368771583809110640673, 177833438973077833766761059439672788199, 128631891325511932183295836033564389845, 264630702708274484961026018712555124444, 332177016836842909214025922243867784749, 33699001223248183660883497183308042982, 80479284950750913341535244046571270020, 310754497088532237390557245585371534861, 295146996206213960408878672391021021156, 107566475533377139402231267400544008848, 315288631201671443092648191146133233491, 228859498211117019078708146120843441115, 10588561249109304936444823418515420118, 233924666505114716429901824134621208018, 333334145016017394279270497693629366413, 246814675903943199172356967469482545848, 369505045782240821915890862690895702, 77459289181082098968430103247097319814, 124964516140236999885369817111466541187, 62956248016842876609513379632768197332, 175485109916549211658723036307921808330, 291859146034202984064876014335045429178, 120897509974519151106339998460446090872, 87974423758695194331137221590982163160, 246272745421814856535384846246697688272, 1544206063030559325154043095075117254, 271335884782577957241941122576098296137, 59950638712062123355075264950535697506, 156155879140698140365492769694972557768, 116578000814813710817612870397731557953, 322876067878946778717097230267317922704, 288491564182463393204179227426207593474, 15262686854009272503131669657026141158, 331456464851317425210780153616929997769, 301855974308040287090127971820758556965, 4304951185163322018143843783726722157, 297639628521203326536637872984527308540, 236454278802101440414026619591332313524, 81853049834729495575479465885785952094, 122961374201102474226378949394872904831, 321084880115159738729608213792233342308, 108457726718306768833940275651067367421, 262757543262317899559349004310898079219, 7910836386298512518782380032535792849, 128068569877428729056595867979605633551, 27945321897258735692949770413041225190, 225635202269860270526101627852114981601, 288992758202261394408930316915384226209, 330572010075897220408256014638406573186, 41840212544935901188199848909829047956, 229718021996726047132942118192643754693, 245919601484807792992974015251543081781, 330946098036491691379113672678771071204, 167650903319138317886980039420879935785, 52687282152993007158074254144649746383, 78777366450430725622352971870663695599, 20713469611707069887091281718347882568, 115265322545567555733618697697791850039, 56524453185532713040835985438610969551, 70888173244460098337925321656995037423, 167412122018265789026730759310078587444, 314089555989776646225441536924544623046, 183247939852982798297447603787321586283, 187265330152575531722017601362269598515, 220803302545743464906951182147173774424, 140182352068273366740364005415571716372, 327054962150562129337234850246592164991, 327364758321373482624017797776900871536, 85987524183223469724814702115400032620, 17459181619618874669907798714527309104, 185894973179500404482046158272895065504, 14532059215691192869551923211618424657, 252707476944380313069934826865600742668, 94270990003881578712448609587791881326, 214037210540061511045129605833322230322, 117624560196416120507669172246189767444, 328659617743612628553071123727453049474, 154939747423431788187509968957916371538, 236863738627502557535927912070078086559, 157221693414906616208630680664163044676, 243686500594520560183864950028546662459, 228297349016473465026010216868432150640, 5674088156620430910694575405250869868, 200783614829599144112554337712823154199, 276214837027927099947093537772769273601, 276061929335118536852285034568751075185, 219357832272455171176952258013261437636, 286008456844394208676406490487618389539, 40899702214733575462451141968141432650, 264144450527660567908011381315867589892, 254276110884619799737166642505188627600, 271594881283074192481787504567761118608, 260687809274037643528758201241065160982, 64531471663360145227175767010628809263, 174235031959602201416167905945629501010, 118249443067366711013909669244894297207, 273141378467400372552876858869864930818, 320283298471281817477726861660343621307, 317876140837254741370647924084874976862, 93425021098668187171567814446610400202, 122555791839736113960685221234156179764, 164915640586365285473575436288813760550, 34235245205614843076412814717670848597, 171066559705381094126357619157399224220, 84854065634991910096105224446268965363, 173239839470503806611328762262685477436, 171960711866655894301656006331849090687, 172447679472824732139937824096104765839, 21675744723015565760863432152955284240, 247315328463400054937048415135639301691, 66461563753137826563380254501029216892, 240318602306757805369620026591520995897, 65026975597484342694677699088329204156, 328555266528586586515992272312327318378, 272153855231187849235319496548693025883, 167776994034272668755102627923847743459, 190527161085654562370572343219077619358, 96012566648293358790672601966609949924, 189079140824421676005069041989091222553, 265057571136678611247325651479601436788, 84898306789411808235326665128365565786, 144121656301269486661463143585511879468, 32683634422552688525161624448856828870, 224732718361168062851810399202451842811, 304295146600050701062699917447469318818, 274412951136416893032815900498315829179, 312356360744969003530290022560459240154, 175476614191053284534574155192724945516, 6025483970894834088595018198910182584, 114864500172864302709609652344193750337, 51058549928845739399643873846155024532, 63267808880130671719529831329241909230, 208363447493534642854240954865085125022, 24542601361098997997651479434982823898, 176713886369278007211700576650234760711, 6664309607286428440927355000064345385, 251546889100876507716384436686111002405, 173367423582974402914885330735133626982, 155723794540402317252893094395280429479, 121750637483107651652803810390515819901, 299869540774915625410008492675134744397, 261211001010255070214733088884145740113, 310453092594576102347244037628048228989, 247204835533864875888613722792819112030, 302799256660299140173249613826810972039, 240538490963097615228298925732680787267, 181228651530633901061419213062911126994, 27210669420513816156322565283225896494, 36403898604913920368329852620506653376, 69376138547142521349601446617988475240, 192301697975064154643095593136337914933, 105503677790321806128229547427789841167, 319374267150344140156395443604376440198, 316392919038303788109781068749810918929, 52688423797714453794568582235113839240, 328922854221035059467442767658596428415, 291752456364376211313455356424686066700, 108600261084541726448654901571044636860, 294292709343983235559875631261924090207, 329585980681690042933312208661461000774, 30951465700467923213055787493448703751, 3060973708855927999835912472796863914, 244598708040871773016092370418912610348, 239144851527847398519708583087754099752, 243418241665664622862944656976859640789, 48975102441905008666382936528142848214, 182080135460036534760338488111982993218, 153351111055712486264266395904525113932, 39272863045359641600153474968251811208, 201515799212725387189747676289017476571, 72216883477486441027250036354555996057, 3045721370156293427759139589446936997, 294461260295731429771436635595517449698, 9022241830465948005387842729184300230, 116803006596272939464529643821855477440, 11522628209578806272679397514628239648, 217879489436960887698171567616771588862, 280462699103927326501421005711028653190, 306179716665976228676762448547816262477, 179986958763694122876598879844060841580, 233627507545068901892580548902028905123, 76566353552917533141829590132528435395, 175818201390503884277569687258176256703, 32975980386073356212554303971185585553, 222210229959519907716953425069828160757, 151645484006373199447521059409365479254, 290394371567101795660037367249865897290, 66108713570175134022471625276472917627, 98626601698531047857612770269340071338, 158352610896904035197879676088406813172, 255450301833455988373455332784687363291, 144592897950903740472069194725283750787, 273540669559031255066147642324833789213, 113799940487022146071407171073330243611, 164205077738541405888026945569728470363, 259256492023262167428219230384913039782, 2103732997935998495690940696981971962, 220813492635112902113635544695233854276, 50154572866642662148593515367205780861, 314877275549096407946319373991000488365, 115611280318590656132995087667409669098, 50587929270860032958469616896637303463, 284390984541559836676100180475169483408, 18256830293626086791230077342679546571, 126036784936008743569192537662212346322, 106356379071993559801690843498063115776, 58830546155722198014533724408617103782, 224490650747386100411148366968989445445, 317245137068606438411560559709394480475, 305643337500574145606605374517767169908, 124899265475679988835470066509835143922, 21544455173834153925403753303901189409, 51685638376799770975016107938849822703, 266970635586949554836706499590156317087, 127802739270346402534621947790880523800, 174918523209935402111147601090136979834, 302748426625045133933876829445413904978, 223113713797416267085543610917732986429, 8353058385596073914189014428001703625, 114296128258094133031642381511116823723, 298110728732467229627159527931667729597, 181623223722216908256191569564853310827, 196804219001158730129119333944427850093, 196181124712207955769565425833822883358, 111927295593405840694902031005481109318, 279160799439799431298372268147449955709, 158801389382597500398260791025451176336, 329397641215282843817245359075329535286, 22140942684366928312703266756972205614, 216044029537615226825465783863273402713, 3430939229359373012806923645110607807, 187654311896992067319099185709976065124, 21496272264441363518383691834753025468, 301880447695731251644284720998267958957, 130524393407261927015918497454480245595, 237528886959269895822812708927596186842, 287351648582105647280527712440300695490, 4022464036927915197577545463195389430, 306531646698922091436716617258619995053, 229864407114588071037918474881424376095, 191590550365652151462766243586660949446, 200904080200540045177094137182069523145, 221797513973429644998855666635184790721, 290399280020729976840286455607606055910, 278288666139006086586529200098697246252, 322942019483272352670025840505905733905, 254172801150915314686452022569554471141, 15799609121752209858996763323765769896, 54407530137913665116573559604279198092, 63719062491608906038846994599726983273, 56974849296135901936418171302781455039, 124879480654442305057503787525167259837, 323759658339823102144625114014103577806, 165878947659804780555061853200475838690, 297066191369057584274046797717508783414, 183214218801249698360529171086430733307, 105610025698465745700759242106769463295, 200861246290633484266495334807442957302, 31870936845550779398253749727520850784, 177563105879862747361967293572533151620, 210798805769425853854420759297631410217, 301670894588565431415759655597555637198, 120058632183178627661826435715254349649, 162331480568022832603351570624801092382, 52989711722942514177401784646882050525, 227397012470814283647764540961727108744, 101731665794744043169588018251205157755, 129148409624954200570650804703043201306, 8367486699727141990543782504476388841, 5715524519637192959107821991196472734, 267627546137814951877920248415278423013, 259930356755381148463358801568621689128, 267860008709647072341457354292904035826, 285548879718761330980198528666969125878, 201472622031744650326141338569187292497, 13219293899345280947930826463505402995, 88915996526802421513552997172917020676, 275024891554545580419987514323056631641, 265408814211150579600278033968441539097, 167074191012660845387820551354239207044, 47693696828288550640356467984820320866, 290680724665526343368542188361687854317, 231357491601274987168089754141127929528, 56195554485232494674621281421354334691, 145931176796492878843123852405622686413, 170525397152258183887892269641164152688, 61829076080026774691863910519371928476, 200126024868921923463162591654240972844, 113337999073086672101605153217431854691, 287564778243766260469133273667598596365, 165271982628362004676351004785570558, 14228603990334534884581192876771901438, 76661730562096128153769995785879733946, 222682512123801239554701134494007332688, 264120453802856155486975638012500103596, 87286610242446709719320221419274537927, 84804219185305873242739662443545861575, 321632003733934068856183231780323230762, 275508462317776786901686665065061433808, 111610869338188658938089094226176926557, 274642330287889836540916209986409712934, 315794403812173193976621618144369545137, 326413621095744274941695290195012489693, 70097507711122970503762805743852122277, 331641684514091090936545580226810811759, 232599360665821968026229475724216325920, 7007578559244574359191684272339733440, 12176757534469693105781789995552028197, 273702162692404521349627906269067624504, 241149670935352582964434897982252728775, 268827944125962064236250757430790412317, 282001841743860800535869866869374221356, 79936267790550558364470100576186706990, 194715178807989384393861424037916285580, 149112542547791336867828651447416499428, 296153658660378372620507719900766202315, 132934165435011232182980594374732216421, 293715138151360839685045812249941809157, 306655174269271724686646870481892666255, 122060208726060174003013075794136739687, 302649943651581766897528696701563253431, 241689071087323950180675023212945762611, 24695453763144223278944710098662278371, 34222697254323473665261900933693226647, 140528332390740388163638668558347387324, 300009683134102468650988527989059238949, 315768174231862687216207349210330310734, 30242567956684902748122636028297377038, 102571370910554462094736315118009192160, 168108796197428125966225285393135572837, 53696555135987482214922650286806567168, 232756847111437464166261855972821557272, 252966763998762555301642638458116732500, 102326512647736124311513128543913363715, 91994679401280682068990764352770416617, 303515809331600833258970797609926532581, 42834632615493235985566346231979150582, 316618658729060299332160580175065675185, 250481147632491797263612322700318128347, 130069073010106191184697438518650682577, 53017287936751814103289677435721728020, 172868401423438541300907218213526144918, 113320460157563296425618396897966154012, 151521376912884022255600586521472320415, 4876465181972327305076086585706864697, 269607842099305401095337122201819178661, 103235199926991543246646684928130954332, 308126393029783797835049482072197397098, 177939072063494502401671183004435693392, 239000909334340156956870069172839208111, 304288806379474244912644044195993905000, 189465923183309719519748543893470493867, 130850743466166121158662988813621075138, 274037224667600503551215703222999293054, 124110274359354770144628939228360267024, 326261451968655024182450706948356274429, 252450566290340708715317383119425392096, 146935356192511436448455617342494843137, 126171308647288723151489181057534188015, 313876358339048719322381389712512976428, 25315506375011230533199903925635860167, 84418385919801263025250873949886741243, 121706073994731630108017712508846780755, 203450348962418594088385820920859238772, 128318911518229016247717437743519405071, 296973497363721656918382967096737669850, 27973358357547046320915208089703514961, 42217357480861730269267448012986440691, 101462591294025067502795592436775833642, 184387603888375501532623503495056414505, 48207999440608075978233634331119566643, 223514324371122006519293030642121247819, 287450051022622779105237433296360073876, 32498079028159953979401867935896757158, 87834215331234711818986439873319341048, 334484791144414462801082076059874367133, 277668937515009682804302664080573679008, 59811670873965163690136950634500108116, 276166345042524115233794249243449381545, 307808567631744876399173797316833845779, 167290119775782817483843669585897078155, 146757848794774827894436592146346575724, 146480644849120183173168508120260729207, 238841036534887214440353490976924260808, 329074209592575495203457491918152519708, 181333673861114873153585437531017830123, 53847996033097303772765941505541848860, 210405170906077286084302399735822517341, 85090405483682583170583355420811271969, 166801047981015586741396627443953485438, 247784668634397872062999495667751729087, 154943208067338359214464357478484030046, 96714498577719299931594503585560597177, 3671884747587137610030276044172856331, 115651725081140197984947174878001721964, 82384797279327432338335175549968203478, 237532114735255277598198832914820809283, 136596226975767979254098639885956512937, 25120552512017599445952531659038455192, 121643896191346132653206689279503086095, 114636888438261859463519718032786179512, 53561198212158611135174055909075178335, 43941849472188301995518203689681779702, 282558417177778532200938011183634119543, 10588832169127234580556750816992343077, 230264102108447496732649242359338660907, 54685844881515613503595937537513738217, 327818231637555223313899936832140800462, 193980893974097885498663188642897583268, 212095326936204512287186855048499004098, 325518522961015476991732929051157007258, 269533645148805663850494528945313277859, 39798270260380770418486183996389874917, 260881296961877824928520624903501971241, 90716691529233363149083982870816916613, 176644785635412441944565491703814747554, 173695872083740443683302389696359487975, 214050416587723732520857158650283683508, 126454238223483416814838888977562903487, 144178337743838149733131736362725518191, 247337722488765388468059673433847231884, 174034341354211089761576437318311289075, 80026180489800065330927348614212066821, 339214281131053121746384834650582916779, 119146770769710478941296509940579518178, 276938347214219354245787747704549172914, 8595538711920021394672706911747939274, 79888651479139758551028089864332945369, 243139258859491895002409040743992615488, 195843858541117975395480186699303217097, 89187791963086564558252786884944778436, 225535929046755820979995123690016316922, 103819480944297274051618202433264488925, 327071967136518755686096137116901434275, 125579209489599904527283691171988978641, 335251057084279459654955636541770643757, 121372873129722420169915290449357238925, 151612841211038318518572450832388295782, 86999539118625719719645704014210402879, 88171915629752899482309253083127274121, 199326492585014199310370497533241794401, 92800531602061205511742324568446358719, 194750550238276656844190600970992984770, 100340714928083121720597711987489474448, 305958045141621914915338332065657532837, 337353034271656649336224610305792058106, 332884207754524770862999454436076362180, 128567231165988314377338542447713367590, 193168248279611554324551343525432907089, 185874067076096866505927109260170998620, 247178240465726441491606525206110962112, 100760780275710135238966459939164922048, 267640921560891320919580099744938708146, 198849678299711088173073115218850751427, 121569030779734321031258097308004997253, 310445975518803998714970157814819252945, 181179470451149596171095515296500891803, 137826509001925810531406301044938413736, 298479295440389716272075918016194031469, 290606105448424540361771192241815374637, 279408148943784370427705096378923369926, 63435885403309390648547614278546485149, 212680956249333350691451017619181188915, 267920795208515636987877049565358861896, 104728269162321928669334056485181330415, 139969414978821325408161706608609437100, 304646672363755996164606969490473398921, 110540442678793617863834137950887052476, 171692503406381612350655049044197776698, 249155645249785194325987108008154614299, 29397731746886297848015893456780967811, 49961017476639112105323956006504233323, 190293871963120014764874143327639017982, 174985833090439706609261422662471257327, 29463558187823764407754862744981262041, 157372500022616137405861792850657468360, 276525561975238816016122233164633459918, 11809031400132928840680597062948808945, 32684221338557928525923511162749025960, 46844467726975144412443746623532450845, 281548770796282640345524456089976857342, 269719928218137812693249077551792099034, 214929370105695568971076168544605479724, 305072637701746894790275880831602477187, 77845411222341470850851471060988172274, 20358172764364062377543427806524767303, 91561265532986567780761409242344238765, 41672382779360052981529316927932377959, 328257298943726125200655522645518314540, 70548042898943940728159715576865437552, 156728660979691643788798830911165250724, 276037317600495228101638241879986619453, 279865917381853812852954250556429960084, 292746366153818075193756778909563380612, 229416017595311763373813482546814793681, 161024548344653361267044629885211986267, 62129763071542131976565094355756096250, 33382654032520386320358388679736195510, 132910414107100603824755718921654733421, 294270221173414958379836890857200845710, 311686974809094744061252159497712340900, 118399583346123537985838686914296323283, 137301328358999482685405067047227135251, 227162295745230164303453583997273357901, 237365655492989095611434876790177201389, 46151370693878107342786287189223571485, 74606749598480073774335515116288533834, 103816551288095685421428759146326636426, 78622469016552185406966176538484920570, 233865472464402991739640558029412891405, 328326581393930987855705457686123815056, 304973180024088242262218078459397354886, 141185394692964761677167115845398451935, 211708744197255078549853668450656842477, 75230624577129199335527278702459164138, 293665982647248221742437646017879222776, 8614656230480171134544752168436825203, 95343883097887360654263488896282232647, 127317608953882202066174427882794206818, 138062311215340450055225882314144760635, 40125460277319320122256722100506399231, 301066684435696867576389591846347657019, 91828026741334315773798758912587719205, 208158706512964953908252976713758369968, 77284704403309652497589576565739841193, 113698767198252657375163956831863067552, 117741509011699895466248097917476742750, 205189197452395418394822677378603940817, 245682454197686438611017277241319852712, 84615216989773663024872548784729733826, 218075080033646915070548212135303475682, 319523371898176914737099247719831552858, 210086528376535651150681425262461804066, 286233649125716034150364927229343749780, 223917759788648197923838075092314912188, 192117829865597883102936945699803027208, 237560320667783303584376642894799458759, 221178732802345722818515887307508488104, 44281702180846854974218685314873817828, 120335553539575315290705228071324560374, 35975250816163449864881494335004725014, 33074661138265119504697717623091674509, 40594251721837742688029925856015938320, 157508005964226807102331537051287196816, 313408559009173998878854217872807850517, 155641844347707144106745462972469656061, 289199886478003845577062762711313378685, 299887964860238045156364153458635937648, 315130726912905188755885010641382780023, 205768468194938281077379208881471025189, 166300955413909612366614302590658285940, 29672115336316530272829402184105351822, 201532177497663122282561745128820517799, 339003062933325153156314617136055610251, 101638200878641700655773142809718974719, 29836776311192529399404942064766298511, 112438063745997888806498665436029954338, 94567070331442133128488908998885589290, 186969266004922956350891207664918086926, 286962923113832803198831111301258235109, 156781915060516018635385845116922777114, 13366997011633096566641515154374778744, 234436203279185847577385353993723576882, 182659389335221704146630016328967750947, 119797361953141643545360621256158678679, 162310459917217552284012210326095152611, 46534221581780915039362210468661288132, 188193798102559403677146352013397880295, 19304792529341904253467276159872997236, 131308748636204483157422851232754597479, 245050285168299577794427375550681381167, 97129844420482530201073457582898651061, 65965332407668628486269226303591666963, 234227497867355319050890010703183377250, 196514207717787710283321718244787954121, 189600406602423276357918299460634619091, 276669748607371536844101171345144468998, 73723314647166082839608571625312307689, 187524165354484874653864248192812169249, 37595926388468003500935939273336164792, 62661131154953222020039224026197405322, 66890443420442480004915514295915350011, 157994216138971674094163426412370124151, 186292353661368485978929205024365411594, 194021320444001772606666147760266982311, 255826854087100070325596740024720474530, 103939134719512900936663226632017666785, 248804947739953542435112576329035587280, 322560811874689750141472686333591731416, 211434045970081516162412895352569587662, 273185514701845700680265803304904533254, 135673738928657610191654370586915074561, 187603898141524766910508221286533689242, 58641193450713866048437730024225965669, 207876151529298462592327821815600494541, 235637151353176729744670339361199274216, 150172647952709711609759883957674328587, 45053480347320944455629990115304919643, 21094529019088655903312329790107196144, 133114991252864376420611567427436163584, 228644643172543059061494214352007951365, 78454180161246102545187745549598664384, 66246015175900687499214853674084504794, 306718188039887768021507535621586478056, 2685492705637745620270619759840683097, 248281888188242083811051811412912122273, 222415494564091005798192094381515998875, 198835629047780855933321744604311139179, 85472465154367820773423510574471040036, 180279453293264147680694754361808722601, 100436410095816543869801584692866729608, 26167691659940382553680991483143309829, 218042090628461939287735390897666355521, 337516183568146684292646130623082556630, 183227821513619266888632163932543480585, 60402940385708358158640050688936945371, 152487190962750435388500636199810907997, 296522725486322739987688916486311641517, 234085385479278380983920336540238415810, 171508937133622243612748054520502595893, 36998017073741701621842447441644067757, 154637368489319567643621724358621577105, 302962416328342950132196062906055653572, 308931940459148111936624962827374289337, 230070399564077941112909988389161470344, 52154579641485202189359827193897969971, 325139497586682435635488644420693616712, 28240301685257104413405261203688131945, 190761525530946248486452200484926846841, 319472961695421020556547392024656914896, 59315773071496999629866640339828426223, 245742063788177130962106010376473909335, 6556724765343484796898907806061558079, 125685785163344493236203111083453688329, 237571518561425590277589861053930589651, 200195044524416900750948879788950438921, 295981778397393433247303493852076173000, 174966225803119168068587640929469699422, 99242976475306396874001866958375496193, 117138780076784951945442256998049207299, 150238608442591548226760810910147481641, 106493897429007454565522993445481713694, 9839080385975222319845056099943032874, 242086252862291393855487524801297005777, 94150416563568962028291304700109992068, 31733393718181978775341476573683129455, 138844965688257516733717258551731680392, 15193748304993764166411885992074542734, 175840834301062720232156668787415862358, 231140122644175522348784495481756998151, 53914885464032007597910737787136693594, 276886294069678254106461869877337185325, 318331291278480325386576667723223622523, 326001583869422394464747703459657282133, 87498118524041089487886016073368325099, 338751818031408232191968738588118521346, 25740794960834950336456394702583896584, 4906926945855731666682838509843388834, 191676394367117166609120793856492891898, 160884434524580484461562311098949688148, 227414308573456977317089199704448138833, 116458080037891896366408494150271359781, 118980959121655140644407739749687022196, 46431747655184643593501357215182336058, 255414602630396738725249885047758915847, 268394204257261976543665963947843084543, 173753260291167348397984350207356887603, 267182734553256122076509813961122695667, 89199067340487266357407496005846671101, 7511749361918101956869562302732217295, 101106752455536152130060997450353217811, 316409104328709158498323078535357324384, 239048042695824106221026240063081972515, 215739511802561281066291520359771020526, 111355880923071208745084773287546885955, 46537197019261720175864325947301668233, 280187206211470338870377480032826215567, 34938789063145112871153790616087028208, 10094072799237007075574732828138638208, 129166567637776062254028533256381813508, 308826180459668347053909231029972214064, 255480344670576564820784347008647151187, 135752770413907223481003782460556770022, 325825911157863178455191729870316250929, 148801648597528110128744160659500758153, 129725688919099038536556878110704059416, 291609383795633026163569907324597616469, 115311027238703247158159010032197380872, 12309721576266339865145568688102004612, 48734483108224800319068900116911804454, 195753685009585717316213543340108512735, 86174406771001408248771158670781994383, 141391837297879826048462494704512196795, 268813798137463841144491630096053583226, 201345866944298540578078411615092316130, 33931297502940300199082112724041659088, 240887238781398766489959783134479019493, 14276961539223809065453865728442000467, 250416569226561859768572217631080101034, 286110513279463272532788235705386258178, 59478401338806829744850018620965500882, 13026992105928941117253958339130805871, 40893164789092625435495116288477844409, 323290964705876208254436956600926538839, 338900431293574211435629675913324469698, 91093644517091939322835677521644918675, 172166735366403216392348928025622885110, 120324730208958038488790201834600459203, 27472716293039963306436432116122931584, 255147620007171089832516186416891468534, 330318505166174168840166838225586826592, 184009645229157910397854715771723231494, 37032956206711304939009932215485684080, 243505514826295497999958830186196797067, 317907159407784694644807143510678598701, 257579890743706975391268166585192528537, 193485772269261526732293185647259974671, 173771888119384751317361304386792728646, 94250635152825288646676910525822459329, 315183585342864237820453486446567763802, 193449114365238745191198101989767102186, 93687076701114319377870292608422498379, 69865113921931621331084860044707714496, 319831208957890150467377865165546481651, 239130406608233594896809449134611045975, 27288352514347309164457512128217561446, 206066969285746593569754407441752166421, 49229126703132946681799556140460531774, 56698392071014124117501034842360068564, 186930957976482671886807657145751398134, 217260109435818201734951732114846682319, 292801702777821338300938397684355462083, 271818721429669262760686485089249278508, 301122912172376142635111581243692122024, 238231355372713895507362023195610617786, 196096199494736490882779794846029282614, 270786346539763199250349489772100343712, 216386913663265524191956442049962603516, 54787920672526870726331662552006493813, 58087783432258998728448516198044231234, 40273285912107604261431733748973395970, 211452605813636074671814190563193919565, 262499106724691887423762178437969050738, 26683849270602387944234189407654658438, 272922773716894243450573714403720584710, 56396556531691614550594426506057428213, 153485817699695627816601686305333572246, 310181917938514646172504303015960704998, 208694630923533842973019639437942890786, 294190428027508637487939578368564137157, 6391889656854054414611333834321279640, 122846417152318039716595500964963877557, 200677313048653317754023101379063547564, 277993585075008440457594029869938336254, 1123439086649836868063524440558915593, 49888104229293095239393240567962635946, 87747397374811620258138794214605240426, 274722051140138105213529280861960232209, 232124330420190662399687733781653910208, 286337481417848501717034159133307029507, 293799042474122426568553355336742961943, 181925186109021416528148036349166384900, 102788253927005876648706644981468063982, 250624217090343238558378636521009891260, 156501654440365812066070423771731238934, 5438705640525520691490361075796432300, 18540863419220732562572173289097625177, 37040960064902397079324174790005123530, 15973884964746862916198299056494791883, 196498285749466028596480894889565124625, 96379279022459424791990527567529836094, 169353157576406474425456815238094207253, 179773468073152525597477031467910026377, 329938938197677700273740110320423069002, 282136403293345786710156703458687090825, 195647869893419171391494328055468612534, 297217502812224771138647909714074427480, 329514223195181932073495938984208924251, 51060572314072380981267181882994750387, 105319704629985520057008773900595211176, 235594441351672568298349078555280220588, 54659065915877285845829687169793676371, 106156637222601559355197670232934205512, 43873886792706577076659428543517853203, 135051648911032817428660381564902073963, 85468475503460062803780830999099489284, 263513176670356066651556161375350174100, 77576234512686805684338554629747053884, 221151407186900615781603022084797433072, 281477004837476593642254341830454589065, 95449930546336868928215887659634767363, 88466777685268932870793004937683746405, 87805001830677656968981943019791520936, 184359142774832837100718103126220490975, 218063970760045493000143459946342940578, 319941460286820920050776142730153046216, 214235138066969548534852090790257702625, 258143750837071482455063066302957853338, 112176017023537367293823770377817125295, 134309971906967998863214410252030933184, 308913480040637421577368293274946130765, 154903982211171389652831798402810514151, 260833205647991183868354735774472228175, 11661259811654237869530386673338066333, 318735797002320349924718021485994515819, 114848208248308172177647848830910309867, 14012259525953434839041220339259623150, 277480275284212765946930180093197031534, 305726232534483201404441560440582603676, 262207210511983669384753602202728891815, 2810940844415527668848708872901709155, 142586288474484908132524799463962430978, 211187735098986114091344303969216740733, 119455033053133001556398611418400344289, 41938357922344293031261563651173907416, 178345142820661695867330846317803302061, 116455818505817751409701142322052874249, 302905215347493259148608953782308032315, 250072623836060449539867086134117189644, 146742200032606823288669524716731702984, 261486988914659123736432821978029663213, 254628802843076190920777809069474385033, 268015085407325813788834487426411843035, 206341435187583125550602129722867588433, 296786797166358397368773215247555922141, 7771488088907766613365956409362080775, 214167330731700550992943834540241199740, 216363077693763491417309770694031182243, 253432204576770373869061891420626329417, 71782133022648463694863136255836717398, 305883248640316615673706919712223769745, 115734051228023271853926665333841956712, 9525120179276788679880979763357168634, 260119887172341932486141193767126233980, 121037236256822334027487169533110414554, 143665757599351406825235686176338961720, 120052387582747141785495159540168019249, 228729134824893594609712023690756979986, 130612547116279840600076782390511698557, 124161012115750142437006072314628793780, 274161448342316606395050702147615587831, 164426099807540265615725108734123221532, 113786349037727402067657394081152410507, 56158047260488436932471144586912082222, 306348011897898257968350371869074737501, 263197250970277841107361840642840336350, 48061961808714826691520885662895476091, 167066549636798860801926271019408099508, 32988132266942206058784966881694652990, 112933580231620899041358864620708594123, 7487312967187668666469511432776727639, 60410667353464063950452879865544902037, 335241116951606266914873068043945496732, 235343047586072487341725285816167121857, 2471722129150189996358091519744993229, 106725002683566709375152161509936639351, 266850839641668945092241058985194038124, 337015574208311233128021339301229519697, 229202073152576714195069297392109928683, 9364612246012939933098685975200023474, 66009345603546446560931864363151775972, 157263701272383362708320631467385905832, 123964923093637934302071330884715400255, 138482823525455244843556657917148475089, 140942191596437787738288355354948741733, 138783634337401210292161227990601667358, 35370454639897546059612741852622218445, 339244610036222834054227633433933098547, 264495715189109695024644436973749318926, 148886150897633499167710189455528907912, 44022547304821392826461506083796130686, 307969718720597274798297939259041108399, 259289083588006477747890344159845183483, 9129431649495818601036683145848146820, 225929721767028201520026167224981033210, 304193495766402320853180619657732711757, 196782992133727068671614241591061235125, 30554157202324273620782473047004286555, 177882895123895887242741868048317838366, 89264929710141949029796458136044681380, 330143279347653507817010455948377340327, 40790161543973908910128206853441898661, 291028627587968463796621383570715900705, 241167243901801586091464823323391310871, 102401473475774000942039991612541212544, 202276250852850997900306828324845834629, 306839448858696335558845888345685445774, 238178517384622465155502577181251449509, 98299677736439455495157940994278273093, 330768498742181740228523881423534657276, 325873983311775222673585578494009205722, 248859511742248746863176294193380400747, 56923442070119413660317186122113914679, 26968679733477139179869707353962366896, 296622864413375455791499108930776612140, 87733201032592055309145490111450000363, 150611357872289303646159162049954945465, 84177182547397235335218165551317164600, 119642295807136298129975003491184549949, 16095042871376066243437794632464810169, 221118313989859055491214685839335630431, 120185347071058376078701561974683358699, 273918679379070329084650642522703625232, 322485525049419508889708164285013397741, 32779953734049979615639523282275337406, 190778133868358123614256077856314647578, 156375679562110756899519205357518467011, 319077238354456174373764602771945778181, 49758689579155360193521305063398495379, 213773328969795836287445146586761084425, 137376107185409707496599588755507004160, 290198556056012795738856363496587665441, 242588530699418250215780442046265462457, 86137669711471385087049200887990142738, 88816202493526758385146975247457642439, 326076267975805470720077672408593778240, 148463651166420475052545616109016673680, 16491870136658639664847720642731824952, 56553989235826004213336147285813560952, 131588516592916707749770950682847248978, 299910759945755019183088174620903486574, 194804119974256324990665860535601730177, 190357390930825965819867867015546114622, 45795990004927897531192759253734349422, 133666501163085934640249492580649680379, 135371963245179700306549974734661432076, 201676290106600869972046255266850459519, 252862680539714697093759991147828730439, 158859900331590689987229968538668610844, 277239644746181736309569370549788771305, 10077173646247354816269642417185970061, 262030629853734631054970365289043824914, 212215037153841530694689003625890849685, 138095753313557533445721172896848853951, 96747714866815063018750068589153214429, 281978129149977060474261210444537846454, 282587990849527862736624263425437671775, 56709573586198722611106436864008306963, 313639829225800262383884883294004292314, 252818833321288340900718949559802116581, 251031839327297528408926920873115227976, 64538719338233210733179805405913158057, 302828989948705107904151879771453283896, 274610937642468619383383074028607978682, 264179364797792072198714299416546631396, 177563508356679648769899059458328233187, 88613179538839817316932329436749906617, 6071097415210413764028468902431434925, 300669976794787215184004741653626134731, 289579333223417630622567723172564336433, 53252201828706133396313098487152654537, 267921147567263349990215714103362812402, 269138266362795174201095117962576814939, 149594983053542408331050095530577487479, 227769265345980040694968407365296671421, 65735019498820933035318287414522598366, 221244619239312781917004931554298579616, 128383368609086747762818775198241630926, 305652753766962080498476234462328290173, 200289747233335194068684968486219101478, 215114199424986108962147599193639745659, 159106181015930625133408360053021323260, 11694471787832401949164103756789221333, 146444184594152965190068533043502389946, 55928954934195566751305757203306921243, 328696862829079332942869670428655341012, 207434443092577588325237331409265291211, 195607341061717152256563506955516450250, 122957172438799534098020340009377217929, 88299231376605316682287599441668485784, 90893309539146069595730713850602848911, 258293098116780435009822910914648832208, 96884236821535375636411922279377291020, 40265551643754192792770922564887427827, 101258475204663339743146385726127445602, 67796942016515401999329380515758179680, 218280131890747280762042047626457260500, 193927869385420834573130603397760277505, 160853203958734332804657589611051917380, 116576227813363101630742845718066428594, 299020669865411317713174509073954726755, 121984388975249248837461484654490872158, 215785344265861343270753900121662871313, 191199709095936719823223132534879718494, 52411479575715984445581385813207660073, 257860432013019710425310437060283613490, 303831665774494934741861442951703997181, 77290647908841593328985187286076336824, 305735606228618368705358434724054917414, 121558758596285154443199366717900727796, 99489376427862679322866177216378719012, 255863497055356473875484402222280770739, 61439259556534506525749891172871737976, 316577100469994719646226939781528312420, 188799716181609650125309073307682164539, 267361026425537699636447782111608970191, 196130215151592909916621915139743202551, 105358742010503229381717220830241625383, 157998006532885465854976494197333197515, 195707562796825230391221560926925799089, 119228795141827712547722269118665724041, 170344255374262283270068978304068062741, 59073275395262135232443663869395378867, 41566271234608899114308969682585690899, 89535798416461605704114377230821307657, 237569341937548887209688740622226471408, 224364162034289963430321101101081220734, 51468290559512178397012737350008532244, 113386449883731164623192860546319219267, 318455218316305998214907175103046854863, 144609928918220001418164632745538042694, 260105647271573852441401352281328356556, 119785207209371705005876350311169755463, 169811296423410222799388916549109321801, 209818985333377615501591799516264476416, 300245680137036720280958644849625664433, 24618777092967109156280558694656386529, 328295472680968475866217131347153860869, 150944834261596954309434281966377863492, 250418984916482509862170132240642635807, 322413903878347631184714583304710755111, 21782268595527107916989670648889613359, 117673755517613985608574886933146248458, 203993110312334748097928721205024322840, 283144625173284621494378658571530397629, 131703553276770622322678220019210038261, 30478930861709988183335746406795578861, 226278921011207451923658123266641733355, 221054323879400021821040209754995275654, 86349370437909415412887041019659901192, 149913174637640597623002762067387986259, 161059398603961140508619130836535439654, 34611117222547107807040949747718408060, 57534585071240252657281445096062752375, 258680087344497146300150106955032277202, 247331144545553020830381127742227929718, 253961180912476398864466291642906348166, 262426015306714057546845927631882728163, 5904307517374524558071314056188866927, 200852926562853933679623461905357807154, 195223136871881376974195182514454135313, 284368447287908015639090460697975565, 140964220207547585303599581752422978981, 226474634108034507222129563742714088588, 330675998139300956123606520304228186072, 152508979091502360093536557596831900680, 22969459866704183699918372998350022366, 10236175740100993614775072183563415, 246861945511968384741848367123262825229, 147752247361006693790553361845428950941, 148969653867576692068080164523709727374, 225254702473058687662586501340330727730, 180729750305046449067445845132890398318, 132972080422583359369241999748736561495, 42246539546629821669291486779301567094, 274483318745512516860058610544743014013, 261535920596982980278627215567774722161, 186109973780325720568476444578917234838, 206731675316323959028975890505277540346, 327810435875966680385102898302263787549, 85832788314770890452201559351389043076, 83175799979083042650985637015568496298, 322638308718638045145201755679355059003, 277461059938439495185356791485854690884, 188264908873612602093761530334897512502, 317782605765352093635438500156669782965, 161154643631503970712270325998288199706, 112613711508466327046576265923301499195, 190479603371235546253845189769113549826, 40663604581399235257864080667386280429, 200400024505132183551717915853716296594, 75338140486992961685291549940605477068, 313054964519284322584680713984897463908, 257846335738042801579033976383552719197, 176984521867769119506799786766245633664, 169135578798369234822910244849679557686, 152711109482362008434337354766582447304, 199435155192482903699446774353495716194, 199848841762638715611465826772041137023, 140494066277685206349287912754365697812, 229611501609025769139817990567716660688, 113884892721185775724716749292344964287, 179977996999649956498381567380583499609, 93166880307693449926961555138614944634, 208056840569500918915347415105584392309, 168729797424200877611091399487120558158, 108203638264116627852774181554316959099, 291681620157445986546564492337346916318, 246798073956939706699546546571523354044, 83803965032416667024915579173102221317, 159464838914135954918194993822525689310, 139817051145938584215847157557439828957, 120113461659537866117617893319989462114, 19113437515911562647787200292117170369, 306596810541926311917402592741342238920, 333783495449128522353952491549354778074, 164768986539307210927070251567502978480, 315677747628933640260365929429245724834, 326774595902579341575376353031192768997, 196615782646756636632923684824211750766, 233247546535570935178776214548723358352, 103124549763347438314790267010442627877, 205203401383629694863076273880506936077, 160546740605267725809419026093362673721, 240032071455766896690972770791654348885, 304391915566679650301113940738251554922, 1952238451938997230196629390051055170, 275396173499747741783140534373710329844, 10063073710029115851250918527606295468, 85568963607512932668510716266960103198, 218994181236814256888025384234657919528, 148416382770423607782583577192844820841, 34851463276333011073034696433683572861, 268830684250261134291040030460886542663, 275173000764822543263962302304951958006, 91253311442935161562143270205695390816, 129173617060914759526008334997099097354, 313572318158872324874874975621109266424, 321496399547642966166277656995079346529, 145499034807064386598765858913633191537, 184185335435073064973463393047748348798, 296231113366021807888609962381578058321, 288536089566115345069293119230462750864, 111080615226981655587458544071632960344, 228834415212469823843455983558567809621, 184969163207344246160097915637319064790, 117827179191836452752230754221188526611, 220654614081557998539119406886556541875, 16849693822740205137702325691357371336, 28196936301122800970985045190433542343, 183753301281096495278719933935524026012, 144372831305310286054153168463479055757, 275171230356973244353522522468340141162, 19248746054129589793128594413589586757, 81454111944672540170812073079735821560, 215744858530882840237094857425733372175, 26409137866410030612844781381000961863, 272946504863711421248885719748355551062, 78104202283010350720930020084089860789, 48207813973814758138319282021350574369, 300783537779849997844957399183978809359, 55381196319854366455701945711339954888, 113631798071704180340408425998969665311, 302465386772338892379777488257651654742, 1706363813159905115511765211540584384, 157633036406978265181561338672419968539, 167096195064753847287316054140282316176, 292490551712504589285628842427933986506, 56617469982883899846224493197546008472, 48863494433864273588434687519441329650, 57149253962411683745753530565344808634, 239017367660423850582939760512028873436, 276250032008836148512944683648681636039, 73832293688364295981629905374379690839, 14340187782574923409984305390113715467, 334029770701989276977356781208927784213, 52237969203055123942153633399445406377, 74003880027727066463038201343583867762, 128777672949155282770010678340977522517, 56710543350939319981298745664732574022, 182842981075776098919596893713077337933, 8922633927346489947424426028818800405, 304575781662954986844860784449616760618, 64599961125965619334695307175792453558, 212336511956446682779973874352480289319, 109075510938126378093703236129193183155, 16066558636191848450883043279168958986, 328274666126688407773953370026078181952, 178722791480323823286154290866180794358, 298279634956546481652119352686124752377, 296518281027710253783214303278972893814, 174846979348874141529992095303878510163, 125862450828147555361725956170443000670, 129928601915448048129070176531084223960, 65887997532915613503991800712211762029, 286452449814907426229832952805757990063, 142694660200111490983594652159766776108, 167910687608692846799456376181056893238, 129300361534195547767943560607386070831, 75743486703129562763470692211024671913, 204670385844674797422493996941377521249, 73184031227599436352099388678319943118, 79691509647622209014180638976559497577, 333887457059264303359451533319922496811, 240069187660340258824623897189284197506, 226460489444394506711671693605093591611, 265753552949754119978082546209160210481, 206070188085438600742274368306917601859, 301644319866554192334505652288583629185, 173424168647316974785717250616966581781, 232567016325886371080212174553876003684, 311110188403579333138155233354226525160, 238611686637050019068491114429992740671, 255231423367674509641545387632008116570, 251283111291443071983023614865177478885, 77505710221071029955835113828335660508, 229593065589742183216809191408333253320, 308263840314504404223258991325742851209, 141341327041132356548653441345593258276, 122028975873645228102790418239092728578, 231801454614877614730365229165702634675, 267836156285941266768408655921754372934, 281324342202608434845041015694547786371, 182033607102332672506570114777783710740, 139595026603150980088296111012155215271, 327128008670995549445135052418914226593, 150994483791420151752975475778116893271, 80881018166501933051156987930898520919, 87842198717422768405299704712799260380, 221740654917544320115103155065976953278, 65257510241528682445004031525862448232, 273019976244623867624844272050132822664, 78414318861767571651139375232174858013, 133776643093518891718300297123080917417, 195206575192401268284114667128425299119, 324673506046014602220845908086781505754, 89882839720076638473232106199233676741, 145645661205124620455623090191788444884, 145150071994559973451566017355103459220, 119808078234809808110859215070323412422, 10083213710577613115332599411988576611, 272891626151425931607457771195685076310, 324446062205248179879354116464166319475, 335825554126916041525771424420463437653, 44284094694234841943708147665933774057, 130256568055143160400485170682216669416, 171357864035681759504934598783553843102, 133548193310010119186916544263052886790, 226833443626273920062550505569341164833, 128631241979983604738417840019739539137, 15748465602185283579964831349387445562, 166859924094108053669107352940630285630, 313198237423328876398295406552423702953, 334513675044735566939995335057012848039, 113674442635393120105030246483180329168, 43400183171323379722162692882723604802, 332068771391359270022616256461361936630, 172646071816290980077682113274663329514, 77049766395771924720985816169790017836, 209845845935803175635827523602444527506, 125261032693325475371978742253895683625, 68378078196146586287665261282322398370, 212172696333158025592311048434039030193, 85798408861324947352685893156165342662, 33679296019342707944798667516405189398, 235572430126225305442094204876525531322, 293914430082604537389120227000969164480, 119411302449017587291966130387909329535, 139497682744290144772885646722325159111, 333126133428257334156658369955382129328, 20413751967735728464318245032476224747, 252082557821579677022031083961699517927, 115564779046534125618181587166798357534, 71631931088608823436782928931648062642, 166919959267465798850132622437070624256, 313519924931397518451008233425154976120, 108767822693552247506596043113865773717, 53396544070027692540556516532730642479, 2136603732061004467375243347799633856, 137743552974541906631885997811341641888, 246446687495356206030705878798254446247, 33254457031865326156134748273438394979, 245736252235204853893707230537015931069, 332918530146947108832097420651503573004, 182278955895827181232343318463521633471, 171550457590194180673306694700699842229, 279408828522062374701709235497017281296, 175894497127502194370914894101753451117, 94079744798615804933011823121470273784, 134527806831912881559605528355862857078, 334477234624900033733218080799671567889, 181454499742935144308882744812719686147, 168953310925707781298586191987580836772, 28978697414003536759545693625381193113, 126478611865260575586389864872334548396, 311877997390027477514253108263358547582, 318836488970568393625963479800978591415, 206490774065114653746467055349565657220, 81691802188417456103445215933785310173, 50408868839233847350676439960986138320, 178465717995079999058998758474990033205, 66592854368721989667123995539311591599, 8592981074507124757084815950235939224, 218058820520864389765176888012457351948, 290711332987948411463295850794737042298, 180854180819801925807892266876251294475, 270039823396461520027227004232694308758, 33566089830703026899584841998071113635, 169977388794458023006305616986808742334, 248930149652651908050831599467602431692, 251653787362523026441475079405853928944, 137712379168416971432274877288551424601, 285440712437036863480741969637380525339, 112656996697169411427766365832639589305, 292757842420479267043177989133609639444, 239246626021554046796823573937092509753, 335569647035002145700858268381992028539, 96044125408228900242057132351676713380, 331584466598728517152014732137876558158, 115827304824555752906578488852946586577, 285812452062347908868476983328554268690, 180395948514505925154872577543168170154, 265356885713843932882083524383250901256, 22458443523184473568713420454567435832, 232810211211919179666701077331197648549, 156073894106635359318930982198999757027, 169083976955961009511286244439278419542, 43158044198431405773737406826246292663, 243031102801438704834977965908578840041, 98621151640085876135420255770947759859, 205273823483697049500805097222359903143, 31429041716121208861938809162500829113, 54563878579808101841793324175448412794, 307887501070776707243038906839968159356, 24433990523869866543492235437768903751, 15235322961626112374274619772740541710, 204779140248008180809121306445091951362, 26785874507834189376180908875635359200, 259015217856013020496728708734968257439, 1888570578612094708844359368839291873, 87925957712319428495020293400858492862, 254258766861763413461480063898153453823, 33953510480972024548822839818483623716, 281158040608023840060512210318614461985, 27458621881286368689531230357012047387, 65284311322633561191261550735203153137, 155734756869338397398447021261768804964, 18634629236914998323228955702714107375, 223287852650893598117215342927546792899, 185896367674426748986371006596465509846, 273498823674444363673510717990979616508, 22520494690085199971999007976212573342, 4882193422801779271085374949331533819, 46380464021830675879793867444074989182, 333908588908833525338603483965629112322, 219286795574516713746824268840338055326, 180822714105881068703518895406911728062, 76355585911543307008361066170867031326, 48132263938821326653289096802485013394, 188131858818245847747803889067192127343, 40691029941502373199512310280446200965, 76787678102079892744212314970029602088, 269883320295722256001640172116972660971, 85817202463276790582896532614901416958, 265430349875041062948487886909276360881, 157788392431735580662753989447384128080, 208967834997005497448965076990626251576, 303004057765277369184928847059643735865, 45028425973921246894799844850909961971, 97672254082342771866603418867578077618, 298360507002530635994722676303044875368, 32934808145684151002801983257339818694, 306112824501801602294019140025899288742, 18759849020329842162211839119909666774, 314817216301728855708343237823888290617, 291199103733447296581348671132499462187, 285083676176441527759978589272572544098, 5744735340334899927268232682848042525, 321927840998616209122655651756211606955, 118893627223848761842197295471951017605, 221781951169944262581912003379793759223, 64585532704029928174059018167087693600, 146114328599658187262886067502581512011, 173716629575141801726023151301636848962, 161857092509260145687703207629240369564, 208507971838205380352585619185695459817, 12303762879914297954245729000345686504, 295842820970269805332994568947418671320, 177758810959968857381307952434249835611, 187540344548916700825925281866838771123, 50399961289789548053298580889613693037, 328645306659380191691536893066324030018, 147196539975339357837695514254830315459, 35708142350500027399127939795862105967, 232131553900834261283421982936547858157, 264260840608539163464558082911190136309, 64720348917461925199431182341023329168, 280135602535947434697499205745260709450, 86364538971665331369136710772869612470, 232034684308496213837101272478445977012, 149359960769488804902423085230846767402, 242479222851505736543293134856840887427, 279900968835574086841551538059845653080, 329862159961318718542843201984102897247, 183040514702216906247854645234141185378, 312460467401480174856879429535916925390, 28760934892457042857427921227325567592, 259170400904853207168396032494242042828, 213493313411079590894254282089864876329, 232832133662762371147891405472518944250, 164208192966719321373138707277941100757, 75083943649915966490810591229615052369, 176931412770036975880116189732270531666, 336295814635553851334006410943941115865, 269680023206383063244923860719148424986, 18761859166857562485781169620549418051, 36903692140322419988226237107069394641, 274055744435382596209323354036673065648, 223249299847065452347925599120889039391, 300636652196274295554677426823242743162, 190930776715561063531659049192456111185, 102138461739427804204512376981757674492, 63025375242814926442837507928994279867, 323955857084973359100480321338688879297, 137619626536591998606242894845054603390, 242917344281837699195295600243179506877, 207952182685110997188881442266655006142, 192675355818756703944766476683315956160, 317797911936679800121934043140544293567, 94609268797355498357660464502471291791, 241602699092514343615168146006848019789, 230281935947814473996219046014710215520, 253662710596051797976998564722718294798, 163277896027362841135915639980872456237, 157109280860832609290170633487559575342, 2859907026935296756147058122051720804, 29520672263292460415959151385647464115, 27605551354328992374256699214973966727, 107790022558709126810272137549987447263, 295010740613825727110558281142286288925, 267585499726825979727414265023834065384, 152023917301384119956037915646637206274, 59335894728300896110146419349176369005, 245462975730672408044163789272556884768, 210545569879116231122103872490249743190, 280148552822340525363495400452835890172, 162219971116511999945463516449199649816, 258582018835744040801919939360659639278, 78681920669546288107511393122958377215, 226842713499099864620210880406343174466, 188380218653570085783951814877567999411, 243149134713025665180351632823044765380, 68749804874778673960643882206545855033, 132102847377068052606377734433505336027, 193739044980469039781428620746555779083, 110143897305764110866865761752081330009, 337199305772517617008206703185841977069, 277020172700271564689739393935632702070, 23224654440616160686380249704613587327, 119966222416482127303937029885490661156, 198181592304838675004100233628698053530, 215957846859151035401539725415734641361, 313448920174553765164223607403288177213, 231659053963658932077395440394503171821, 41277219556806540507900497128622460359, 315093310788446262537207915972512427845, 315589590348595679602824561865122376967, 174176738171991779928759799764101571826, 101787016260780206458382227212021001442, 174540766295525282170142068644597046494, 304362472531271411817902140558345934131, 315692887465687580086892702084953087505, 133334744500953262815269267262348757426, 335736680271700829339858386612508449733, 154563512805396874611726373348283940559, 125311927935250785125554828855731718366, 302604064224444101760722362770474607135, 144400065790447093305571556762875439240, 79499127410887499634746497305477666359, 241536192553479001361016769833935063204, 270650080961498159478605469958869679953, 315573452552400627552413704036984031918, 17892844450279191403484800127113817519, 242709885486484034779026668384118945887, 60811361079900763260574686096184605814, 12456069043827500670874919319906151982, 225827867376682845845337333573301016042, 57433987407797202285935629523961859444, 70705297391807815151858768295741633737, 172293729077992591561015682302996361602, 194960048929791384621159885600048064952, 6129413453707298594890552756558976525, 241769760477472693478885084380380437941, 42275369990253181092245989378706640914, 312005024305195632654194686508362462063, 122546363313277751722247598413871555859, 171444853803059511934277353741022933160, 310247014164722910600788641199100795189, 115474977716260597164970915458331480865, 223341419401755039409233637966502211985, 179275712034579033169026380956866163401, 218636388244762422555337346349426398294, 331527322672725643890724170504354500244, 57426489886236848323466684069275932556, 161302164206073584048937953052007480559, 227836412509892411519792358676789983637, 326395012864684930684527917977468444829, 17306146438918267172213515147651475812, 185013245788192914482788480984248489422, 206052957141741067626602125979560388507, 103226310950392050044205076565307118276, 295003954446856994492667198052359426271, 156686178971708632116266462965073444419, 281379597076894315181415144411223081335, 305224873012681737401415821207165091214, 48578565623666282927686494485465854058, 15144484973662892936370382055358050499, 295703431550150357520475098719262640140, 177117866787960365421123195887561180041, 281744232813596074943299755067820321093, 254766935759079397792537891313039274905, 86750303042532302682084411311596616467, 160027252870146614574904372007047118541, 82278813381885470777548326002855203516, 201753143391682216721796125309489522064, 70647409086458618282151436644296962985, 249167727678103280614333417650511641307, 304048106573677094660682771784109634742, 287089719631881731819986223957543666454, 63712798219759061216995249363080474916, 37029516097783205109621697916587184515, 244525039297022823080645625713526065990, 230782559049370580251939498308692270640, 262383563732409823327769058593780808816, 35573081023142461639035027319656419613, 132297512577429461091022703819052612551, 270710625241749460903087124260235780835, 77771966592943253650375145380207637634, 47892002647464500743131042811561041369, 95080912273522518400099929294525101893, 239348493726414057264562407323565184385, 124669465848431613317605231979081736349, 20008907285659371148645975271435214271, 101859746172680256986069415831822836841, 162450637223374198028017183377532762029, 143064069515996042059196075344586473412, 307538448932355827871476860064237889094, 291827258050621059605743461669093864233, 159078668511853687750546948729253261739, 291532576561458732055830464141906956121, 293136212193037683271746280876745812056, 304159905816842754130723629463891318787, 251928452559021362383241824677705107497, 185068287960233061365768534078100850652, 205669969385566582813804799771721021223, 296394144742125696024082782914441563532, 271041046563172208299424717457202793256, 198637831422539833864072208893545389431, 164179246306791730623374116173447058468, 102669382928988963713951497840046071768, 210710440414358554108732626708299542100, 320826191474099481991951979341538180328, 288784175281304322449587926678662430373, 319479913420968868326804237054974552569, 257792585396184563677424140397662200547, 167169030312920415975648452450810982287, 209498156699908672304035151981834864298, 227805223842805297526904624014406274954, 213162599285425236061634472067953504636, 117187486252221341111766559160031828264, 65247327925961385090157338980812513690, 72106748084323051404703512824270100075, 200358626878709054996699118225232656639, 9321338531507248984319721968508124660, 10302890844132052405692320771004913903, 236683080457499648896960331058265587774, 79364824892438422372567201506925044712, 197477108585722115414215613727860372584, 257050825944370571491412764388366358666, 191368222929748628126935739869815146237, 67467471057823522460539289815346562555, 178681034772169577659894028722963832927, 326642983239651822379699159227536198196, 45683181599644739499583876719151932694, 125757202676309290877511126914402764533, 265197395837665396825381786119039881300, 242432929026487320069725809140638540308, 249395545291959974709018630261687917123, 339100854684292282392664846004431894606, 305591714496373214733509527228886392406, 336505527745133612823545164044125248591, 52072478372843421634544273085781395801, 79456314208603513819700246600230820416, 105101356215971782331300220172959739376, 238102994728487605164217171653508895321, 183327238692856075446909179406041054880, 81222782833227852106228687612317208571, 196592655112020709514200138095734085806, 6416696912180331006047030226271153192, 261100142053461604811920582243504093965, 11003830192144606736518901421788136748, 40579306997846782097896474443630888089, 327881966172562204134661603561232979523, 187574003936383261372536151615730656022, 32018641043708247725395312580745117195, 323993425592539813700765884417158289481, 104070071679188940746350325129002166070, 6459443761711875698535874091893064766, 335732959193273439093429554918550413651, 75270998208501816505922496246841216073, 52149583934747593826937688534510690223, 46272852519364867229718512939344300101, 220051378028923063965455436161102803044, 106853078154082335347552719094042541808, 18276117035877767188116863297614389528, 252887703317018733820836015670279471679, 336055882808228143424284951201113156497, 339801192563631526779145685025850639347, 253859009291731781019634918009852340050, 29651835836703226584314966757929438548, 258433299695912134588496293195840436516, 263572003826488877750922973201037098394, 51012432693923582811513763854328458947, 90471382953459833411620396628342601137, 101148794377419053758071945237975664539, 157352352876673775532243396327791730794, 248153650641611871999289524487622237169, 169787925588036502694089594757432325877, 37738901202135227235870786242271905295, 167235626176227992437526893690950266160, 216668089037317512880048787475784064298, 83429506404759654245781735845911106326, 76718976138388717641500151838356869451, 43425719431886883591861777611922399440, 50441302434610684000663475436229561844, 61281258026800628834660362955549533884, 281873374942221482664518450232602960927, 300173725765469931677597690302717838995, 216653427783542952452129810353003775328, 213785842693131556291967561426110456495, 28103164264633226525689355396575008743, 187175899477313001122694962821319123809, 203417643724067529126533552436232952421, 202566340322027532710696673556751141082, 271023675606865410253351862440141084481, 156547954121415264671056838403894449508, 211359366456649922613765641295224434723, 28744824625587888201899124688874952867, 196462352484309028781681179589277443610, 240914234328365956932966899235306997422, 253723173248486263382231473538043872434, 112578652120550224063953449082686793529, 314028630023987523227241258481077873307, 189498212645836205287092858815542805769, 35428836603264413961006920695517450444, 267047956289167894837852170206211795982, 234633029049463033700234891780576646720, 333805993495823032911359566523625371527, 92044388917232400507704545266372430917, 39475472080447001757397898660068650558, 255995800947174532344765596523904846518, 231668680763450787461026749382349786178, 113857837067556904668965579803460142393, 85351949150132320362761232207386896237, 202998831441454700542088683513068247830, 326490366119044271343846270779539176775, 149438533859440376969545836287285972455, 297095607345270808946433307131086677207, 165572249735914924150300131004601171832, 257142555784732022024408623480662456293, 266441710624386537172009137525683920244, 307756108341132125465481249592916572192, 287730249767814209843211151647449523651, 242787359605792219245137980970427013644, 27148806076500620837361320432816735230, 60146614321169701425557208068906863796, 204000021829332857395636097304309231661, 61504770492684633486967210126247061075, 146014727766264995613760373540084083843, 111172351593654358168339910129833635195, 146943152207310852663389025994744740809, 59676530000349628778037982693231401155, 302960536806171163996933134594015818911, 41668095452349088593132547195828556765, 230799771924147997988028196330610666979, 33205690185544165212857478295946318524, 144429515095273087560667256763805356008, 249494816484888970673885721763347969763, 177117374034992759933961169792496218406, 167526596396823694357509197071251222623, 61419657185300697945984993732700729255, 322825445212654440538559460599556851691, 198089638968301742848201283194963609762, 131561549638638150299733814801200778506, 128323639571213014544366112888160585482, 109634782018755257156404631068947361407, 15767508246370585428667560226925146004, 173810310251813619530701175832412896081, 170794886294750361373815970524040701743, 193101260316430455739416857355028913354, 54704444120568974274735282082990011604, 89328896806306098151015087877877514429, 103228623106652656454570195640969365338, 250675260599727237238263129322222872619, 252375525516186554471792182908253386387, 316604602145412416413320415301451900175, 204744161728427162323115876213688797575, 148184451619714694663241018449402678139, 224046394192369678398148178370757599880, 193345210327409150690610638236880608164, 202183058698251266989503856204831578322, 76242977113264998279946414767333733544, 102549526165807000409924089551545435361, 12252990184499692273591488968692348943, 24202341344838714937650489202481892206, 248070338800247132761581285063856296179, 164196364087968876122789290135059607720, 246931233570530038865878722120717850436, 335243038315194351528354756562871008171, 46089553730406009049712320223151799754, 192969181829860301396028448911097599188, 151623762847796534622397574907706938787, 268183395756972422087909072866211915690, 273325599101087883574188409567682024101, 302832177149090509085229980462935446943, 109199718876533760686301881320946030644, 213377668930035784069371321820754770218, 176263748528165746945589929950607531867, 327843507005530785505629879065419406192, 237029574153106190414421275359000050304, 260501973647825505254241763365495532797, 284070746822125739390103405352898904627, 190640518696069265100378783976711615740, 265834086195161429943243801589594686696, 191050905582675251349016577367626768739, 22239934995561589718835314063287437938, 21624258602875753922473910327315504823, 319249693833811044899635985841341504178, 1657445378589943527413007873101868593, 195990899820736961664132160445295555923, 242916350915127841187745705046358294029, 175146397633435419613386856030677832451, 27180219171877890236007903886626807950, 337260698432828830381418827043953992829, 128373164757104249120077587515198418349, 238202157315820591723945542914027522784, 1912236853833567996261889712815732291, 227848893458553463865409845736175978693, 12622509334332711945762366544273149805, 284048858690530173037741765476792780119, 56626211218366623788921643479870634520, 319629753269542236781704046594184937506, 323651676236565697495629987816275067961, 178064685365241981179467531230710893931, 298663714816183334456614341696666191498, 79755271969212928550281171795381626190, 214537371744125407076324914261070495383, 162142820153116356837821898252227382705, 59641571727189948739323596745444856074, 224827960962601264125007283053682845253, 18665981807273173143407745009472249908, 285968048771247489118345819122914204431, 302328891646937889095000216430459028801, 213703760147200336189465999592648720900, 311993165677828708066510366494053015051, 328522367457821616941214076397258829596, 155176366955805118489021988587665625424, 146830077718013982766483143712549026925, 325754763462167323530798443540546521804, 317785911138932929416392201029131021960, 219365214999061245572803254977685498245, 4861336569092332104550968900056837899, 299491700657079037754713323179651279467, 70664428373379241333216980875624999041, 239576737425896595296176401770987168319, 49484126854592485673563882406967389718, 266933131394391373350981155282804509166, 205516525173639899991927107236544422927, 267104641870203228617106453411635212285, 38154220573084730492551387813167155402, 23942054116915355539440728846946279312, 175822585068950052127398547984882258614, 15188704713321998331845281020109042414, 21585413419683273699907978515774678730, 60550518435395560458401246036356457955, 71773401154874068300938595352525868908, 49896827951166221441243782305724185394, 162166447487025636414458480275855319722, 272100146577595847082823723154231293148, 95329709984709422131252332194197915826, 339511164241675975076905896983547732193, 64869062697280763624319735624121009351, 196680737401627214178478147568941398816, 60759148748796490326878046641700398856, 6239571946662995472227770465683090115, 10617651896278456907272231010779360381, 80068331091983107110601210729712911939, 221035507937675741028748446452702014412, 160613745384349871324873229034704106007, 178257166377871194486268308600450474211, 78589662389181190689911925696816833382, 16914032209308137799471422458151785819, 328517839893077027099640544086632946670, 269234984152404387353672211704065452503, 22137311206109209131120806418278051921, 180632696583671509946811744627287370070, 24526346884837876008157616739107881081, 193825180206957871702014031336808178846, 171645675393566325294043032737116964142, 151086181736944302376763788185650649307, 115329048604416358579614645629215073574, 146676785185459882746647193766304432669, 128618039177291486240463209241687279683, 302658930331580952572761310639271755653, 312678441281692766341048990398355197285, 172660099385786420376433443932238391245, 184338102086116061266786205491471867097, 67345106694843973188290972947649359422, 114475053178123199135442493220036978691, 235128919317918302226406289840049919002, 111953703360736261167382393034826222276, 137532601175262985304313592088406346290, 319912707951909893053509015638243108110, 311569899955841852295209604780530652862, 58278680827775991592729970474462827025, 282378308825906567451568861468869514524, 17941640769694536475984398699470481488, 205350132625014560597097148356403706692, 38623826402879983799661123459709041628, 77743828106315272356662008658109611317, 247746605465941775777358600490871344890, 34547457509356753617029057973413335670, 160869091746507287256700889362658323733, 96490999848348201280444850708674829212, 164068569490754482747726230135127028273, 43668015170954859425325090213531281896, 220849856981335248339994428577536154226, 108089202231582233500957486247308035423, 98339773309513840647898162839369190657, 295212162086980086836781341103288595374, 127448364213840626669478253005717673510, 158324104519895975329354039379190792000, 267413478500099846004582067845337161248, 335723949819878106318288451021475239866, 113999856330309327142968262989111980476, 175781957682845590456261313530642283397, 297780282032793353182366229859754750726, 107852338801156318701908154844669961752, 299688136353018465622517994392648485008, 46438640385536961530475564875734208979, 319416043828386593546745703435044691144, 196560192090782043995538341868713586763, 282497898445525041541379671260064929984, 227146596932713046206669714496126370134, 17480497571184854231876995716091174186, 237345385006017055982680624469460999905, 227783122267209862782998989813216165340, 332531000425229888765908886015066962871, 194583603447345869963239284365591058112, 81658204700669185599279272462702400318, 237637864794030799377516990201002604839, 63403168028391034425842038347692950350, 14634086051498775523590626779661070684, 257724539138358379999056250864039094057, 142490656263063274385112701479143335544, 109252575140357036004755987689583314698, 335562552219613767580726167200327691660, 323307814895186193941368178111314348500, 282378135980522350638102768044603300420, 338462315527251773203556696122616428392, 304876986819485635447041879194851979580, 86763124748813934685117889574054573851, 148612898321274489186572331404513632686, 253978252217218561347425167400791083564, 300241927228112796334444126889984870545, 43581839020254013656052950723519176710, 116112602435199975275659813772538350699, 157310683729449104885484203240022099438, 319342427814671133455360644687356446428, 307636040511239081063773212180578434165, 13347269153212655437409671727614852431, 224947494468219869007989253193790978242, 204410466288833631525408984358838426835, 161828897377559030672828719654368535726, 311908281747069618440834139532002631907, 103783848639030141695138639273439649131, 1121875589399096202708675905098329288, 187433095753179990050469417668918380169, 246152698958832988898597448946499658388, 242745428364062551385264867899067822502, 332457548137950264446683300216012173186, 182732185868692064421798318198240911346, 56115926212805789485842647774653274736, 277927884616593715625149470721836248674, 330805278536744085172626118016384450517, 275516916433903970541827902172410134195, 49861370780162286754976093961816927305, 335847818955453916012427847224123636443, 235616623619267191993684133937266124666, 178075237996364100493518039798171831283, 258878318470385103294187429690504772143, 255584261172716094208368737803806897948, 161291988649727487472169255300960387412, 248480538056406499355280426670681847556, 325433796848534465403402178373785749646, 64822743426344642346797431579565641836, 30898663024313664669801080099991064804, 200677465081988284176518731783258213864, 194730431651118814659457507538695661514, 54859577361033896179637374970208264928, 127023128211548845971506993481475264262, 224551755223845238888089400188777071632, 231061055138606211337520125978476570546, 158870341719839605409362711903846983771, 283255151245307933704693766241511623821, 236447453428585952258640396652716901146, 324547136337679582166079410687388637587, 156468849591088803932087163125556792992, 179449795474650496156233532906168550457, 116043915968967766785335849312198413360, 136656317297699235889067930218062746082, 225847166798785226516231017264276939259, 303933188951800730679129296061097842865, 61321134805610931834657780692739608130, 33402879639211581070554583289632533865, 168729473602695613127236922757256235108, 284339410288280594251679677380456277743, 175409666201192277914693405380277693281, 253393695439962026755065903296998417282, 251874494483886729562605072351184545641, 24536965877462840247725756138127815848, 111604439424515533054928189350677285822, 139209687437245295273615166122429280992, 224587185203227434062305570969991191312, 130028078042016630921521904191086876595, 36681145153894368176775299095953495180, 253235982903636044647589493760384473281, 12962603774631655746661335727441092178, 84412306440322760442244817001318164284, 263679843545938340943019717075695367116, 300558358526684094045863038600097437743, 114745148606675684503905901622144441305, 282582173637765787453987779103973316081, 269352638353052648217485883687456672770, 196761603226494788737256807089498119995, 23068389099641623313190576549290191766, 273430919423204748087289704314406174851, 98531398952849154659834408257206993250, 144720415425152205272869569180109885055, 34872763936173655885406653160405066381, 290098089315006214182647929832238882144, 138901847938759210629117273069870438560, 155512019295700815718185483047462770206, 116762773922666116056961077688169388101, 186595815546821055419327413681165624259, 71978298765206750589622758452263043883, 337499811749645769536311898809584634618, 188806573955353544303461634160044637474, 228527855250412340376993945517011564349, 207061723903943661222374442632468926863, 2668629100125592954260059738219988315, 259362299649965595876980528856015953542, 8372135418165219561347502169463772581, 170620275193754588521658573978916057472, 41280662801651164232181518435154386916, 88801391285808117549810826592614575258, 322493601787715536491819597177095972216, 278053407724368973049691105087999781990, 72969367784789450770410675791918822424, 91769388553299260137674858518320977815, 158973183306808160561731476570739919754, 90027727057814442102702653502928179962, 129034762655510843088933475609740453918, 181112981641380753384832606659652850945, 116382358206915158647413991616976324144, 229864784559871315564715954444109987463, 302063199273947195151067575968440521454, 265159398465341183215387390288670654267, 125193481259968716273243692991173856694, 225734190999918360603353313218198874010, 130618818518734368859989785459928268609, 55817674352565543305244346578160396054, 117828979214060292289375032949955879564, 48790454427629583685706649171056147281, 55294433776276470794756536523818579095, 53939240270168685801128500558552432330, 48691319974506681390267661859389353700, 229319272250391711876800036642288557083, 16792399597719310072676289524533231693, 150015550188409976183425300895975267062, 32533724462921865248829210585461649577, 246558232421300441235212626418724149317, 38682926203664464105995026934165286007, 239185122342568784397019496523969270655, 167558619183005992764299622681454005662, 105087691541484349587768037803360900380, 180213638384057659299059235063086961528, 257662332791092428165775571421132256347, 261759119501842777393948293103479424175, 251463009468881612016003589941828534742, 62830897306306558415914565894084252557, 7093728934477077618313790232548174880, 267739748014793377838498836440925478045, 104594988998168632585237146980739924976, 107377521007194616731597284618523699021, 18742173590039171825273333261467650553, 164698521905652768232247984057589055262, 38621755181372324452043916990187048068, 284779976408367367462540655449400350152, 50242557894021671785003390223057612213, 258595367119028636696299429357111142640, 338687610730600659901233663657266916237, 140907697915765534276995678903631339509, 236551747610676090594637394212380458757, 77443123141491190326210913008857370120, 155774698154361962823970939542833745819, 254657014589232561315350739579393776760, 107990561029130943971592759832971694964, 156065893482503235175464796536170544848, 49992337427682520565310984048514291859, 9882399324056536791178890035117221004, 90001803319781117844596302273000168135, 234860595311891523844286961920036939720, 141182176923673540167385879576708561649, 277344647647813352148656238150955158416, 250586582438110827613641693266729856398, 125235461304554484117370595212475507934, 325666063769261326756784568369170498094, 280359839795935959965248866631495248116, 320205585230375493517322429494651875979, 94720235711058749313075830706280262626, 141623644455681884157320095095569852303, 104787031806999314272430604837038015739, 161820452714666047357742603151664693004, 146280772518487695136253541466576391437, 138954826396585247954221049613619523779, 301442881053451801253229804709795310430, 260605642150665794188834174467321162831, 67826917142730397967623174070934448049, 242399034596691431693658771088427604437, 319535617709595907172369923552216939716, 182775039963482696188094615079064766465, 214034783211874808699252355956307480560, 191067061501198915314665250294124432601, 253465494552181413309531676721774120086, 218294071719221658800177883174654171606, 242986918968971360738392032508684369848, 328533897923660958352092332774232355470, 313731321152964648762625445712928020132, 68109047855935117904546470315162736272, 100383630365415842612060858767373118447, 15414422086775484173451173087585551000, 235585860935285094798748471892986879542, 286419316748924042657938731298263907407, 57165533719685430970231601175785035083, 241838517303104274841060009577866475146, 88729837908460479689343629779867709882, 62622798564646868670092277974276951724, 144728872862119735282384507580609050126, 1907489547121549628335693256867369080, 73553909515051001350545792768127154380, 55259799741384119954941323448598624364, 84038163870118934881653159763308210653, 34457544480527856249025234064666044911, 242501468987380787674404398750572160057, 42007005747069811986006750684437919341, 6782573868715001722591729413909469830, 278081755710239738194846651849302257208, 140352776880445907155906201597068466791, 224920624559417024982158754469890879341, 127077617248339355185509757364693388058, 69927669805441449175244662974195386249, 291348904011776341210130495121226241868, 302029134891053458916404833708012469352, 159542227971844938845277887827076084151, 191345602506040851052098827530818538019, 48609393960744727302719780823475251593, 74079718546278877263632082589690554832, 255562181044499682180151899424308694612, 166896972995857908362896525152103089627, 117130340355618871389385650060379986297, 88521924108767092470191512688321013063, 98733468990484968449180657167015945427, 94254796907777957998455199381261680261, 123906093881476494981468329104146070684, 71288903497095293028209116890162394355, 244163050045364874672740392285115451970, 261042420691228752991515300613775287507, 279524097864972595348679588159607086705, 316512301066051770007291973629621969925, 112031445633245382819386728677158111494, 197139801796572685964479055168643103325, 211440931418725471735137734731649287737, 288876610783780856485920265622057412442, 135351717734557193738886427679830786099, 160588154392241412407330271980365979281, 67949481361297721239424267406041597559, 139011984597491532973623445971884934623, 76470749125766628645594705924680744368, 248809940285010355145024963140740655191, 261280019904758584907767374034110409671, 244675670595353254570542819459498431959, 83656256406643699248510933093553443345, 321265207730386486251751642063980095663, 194039029159111779219994018351463412927, 128569267217108309377216181578062782108, 61265175024112391948004253390270872036, 332959718145381187325980063649845231891, 248129128398886604321715750089807467090, 108317440549981447660911899058193242930, 323042769590471938247250695677707430526, 315120435868035704889023708594493797069, 266412215426608441476719252602420728217, 68298494437025773839609950390749368907, 179885528218470547252487057226780602969, 153515938927974736224018097046052298912, 314491269922208567574270163897000156267, 127261595876532367557898849124989846344, 313568963986292762144127821140096542970, 238308903324359533190010154832337713071, 95858847379605158195484401081468109920, 72813422357991688061537690980112872536, 4847958975908953663177580353871756037, 298678931496032483199090250061940044957, 294061651509940891296287432102531100701, 166903523098143879055162929457934832457, 314635258353070358841088552430021992066, 27381209377515776331902351085950089673, 145381464394589506327242442115382893756, 180071891604201140381443930451257818333, 267419931589631420737836245579538597417, 31784174868734450691168593730711910490, 177824900336750389518227810369614691593, 241651954190586566164613165800926901876, 227529530750733979007741307251997596736, 123894948990177051089632339610174806413, 339731060915053071684535830412266687312, 57856799628206771067274441292326541811, 308940754206189610705635129158427562730, 312802939123636450062983555916158976596, 300665899187583206577637697431708932881, 123662554710933923527165529837291221766, 290944926573407377625386840392522101363, 187817895184065044939414238514286595261, 2684985813621751220795289754844343860, 276249919840416156729578176351932198116, 102860819621800629994376646369303953917, 295700923559977057966554958842003616841, 121476235943285512185787631750777901775, 184912021144018470192070704597624199758, 23365467696376432721066858332972306470, 244405626896121494161559194664199368063, 244385943769388629006898234905571587780, 141367531699180380933948630996347437406, 242970845071899166863330663546283739340, 100522677517827319193171178473635769517, 144489025720263640411952851872343429197, 120263923643755310459532478949603343566, 107233479709701445090063331529648649409, 273806436676822333840792177461685006602, 260148821015870730549674684417238707618, 44920524979785376708055829293118069021, 85128618850233482225271658283696237372, 119847712525391416405485401592640461285, 236984532802251653282965974252692669779, 95483592119300760875841952714983962135, 303790704460888350931633100106711711871, 195783483785375248098463796565034230000, 83664496247783235081620539931396649636, 135222482212273667160723407723067215909, 216291934242696472043307127588503467894, 262717995681183798467857313927662210361, 304860851961287538232335421623798896395, 150823889432220500957967223750611867098, 152628976112600749791340478504405462640, 152911898182924240932336637747624460119, 219676788419013681090089785079544751008, 42861538181953643358261719109596088497, 208110785456471944544182887227814148131, 309346557027558833550223684413788619820, 204454953742612321330888566653864046814, 13214806306419995386536424765896140068, 46958324294217064179645003464276763480, 46391037145394011402057153316609395119, 90043837856371171396087040178507837648, 21656200272535554133713250146032233649, 124188793551182512222050812398314992050, 69106626209801212070344608219458005072, 122391708164149562109639604563677528573, 110573780643820441413700028403495621482, 199848189422738304659812653706981395614, 96675307593273851849468468361006566993, 110931623756893150748400101818157572802, 333795224611841417464097826601056268850, 267448070570208821789941184906353520321, 131930593356642026621477169754729790798, 314797255940665478527933265380079049281, 25541218211533540544376035048746835037, 42392906732461583619970103965359240242, 145280382440050228117947623342582175072, 14481320386178041472582728406627997111, 8739213622975026420112726222454797124, 58022291345487091345449232362348417072, 180680662906034865172620260267955457509, 225178962681454703200901221841567552137, 337257454662682830976235933566486813632, 292969013360624356308845665787048847330, 140494153471885056161643094827750726994, 312879004406022752239148577709606359421, 322518469760883783273925579311509685131, 21039021744686288703929198470304018832, 260639000802822381774715756480396554409, 72476137879435977344488233436189434173, 186078898264981062424341064626082789335, 235876338472969810477544149654893601125, 266637985538165588667739535472911276844, 244390944551524006017549600533317537069, 316708680337586491222731513493384006631, 17501158292543791478859725019395104341, 82270351604451265900172652046469392535, 97909144771662775112189706536278628045, 298409433776054514131805789656144871305, 309716117305539828517357862395734241242, 73298858146143413115622535635986138399, 175170874668129994136204824619854961419, 42067813494144104396597469290594159109, 334582793994018043947798337041884992653, 33180328003679449781850641145834735101, 318745377021421649018511331841305119606, 289286515736731527802713549514499964751, 139309333349755893323622222896481061162, 258903915274098160888663783992151006467, 256024685882235347970074232316542443159, 241857414540737462653506332249976092818, 69111232014575964573268312250084902978, 5428897013746123006584704004844721717, 307509604680752758298851994753044885218, 251189212540413088364346536300953587532, 23569015345700460482070405458894168117, 118744403384657322337750854893223708016, 40342291902482223334949198160781605069, 14162430568513912951485063907636096095, 86619293712465264000940915196863801072, 187265402962570128911724028021814293233, 265646772778846119126211842257967016842, 245367685801391775898283337762367105598, 116546679844269205898939565393786189186, 276826875473990699407637013475911675032, 142270037186494391822266786417768393086, 29574800131299380904355181769766617240, 243373749978154457703757256547045726176, 157275297341378337255275537443774628643, 325999161616882246215616203825206041963, 332951240849141625278854078361428834763, 179818100837392703183576820059012445466, 261902508995489830249521332543912424494, 107304004394604112206718095066078611087, 128739025286111322411708242768584089092, 269159938828279383655964799988089326026, 283041510260874293761486046283048711826, 128464406282861569776642165496831424019, 185378472086502393486743226102933902390, 192404766782942241624131383010236276973, 201840208459607041896922529255576355774, 20354399458109710617559269886546460095, 274315005727582399652310382423487508060, 133452071384228594963524475424697577431, 152400719555239261330896159054644388310, 9258422971627575658169149586079254245, 158490492681797332878077769862709072961, 340001575393228508773759418137980610496, 271196981750492579126143397130108707110, 115808284104383809351357273764620996682, 250381255889533516033813113269745444500, 113494808284500177895569988998589635961, 175086336593855210363666198438170378092, 200858134885199100256100072448158934528, 148575954891844321338813122652679204235, 28302260858350923727232784463057106232, 230127350388742273273086320321485826618, 32321888467009866547902296799108934689, 2460971481009524637771586944052592371, 109324096694466212266234904553853929849, 180827950435727411035479795620484151175, 145541420830862660916005026729083635474, 173137003334134372087650018513019802383, 71629899436780158445606377802076420113, 317639504369092844862883640191786016106, 176856573090598592653040283685632523135, 85347494457586388322147856623702075557, 1041198959530758461005103241600163271, 244685798023671205497147239458402671391, 148880369820026618942553952588569275197, 249203185245054954834309009707230069927, 158754029971765701247327239786629707084, 327914503117070790534101750287364706285, 223246940441045989979744995687087985797, 249658122836379881864453357875983482348, 38352884384400361660720273671915570868, 71944020172872389636264111468219966574, 123580659037151858803011310611549561538, 203368471374452551797372409974296268464, 187830657273833242154288894320011930899, 21338702600417619915331055572654080983, 97703253224188788611423044918787719315, 250198979354289577781714196352253502932, 246513134130422268585166051225332267671, 324998171955330443434828888229389083522, 196074390936401998954099978242535785544, 258905659273068313082448163370088422195, 313050044078393383752564866909592952501, 257971157384459618567439097977699423007, 63504508407851048950593875733929895097, 72872862718087660125093835547646454493, 8770897769637479441542720816437110998, 335217390922363327059000482568096122954, 105679825349516920430668233799471564583, 117065572889660138014997838997271517834, 263418376231763453362766931005171851682, 72065796241530564324024173457461393749, 224869236728366995019424072943671632200, 299388963844288596692821068095935277769, 1580564020783743614861399940634717382, 70807461066237141448885654003118272610, 314574709421931868474773872324266635089, 219544778014980336578061386647914236965, 159512555166667879063838952418669136705, 271126923360038258785243180158177568085, 241832522399631163069246768822143758422, 326191726731719615867370361250486565977, 154972531956648940786062301288797017160, 34819386202929492095344291010654751340, 111592482654964423954481501727621473082, 44697661009955744622325967816371461787, 266867149490096964126388134421289444236, 298217849131919714776137330537333356310, 115656827153716423742920041513262521214, 29868390369977066676187052930076525836, 95299951275692154458735642707669024316, 130051647430011664794292214658761002859, 22344808043094672816985923676972883070, 76064906594896607810049164792765391148, 310579792920869854070514981639884597537, 272719518739345673836540802026401365257, 282482768817473143498545898359112956744, 326253387313396336678615494683172642328, 295212198579767366910599414425287962031, 86455253913804099077411247819852383768, 236209928054910287704723813476253875223, 58231084751267331913724898293052196423, 3116145240184380199760714798096825855, 203735610959142382001939785369515477211, 145894074899359054797091340063453688465, 235786334113327970816228574096098221961, 154916911711477202632147912073197971652, 266773552599602445372004097018763330974, 207856086756301798115513674201551082936, 159762682939353595998517356255750091753, 315926011186863433705865370477784565052, 141142247541555866870586804735398271022, 46002466414224144916977577972738378068, 264527475401273350896696392978325106027, 132394096877116355876117124959172528769, 262491075458484812590785589717576821695, 182593160853523531432551157437703207296, 326147396061077744362779192378074182444, 126308851100981769266452850263550324955, 44986287033414378753133995438519724111, 151348349111140803888598665835389052957, 296424222198396911573857474852261388560, 334988071964730722990143161234120235942, 93126790178465704810564606394442836, 336556763759591109232630179606420966965, 196846772968461216964401332416193425366, 169079751610714246803594781829480071641, 318278443303855471277026693910522509245, 233602473862771090244879870384175766483, 82900898367513681105895054117366061070, 35724842968033475539643409560552860590, 179355854650562280998896488848373362909, 22921448484579019558316284664813418808, 39483346997679287267620392335987328422, 60485665692351034843571245757459646443, 260210025816420291542687989256483262111, 288441500393203617348364342463341185538, 198825619766602529576628956305649437375, 254565715807876610533180835702864578256, 84715482673197943423726337408849024965, 30108943529796552641544307727620740021, 31128341592638579101390202518611172940, 120281445229182261637377709773678868153, 202712694574991601723923107337691011400, 244976253592934314985262409584497090963, 230567737898199642888522028610242499478, 88262020966877500989240303870446289916, 291561957694786545420334707173196183797, 125492429779140053273310163752300979783, 136776888234813649508518862724345755449, 149428434332311219037534788427950499492, 146416929864975643169873033644796927579, 28928357153150619536165816572408227315, 257356097136379584777319918144157469644, 205144761254811175570935994617521231636, 335211519275673531069804796805845711830, 188896461682399139575701258345708778188, 141134440452854083074335468428872326458, 145008543993730625668577192595991505866, 179322711780518686064978226597190845745, 48762236923588144697783916374985708192, 297655645776458415966909920631086856529, 74397045296909004460716077500904075330, 194236080957312112322815457470238183140, 125602972323720980736763193047163480670, 213715401124334210329529553986950374072, 65876629816023236222332875006307599052, 312515048308150297812834451972526134190, 229127887859886295688841278158464501784, 325627171417029535908894299581150706744, 196407207773509012575898547640229212640, 339542360816073750109008723845339040587, 169860797193540786893261346312223858063, 257266932398417008587483781785924017957, 181779465845627899992501939190325718079, 106590213714196961238411785259799343334, 167564302708445829972640479257406591019, 141151358839936107970269290279206494834, 12936973644970392873486857589631195501, 212808670858460243537760241233917537136, 255521799278523283075547776094562296059, 19559208062653124078372678243124615558, 107651916524882359478917226590512068005, 212072068514771153700943758915300216278, 52751280139376158988384003375610954266, 127783320577653563180073856765044477603, 114074231293282293010470646560811953184, 65782898303734013476236703617140832690, 292522765792825940361520623383518622643, 232913180920734473605300264934196908105, 127921151737528412611310937509138135780, 49738243797537271618995952200689668394, 329903243906543222109355398023596902361, 224540994799568372613222473663160086820, 46150426001738075038430641689038187227, 151765637987918704886249296757308790238, 91913816018715392501493864208463967497, 161156534839655480013406080741097215999, 224641518157442195815254930972450927985, 15923411156591679057794684516666612125, 227306465518976789380899570914617838055, 12075579362729143794726203037836624283, 10950254585866901229433028145836219845, 253057017716030930914455688658478538698, 194953117898490881384749724901667836449, 124819510165671836033587631814573083072, 119027165118607354945725236596628638763, 138718912217058072737386709602908836791, 162391457355734422362627732914176488726, 311726322553890647796858394983271922182, 144674398308120968342972497433817360998, 224941228531197774511972687871566908562, 329148464663138062616925026640820195870, 98213697434061399427092562854387044487, 146836955268102742990656278340922377379, 261995133042943233089416087244963216776, 242737096933327943705778669898415800242, 158202064241879971600676814205511013940, 3450857021565814679403967136094012407, 337270534304933782060664029567918605308, 17230447167679318838011040664513882156, 4393883723138065554898209233675953515, 103887427169996525314536969154465693632, 69419702433044909670575821873676536381, 299909709654454554707083351466110759614, 82833165425761969845733902154848795704, 186846785573881945336889063499980371852, 189991547177616996379455789735989851327, 165628915052945904041559710484705415500, 118748065289429331092607393181787201033, 122757002577501287857089193048940721400, 170685949625789277848880673808400736960, 301314301460957375114288149418088279744, 278243827666775757542090668246003504754, 117614572059700749059059043586672931022, 135832178516452659649065008405978692216, 199614852135592359850567440261921274275, 197018956405516345917687088016809018565, 62415102404037551467937694997210649875, 135018451299079796255840651946071674606, 226316128917637429917000870532167487829, 275168969518210087360460953950790387926, 168235179637332410034750802505825424812, 119815101840852686322904081625825729974, 316893475930001471347388644836447039673, 319304141770362143156933776438871510877, 334875431744708300509885227060132155210, 304334519293866390777698735453572709909, 318653084327492029013350701745622993021, 4486143902699004824062125781904052415, 194318236286963831228566279530933020181, 20883139321784576917518790395348963838, 439143124759774738055608466830817903, 174975649209133963461400129942146820418, 250553300623925686510489720748277433660, 143482571178442751491833960613027849122, 338599208506063541679994484245829524289, 250346338114417831396561571134819275519, 185457410006838522426733991534885461535, 261338675007463053220231587761566406661, 161372770645024267837460506236574122626, 141166151314193240676912993911027710478, 208430884134117353072064088080904109930, 74048258903024649481542058092417265157, 171746721568883184129873960575063880168, 960382483640901111560635925030310145, 175262691250993570762612981515542948017, 221913612489456117461429445630784363858, 118338502013261084366096929600877649476, 88592126763959895275235687195452501596, 280136790812862978357814507838171311934, 310747515925687410793402963161275551526, 29814061901035701379563418196825257639, 108412411383175639946854649188793657854, 23559156503168116042964921808443022620, 297234795244410396126032793084118190086, 13141896245622498969390049300576054802, 121482192956129831386518008896507844358, 298906304550707998102330334689363674873, 285926112581036716490578317843438932599, 265986532891211348028318063458849370980, 236931619912071014148825843473356073087, 13765840087643498754634612188111074275, 67758712206588454625971615803537696129, 324095432901210823642849314484439475446, 288889964333046303473402959270819962706, 102941468606592539331158811646005980762, 192668087420012611188751387609850176423, 107034706994225722803810778928036521851, 177118977582816910327201257533031324339, 145201022113994547640050434100585672358, 305954130794660278141598092769495468464, 285733651838336614829687952094846953563, 145145241998163450933900077156679135208, 11159289470407295290320426290977760627, 284453966487771901303641110085486342580, 309016223317303257763475472310665849104, 153846811970443556581616158610939379482, 35794507955053954277099564042525285473, 93790562417257715088824255888514693989, 23189062501826872119090552562803299465, 66826302668947434219867485364729130799, 22567890928540951560891000007648687496, 261063894798708598422298649736492920205, 18668108529191086250996945464707860419, 2123420057553954208461814714839070764, 3386477651629799581922566729940507665, 85239101365639837640638759143869842067, 311143705990394285911073063029475867555, 199330492521650033480983275267961187713, 140099253796090560846215739375762939531, 267190372541335140882306068101518372900, 135596175424848620633067629944963580645, 111194139043722038059292547635982622434, 216125640536868326597315706663108472506, 171334166704330966137491939688997840372, 252352672925722550067337131199978482546, 260445045034794499748410418827818750770, 307772120125210196547389466768180734252, 292049467218614856578892375278057510918, 124949356159631021963042420384466086694, 241000476516691834412928703128207059410, 299129699800929834850765814524450618878, 143557813584733049727100428847558966746, 298987709576067351585246999643487154090, 125606471529892985433237067840009762034, 190818928204727878961482839613931725091, 315061131585910379132967906343509045912, 169548171178297589745724589463469471826, 47932745108803609258851628197970548160, 230646950986037401806532624220612780367, 120708732545699881682197987253080546391, 295823451824218478459294836357940407848, 321127838593834721041826424181811806548, 196449279841689817770743917330531455332, 278326748698349766294944593716665308342, 23228751295683633774723128080749214725, 161621894415970361333792383863542688183, 6988589593628695693297230331094423694, 75315553508511942698362978173622157302, 108083138917603984536230873868576765180, 262732271483532001700778215378546215586, 244384522899310987837572962911243338308, 179517389620500538268168385983302628373, 340124108168676576284594353717115635703, 43695789996511337648216101843780921392, 17565494223198671505038174938304925034, 120455981630330619586042811262990086423, 78437309063904887563446766822449922373, 334387436652561438496747462951212440087, 53303145518695616675201681381203571570, 170030720151616038237195997455739819104, 137084558135822513973904000762333577196, 100304693924263895328841542530715310463, 60730200054086140975203461644007299678, 146547735845044956652814238645081364351, 29177624382485084624467494869920131363, 53938226577185730073102206030844558943, 116583705859083583470558496562049156002, 201280849858314781588837979930340003343, 326340522696516609979003058738294176113, 319073462692324465020885581259827688260, 60488194096999089009516877972774511885, 246343843655035554008001947111558602747, 213255176612081344481816857289243214384, 118033243422943786180443195379574213386, 36315161788632141115892150987770974556, 302494909067627686577806994028594343081, 221818747237029012852522446115221358595, 94123609189937477429460781857745521597, 242116089152064057927895458326981237476, 228405023692691891516229384403672144958, 297127091033980957450969430777975769711, 74752006188424052230793376513683582285, 82673727828011445012043842535921181840, 203844391799718701234538350726933012791, 249947026262953195511224722843346106113, 30172266307665912509887614336986471035, 267066035700726488368789050833310136537, 111088731732190721752876845933141945753, 224565564214293421154503814550981456575, 17318529289014195130949462378530989616, 312963269804457151358812928251479153845, 261058835518865045508176177654232893636, 84328370461743342873486566863719248960, 168448457802377996193972352686263451347, 319345175088140999243167864514854923222, 176175411040729858833837226776783969968, 172402601656859820662552555936020664099, 222960103415126775980453701140915739002, 15002085280458794532553596056793419534, 202882378660571965797947918613664431621, 86229808552453816126983282177377526249, 271963445847075695162405722249628222804, 225906120323433184976512823721633282069, 248743511337780862469890295419338441033, 187962953399205588067200299217067159014, 17067406276815672028297661517302515646, 324335094351710793615975870053703397113, 238023349916462766961222076637988343095, 279702836298109582517434226497513608484, 324430172722749437522200191673165093720, 115332056985807838156136796327621394752, 267926311082613166975297632078732385724, 336588922014476391334552223684166451499, 218337631253852814065670460819947827643, 63560219598878166919213932582899531999, 64909334903761377737364126416555521267, 59699145267730504168327880186271364971, 180800060693073374573315968570808321004, 45266877253102820968601633996321049842, 47340773629403147736322766682887198379, 336493595918264695033372154873405783520, 214549528350567837245685221054044867170, 129404100770712044902008287447817564245, 208496839219996355396734070402176348184, 76708714090795597078799490159427586647, 63881019319815974961040772785071969138, 79793556138065038936873736609718079474, 325505013762872325560755727238104723609, 337053404623417799967821783062978892439, 259707247406525194278078598582249102732, 248833410323279151613458072266232606365, 70368756457788203546918312411238589854, 337661993863941393648321752938999517308, 8630687646810692058640915987361616325, 289969813955445373175286616598694925693, 218082133809510554687229099389193594252, 219998277365551786949218032399862335366, 142803103612824230537965826550803590159, 90204155338478778531341852185211380072, 58565337941733005482205662259785975130, 244173988508048142889937409854545652613, 323179090183469879001120693117570507091, 104725105763522740788963657737467747974, 211571972641674144124832860899715036285, 142700516051709378218849795457035082003, 128168121010846589587947493994119811461, 33525736392604671854693225152171770936, 268167491276290812533096767579559012656, 319863451142328962600115986007101507243, 170872670762177142866142561907122334746, 172700456512082947809897139986946598067, 145951147460219493882464009737155137007, 199496931424842057734023899638764909189, 336235315833781699135573951548899628560, 158628057239366928071773134561851451502, 37303722269596173793075626598702734760, 161013849573125718717740447040573255087, 22921257237012844529592280734114972373, 250165695539538459031928308006912697172, 53991391970062537824579609364335683364, 57599765044700628574803633547899068595, 304841887881610771214235574788084557215, 236473203046874080105343324074070023241, 168514306355792039129479682030651229781, 271496334153278169534383799921922542709, 312711192732674644554079572106970816649, 165317342055591022130491635916454729030, 300851372191660973696850365939086044163, 13870901229102821117293631312949201751, 95122791818800174315762533908920687995, 231807033452124426411662320837640474763, 241722856541521513590482418639832334656, 307092805646520248080752591101092213754, 318315670296717351511886599945578679417, 85323631487706134495062761832759240278, 301171886879432230311412974371548215175, 258337744364919680374790648539938939822, 203760798069052714967969832928615929139, 4465572268886993057536618340764869619, 205651887429310007922193026488350453758, 302014764817692784238889581665348756013, 117255278231018185838371793239698473915, 224406644249702141561935788784751410981, 122518005942467375011317810095827196082, 264280276363351343983463583917951370611, 86897308546401042125718749886884295710, 35618958469789007017950854713058363873, 224447631849622696591233112991474124119, 27104425410005296380860181757689681704, 201191585151194139205824873491819341808, 233676864687524432521934341250423530633, 214306419311853771383233230389136652260, 147724397683593232723455658829271191571, 296390249586620699633916791311726558583, 45053021853809944691000423270494979250, 270923919866789148064538918301624501904, 93708975664362010523678374127274232820, 173192342104764791192634930706965028668, 152181853741164719438238849596132711452, 202564342907445814837009170010458895916, 203352930763475840082908715983199548731, 78289492673287052947549647851745014373, 141412194060353039221141416445290081824, 275911957431814353595040614962649276156, 107468555290548955553123883069131508713, 88377689858953141987490107583893412264, 261206673383625916829653247704460364650, 220113608829357456248003841066831004698, 331987024103692871204402755479129494889, 55160706702102137020124245212462125508, 291283718927523227606675586976185531280, 17367036686464969103865029889516859419, 203120970038614575091871969785800714243, 336268586078162668021839640332818819140, 110465047680950037360823693928496436002, 225508255643556940837483360694042143804, 193579939082635373424136818320676352464, 108733485612540634438569760710868975491, 212045860143272766222786165360807903842, 242551128266031764758720252878477813069, 166066958826711354064388208053649128347, 84688994471618664152719738786748911123, 192330979805148691077075692748261460628, 241514399017497303961282001595370344787, 118603781959444633176594728478906307222, 176029896085273770196566954105107722641, 112532821669709888208104809222436115450, 80723834770653693253388381122914993494, 187858038964622567219472009211142291698, 139743200238027124205659008580850701939, 175238019350456751401372635220253369062, 282988186843913558362709222168338476167, 279085755221228330163077488581052489179, 126353710952381354216318191493933392967, 227006772262030868112479007804913874813, 143539654523170153600349989686404599478, 123974500992511182604210568931530665772, 201022556377719764482642108153331449195, 231750794845642124512998738159764660370, 269953177378932081822460151226858605665, 96706175059430581810291267970999690687, 151308904520650907288810767961838406976, 188449254803464780307552469180841092185, 99982606163898060577076707225774426195, 195940353272628389421176983627767814751, 161465621624500673091915919502007877110, 322558235312019909385697363339390890003, 93966867649956264079045546978526826208, 219015924349078449960264278990872223704, 157348685818309555425270483281031831602, 203871189167201918768116366082226797543, 170670362203401337128215149297656122823, 273598296948298324183289612365211405209, 240354548895226287906385843104709186549, 37802899861756032780034342744731303090, 280996827518031178169642662431886998041, 14490753420280307669528160464117557427, 98183035514990826749920934594477552643, 270118071147848224861055877025678742780, 297243522765081252556962853358183992306, 228163338651796273780328424060910600817, 135139360609250869314234386992337756977, 58532775095788122580943658506188878702, 145685121195201615689026757910978125702, 111921738063534660994707481654460207748, 261434394336711089801493162928355084656, 254266974317314789452342414566646946458, 50996083901433742958255868993250513177, 60305892151434807177283796656228070967, 70273456126002548613781152038512750422, 178163540877057038639325926301795469661, 338132435690151717559860251611530879594, 132792600839364842700891398824740260390, 287442498468859593923943330038913612215, 268528951437630914381254782797749442696, 194411715642879056628759680817111260734, 38988221527178168852116628777237307779, 195430067811068395072410007037328107335, 38943278943182175637709958273961315528, 243883751900788280162054121951731207750, 99746785837862349652029194361962400099, 290416568821139752214415801470002658530, 132550812759898654726414322918796921331, 240120867120066132128028097542379110102, 177945633591855863515435056526379322902, 16816357945940698596574236977428875602, 102385180239805855604054740375560331961, 308040153617885087999970087220696715355, 318385681719276635121930103590302180097, 22593828138578719990895892985334934119, 169440378742936672408914714824388637127, 284912289888444777767889263759059952887, 294921691895362227642127215735343515780, 125898585276965710064946524533834966094, 247250561419820839246424014449846851563, 90665822155582469631337180099106759646, 259923246190867307430340016118169155925, 327469503599273343935542838599847745520, 293877457444450902416699519939118080130, 99407214440734362087385036989321975835, 178715634298621968670203835107456814288, 289658851442936548964740254014349601368, 105220036788663697078067500406478629778, 277798449446245880673340584804582276517, 287972065143407290894753702857538176819, 236245606003067853661509171300312205975, 252835329360950429654026019507888115741, 133568654588053847798913610668174923460, 69593196823397798500279896532986052477, 337554885536124269776477323696642245176, 338888063488236126853288099308014864475, 177965957457342907265722519527187932339, 336141727398954166079279082810882075234, 284191245903423913934458279283077897544, 298073514592461458296455403644223439530, 261030524622858805228410068384955904386, 211955306099136698775636753313848006437, 192525383264888591773631872199328335732, 303148216365211827537892139175770979132, 103135702328418936337310912899577898988, 228053614821231196518243664950275514776, 136691139494432478736528870226880514991, 333008502356778289327064007249935344481, 83254065551497429804247074210838178148, 83028305785507521932669530466417864790, 143059438933409854706260112613783722065, 172122676031410448197441506550991612487, 196978893331028556411823228242255581701, 212066445117345101855003157880438171763, 55832154528352642230795107574841539563, 185052067218720109149558084556532947535, 211251222659931535837307191516860313365, 147392848261556350075135013643450503568, 20754812281641540621718502401670410105, 170980262971333214020235439056400387912, 31154037239265384195369309841238145183, 326879895835148036156247198043256970479, 21324691777801656307667471720069061585, 284126250340026304606435049743897649287, 324226927060454617812537833753856050792, 27266708200556202773405651120360126969, 46982450687828932244356470047560949447, 74955568271405811983428990572582697584, 315825422911727679564060000349096367477, 62474076802708778698190517855034768398, 190526182971415183774735942234063455953, 100472355483673738895083115519186472119, 244360205335067848031981678903949165783, 187781179407982546192507920672052568913, 290732889377486543516901197330281625421, 79982996879303793339800924040884438589, 201884731033621778484538217323387406084, 56610427672915146484050355429596850768, 173860852197621860969429301866942496039, 162030112076577634020321667779066403628, 207954955193013008576314699328919774452, 12621806108487558054380576041929677031, 306048115437595551302988829151100497215, 261867679090730124993000102113868117807, 144281115848249752133616044664975614980, 192671985565330546664294170945800509690, 249099807230172618893533937813258759778, 140738762049549815070547675850028992502, 169752045973352105185381381962116579461, 112404627410325936366054261480042409263, 56235333900707794603482179890694009696, 82832626833448605043307730541707373041, 145902043735815246162897460797198063331, 316016623155155105022672959430729298987, 169291460546538332321880869551394529827, 330090108406664747109702045065959425358, 26790054914187832737370623201527556810, 67835768710568700624208493244362103135, 86702711124616941872247831944989142350, 37760708816159931186221479144869197919, 32833147309368640756776744649653476940, 325564587803892628535302913144581191046, 43814772543486044012231397053535330519, 30251540844214858527028565682506192575, 146802811591485982198597311575181249098, 166612232657041474497473453547390293464, 97913841350614173647388407820993860479, 122248844465894974298976450563674831357, 286107239657380739471303513581469197247, 20791857479485877998850324932605514007, 240477181976627846794905093067616401345, 239540666903237729404853566728474474762, 187833240820212210239070883240503402186, 254520200724369840093924146619844491360, 192006248620593515487601516358556249978, 289572623584453486776124464254297514244, 244740502923493157923287375286196843096, 92704963149860321253386379612348800239, 102506673465011926502783431361389509470, 255067281728136010139133251222533787550, 35170791090325787633920182778018609772, 183124204903403823503868454479969596585, 321161453069664989288635490703300435581, 157392316577696659536419602071109022086, 236472868777360098030196514866018836456, 56505134160647577265156121736160101640, 129217052639795899945492657044287530910, 73192869696433529793679906577462353803, 83603959563037974850178060376176937589, 25325895220735079133519989652471338242, 49999327979167289555177392830417678073, 65596318680925202143346977125353776551, 130533528931157235831317434436201273283, 313059663239558053103235807068503184582, 266093176430918447925483779372330223610, 296346836312294615973546467026126855807, 126544958026088333018486668621369835738, 317367292692101521846565077404203746592, 213854825987314428268094664693346663137, 134221398222614523959647032543715563556, 164773253256516608670805462544176063868, 65018700163183449926011506337779002212, 220218982008952927178750314152304481564, 161610890910176683338303685002317081322, 22195039181690975507162758986319524494, 243499954506759584554774870468392568441, 99675863943546011031918432060075303543, 50124482358701969689101601447449492542, 291321086481414930231186518948235527517, 337413111638156973865866527997613736702, 2317265719745365310920521870266155749, 176697843401857889158930643723164392646, 319014030989897273133345583919907367861, 306024420209780520093576124261571089548, 226972594380695786813717011576788474057, 157147228768582684452884126422782228418, 75834184749841945829376621033488244542, 11756233809921765333355308444117739124, 247323594253312333079377551065631665969, 182073021080083675384092696304240454006, 151817536524157616101646045913577820285, 209764156788480066526174972355951098219, 40649262791105598112569341483088658535, 76495670724931358285186722752510155736, 232053325524386088027095329512092041569, 263885819192283183246132140572694000282, 220432057358846986897659712595610701678, 142711319398156433121262946386558533141, 223005552135906914452721829041855786237, 273663206934939524135801842118268556908, 245677159737482007598211540327025275226, 114018960527353029787950210491149515496, 50329983486791317341340017097629170848, 92805945825433528657822234863957930542, 246202339157260609918608024172020097530, 299651499646287843287491683191866625566, 102738823687097434661318912055444348872, 230359685457116465712164362692337510376, 123415917822629593570263969937565557170, 275066677404016307818361085451484338875, 18672085592736353654000871396816984481, 232546586362322957735253228976879598460, 99974838037567392739243970694625503309, 21983611118862531309783942132115784082, 230688622198055074236713475266701639878, 159401208048069932389763000739933406984, 130983660958375400231271143712579161517, 71377396887693881721742320900264205204, 125620057297674336824685985001043091106, 171969536712569444447828744274588253956, 41232820467619823482089646267213059294, 262230581185924798660858308100307333040, 322994847312917505771674376872157553859, 276416533848094618838419081840841205119, 272299576481230928611790256914325057134, 221640281133270106926840028266779896484, 148831326338453782634271364724119852311, 16773049653768947865032964865318733867, 135538106526719934901037811476538289599, 188873134564173675050612338501730158863, 98434357813142757709630893407699589657, 176885109882888231019813063094603815248, 46276341924764055496313675420069652404, 238370033018760002704990283924109494165, 21874376580117415409618744235750905708, 318507142238004688391843379613668797141, 130501467704427215586981825286468674860, 71956580291756392436990353711473283672, 159083589145175819502123045128812711933, 167366642136196737788179434045844994459, 34028282227499796833156700539242855152, 161712607999295724598578331552858655102, 163822211179078900763712614451641654452, 116446601399518459100338804183464624804, 201259164349649761656788633848992292149, 232600353439932824171738379736872821754, 8132894460987946668213817351412528360, 137995699051276460257627769336061284295, 30768588965151231277335010092545072661, 186582512586942145473999599693029609739, 330677917743432677201020602797813117248, 248695330599972155015505955977862210926, 99419389730297839167147950573438932348, 56057669078219710305979714008388652569, 55714987673998313010936266064201240366, 181755284036879868179229350498876219525, 170432313507404595287080877597578549742, 9263670928184587055512507261742522318, 210497273051747561896880696276125529331, 239448968364678285227947697104411772214, 67087507815417406740284929565434245318, 317261408868431072646740575874842022940, 230747374361386992401629298931721570127, 220470848941212078366056155452021317948, 298169746108242834280763886020366099851, 168487505609574952360333655747032064541, 320849762132514250275843305683872311223, 110763304762346851748383694179295155536, 106043094948403192523824678542857889939, 35966080831948064542045175104729414759, 294964256144929235566729335439018834867, 137464128401884735456211807141295602505, 312169549448262184464598398982833733768, 233153760031585238174034958081892866799, 258765425410221647126762074548040551750, 5524572271381341897414351118967296434, 284129859194905471130710640480759107427, 84118546447889397107077651078382670416, 202722865346294362092764383885780383216, 210509084733357957185329710072418355100, 44950138051615520483590631453038424993, 297359910239273022417195509517646636901, 190332548223803833413358464677135209368, 77030356844257873017200715735833319387, 278688411259897729810592379531730674674, 37023755358713301643191322207978886293, 169652162504680289236128579636028485142, 30803397767999360380605416742841738250, 85485425595456221879197590445745033459, 273907936621372204102282043521434420099, 108500414704444564262845258459184505265, 23975341078766725645960709142166545721, 236494744228592354045920739714968397315, 173887337484855326472525377384999915905, 45524605594706539661470600569940826679, 225778545963302493406103111665528209724, 170637039541868469633809269055349099759, 256795853853636447760891895050484511382, 47864587319418583786197512941843679125, 252083897755592466400416443742668422819, 102665270537875822334403316357359998372, 144566933198572540269725169570902451916, 215972319532659992267618822501831219646, 297381394219871625112335967443017582912, 32699414562823056008433952136029265952, 65134884382698936422423963164474559545, 66306679266164722206150076724447071508, 258593333521091644953416097138301284238, 43932146082014512628142020486632915245, 4596721676073437694707816194645011428, 53419866226542842446799671748042146985, 141797441093901932413299103562237688473, 70794230870459235537342412448930144040, 1059118040260416223325879185491904972, 239224653788590272124701234008312817681, 133220303070510226750928380666927316904, 205358006230872604244958211964587664169, 280321576515205401177684770500536260111, 48203955000696612087011724538676112450, 42946107041220040361702103034786065708, 262766913990188809129011364168548025467, 95840253380465297240182421662663968283, 179942755280461127539900543053860716807, 15548521346806979253249317172207233972, 324131293760074492806418845850092324156, 183120679815697045617500625422979207369, 118772818422190620823427080255649356521, 285354051748064213403237547225765771480, 32248243295270521187231367277555393989, 36752743566285579064556110747332517457, 201142577240276113522840558127316442911, 101617255162773479898215169391232583008, 181288809435386959912374187920637097591, 45230285930966217416044592634049449351, 279761709575416678717271456006106218177, 172582849028827984860692834335113239648, 182864765388946646028808352520798562905, 82116839442297720266224003132097388451, 138157996870510303085714811436067851442, 289599011634125174413701101314819347930, 20761047215379349102012676629547604556, 231820746346409397824639611541450758000, 146859847134295364541568631063871455123, 62704480446017482219735513681790210461, 63557105372211537716336572073167382790, 189414036516828612755469854947641509696, 279425375195168637405181777861217691388, 260186085673436395338495520897504198964, 107133785226630884296579840360967567702, 325358730814262736922624954191507802466, 311931764293711680970847369784660069171, 183278932506518552060853167588696640197, 251216851494954573223818976414751215842, 54800506649418366639426250891112044434, 78949382972616750883780075227503759630, 315143057799996720361159775408934696497, 256577263607099311234394715166073555737, 194750747832240581275744669474470730292, 49089360222454897904327839837093579641, 118360091325898512410108970417864765224, 232497472733182986610600643464891449879, 91159448385778525672557028773257412696, 332405083379914525469063052571850932693, 157655152668299415458998306737064325817, 215426753568196631930408675179869815484, 172176722878503187522973884137705680165, 93872311607780671351699950917939195552, 246675560941806226349750148707317688308, 148533923033935467762233489866925954905, 39747989674641352003054534567981913470, 250661787244621626873637387913899953028, 197402737292239995080010086792615574048, 122586021247680849010166842452926202247, 44657362238822326431247506101944209228, 250775902329889604871796026833671291591, 326396180181561761088270576879543703536, 59613219366916002105052189115406361386, 283676830651597085143878286266859883749, 325419200450352487825947923533926291891, 243814719985477105493011512425667719936, 32062054464532041610640302781111312193, 254340951363978046878913623181781724169, 122762409944538129860128931705791368901, 117035672747241284471591480482634891947, 145083220573624683442439399056643425966, 196385284471021136203441341954496152374, 261313248381757370174278624861065064392, 28208356317449315164549837542501494739, 285385778871256799906675786600110104718, 256957748629842432453936505844075060711, 175632617597513309848345702965799320949, 238543273451586801323773992127813138935, 28692962354239471019892643706985289407, 330897474464391933194778242669192549409, 101366934502346494874912938482416609836, 198240292008617696773245144810725761577, 340231541198948505325084366257020564445, 101926414879650306288962679621651945953, 316948622478833885278287978196060223886, 89249868829542348696166195064704659264, 103101940643382209933231984751132943337, 108724688282214591578954740145189548982, 284502696487367965573593780663650972163, 35787042602058508895561041733024475728, 208339769556397478131417106829330534631, 291356704097143676266179705208438651678, 72569173941185199516279099848717194425, 252332105880113688627927388409532145701, 164085494041697642633066658577457392763, 107694411539502531635475369023718569664, 261680131844810046359207475973256729872, 161503748126285458069742343831439218702, 88459497882124469488578293438225102370, 116966074312465355764951785857542684476, 154594879285703896790680929636118114351, 161998934171723218924193749983494932189, 17659099920310044773095781463832529746, 316366003891681982007373586672273218923, 21323970484340620505884689667622072920, 273460301865363171860589294041613833302, 142954890689585465747018520026942394235, 230838761022495809454981682629183891347, 21780895839183245235343055925371809225, 99889373209549502165354017417064616699, 235669084519567756264926397515233364128, 251376167699256334377050777550569866822, 8051498550418704003308946431978085270, 272695480953437227944162877245342934246, 24943876154156912181094314360144234680, 307932885066393450734453265543395477731, 143327593567791887843121209898703834648, 63299830454202210917708759574360887396, 311961906760821053001952264867224392056, 80558518316241177487249698758303313858, 201194482877149308175863590092698316852, 293445131882135555974487770535512248518, 239108406754831713359586452297362901907, 93542606846683100535370909016931776229, 310454805466118261692534259459257919895, 256095618160797696265387730433932481679, 142844011552807032835585589928759766981, 163968718870738854567865654244220760067, 317778826011207626172598575710484766286, 27158164343523611177481776469837608760, 251519714829286785964882718614012286423, 323465059195984100857837044145787961210, 140647907355642255041359573889093191405, 108373803752385489843603394939802990442, 76750496235871694496730219955445201300, 235637454294556817134442180962333119230, 231951696738921907886856659666321948019, 54464901832602852493294423618896798924, 168914304563452973715878475103766887011, 187061098430866181159370669556894907030, 273854075601008193651651827208999328644, 330786123638437009222146939356996629529, 129188163191375544737854264275986316295, 106547538655303998309812914566190166558, 7533758614757525960822054508090915394, 202169995804640497553400611729292059992, 325007400450105808311761395928075824904, 208773202936469825271400719538832822263, 172050818292846647893231449312842833991, 189432680298144449478723240017723335114, 231982994371114076218100373020786094737, 89745743911790421461791988598027542017, 270670292455906343732350567387098761776, 120605343132453760600152818889424298115, 271396286997508795411794146648109669423, 339744046355224304789076932067707005620, 273501932507087222225096398228832402027, 313045002459880410076459946867858760274, 98042436143763519522872524837110399888, 99606322247550331589112545593137804063, 193419957071961050365384596073540325043, 46095664036624380924732449396815634643, 235301116133346979398764655723725663255, 121072033713249366807263515093048394601, 257282088211388601105032183251910646132, 269064602513389072934438948911412459279, 55474366530045593984868128471227407202, 72084214068303651281516962396233424230, 140434246967734598598427064517888933970, 53612429767687159649865300064689973924, 146865939457326867932069567600714525656, 110643411339568966268087087695976226192, 21379795616325419472886256476225506655, 180251250317236865955192146260901854892, 308661608683032663726698617877042490642, 132322742370346578289626910187872560453, 235022083457120023505174466643137399522, 153462103588685308108749370591285871077, 123738383101575598408187410906504833824, 126231074725015954884631786272013061369, 99823667837016563289343932076225104451, 193728934037639825760896409118918996980, 323162650056706530925328239628229857810, 291142782107187531300602634782366365657, 11745435954976434041449940643370790146, 329149581759573349405847356944794503937, 92957297606010845515649356975494895003, 189245287025827902691338974587054542570, 169727335995807596370142273749441873120, 323375864387605435567459825115786370379, 65984886000734123973242211873511001414, 294825241552542447082905394966666079219, 236100827478054541473576287368258892657, 55275087439563019608764811912733559868, 112446307331484353345288593027781869907, 192721542925418574088714563753754072694, 315038553389440525948780172140240043258, 186914296236692147418451655075766933226, 300811773638491766936979773539939113946, 266522496605162555943680518725122948958, 181751789164718593233573660357025500596, 192699852186598192593343531181732227796, 180692103948927869528900305591971239096, 332368364097967423535146758047220105442, 36722617504443340353516399156568831969, 328308364814962120872399022034721865309, 314121727282901659132149866194045427738, 34444405831529736823598362772251239899, 20266710496234189479044232565434827451, 161922157988903601565662760167282329521, 189887280859946629821350163997178476839, 158087041363031330626999708751827733341, 15524500709917727241677795791895374851, 251520761783200846203222178094143952032, 61340902770913845244882242927789381680, 189585986132738619824842673886264376337, 162840839597567942539515240669097100179, 59925671396488233108514450520070230141, 308995811835063465905963928609846631348, 675306092957220540364307255079667468, 2618158825746590391375050787641585611, 223352977976424511486947315251114929439, 306849004558157391506620850713064355971, 14954164288130674096486429325356636652, 72436781701696247522485497168597445280, 19975033546727457294750189231556607823, 58889542299512772921114573574064780209, 258902954453561421724535455174884716956, 4086567703757486806864714585900615122, 146933681086431871329164755967574759607, 30580076133892925015744390316789844715, 135989713120812160383011812641985285517, 166498966444491274405299906027998315694, 39689707359891783179083630751326577834, 330483600336957087101313402316182725072, 38441372280678363127051578757036736713, 211861870317725244507164923375934844410, 206029022322642566110294430127233032577, 329302426891560494945446638136386470248, 235833210697045309570347442479034283593, 175912673551101537927144541219282284641, 139785335271901524222464617724673874050, 298817423433664587564826361180950924796, 89144295726891536617089008123033765858, 50143786204601811298581395686737783054, 177144496550983827107625559207405789737, 42508685680989794697436957232679980967, 167587224047832139877578830690442654890, 288455175353358185099514577103406483244, 46957618771526404034893273007065492192, 103663299065125050442756740321487258568, 184614795531864085054963232035698861108, 306384415835873525041129979141324015405, 121864620427165469491812168836551929694, 104892077645492187304246818742481947929, 120641250985654932509383282931878716738, 104572164963124195662547501556673889662, 171998397349085780350315316038418983544, 142825295597357359783416710360506945066, 303661424123367576442723766051393929397, 243325026424987789469474208062097795789, 219565380917362784386097871716558423529, 42442646304233179985088473325378220905, 252372096438115722360782544603846747546, 90627466592921937095751897425263066076, 107374997564419391829942373512950387319, 248101917320908894176911417936416038154, 126831883048382611574782418699417354301, 295847308216498437190645495771645709156, 77432580827422499951375220432568342967, 320637634011296860013237191390092370090, 10140719043491481975015346796026064770, 332549033980357463511783442331405740830, 138075718409251695542175854840968633562, 324737932107883881370312927248379596481, 200459937891170372508492014092817391295, 226623367499057901168058691040846006148, 165973844632113323402875865469571496988, 34621959876095818078939233720523053571, 105628678571675989459283628444758118991, 151588710771788424880501146770639308604, 316757159824792217357862627676682708626, 327662628331474990091164571544764369308, 98670914339802152357590064758131811315, 183102293986069168506392351287293297880, 68246414114508526741661247987462017050, 131293515877192841607277731314429904359, 97336517017640283041327167083051160182, 97736014459312830773832429741747388837, 203669848641138035324837558520637235237, 66657066037794558763112923803262475553, 259355605970801261723575689671467436045, 301849130587717873415868912193231652491, 184017035184897933544124963883261148980, 70071742346417505998056581835220154883, 95611675079853382389949355446366069162, 84981377063122223542675856736753032535, 127723738615932824161327414220943976398, 202810853900852870317424413833548727391, 276107487773417076480587001650878005941, 261788442472647713307287655675955517178, 104316141202880988394322275693634264460, 102783560813713341188189447550707367500, 216282748321602424199803619653339602141, 212253904588900789562291067049053879813, 212529342974569573286831212339479785394, 216743283779289185562198058166975201328, 171943629708929483849589310951750492805, 302560155694795987894344400709798636994, 255468508422994827716023215141593822182, 88645737823106942646046440766666106638, 334594454596777340415374524165142517152, 19369712918241008965994468177535832837, 217852896302139657386878974155420568733, 77961614659194998452290662546955204566, 98178457168684433445410028566945867346, 222850638099293657754832435349181372686, 274046225755186740097250402432537474477, 115297265922239712277114856932489114542, 219896855480314553847575872561701656646, 289618140908063773652060641583082552819, 299538723192530645744436979502756276624, 173820506869846454665091639866431023172, 45593637600716706174399624094428454469, 322015698664749560990168766525472064836, 279699754207191173291232026078263543940, 70307942200425028750165705685187985453, 225485018091632731621256478099798415614, 280366954086250060678115384063483574465, 90505403394532011142898844629892543217, 280341791038062901334166536311511919355, 244303641238230057385008589136747719902, 319525881053587198707275692083704951176, 263079544203540893884689250341340975003, 190042754443690867020089235714883844252, 99040866729685742405432522700515831771, 119091945404297177650988590591720204671, 114743360383928034143414199974543003964, 2280683351737440760670151097875748741, 191534485696216626460043686026570018634, 16499566962272605057579622975982030505, 317303159728844362606227028494034744923, 335995374863558335180993076867909173500, 54426536092031659482830936786537464341, 235001190474248224681139397235290544932, 313808085272263823530424798806751873421, 250444885128812069933990796353319392662, 284904667538541095454154192348037500758, 96603896521341752461331552004659498624, 117584240704891114181774881540449654110, 6626608451694354782662493274499644028, 332128909730117312835166124416536933647, 17891642828432570777789873984683545211, 71373383262575588476547980223227721847, 168010484677607527004379563301790327322, 275117812595223315948724753012075724868, 96113524062069119793165584009365507144, 20444166001695006733915920800238780956, 187512395371545807599894853432401073571, 313275251201997633785127162940454859592, 227260394891871576396817614143181280932, 310136505141530103616408391138399951068, 308649888373587295004797417009441965327, 288640649595378747016666133865577147259, 101088874445264684947396872399882206679, 294822415995796592985828612591847133762, 8606066572987073144851620704716080947, 289448116555729494641164217014352799678, 102242211596285470736246261744515817331, 167247676385435419732491988054452453339, 128121379307769490787528615607180076153, 156145108344150354273504609481168421205, 242226197002759595572791554056271529782, 331787234191222750174953374112373183665, 196416721214690013635573749696376402776, 220400742622773976717780636796811469023, 207285929419342068205793243312452428180, 325290607947712963607335607359303812262, 126505146443466166889765951588712598692, 157299878629406770475568163799337688624, 63781732672533408363710203844330597101, 198733464568076937281942294593261965571, 50162114771837177039684876761953746609, 191473041657244625763064540924596017871, 258043625400284085686686878970831187291, 308891245919918175151791484555875163604, 258343415935397492897066475548189193478, 333326491315001216081076852082032080388, 308073525982746331333977882621979948427, 232621084341311271250455005467539411099, 164131094726215314626388434648618699325, 165757142530511975779441816372836926553, 13087065220546122527301667015746005643, 175255854620982815749243834437102725035, 185702192598371905770129551439363852728, 7632280578961303560470736320494372358, 265916874815034109534515735904733377854, 50628667021681596848840883271445691268, 192031654812121552151815265234621635134, 66592179178608763315244569696083807448, 164010038194972742095717757986533870555, 238735658875196876197034383269677823543, 29929447578962317067447537984689848167, 303867454843433484830593668756718593828, 281798647238981808917539013064120458798, 61187084918466931060071540099079471049, 145750203936361598616348144316847102493, 148226955497761055247475889398365090949, 106726166699106413542421225221086824740, 126436906813540065275850295409110179947, 24136353972743560993773408917723966691, 64153112396141544933520301959247820233, 91027224862998508600006380416008780849, 49595168354659074688388689711311527167, 235221008535141131904791421394981458414, 289668665150280691998728361394908988400, 150762672375064166493446433790250618144, 96251226197925378138578753384320411519, 301041920238699200360743698037082976387, 301137003428547827124210451834318911176, 41481968709596678505015344669706251721, 11275613078761905346456227129865391454, 69673512029460181830110810136063461432, 140591719376512579478794595453371935117, 276050449251852596875875217890504266291, 91592431297589489102356901806347123485, 159983765487414196082400843297483140346, 139994319526307015117129026100826596182, 242051805128489383586304119854545309231, 282433353867655447506846277376134419354, 39837858364279822120426343685542152596, 158978257860097438221574828910195204593, 57044595184539642007905533328501627531, 293233336577593170040435259154516781751, 18937797056844771305919520607596053495, 53851268243951529084288502207338920643, 259347577298470284142642790681702780104, 129997281914865383801158752881474598842, 160847590773503394432156639815253521900, 161554894493677750969006017058354541396, 28887961828541966290392547665884017679, 333981230541081879654931014850011799954, 199753047845202235417510586997284211615, 18156761461138556050728026477784545820, 146428445709559774327349922479562655745, 316450277317505541046323134589005026502, 259724092830092648283328266396145857137, 116285604709377020051855111346648765188, 175675610379276430540748792898733067266, 154594088167267459330595728082156094318, 294446008300369286653988076704745597949, 215272189339066739947800713359080994691, 49941598465960906727462376503588576290, 81032195174844879775216891089165895666, 184707940040806608194846786649148170953, 243922166759318177543510609802336652238, 264209607550741439327942821961438034030, 204466783512158863971290066022572541983, 17383928705423951102100860649731353735, 207765626071411157497453637116155619942, 157911058222392257114267168604712039, 108814225441612708528051609686354301784, 308792686795993845600787124630571410968, 323685352826864273911210314232085160559, 57208628055608213744190004209226746776, 255877532681715190220536519345271122168, 156337471494538242201582960757291443173, 137152912207231453107683631349443826642, 122374525649628664740299474471483880881, 67966757372216018571722998226095712869, 296628923864291258395726176678314245345, 325823579354823667792049305545227092255, 69601542643100286814936743432971621014, 287587065938150771331204055167934184854, 289628994040791271010802085014131162923, 64105030413325904372819913228519661805, 85578816743291977096923878222087591671, 119737420349086816819357662419699226660, 269500932056471621090700807518820998423, 218243022686161588703101849687285393105, 3714715048511169025512689480272208125, 216254994090982685271295026408168315537, 57273317667578746888765516507343851211, 333490710880686265569589116509246641658, 21744995010444573650627533366798769140, 32643133624114223316378423130843264648, 319223335953892287505132239553256105611, 252828844126185244761947146090022025208, 246291072521128481671958215630120795388, 56841167788274499143035554574288085556, 218055748207420254539234841830712709361, 286609129355570264624430992790147279419, 284211639923281975577647186930023920003, 25854514209031424203829437718178723482, 70274939331862993618469980613291459351, 166599129147481887819552483559699579812, 226739353074978792224408672424606525849, 295790393383271070066618478618241424191, 274739552542673564965108767664037094789, 136196736207184989232407984271061449486, 65792022020934281301280164508821680778, 305394441118281382728327313612399059423, 338484314284814263423000568424640522598, 204721668411305640745262199178189547550, 301698239619060787792216299973077444636, 302637780104276795582414316516842404013, 247282353117700563288922514118561141982, 181451726730091270560673870785317041852, 285208416205216315924667231690052617427, 186253615395778182856486202852137232612, 303980978631478746434393839344151983337, 241977740827755951373804080861218236283, 109795453057663947171657506512803761511, 67836760635784141080842353138842380886, 101407072173373538421897310880007648551, 248532461804757718321126944145342909389, 328692741883381837668960372875363717471, 44799525933617206424172532025132791058, 243866838211155992565070055202797881776, 169230545416977535076399212858062074463, 195723405031481686888075056748727290067, 185783359632318153409324131953790160340, 199567027771637823979235354207360436767, 137958117850415640661581809058361302382, 22935884820812629223316800169984680310, 74885027082656185253145263769531971677, 296002364579548159823787675477666045671, 259540966390373460768806262382009778794, 239496158743002519734689723090619988808, 156707817525113836697655443218500480789, 307484901194481528284163539502181034222, 120362055042352160211754908470838089003, 110129746233430210099425648293092317935, 154617158579641711631135866886021343637, 245708037340705619370446515873965390112, 210061216023665168941239482088099496086, 297216313564624289298068320131990291276, 166421507000076282872807512476317011614, 119990361369737463467050461399907113840, 308412680336201456033167235139126058996, 254834300038862919231334191726935618806, 100221353208062400569892110624167121361, 139129932315090733821311239387155449406, 314310087813249621088898667879983023486, 27909766409538246028333807359526235960, 166089494822381695808277988940374046274, 53300016255320204442502624684506243800, 160553028304843962902425323919075750031, 24965965960212749632570529418536046496, 190962308747422344817040083517517434268, 338337755359389391164714966222031018063, 230782312466977024464940916624937027438, 210946695018447020407688790901297803016, 179270953714180259177370757678790133990, 168962573789768315029330271457787571434, 3303595272144712609443184080274469128, 233932247407874281182546888818528871861, 49874620052583637601119009883215205634, 220752247167115108918855553394471428841, 6820701992895640295490327806602422321, 265537385965683012353322228569391828185, 68538223532401360117403479858049769303, 256246315115982075971520244857070431620, 148772704757051498505613258191305683041, 273192827393331521702438058900431956439, 259063268407708110827129078767152192778, 55284528637521756991705171917417620755, 116014462347071207108329868313369493696, 535886454551529172269139882515482031, 278619016556522775746992942320252346977, 293210498093435240024393621072579257178, 185322847855234824054302525937320133594, 63711335264866442945735996482451586918, 119091898830696539280858659376158395081, 103267185166306275671821484007648651688, 78550516804105229821091103881009647659, 316174437299509444032310309112703438097, 132905984421475288148170543804906660541, 108484974623124340124855537350250709071, 96475008961359725683520175053046027894, 232369008617796461429482759605659819510, 66393951117507899827736232283697482742, 279192921908434942216093714281674506435, 176747007548395453443062197484584414764, 292165347249286371569291577026983824928, 250681381073861662959722679633602593599, 77314282389922187097159051198647955235, 32730269072702968709878177481188913553, 268302204042391662563633769462256452692, 73173244293770554102524563120419270842, 25384304825642096037689968961161087903, 167135748502909574711328882834188768131, 224392214285481665541514474911544472659, 197181624221165497773249941045934644484, 275820869972108486885133071041665750504, 255667233527178280290280160326893044614, 82408844098984294582922717333166711298, 180608690660227941748916954163996544837, 259068397866011752562639179699944716423, 21834544536116504189038095009400966697, 157392157546974589607636829078120849095, 203128715472643401744460258983661028602, 249008243281984197341467131595985682465, 144078914403524368516089415610268283814, 98305587811147343854092095595702531313, 119595405742320621369620431413131439472, 290429158741100486392467173637354434556, 54928931718093885971678650630389123345, 164543844750630925233088515453563269584, 82055825071313244280883806855976514700, 278509451162292457367870304265882946631, 162435386431603342174463888619283502022, 66409807765412042159463075551790192969, 93941505957029229470153031578667761302, 187927557905787267698940358518088207559, 51751687097914155573556758361815637403, 34510811928326875104245296332958338583, 55332590642192860691382172095017365370, 66887567230758563964792788971969213864, 107471712260345358689784883559001127173, 150993912369216690056301816097751812716, 216980342809637869527263917274919596212, 15125719458706843037201006839010149519, 291534583924304448969879543803481384246, 40763372706232940316577417351780617986, 210683478407036565323337710873714661856, 78046054635285275008294573153363021284, 137608756264229730865182586218372498956, 130418332376644140067823342635814786256, 1251861277669620173563238802620859891, 49526101790620358316790536319104891075, 142243822778287113452985779725061208483, 48038559816779351792008694649902247518, 265928985353785676576727861588446677081, 51117753432452144185580000332860523905, 100081980937202251329861041545651407818, 244955099623710299207381823151871659054, 86823088029966740196838053648752996174, 259013704627571562204995829894018215995, 45034313185869038166212743172457144274, 7494243960360901275456419421807945407, 116979500127349811290720284676462711831, 248605509960320092286620079614858320480, 249502937144997056930784482864023452574, 229596113838308903820266118475468075116, 190382197890284690443946462907228248680, 133638499545994142424785599395859060898, 103971474540712777756831384765739775635, 5324971412978723174090938174340889832, 216987806232036843657943879153704718437, 40095290485197256375369098781899061921, 78262745620506704124083405851602848539, 101674225121065085799646864385068612844, 7088222471639062349656204442177576716, 145888668295895401510029126726697288076, 217977876361965631196842453895253837401, 119553998507449063669491321473839140389, 64147658875512262770595807758484171147, 104929689945614918249014885089354610536, 177904847258011969012607823036776409249, 89311719739412948111150312614951159699, 15107533745501824665940569589899060511, 317177675104022009219458877511540979848, 183208647237294939085659351418826829348, 108935433321763997717185831554339310383, 272091110180477775403773526888909631856, 207263664156420835634252121642892739797, 24971620925313934498821220313148528466, 195908896495177725295522164154803974182, 163309204971814084307911294426126758186, 294417123147673782433707745232546703161, 175961320985940626879727792601250739136, 226599639671676774195426938124181290436, 289627541534164900359745216831138147846, 59631508171957559878009356749546476452, 309742255465919756946484469474632937407, 82609712613132919618189018648685309184, 271027046447324337514406940983260751331, 6912435319426875364325697323616837433, 275428859579270283649445662072081286155, 74985637660763304635874757268640849587, 115652581856661417819568934637267202755, 140801549329217863603240362975115873498, 129449966179235574105685584384017568240, 103204811402873772248784195080766565153, 206478151810944740768998603899697573838, 138790697385847370745152530275414977286, 61198093085622075237832685658111595994, 61851370828201394591127188713616700707, 309130083722766808611761151409407929235, 182579172145167430240810867584269940913, 8908152259445586458328550797394595691, 111331105009389715478222200317077484897, 284003655574891493537956792664977904967, 215932848179592446632411136574341872657, 96791482710140360482789455397083414425, 152859582039018565362480966254166847082, 256196909765051553923740651844434528533, 152494236276964278429036251422994915125, 140841014152976432061146660062816069568, 303528893650472546756529865923709850767, 183683897128421008326379580352964268953, 104230113040818563942674368640276638947, 301901237534648895723320653726283586230, 153653229093186707869213349043975858555, 287098541882279585969031342014612377289, 266102454163004774069486952993250401049, 529163240475199264875856023459364762, 98581964864421478431803584122015889426, 95907124538964632955096436416566574456, 11139414970393255572753851725040557881, 36341378689933683704092396417630151077, 299295995688500548982915829108755965262, 185836280350979201633906238599902973242, 98910552771258298694745274822593988664, 72891870765842598860510497993582850413, 153929923190153217635844092732655145596, 270658506666265537076871893594540378673, 75745000453760003859346443018666520969, 76599376815176495592541041853142159677, 306412966529130275035453547304505842776, 35322217962306257006745262994201102801, 97502305404851971866256115077268427098, 100059056558607712807506218201625210340, 70594474397024706088758799465231924379, 33551478104968116937978085054548125743, 267773049022920509874209552717440322653, 243986837677318726687865408241160335903, 148064052382922227264855101542225775388, 9764301608243185216123601889967847018, 166936349190157552148898442061681565532, 158507236050410923791724088097618103240, 232224152670406447202788421238702562103, 36254436628593835995870718622930457571, 133532315392799296960207654778177581479, 21110821382166173423841265148732988498, 89262924781269303112942347409055295908, 147976889617881741056776107701501166825, 306769819673674936822500485192243142655, 147396646514077460187361319820718737113, 127195824869157765235802920064105101318, 221390108793164035955509850450316497454, 232938058262562118612455621059597001350, 116836013819271638659403807267989873391, 197932235465959890036267311721553781255, 295008918850072793391626665453851222124, 152329179413199138474295301544868517396, 228138498841663264294024594923010955704, 141472099724833159074296395358323311621, 124595227926839398835562395450463886152, 163436215868522081674985657058502017857, 16477173848018974097742874614530383320, 191754550741557047031885424327072745995, 154989453195444794658830337887163011281, 138573286795517370399444304875841890196, 60951268979851583802925479887101978730, 50426384808763419954744167530255141968, 90338472348065138206666146438850715121, 249968531932263358672104972269334945464, 180687235897263722409747714949994259383, 263300968757962161178533297127947912895, 299797450299790866684290905293755032192, 31602404540341471636560187233154858401, 319760063171876653902587764234937026365, 229961239702824987129774281584642366153, 116795327402191148829832382807339848409, 255482259501245143063191069603297872643, 338263537565450433644866205218317252774, 98601097168109706736287289887740322260, 48307233395577343349030678105977866872, 104804753826175325473496148541498609175, 50580364745124728347131515376067718206, 339181213159051117446950086016391271275, 297758929924698732222500238403096468444, 165344583370586213750114386851554206107, 227790493439108814457610776354730517480, 328201022462453661468051086038124684205, 169285127962095767840751688015688459398, 278072495026061257928263532276775250295, 45706894667092519898901768646116735571, 321516548709361534972678284006630382733, 54186659831717899234015050678318302740, 192508765065754483792457709431040210103, 224560725351814259480543092140267610430, 94233595219629463264150880186697656876, 55341803253385228829910999568201195881, 304815869602482848721985397750683719913, 188342484506251920005230547044942812744, 64381837536713865886952014203092974252, 30800298783316789975642906874755395440, 302407227005117568466809430084805754881, 182747245326058555158771527771427273898, 70392424612872717553112712421492350486, 204554188561255397554801411927730244401, 31598866984712708403734594803910453040, 292430471856259811924732324800155252003, 278920400715031205885292651064025142544, 292785105230920917457171456620171250285, 100294320561458377546363460076682432664, 85202274764532857142925260835890529363, 213693871048289415638134785890248913799, 255013639344175307335374313876332273541, 249724615697851583436218805441141249231, 80539045643923455437461706175107149482, 316830277653542708643250937831774838646, 96654548451971948687197286056356723433, 321479206819365393677943172581092051123, 9528197868719694173962267896310225220, 25950904971453395324058445045281244751, 268433152688990813849796321678099654821, 197904517785964141357271829429453636191, 297465305193535744736250726394483795530, 55934032768208721123059829321941122388, 183741046472364929338149744519145428296, 307752837784503386947513924440437349214, 177473935458127535291704223348973377035, 102164425937814698484898792281375526818, 186913289281048392844961153338304246614, 36992563992093458845403156486866811499, 119208094155434710590667654226169344068, 29591321321888379146309307060873801560, 46496588964144801545950815571032842317, 30533891705566840437389048961570083036, 71007579078311491072394354912692740459, 302810474853045008628182307191744842991, 305463229726369102378927045211280576252, 96529409942901934709938121334508276788, 187461651951285077712453951432926756051, 167451662434218454367392182595810957277, 324730406589424246439701364615345946415, 257507316818637443964253707620471968348, 90660797344977783266954851306106443200, 140762175897534334882217814849846034653, 55881034305796750464873068200491347427, 194512353629164589333331218436639869353, 156926470051193158480198762072918160804, 267287855327190537515707827011998259456, 218768812993223993921592602675481923852, 90928688550336941295358918077484387150, 329552526788350784078604283430006206136, 293954778601974364257886575264394725334, 169695419554319611180425214916014055415, 134100873852093312488715197314245802376, 137950535122335532789008802894089852618, 241300784516188145943053108075626175782, 83326776662870132101703988197697681044, 291715381663794269697237130023105273754, 8908418286564284931193042981426343934, 7343078817223995379342067455867812151, 190155561558248006355775818617915236512, 134412789139664993024828061024148640021, 255338702902520529735161151836213216800, 189064980512139614236979470644994376073, 60608990287976089638157042035708387175, 247015437671244292971438843616424694451, 254533193272076130126828912805096917039, 184556431742616568749770551663443750503, 260971342581963048735876655165057908871, 296288653383711113001988510168008292589, 258333223252387519528797877629823371991, 329890518177115213559624173557129726690, 222293603615760236502934234679357409577, 123213538747110456952497115261780607111, 180879727609369646249528100557731926336, 21589866261933934968516567929362918854, 327158577578741747887955606643674568021, 155114600183349478373291834030850496547, 333049453356783417755931016020070623896, 245030403239067740847493258317080556282, 4963813210904127879153610683989711244, 308677265090312809587788164198474867640, 76056735263637992972255030171699288734, 171209227174068036035819823655028090816, 190498448476576463519748179818886413012, 119951500374441156563482390286523293244, 229466194841643884482827862453937922210, 65880287335290059388104010337217977777, 18585680099147252152126512416166456087, 91425593188407265394479454799829912059, 89237114235280913076000903349255094676, 272074920780663891520612039578503547614, 232074820270028535023564481441113211735, 194673432965737800601801494163683819900, 163690129595400704874780598989142790775, 66546305851423983736314982926313795700, 289094823909112034071966324328045562952, 60733801205737634724996554814027166622, 130749408348727022807081576176790842099, 48966883503006770286217112898600028499, 105595443940688449418094381537535101339, 60406100057085722049759665997321459157, 294005600006215189270150890592052491029, 208843233310336188074855308671438089174, 62008401266421738184539112615654599572, 39773213572395371006935080237434048809, 265080202089094901712417111045518145724, 308294492626567298717836946013855782710, 289842042081068736682061329346582951349, 107152624785881293185064749640782031475, 97056624308646370649948914976378795727, 66753074620153288691866180278807394360, 122135076879064992791033076677080726175, 44334083671320745708334794772147511655, 293169576954046798240270927715226183094, 82623539651227898058227551895129609319, 96827242047175971642900250562291922070, 316435596399678162242488080498025033899, 14654922588354916404557214115578372489, 119877946759257808237352798455757104443, 41892408589915849953531907689858212579, 37768491178558030323423156623948802839, 247755750101887901739095675790867744196, 119310952564292820772225951744194725681, 271835605272431560115138004863150941658, 114754120721804265627891361771464510001, 128517719145792230641722880882626597581, 172222821968345180012311042857803982247, 174716438285177760621733885328304773526, 205225580378106666949644960521334739, 314400312616152026414261925206454602389, 320295649308602169328986605754936816784, 20000487043801332950919154147877941051, 194348328845300553990357850747169818871, 101817386656333416246908251372715089247, 329824587289701291609872636662325322581, 320246317320112310307591969352762697915, 179592576925674567943629658116362894886, 133021824022945338675803607377505331148, 304640744296944995378558620526880331661, 208577324494990501892065737834376225041, 334072687715636813160052994228008842557, 50498765026825561038698548291520038573, 112821503072671319945690143340241062650, 47405634606011393240910088101085799629, 144882751519574874334409451714546878751, 58274415125477717879704639988636242110, 295884703583650971978937728240496976525, 262437196924175056569542673462863281649, 317729985959463181836237312151426649217, 41921413033342627270640114838208216212, 82776141444244378507313280271446638022, 254797365082399443163161762262059598713, 296089174218442676799641795015669315804, 330575062349448236285337346888534002595, 127321004334589025256961766416647485642, 77356925074710574914050232938545163350, 44509957777652661855039035605857147151, 5342038422371089573841642117788318005, 41281833579133618588490213746646999333, 64810152148013331137552771092123485349, 240873394056959929474137838904316595561, 199872902598682758715093484371387600904, 79258550930514644097603089631176645748, 240248988236135180591979442606865744810, 286255951318199259093734764417305293939, 25473737613597729559197700506085451120, 113284802604435514139973707088046894328, 101466869294699277414754074245334362452, 312951658186247123551133433576468681727, 76028754373763179578923245600071203925, 261392778094298456902039277972322499809, 4859940836432003909118533306596983798, 327656643358099160598525706126600127535, 148080814899936025768589142257812765752, 122601135946498045023065535516095888729, 216241928927082923750379386022254927186, 77495115004672101355445307276815804207, 201600931135581790898005951897145534127, 6841685297644682995078009387374310596, 20337325043351614985571033680773616421, 151271816783170617542707467687618600513, 303882042259710851427555086064812781873, 188191506526204581121676876262585758616, 271285634777174032704898047761192693014, 332257885121025657339031441806440823664, 288298426681254832755405789666970137953, 308237165189802466692119214372942213686, 93639404725888193190725520478575035027, 193346329765276741054606344509618957400, 140684478603331948927401417117866551826, 316650521866568474775712974991765745677, 143718221623302700476802930922277488692, 329275640020757390085448956265355790421, 112384221323072164812069343554226741213, 118455787397830000395323868132826410827, 37114448618277905953983089021310474430, 107513953013546058792803833049496964418, 132767295555309879807558219030815776442, 198089481720764474792509985595378259454, 241396697053696997499735490432696135726, 93955591704048626001665241277839026167, 273300606665649054401267585327532331495, 96396781964487232910527024414158487678, 177672160246368881845331351123004552161, 77716915525561826906089876839316086235, 250329804278980716602500696458632746320, 206812638767051732676299282352821527150, 186008719235650289260949007651851756180, 333822595334832496126866445299085309081, 286523939465950541703792756563678834838, 310095934369992634835005042771524178074, 48243633818196254375020749602463415039, 137635230354708115279286205201961743159, 304570966962889643549329350211452968973, 258414516655187881836172300209690072227, 143839256486806992674642694749281196796, 81808422995079984027914361310058640313, 142986481551231023081814661840562995295, 231477617275683283377332041153465941102, 229062035666967917841032794175155615221, 259568300166215818857612519243189861399, 206777060527463174735943972520890148369, 187773205055181951247162617635680061808, 79741087228998295431653332646735754443, 181526091622670677792368969469997373636, 227030549464833578630665444904598042601, 251571619137576654955066079514514704630, 36357494653843465209962795815766806926, 305946690559240770401121973609971399099, 156541086723857098547448465063266333226, 6515823744987229611272115287054810005, 268656107968963332856527267862081498063, 219974623409787182221453204756053171522, 58911476819393696308971781649468769689, 119224421246592043400419237560191924002, 134638566572009785038074806795273257774, 231998454821421516429242297423842042897, 119107204755682238125672810253291626362, 139032117072644683521472187338486660406, 114897534611281394021800458608066021143, 230204038373559188541475260186527845935, 268843321830785195159464333063398187978, 114313780591183269349387217457632393039, 254817520949811076365852902730011608864, 83657405394318626472889942829911646313, 12853800345062474265028380540056624714, 324498706264469717949291558479102949721, 228237931534709201560321635616228826087, 20366573053848386585895456569737197349, 243386941998596928166658807946266142198, 83089086038657436333373335366402015713, 313473309658573351336594865080906846451, 193241101594582379565503063668963359886, 138353687763596234763550899062390435284, 213595428783075907705511276551572728569, 118187881976854390868338673135431043743, 109638191030811300340465977744320767113, 177800463579554521508888525963011867388, 274593437433445781997600735457733444899, 267236377116545630294336191488989212262, 212766765896317498215768940571676885536, 168442923787794978695611206405733555260, 104588059094959760500318196154885572893, 318248982792309000949242638815639248144, 178582423408572727747531934679611162612, 151437867291698272264592773749865621782, 314050536707980397684459940124783690248, 274978428508268187295429967210296000854, 72973763577217965039477574740998357247, 175653704019500070766900679521740601605, 51875383895145873495087300198420465577, 32529420652702990937939512233322507260, 104448752082861403953024801731876472593, 217704327742086903738313699403555566528, 136592114583966439554498169452747463725, 233952005985086765482107293106068491650, 143164154154949206682886670280628051674, 42314570030624209191158045450099824370, 160726665054754745944496187934811623049, 258400555672799534030589217832405560528, 306252523559835933744822163780365658408, 176661576452050700200719581712114478931, 69332154470728653026826472322810026746, 12068271033567996544327737985111524768, 306300243261305147063823654786224122251, 137146628847559092316581455929785896770, 186855934015885843980780956046559483737, 116855593078359497505519497601890347576, 122997805153101570789435263426230170235, 15732955340469107189371533198193263025, 211409341293307475227882124041344260499, 241388364304293814779686860065303190966, 323180581383815947434527512882423365059, 87054893184670573755890536538101974550, 333002298831063031732235668221927384928, 192180523324622596659756917420822554640, 73089358530039066722467345707467232986, 171740085465707404700142819007574319961, 289762618838637920569354465674905276699, 11369083687707083238696693970353438880, 219890104305625957589315632903500647562, 231994213919444267541761644474350314618, 129550016501456535229295803163731515345, 199199040069470729617202562722081293819, 88052587620350941529989918782998409441, 37602281132655516789507624854265907123, 253809736576015583614682235751334819799, 164753434142082855612157528034278373443, 308969012993001880989471294468624615092, 248488312567368788674489939318321806647, 80527828484251431182852580446047117829, 340177738312031360860711422917564012678, 24597727657337562067866684068421500661, 17050940001736145587692118149955169008, 293117354289783593819153320613891156145, 58434128452916440612514131779887662843, 269146946277665614892800054471259998038, 74710376770709653725323640692219010725, 64260078515942501485781032534735800186, 77409064356160193090728278649221012742, 329423871327797238807771284706145047412, 14526789754532185218742281407426604139, 89568618359420007971291042812107308241, 179507857177382621608464741871143693980, 40517792738653995414361681369794700883, 24018181284862202992271847149976950303, 99149735347817278738763833967996294987, 138229513196763015583520841838472347111, 166405844518871083258715970228659084115, 130867173935394255522683342240811781124, 132044434320949509752367703400418878357, 230101321348977302825467639037656701017, 308395988985440736617909553023473524871, 263683567305321848774178723804248064588, 3403358719903327227886629498113558649, 18267718842474595228261537756902041980, 334359956800879634503729025483172399768, 120808250854947104813684918544140848746, 294035583178384574811323549265138076611, 234277447395392244621346475927687608459, 81623186807903003596818535663327535114, 329259541919089137845710177950819091126, 124603296832012443165941162910812330982, 73798385157292089443676244230662597924, 288883599902479249951877527615225267620, 197747724265447678901139041478109145293, 334444467772730157831845460777017840374, 77612439384408183602905547316422158931, 83074618849864904022880708564080218041, 5345780348439757467156114806685446763, 329611552982964148075139804226916145065, 330291573974949464301141518173264170605, 52335317689036338491758024369054309921, 178481786895473754634628866532347644704, 184405242545987797437154363034364790403, 70196194810102878811647232463546633855, 277186223748788539888048528240205813267, 243472729475782696641158401590977881516, 162318150371136205982147535141103467832, 49441714019950435249444982597086430847, 92780899644919032675791138604399978295, 130054446243635498587449189006969379845, 32430068070630284559455195765434840616, 45520871738146341526051340877728043973, 229394962002229835933961936861321413338, 108593505227677182057211995581457512505, 291904850296484012097604787495861453841, 189452235038882469555324617094584453328, 124978996887489073862534127727462142505, 153017678163487085182716943104880515368, 209150460423508499864630609574736867714, 39221625496551988402441970794805759902, 199456207053388380875690621200936016233, 96949073986670521684997836359592364889, 103584122737002072736630539863931782204, 199243477881320460031906776622460549631, 279850840121134759855633575277250205648, 14251069494507155601131609759045987394, 257161078345223548649849906274009036630, 316334175674087113536576601605044937435, 234517694728952995021382142693084709025, 264339170310261369392670269728655008896, 62930901217554204707981552077037677528, 165784045875642622559873235441657890387, 110138523021290907651026362814541819065, 127516587438160128511262411720238445029, 162871132087891203216853507624551527076, 8611896167280305659643337872663440090, 154544705361540852614981478848986494449, 284881628865938484071182023248165267579, 35231966095724014492232597210232438210, 293243044175970880110948119832093833696, 196464331487309328414447031721866432563, 251165798030851096147496156598552733413, 99936926778767261560332737118316679352, 50463711148734850136951354481442813209, 17878151601833300631053974993101568770, 99301669277036934977523340854066619128, 175167234385766953898566553751504254049, 228580766353431933107638524096371223635, 274517782844956546413352666535365504947, 236778002932855816910410358672329120585, 281347848264546314970355339677703155932, 180280700489925325927999337578767107952, 112932725313741424387861754004104983665, 319595786334326316008606218173971732590, 19572907409933294449726158715742562340, 313906598204277522440184827944835351018, 330380731551877018628989054820336140534, 93212877026753270996171412387655858407, 281491658103398912320887309947761760277, 159809128616239825722132259215837855001, 248474316386548503075013858982352832679, 316838271694422409773802464788737617723, 139253691842829165278619216585526912387, 82066660351904691397439772455839703474, 197693582157157371566105822847476537545, 198912941271194577320792800739279881613, 39022447886902367172466459375460194355, 114277560283719184517430897850966210382, 216560010394840289827487874684235859320, 33403289768761905306209500171052658355, 250874103590240239072067640920541477983, 23028599308633999072546552177478744119, 8187025434344481117596041052359046225, 177980302650733894608974019390101721592, 146349566489375617835742291300013555744, 143152903723478564598241543337470155275, 170113209211253100881022085457333691868, 154562604788490585017062132275501586459, 82274493191224213303028728627656802108, 132044492562379411531431098123365213112, 266679462530866966827413258913733220439, 92847653123195915158137193147867538513, 91046552019184901170230339696121917362, 97675724099843172602002989493665577970, 172400930435002497237498187239728106082, 54001995641723109029819689386424782512, 315608674103913099463844883596146290115, 216460656373277794666504702683765067145, 243477160244922613497333453529789547436, 77819476117606521401828090711181001173, 282385219376186108919360895233025230790, 23322091176536171207120307961423971143, 262280431045443088397593110853220153714, 27112547827854136414944294418140156502, 273694665674504821325173819563259307568, 246584008263982922669871892075724244673, 324216089534873246355723381989908180904, 159684530090744629417629411556319335829, 227677709715280111854111695129554552283, 158732589501501767851301770222393557806, 235819379487525372120441396744196075552, 63969249168505412141676035317599532621, 165934456835124909667428483252783850427, 33845963604359218315837183334042457059, 208492093580539974544609021337293989301, 10790723639128355669680149409264196709, 165743021750164094897060414546528679780, 90252018394308854328464286710232621932, 96433418037998411680813034900095697755, 274370240868555956381129610375624973436, 95023081588690620714646720664563612970, 152405714977388345950493206378674047806, 312864878812514874148422678236226918145, 290492779976790379273416105479082762777, 208049082266181427377711348816933750735, 156818621188747683534610936971057922272, 252057567731625538036378978499394400073, 102467678832013080596507708438542070764, 279203246486115168729735132546463826765, 77792222219908995333548068046418016548, 329400008333125734634914104120337610837, 332361813887785880723630508694957984523, 99311658921081841836130750825158280062, 202021006901169425941549399484398340058, 136682521678043757871275833241437946525, 126138450684159295155624019053785843451, 77157669173345503883138514823825769959, 208757697852195697435989326027321979970, 298750734817226274171255369146347988368, 55689949338855522947724507088273553550, 188012216228173103090145430566189557351, 72335971371962171785578861408907069225, 105600335748215631014657120909826641165, 269397754668349845844494496779306438094, 7244360809566039109766863348401157814, 234125617491908432278893971030002626704, 30835073803411279113823020828396389583, 167732057072936950235489123620756791443, 146573727901030188749646761067276222789, 324949622649356280101326827877315825673, 123812786292793388446538297546769131798, 19437985969845176492443382852569159365, 284676568468974643181867838938155722831, 85096812267830203446130671205256553466, 231630952095993555075226394019936225215, 227718030661148514918791166965259263602, 338570159618803456400156444372413436558, 132655641521689333836576039611299643631, 216393709107799065553129219327633259004, 16512022670338029356319535396942951239, 161400253891002837160967920852521264049, 184242625278366503897953619188294789685, 22133465786613638316819282426668172512, 327739023724534526729583159738397344647, 193213262739857760918439808192965226160, 328212242398949258794695198080342923589, 89474340967863982434251502461852910466, 264257777236980397411756613812898832675, 94504729464668676675913923260291230016, 322263864942737589911472082851520902101, 159277682175664753398377422580542450507, 230420162390794086923692968310551925175, 289234212997339743174055984779286346633, 178265649547461533555329923595679055744, 190408118556088640471379145585354768907, 334303398640007389292173201344248362869, 189830053531892012183379004480132067455, 187947340539905180994906083812726020259, 195608002599929963322834820744114995404, 253618558593349814406769947139229567742, 312466805566058159919685231786339207025, 301953351028120328836556697091632301456, 50365856957187446185664774222454280479, 227951724704059541208169017989957520511, 280475887229378377777836236227467517899, 295808114632928265738418601940244662714, 140224101317014288983374690809706348624, 13467994960180363884947526905916528314, 48190433468951250103574405176372281641, 143327595007044649361566638921353883392, 286855803249202799084587220153466597099, 99586243773333557005964061344070802591, 81755470909576244844534971056233656788, 220619611176688535066990188486829213975, 250253162141016704677297556060052874448, 289455870110966158149222524825900404193, 177951175917362781666659190386976055834, 273684726498511113381570415185019672772, 160321109057429712142693149296054269700, 165554849010743187913860895061544728486, 212381453591287059920396917413754248628, 249498391544397856234703705056353674418, 184919023457030293143068289130408542992, 260517098479615341654577309397798308784, 113202944563200256602797705381934999335, 166561425236589820333351052882515372302, 56419906036781366747977634022753454965, 35518836161438552923391574031904779196, 138216886028273175565423326989274360717, 302623430143002864392888891117462615389, 319197634241963014591557392701333519061, 287842262004320878787953132954075543521, 38633224202243473805471741487101444151, 307162589238119981600212367008141901805, 20753311108317364417693458954243650294, 148271631948204159107696734132339619561, 238686206283506754558381747011238805337, 72858775931613151317481131179708200274, 126528848782995963686528949286699519445, 232180924500019825706903323036094468356, 104718654875522959742131594214209781437, 313640653383056883454348280382102059760, 291629054369537868672171886650500902978, 84956608239696978074861041703611116150, 94182758865926999829915865197533964804, 80096549250423075774322776764781770004, 265865333587942766839426763722742207123, 263021620716141561772542102869708653285, 280395123740927751974704582694840140012, 300256450503867566816428341239377850640, 220384153542894646873710414820820619740, 333912443550245807421014102249901888056, 33177211503752106501169619197900962231, 12514738076721030556463879149466648357, 269811716385031750982864063477994546572, 331256843867284038051818464482855656420, 253953201638451724234635771440818359287, 213894443316380336620607070086236562294, 96984882869420260799798630319752189842, 66187516608794575779115526084354408010, 132425836413407368047883479621053088042, 129407117787088065501719356586605197915, 300494542254633161137029827705827637671, 16660197205971200191803552407704673145, 89495278717083137848896751067679200346, 40544364549960650017249434227528837602, 134021670947339955643696407809913046646, 104456929981394357362361584469287014884, 281517655020070087870775037434971885380, 166462687548990380725739815901197958216, 124522475477021755527971815268330362237, 260632642171074553404318501829429557683, 145535066882776661445798231535505438367, 99360927675493768739715918610765925887, 216816709187042671378601944591467240740, 319949625908581094414813425722373190274, 193714847180885858556494124046764167097, 114440508197509861207268131622564464091, 140027373334985040237134097598106393391, 253936492789427793707310391794891921942, 338984585142333014774983099214516062368, 176475221663871110427823481495789019499, 87405658112073150267461728885476220867, 252528398637166876706268642716234167238, 153561130008002101259953329104436789515, 229615237097201048043157410167168171859, 13262880350916148282161185731892540063, 236682814681283519234228767441204428219, 29323695470325153769869626561973622003, 267286881657209642055617392541731397684, 21002648962864709059603164571095336857, 207168913677302130446572364631530645921, 338054557262160049251769675688024651528, 254121391227770861181546553462109122035, 253836110888564018889996572495857567709, 281796487191341412909208450590670176966, 281369682523523903150250912533278820302, 94046317302074132814766316378640857087, 34452678402643223935009859549743131771, 50296441623812461098599684995007233891, 225821457604001141244263718836550527083, 84234928621294514940868879905147091130, 183239064091612417404959418370516426588, 209757480473123087433018486631407772169, 51993563169542908350780057932104148821, 121841242207751595771817306915112377120, 131418528918942795437606349493419574421, 196016454818348068075491476826998418307, 124373290707625159879662904652619656300, 74350136342678263660448402851750758501, 113381268837133945233860029810259575755, 226418790369547179007978742588159208818, 307972628424144300159752891643375857877, 42317587001435920314239116263572321877, 287465503884524641447788433657410969258, 157000234152987780033454931991611740369, 108053514990837594469589702849255891182, 199076821609421493835699851260486683342, 65809323592846410933343782899400109687, 136185677536209703903191322358754870098, 66268459615912892036248284792593160986, 154081332219997405758230161283913425405, 69868248318124490519221242705809746482, 210356196467959585133361646888144603343, 102240896644371144977440181611557385658, 327496013359609319993040741951392442400, 208858649833724982729311187479629579496, 171064771242510142162593732444025353269, 66972294476000973094338038238159894020, 133685664876721794185777238937675031479, 222998632859607259759969415538839687766, 92025010791547513232619681998204366929, 88282744164726197436833709064730056454, 82492906745225791040422582032716369884, 38179208704362853960254870539633293371, 74148964801025415133877486352724128731, 196627257493418967396963768679334424188, 209916211774147953976746077421075699709, 305956395435582477820917959840779527504, 271011556270143713567687814562171322568, 39665298831816424614225975258134783094, 167750684537602237044337017350479792131, 244930105144254589282453028049846925989, 269179293010900505825460641359150588808, 264389892894385862359608699050166313411, 287221695323336414081443853701514287691, 114971660724446774154712917575643717220, 270808570286408877564863353883567012012, 324682924639887197453365056048305844130, 50888087282011694091098073573009095231, 117200646620483550159370074561858507426, 309128770775267466409844517414987600516, 244177900777643249840320343761309723804, 18236400814754170525282755280580986021, 243315803252882812593893750187062162336, 308067418426038719090417862507212437723, 297145331651275679933202069378553832523, 133720293247137103618189810929033526622, 137007070341671604191451859715638282463, 71187101657996816127543123256298870076, 272883733219148674373880672658313523048, 210213879309895060378204533972013541455, 135004007978586724653863377115022386285, 164144903967917524011099575238835477403, 24852048266264614292149558992623541695, 2410540657561721912610109969357908016, 335597343233363810806991464494127046559, 89134231181663978716296011801761835026, 167136166341648159463638881485992987042, 96780105178853411442641197879257526049, 157398644776682768664319807855203522430, 271690157276454592465902299167784477272, 3531473578516086084279049345270506921, 163068865246199262941768570241854165329, 156047272914891734016978691722766856995, 185594981345042999649028433988428267119, 13221065706817409548008823221730851307, 162146889284677785965684970924625807183, 202895093962149707278373050515993245803, 97238184203193315259203896302983283585, 221870467164081124559142089490371979347, 194864850723923034254129177313341909084, 300029143775391204327917050579337537566, 318437356067469106553275223154796989113, 172924909098353166318159553983467882725, 173696642554397485305861842116644213499, 44998480455976007179828731513140587668, 64365849919586483547164429872084862341, 122848516506470693510955075157987423922, 123124795755884034389355573417496606913, 238036319753344486937228404957679387673, 165030551659089542006802028340081340482, 46178189649678246013635806734345627503, 162893900106268133407232985764015945970, 156424343361169974158360992599398188144, 323878120275925225044674215797326919474, 69790582052617572482306189026240419723, 312620334135317789522987620857704515936, 251373850527080533748686849307633479585, 297484830843666581413083767157298212835, 224955089460275058795408003340211006130, 100947271193798275825406018645076856654, 254834074079022379935053685068436616198, 69407135372568540857658117346038072258, 17809152281906771862085290707580255762, 145411450607919141834401102914509979280, 109341500245854872779885005863508084787, 242113215793223265795591228992629952469, 269102849426926942549733159505008881120, 192340123157020261491239487878757302809, 141206248772671613755074329755220580546, 53332842409144214232994525124705493762, 205333601091036893014691569045256338791, 267652563976100394177521103302974683423, 304459268500967244771737246212154074795, 283151602426088631529129759662283092068, 279840219432158363307882980086315293830, 40966980790204412803756307998858522757, 147569308681890815021450015224414702538, 143514614941345133859516984293056763837, 250928950482304481078057090072781217626, 15309248649072754613256040947205432238, 329523021718353799047097442717763515390, 52821756374574637427412138004469646233, 217635510232001246518683479725021312182, 67378167229664245495823924196890016325, 80922598495922504836521645837638024493, 29829631341177951960470785713813651946, 155845224330450624785116908375970905078, 329395537562215576175214977254465623375, 339035639951578941663973122691928715014, 249786289346694951233485933235918964654, 146655406106800391206454899280228702258, 110502678026259384010510926621727525756, 289767857668970315410174859121364397631, 265601985277668215612977679132590660644, 180775075594409736132937940271550607966, 90925445229329796571030332948211309871, 280855523581979037411647328645458169074, 160738251297164557160651512008827819172, 181408370097528113399738453793724605484, 211476435219072139275041536646901598000, 249177678571264592800043495290552137214, 220211559754458019312670441536675316250, 116420820396652673686713538644060622142, 91517245408652112089178145540731232115, 258492729936071640587876024346875621085, 135398694141030457949560933341930248207, 229375175710028672464724682838072940341, 153581030980411567466019502193980987420, 267490548967056423944861139045083032600, 15452904584682516152422420333930924876, 323961279823615490149408580687457788953, 276586309191562199876025169464886810424, 317577646847607179006665028884246378983, 48836639434150828378164607216660311381, 285735156123597627180137669771069033684, 117281768526859810109930149120864260548, 121147586328032680013403569336016250389, 88446612615082308752430418346265938132, 10769487232661052592549867141485777873, 263831059440323936664063178385431331252, 242165705582864534201752806008957868080, 183870801490484124410234612986922623135, 207617201140593906807556983223482433932, 65471047336354382958051472812006762579, 142087140970079025149890957237338391712, 151279937803356211468143557228033534272, 237205913643214025117023383007067334644, 22481619156558248369612554258496383304, 249431271835206971068120699925526129121, 300567457007788869579322326044956125572, 125450399262056282989302748537814919316, 283096335997579481672120128892533606129, 211057142562194555904289797606652547729, 65993674056349929772622439561265670011, 72561534433698536237772365207708909692, 15518068184844302760988282315606241261, 44716292825393773039731862263183848467, 334686522934118110644439670638256366677, 37952066814113759392782228714404399171, 267856978787344187049167825026052574519, 100270542674850573333704855804951965013, 284942964849996651463677817794086997247, 40350419524291948953440190440676786779, 84946984315581812940580748116330799607, 238021272331491706559910288501927864579, 100037365280285735358900557914300273089, 327932360096562180429440010870479912635, 320380614506769599260533868185538237458, 90989114707403010569066174838853035302, 249242657417274459503225668572353437213, 36845944339739801078616535119406389477, 97066206003574103953559771809082077117, 182545831921618233085604953353301872743, 13757937420542698647087252221589834771, 224838222946553655013395964617557286909, 118646913650117916890260188921828354064, 92927838523209215257851634234647710405, 303839976702583979069946471225214129882, 20146451106499985992099200698658042634, 337333770147275182070585517143742415273, 324748324609095952104936540339806876807, 260778499216668251745801888748855904759, 266942160513854991035536650703768634480, 4147502368899887801136670351376453100, 136125148687819841601599066399637367097, 201379625201121866601874734148269905136, 37020357096153302560917783209801897810, 28349324821969679153975518055050485061, 190981868042143262285313254388417911395, 275939649952431174625077988713202241542, 8350180307720590741642987455301255747, 123455460767927166865806867521382758184, 92224783946752429243470738535472243498, 209333408331828173585866316903378407338, 260305295773361479423375376503301837124, 43066647498682935203001571908916114396, 14278787078918540152644660966660584827, 1501022450771078197774637572267119112, 263859134010584855232812029467506120923, 1227171720227330085124208400497204571, 104619770432806901332706530120838523519, 286502922438082175380005935557757068558, 110617676835710488763778093889670484778, 98491278155485050283814105809420226162, 170846929754743448097654498450266460370, 89017442706609105255089874231821881931, 157532073563705050645329529885493571399, 104843255040426251873598961886502277619, 41443583935890849004708875526712235800, 232225572240375186624912921990433608209, 291345770037962523214710943429833813107, 203210539919185284168817460054270970725, 88580291214985236491833648295932914810, 212008886284651089140055531126647694632, 41790757930844349766550387441885823399, 313928377216923706880927195591985172302, 308767948723591928124150800180718775435, 213815709308939320021951283176488630308, 78761179355929977492703581454639953008, 150465121925285089069136580905801270302, 226897096459247696599780904550264974567, 148624090280140640121967461982820623909, 291811699667755516305160134804506673086, 183349519019190383021379534965029266750, 216316168054108354931121119469275848397, 65846931113382717057129374667588962812, 194662559956010671882333750934112686069, 238273691315834532662308753819498673108, 58155474280866065603532762930415419715, 234972364620600630291438061681524913132, 267482573167025644831449578275138364411, 5516717703599343844971338357021793252, 4723775305422397502965087856770492462, 272208353469593509742768775418741786453, 215249831572207972715598030682863380025, 113513522436709100513294436021709851157, 7869299164547922179615497838138308804, 43874948436907043486494732714842239198, 42892952698121709452195510823564365740, 244989052465449158227454488922711201837, 295819604327782517929565011150954834043, 5019296400028622643808401954487629307, 183148555081096825932102751857097043671, 215897373500419481271903002855009708475, 327662399823878496218547738145101932751, 191212001847524093432869832417904394469, 110204013784165298785569305782053302374, 36432021124941915748642653530780970229, 275854869156216912989214480481637273238, 170851375865197629596284345146719308172, 150861887013373008427846410254474576616, 191525483268140347677065772177136448100, 182035546101422600318531237167898469454, 303104613938543953215729842822034362515, 115333932648127603624949964872009280123, 232828307745669348911077407007428883973, 248335774837440659194070582461399614682, 268109397809004743017815688573547788660, 100894277078551658961676782062571795827, 63887313248570668082999255067093347663, 256060007491565148315551238408385716521, 148964749026910190106566367009408726641, 278318799703978163081285759899642678304, 296973998612442588422428375350340666069, 323414311450653208182624874101817299602, 79310132455644420788092060160691294856, 334062361753612988846321872415763045426, 56251376557775549470937280262124422448, 208786287075757264895044880454579074890, 279244864457887467452408366801046207164, 66980194991049705308527977437001517562, 24969523688698265990698176703669061789, 340071448926456319307908900821212595323, 228175217290013910565152213563818968837, 77617978059403194487555751432017366259, 319038421762311247926070452407824965172, 288900315542382311639411570923875306825, 309050053954402332328537509596783523376, 327920134454368322186460619953120317578, 300948683574408390378668682121865627598, 109851753878942542923321283841309202253, 51078800182536503057646569287581699921, 175424342412417450326525620148796801316, 224681639408381222712459082631035235126, 77799312618166109660140846596189294130, 62483932539462081408270083926302828615, 190594346643864709893347428472051843057, 246879384569917719202723097496984393516, 44676328455736540363009260326771632059, 340237199761159207414121583194764448730, 306420625816997731233247500289940031219, 277445420192571470193689120554108908257, 82634739638553663883709435650924055044, 231130429272112822873432645254448300307, 263609144628142134525502447746947121585, 255852540056821830424842185307457185835, 4024038697419378405206763692295624241, 319293453470177493620189402716046050353, 47198237302471016494023578777908836085, 221585773746367942969097868275953333971, 203327235249459661790144939392459303611, 94032587922885262315668504763259596230, 16051668672129207406285216540376947022, 267064561415551347091122320875495637673, 67246937421579308530866401923813048215, 197938808529000104621543823546346410118, 143480475094843695897840434666576740526, 88875999163546050372512152535186380357, 232090009321315661378448398969514443366, 316581134535687830795762322025165477956, 98575932460953776506620174252535518248, 222649734930838825064513097130155399048, 81793834538947307959277410456504187545, 31286418449900893219788640627977862338, 154713471090232009557756889299394457995, 161333819123912371855157675408098429534, 1969959573331032189416435597141747086, 101286616644528219821055924969388534086, 109261156576125030449588333360171357153, 220037848956463198394354381808508690749, 275029721928289556556558586775165677669, 58338190194229339259958008514250354671, 55955285375371736465964083225128523892, 259174074863482053611317661139154807851, 92014764000727688061887904334441438842, 132874678211355361313746926248374977881, 324304456131963357104765959158175909763, 235119572305553473876269673715977437158, 16153178473184830178900362277991796436, 312782380603922384604809454554643703278, 225898360239429269333455463311890677549, 10549869139052997242663202573941731742, 169492764791334708957610893332106777936, 57100981778429485744795846184034049616, 107706460932016368667763318849455523119, 209598214404064453270344762324283794254, 78460556036720850917811581676800096484, 170432905649161072096167584170861008346, 179874542460319995093074498698354234270, 108280094406364990513848538918126040492, 199260740094575518294834786402739061550, 63002980450287044088736720984382342252, 73111476593053041535632426879499186796, 192175603230187864594409569931511561843, 90269635132774890499298048355057326833, 198235817407567094019064305098328507608, 329122115483981914201807351190493199893, 8238335279733132942835999936951440146, 110744443609378057966216023286416125682, 216430886171813385369406889587611046006, 54474099424245906150488340868060650258, 162150145529796242236403877970684539453, 328497212857085639581723955934737259241, 8074887158415376185570539842609398408, 59359908958076064534662942282581812811, 226129646417190014327921960724839651676, 290637315934751000187250266750893935969, 118229615822531268893405467337873541549, 248392955405206628590156711838228279592, 204666066847412082805199509668046331391, 74723384282604790635957339724590824505, 32556812079614668110897879526092529415, 299692420539433811369102779624351898341, 275908254607410719366805728321407788043, 242987653721787821936189908463008520455, 261719097443440555558190767821793977676, 155643219170858568447507944744829284313, 239087481250385561673133257060697266564, 27780329839126517679852739612568405171, 27156833701965722013200021292500314994, 119861118032535688727428227212911962298, 222243932501313839461452176705404298199, 88845196662174780867289703035904498585, 95040986172394643795553273602134517953, 180719806096169992825616909308387761363, 337219496411662151639873157671913274721, 202150163794769559533397294119739759275, 324387930402237124074448383113073145906, 339256869438530500027308995194855216719, 240505947391795110477531110439405371177, 119106759145609372850072554951239340120, 181802442768222364809551982505747940099, 134511226874537941581987844103974416730, 138256519400660803976163287663186288853, 9302729060217294123770614182605563701, 254025821649191704418284003678640088290, 80662722986537462312681490230575820001, 262772073230763270878266947661541045738, 272089074197884075565855888292099464134, 49926201077699045586859671766176868571, 265029210982876814408090857998317714823, 215661862791280063291078930045648093254, 249504599573587706368179324926466285282, 266766857922874725447023776328111451503, 213482724379761524267041201615985262310, 216192080107326711400645805659046292268, 61042220468699384325842809104720010748, 61251442270381929304278029875181753303, 60060358219211983152073711368411857761, 237608216868122742126908930922180569908, 71955719017896699856395666122487599708, 318642958588027090908994865321084877759, 83967931385281907561287489527762396442, 173445271365984449458548596361677733041, 260948021397804860388107465233140744353, 325554231966926085466301618235805352302, 269627175454880916165776878442859738525, 295474563669496938477331009040058238973, 273208248046876932839204294077350515955, 212774752209132353425500787503623396902, 214699938144418140695934786736247545260, 324164201051086149977235448915531393169, 153813748310123455992184249880069491938, 73993203672113212855085247199244109251, 58256990471575897044727803236044767705, 138354751151880790544296435888124136271, 295239694086990287709521673566939962940, 337286550809507408335431786579337446195, 36373521143786284315331476428413249522, 285916877278343890311951071545270981395, 171537874783283721750790699280334485742, 174497503393719078461724999695677611461, 191884753992394730055060163787397468173, 103474591843195974471486753480984537574, 140756227487463682241916868234408100334, 233504965755115568262059353181740311494, 109731206721247384092684837554248235160, 253404578200533065631681731862523654931, 331059392510288003815383013988570857730, 7547099230590874370230005306965704125, 223012660460672456004378654716499563284, 271267640635808604633808488243218444811, 75314348195722675864981457307060786605, 221886874767802132195200733308232318075, 156220461353551671783048743249254406065, 281639282840552629181251890725841992195, 37035933336361871795195386960836769955, 300656396610869335612764214829370302626, 139473625624351775717560720882162292729, 224380305535195846137687981112916403482, 197012888802297328409852015601740564021, 195038703541845880727177914183354385953, 44066815405306557003399549912079247014, 3751402859408126300489770570034773006, 37508565377078490519017444219064166769, 339359833070785425991905860230166798670, 203493483781306466059831103607425197400, 136178522239557925617676899975784805365, 98810120971301455937265232988397157961, 95047401551922608253067501539188107859, 202134976197266694390565905356447829305, 241220360866737284667909795001252288113, 238823577030882834174816836528998119372, 27253383714851896640789456815030642303, 25245455350169831870945617735865768314, 27591057707933410504461170885777719441, 134990588153534847254749974696888114507, 12830812748821806931617310144699051860, 252865849113708741755226605907129483268, 245072485884294490548379295941842261606, 279352724193288187467141549981846959333, 314796760354658001168047614709771243480, 85891653449538718635084392402640647206, 310023033497125050463172083286990543750, 236926094620759058385762462776605953903, 198320432652710170884041879047500118721, 88210111448382669873708790211150663588, 85855308016936187343047587239866808529, 312520017831468159460324532162856079235, 294001472060132056243184428607829568802, 169887361653314570214978091431641807727, 146492296010499591878798986640959608524, 160241783728070885633625629801160249934, 251812444493599391792269699038329463852, 177216848480280032429585186824885505894, 22809726052594272920086107713159664075, 248290767800743614241120958363636576860, 92882684913019414341849633996518340446, 77117686876316371252588040978803403350, 219838673437311381046363583104904844406, 219338693125823724408941489210177516455, 22356915641288111703541713804768855991, 120370685013032920101206212032824062821, 202797993608663890185767607798280795957, 60125937768400332468957547791942706014, 315349970669912208659273827977433807396, 125702883945682095914839942842330260561, 202695245242994433759577496897754611730, 217760896815749250236407828756088579518, 264754754685251617690543455771784895397, 315042846433230993427712260214136214740, 234833449843072503781884291225481582102, 282842223111757734777654249352251976885, 15721505046182162030907609942208059140, 197271298518805788562211826771703070431, 272633588916474605611022673997593162497, 138477154008059715849655529697798342649, 149918897755401602620375073229084553954, 274615505289065271684819924217089641898, 240329291546657589658357182156808234645, 273644255357856227477695456570667314343, 229326480130793412775348141192521896276, 47576672431191346714641802494797694183, 88860379631388345892248170335196301997, 288813953151344039627027201971468495043, 250925346736677987796701253817322222939, 116730107629358748578641519711023670777, 42466012941979169506591196399686961067, 22095736522079306085809280048518734303, 78161121470121799506833260950294412936, 66652088790510444483462721207408678722, 86728691035086110808055159418480443508, 35234893359196193485631909722579197008, 289089196365071473080519411507117607830, 202733667604670735252805546687094086086, 13243836794264560235771696631739743545, 3588706105773549169620378361189724955, 180618553400203943010348876718576571116, 113139291051960291415296322296893226519, 275408076688864578460847006321142871074, 155590782313896501610409286335212293844, 87160695351307006277314405944175370548, 280685244335855078209387080298296028445, 99330167847156227216203604598002046470, 198377400079549574698921118976786841660, 254318134173556069137978111094104069325, 195681217672521035846603773398697921281, 336540917175849890841322434296446668966, 194982407030561803377776670159675145803, 186674927812360651545983431470964878717, 251996677845864148775382837204802414816, 151033416697782380976055613124155666202, 28708132773560950110253472078866531541, 71378865503836997997659655726055225785, 231122153384929512725348010223597522361, 49656508547070041999779799778985286389, 6397831950786278072313724225983052713, 304109738531434640868893653861095095284, 67278251827827017015773260330442103037, 336531420715305517001569730570440925210, 276675605913483061443420066887160500659, 122556976028637019297984406210186723396, 75661194127015509848697886823283619362, 85638534770471971123088061819214967918, 216652533284203681567896678739103822617, 49953374788671443765997210736280845001, 225394483734363920280878795238674419910, 21111254582083492392448507406757308045, 270743601041394259473434513485414128729, 251305838486467603477797622078640622350, 63954219889169421942647675721355759624, 182641829857013663706612007310913326002, 167007278071313733895103105390510987830, 337656082214821357706235493601302134221, 296504730603562816074142739414473795570, 279348753692254248737128014803108021231, 34214140583994870179146757884453597845, 280469701617976380793588908308556494700, 273577075790240674930873493931070591272, 337820649948863539289168570559905758498, 142834196198821934435130290494145498994, 328807524081537580589761263645829813891, 12073940174068727640356642996635694596, 118312945991381983512921948074772116474, 222335333542703736392257437357491540570, 149690308943842532578478268461530835317, 8606654427479880300755531249836816698, 281066064893236636641430522386217601860, 286804169087291592173963403108117870351, 123411986735433435044485438696553364565, 101765180860338753955813225049805414829, 32992391147701010665806613869981348643, 295097038569977620325283247961617806162, 259673501675545649294758484298735155736, 68640891910609944139225067380554088592, 319269500871366209775193612530340672247, 233846168008821074076665545905389024470, 56065616902583476295212344007860737912, 10651769007016476785713605436901261983, 50765890912161999547355912188217633113, 259959690155399265947935932977803472620, 330620098910059400694267341167380016972, 7845001905222467507473723958084045843, 167356614369071652797435188232160692400, 278280560365995847408268765192591432462, 289820826972613647112871144860615942163, 243150689648739361816743413417975534695, 210967277465994989522989112155475408854, 283544782183893327277525411673635372471, 50654132995142937822100956515743691110, 262954701933618023360409861193807055016, 249189570878637174263937466606811029025, 260026798366926714764504738330934947255, 261165483862497556706109259770497090668, 23519963308048302301820537516145652831, 32735347709958945214881031045008675306, 141430383455499002128848410292790789561, 337811991252529454417577911023218483964, 336991413735399378944285706759089334756, 137047983317004362619623579813541210558, 59855791793694032410052306450404936175, 237967568014230128457873104017594330303, 229971426569655503742885913887446673132, 313459868619407177370690790331079619970, 200756327044681854757825776024387978846, 59887042796311395574047115334361636088, 179596810285595161735165809716638740671, 186920719698804412671816263162257067892, 259440116409819359656394392291515066290, 190619145868367074235385490792177619197, 163628774288392453692285117915713406820, 207491254519292818469944404542921780060, 127121108197473307093882614373264161661, 335064877212580634650864373736887606766, 250281294523389806570567606135185484350, 232177493995405653225398867628180469135, 157713625766828508333436728841232077162, 292443423829747636793198011612754189840, 244567481090720432110653995975661438688, 41725881910304369882648619946711681347, 201661596550531756841747227420946978720, 7610768511816277341469381017537448413, 198378956054966425778986585609215727428, 137022377661881152321386377193557167280, 101312596899244605164225894700731413460, 229957313662122327959701017262487985924, 205578446498639099125837942773545186685, 200936009226677320608479945335504300361, 247726025875928249862201852931082532882, 283796611834354786622296561114576988958, 24380163897597716920281739241094839333, 294099647532054965188137191290119032768, 236289237729008426488579082816580900731, 100310525017805633007770684107124599911, 298814980025051470446259918200168759719, 222385215403243010500011260322554044921, 336490763996694833167770381451355739067, 237973758728301381690808588742170074398, 292458837698106331974626866492977049560, 299798654513518900813856438537183701707, 56961581342125809796559897076139305327, 278656785704816029350655773074168954201, 249609815281544985084485807879593019629, 176573670359581117543368263808918659820, 171177124898295781377534769235913500188, 331802166436985760561027433802168631360, 142073000175579065576960023360602027822, 10053810144325823459871510855898194433, 337884038601044902371029620122777951663, 36712888347164328026298330410065010487, 295271355535483310380144047347001985865, 283066911485044193566038434341082889566, 99794359278886146056912917094601153631, 64295942830337137718799722691344573525, 157343204605624635403209934872233829437, 89978039168764186967898204372381014129, 53845860150365337184685749300655734497, 252416474579862399864270575594289080848, 155165485142421591515950722041182441880, 261571599106126311892651571989670797606, 132067494763201988058843728629694005227, 266814021014655046978114863975757530898, 37725955859600546562552496504365356089, 15921522639332032746520038392381073459, 17952674780284682643012632770764279871, 110087776435778423230449499187129396804, 145095409106797599670283601430083567703, 49055561815645849353898400880592953719, 154646753160374481916451531967630841754, 338448153007304494938950606598546009105, 95352439305930950167838752554692574706, 100321582836284690873719404449312143511, 333502772711516541292023904712471763862, 69990639814547072079584745828685110711, 35970548777598753513713753706228412319, 80402763492373504851966405345261967983, 118009337465277115102435640608012093202, 158288613369968462681338390333557353303, 194583859087707702949882640871903238639, 270752608002373189574456787727765416429, 24116620542626890661733942559932349861, 213706211999378392275066400489896530036, 84213092376184479457907989575651646608, 237437649035122372110598230277231668533, 135719932937530563212483095341997524634, 311283261342799775674609708023065127959, 282867635928015472048728083874997010992, 250529698051984045294285331925098629583, 35603530778368582856052925389090889479, 83613860233353322063568563870184755833, 146530188892621665671446035030433480776, 253493605335096780662636931900608528401, 257092522422637872348551853773071736031, 239917649962801276435783678675882865781, 74235018173116483709810971970968209089, 141391474622170320823814729519817928042, 13041125503455100516255703317227364817, 161761583862366685900295018689643007833, 92636225403991851948633478301925159584, 43022753445652465500684239815438304176, 225816212780864824207990914861984235473, 226079308777973578419882394836130814369, 178002306262304114229571313519305515324, 172300833830478950951234903084116427918, 95358156063483455108883989665734384730, 320673378428462842632176959591365894859, 20367980238538614457979141427994646173, 177545237495071496076575629328989319495, 51953272395760514275135193834595355690, 188139599157378877205196463411749029843, 262880940954663826858437922731371037719, 67423924100277846339602614619266462907, 92457364316862620906241921202456224433, 172898514389675927914718199277985453164, 184529047007256226647045956735173499631, 27182808144583948077236667150682053318, 203477876905971062365395948613706574634, 240928460021960082606304466119018837441, 253457886075670554375374855895725293140, 34226318587940639535350605290379096545, 299500493064912922889783430164949792580, 178731659241444380491869614650219181288, 251863594556675266928688400585851916689, 187445057312374525947475758605387724080, 267007034430349592431018062700107091736, 54001086088994455508680366286816497881, 69062239540460257507586047729157784186, 51256981365761876622813166720344275124, 61229818049585978290729451268405114940, 145409654622711250624802588001678997022, 218843418207348143801224742452513677247, 166684383974409909169094541520161107868, 59123426978520457415448710273038864412, 182775855237632228180771962317171413040, 109080452025922629201745909865703469075, 207775759095672775026087390999659614137, 336044335932806884737674584441841918193, 312458118925772466249574311699373393897, 250882731027138427166641350076529222643, 248736735052416836844435355625813039703, 236343829697655117525878092591692487098, 131747466059828337928397088975174786330, 73644299395412802762360760781793758882, 7018653142490571493143908236171211552, 320248089976733055040001923026242957554, 300826872851229092501451488398843078094, 315694096443472836467719016796193387971, 2428535040735001413133829793502249004, 166375366564850039699325802180156310383, 1648791580156213573727481189982523976, 232246222606741266877152946732034583520, 234520651171185449575421252938688420457, 312621990387921965408418856556579325761, 48679113872954234602527617854831138535, 280678083797713737920802243888462488830, 325156058476233584391709451176260249803, 84760993179440139949802717422824256494, 204247248397324632782193935040862507553, 326224651039241339531926173797878256017, 85376195126168896273782830106179334101, 276835192586879279662858034677595488520, 274724680652577644136373802127229710350, 328009089888321697573747588510155429754, 247061261048739493678748621177918103291, 309603320369849908882455935600282326218, 260301845597707939806956943589042652718, 262026043203245584021721599183248954217, 84382038910076508920976759576318363493, 311642354247474785426298251332968651617, 138441707212893783179097115928866651865, 336515794223745003050943736816127218325, 276429119901888866935294446950890171364, 107134466060695364099281939235853389836, 233658840902128775906128471175791297699, 3820682478754056744755359138120131530, 234444419622600450459150969358599148303, 280122634441284283362205948005634871540, 198596619454149293738515074927419196917, 335782407815626248983264740768461205546, 64472112653192159026115764428845323861, 76831283011007338391019668238278814224, 314694310382563562962305852835374416806, 183742851486328461004537476044511510294, 284286711781500082139686370240705341149, 234377649239944635538467657940809793051, 142654194278187399668682656586152159321, 267763589965609235886905218600106859882, 214927054008698539274397007558813977411, 169372364635700375676776335893880287943, 106650524095445643968643973960217386793, 182722271921191417057052280589680440018, 75389816808903045277435191780330594909, 136654561605510879726335624928164328429, 54505075431780009608393038210469664207, 19858611264859018997768295162499115639, 219781755485334491387748498973609327516, 122548449292079163608645088138598638585, 128301545213445069452001684691470023585, 71054258664656863175162406806022257479, 249436480710643369436822736431029448937, 12377755476009606106200496192274276272, 291966989396418083233801218142412187101, 190263319512809168264233580393964400585, 264641942420164272982473780161573819807, 151983507995428235514866300593848847333, 202062567169237723477829800401160886840, 332545597754809868504831886079388868733, 130591653544586477503327964646975218644, 25342501616693166165161031156573830489, 247285215654820589955118322875853533314, 100251413420166691321932568852615486047, 79284056009187935774334647323152066269, 114109987631177438357811585101221226115, 265052489481407313611693512723785022727, 45593176642447222893702496337742806779, 72046684501754028829632266802628053158, 50686584891305646461149502403857969606, 127513513848880170398683632213430025975, 302044842309514493662033684244390590086, 221948085352413733558282196702077913819, 84880931403979298513925571470970677712, 282213583393949324744484553335411366765, 79817237655684280297205769454122604709, 120958399493369277679959513661184173201, 80741636541094449628318463839400969352, 7147325620805755012999923250918725954, 207260594058920560337074151739472135869, 294672410061265189910501135489065044938, 191213019130806342128204972374476350093, 117989015916864810020435486086887460503, 87103757175520099345824622128285770986, 285241852087630406158660107218762986233, 148795021294257592849533759407514931922, 29207995617464834859080756094275133681, 173914329610919723189446576847514152718, 314650957985375363288492309162098470472, 267229853225645548659063677996653332681, 202516472017207988142968660281854097249, 171326864843807401215109531467852581762, 261532477930256289834217737210598462305, 73816948063229173705571031664722265714, 10483206010906704857381861911835151, 136606731232881672826817603165462550086, 21721068312053640049737014121746067059, 178266287875985462190661326571883279101, 45017100039205108497600396864966025989, 178605195546848824880625745092700484031, 278217082485336651607338680261313467291, 9781614887633474628782844638066959802, 55533940619145565459520698665001796778, 86661301196534113254450134356425406470, 178962647048979767247363140887538667415, 110514458622207751598593060305990138751, 110153303965158188650664839209730606715, 119918068420788605443486691779886855340, 219364118685248810821514157394454821034, 119289265126971362077239563329783892307, 6352797848618986386348410192622276908, 336102185722598785632952898630845804522, 261285357653861419657467392426150821823, 2549669478361107282973211693262876430, 31207678469656680494230458049298656702, 86823837857046122966920215707762616023, 264371664768545423415487187177760659424, 167742824895260106681284894149425045574, 34860132978087651022923583825599184505, 152712906751878895607494082076669654346, 5326969923129141940372163871311878114, 9622633272852878745212717445224860511, 164034307918080456977691880693856191464, 80291217462758624298394532368487026595, 53660729948454807678181251873281153929, 137007656293491719535241980304440860278, 119800402971834665937669914253460046572, 289258348007706323910828052910052900000, 248847692028263901751496622421546419457, 152160302928085303930302769654316777931, 64787674852528677491642192607827798049, 267672180770118984761850952031916110282, 143111202180993452574259419265796971372, 265761083529079860235006648125587282499, 288972683618832171584824420605264018118, 260003166032788442356514033626585778145, 172695037239998934006473720047710654680, 147073304496572177844977106564393944267, 268042121735101340357966797451446726047, 308587589933053108111196463485585315148, 48210668057463297774185535795962356008, 132394576441729037810007562513430937146, 135972256651922338791444664838391818969, 310478306942394199725617053119007158224, 250490791378839586768097171251170314501, 327212417838348763100610362645279816898, 327861706314122360462147171217277703315, 202046428870717285940658788605124058132, 90326102096358604390801167628558094045, 301030537084500510095575429508705481407, 170664894701587268821001377236142785411, 191513661064483560194312775284980238838, 80589336439326683102922889908880289218, 320776283475513123385468708308930056300, 242557215710838285403650442554856543691, 62820544063619455016014897158445269661, 262442393640470625942485182494044123283, 117784199231592278225954364817636295655, 250552256330261717207075300922723977125, 143865016515614404163846273412070472276, 6820503545287138301644178451190054726, 258419024219415262030774255876184070149, 40785143889073691451215911819002800478, 115254758947228089415552379442452423715, 134761246475948842020646384290880984564, 137082266827895211903374975305174612139, 16265002818461073202212921861849195237, 28420282015384091138590750256360676013, 195907048066671227265521504476700804150, 174923713190189691085199038731607629171, 184905575920589458996066728323594848900, 165932268978947291275605011797486642532, 138264125377154445896999649265592631834, 78570805519263953098319425258753711305, 172469177355985904631827105530044597651, 316025429511044041595607470266764951104, 222207010427968111480765100441920382285, 77676383045110595912074280443626022131, 174646134460523243114098537122557489587, 223681973487854020275063277928591614915, 176819745250989847282175255521380516277, 83113869255212221753732496022000551879, 281207534046150117667193481388157079547, 210047349165524895553652246401011957125, 248368255784850093669366169309982741291, 89389357630020807305053298603565405267, 97998432003597939167273339960932713412, 52630404258723569052352064871643103210, 315811852512311237713756821790987511460, 248326548880430655448527957493069843825, 2065086310751714048496578515271876119, 134786739469719284790735568438271414841, 131962175475752328113354261153921132461, 273317216417772716284554845558074520917, 244303838019316744577023295032153947151, 100521790760150236616860088778509126648, 28812662152452950641084095276899263279, 22714191940575647803749437818018834544, 246544336499700192787891094652086475371, 286060416370484374838164364075607741445, 128523625348159354699639151387232464408, 109507080082841979806281419768473509437, 122401365671659888047115175406055886057, 305195723193971103850870821849582819302, 153164786195237194209605714204694214212, 178434633595165988756175153056173772285, 125658830622784520167854022907194298247, 28474456675476507741372958143133376274, 126969042483407244706666213894489505211, 24478266099149049134129927144555454003, 31469571011528931508790169829875937794, 112363855945943065722781613820311328698, 316417169298486974997001857930328070624, 201728819423061547927617785411624105433, 339452839868440841283122066910459079059, 157561624193977376107289017107683494635, 163829072217508508769901924536887669860, 78963842891653583889571242535892529288, 37526932346795833876667649771718088159, 186505675677172679427057909137317078706, 40059379574240591670312587995976617656, 24001897093846947099608834728472008971, 97262285948074750340110529721139839160, 286325105005837240641539206028324964416, 214531427147273266617304546156673926270, 185988870187028098498572945541602518482, 197526116874871887993512899851640095304, 54301456692966661018807062343972593927, 162107123136152809981553812663074301046, 176776789720147230406028722144500259446, 298144554689183485355951597898485484414, 324449017164390204943590461413062663597, 323203777140868450268012360762965581670, 155632109293405125327513665263974882091, 285738247678243344935653265494711005658, 100006257362854352681359953575344085814, 340202537323311861875788361123999462357, 289269394036366472783453985599156397063, 179251816948636348029710627035764076726, 184776031245109211380265235054061574489, 252067886993133249324155128523757491719, 51941994005416872681761321096526710167, 159966343039531147541958750493913915651, 226185127160913362918742482102424555903, 304931734171664806105801836629808086018, 118696677768345097490672124127675642597, 109406204833938721214784507208616502592, 104653001670737741220348985313707910589, 160070425230550978462856955190727097433, 140983433837173416284087216480088453651, 124461612958254010465621128804798147690, 283657319216117368694203811325240180176, 79734706018428277601032972754840337690, 46841416786710550906013458724291455814, 209267930604149071388371450234489833781, 316720559147587801946129914741992195337, 46665313187385465554143071732953023706, 172504218155485130270087467182588041744, 113954430353557432978804789842340952396, 13690876789365053660579098424900347568, 78396836717395364296059961123229707128, 171176196704107535450932118528625755187, 235158919789099095248014651383564642056, 41301769576393946581511855539041731385, 275248214974059769258654965345774194545, 71165682077944774383418299928787151747, 98450620873794682251464672695006328416, 10803185257084695148209956339113237140, 265542426098541710928806763274963787750, 293307923790174126324838867959638063738, 145993366518586141472772473939607312451, 148326923151119060578300133243446399520, 229710275798822142460661094852870529200, 316793827517211235024295925170187333100, 202006444340280280223905991823546020365, 251280742133831415132565644820997516770, 321024066615386127288656232773570677665, 292656410242634877550988228740053711098, 329262669827053061107678545594901984232, 283432694265801558883165302740543899643, 24426147406197365801240892385849527241, 178474847986185177766057965354414894067, 157627311025866746017482696187819538741, 159736067057508916354974877402425580724, 257774997181610853222202070035785466584, 294155983205756039212544923442947084149, 182942375975541867311867616374121892444, 77777696151765076744001429407732724126, 234376532742038435299772165415713224133, 310394484903536542257600026382837991333, 325251558474924889562386398355472940389, 123962109895822498887415896051886554021, 315441686821248210092610183375083314029, 48901178001395941315351407038797140688, 41307066041138629082195793439627544619, 46263526511461334983900218504582967799, 193372675624878563518463232276995969464, 197219498822227976325407157358153450473, 52128081519270496243777272420572183873, 256852717168225160180195447134421568811, 110963515926762794559731210217634641481, 13841258508641397921297331326384611675, 306533124662074916316523241478532716208, 147196290978682474965442097675921490481, 45159914070326310830216387687037930921, 245661771514465126299408380319677512847, 315548308922619114381189665022417766952, 119501192076163899931369807811111048415, 81517984292485243546546742453504855354, 295435752866127867594235307148259382717, 91026696993734875924471215712770918733, 33018799017371548457562390188517727890, 272666656163049431387064837699991714277, 248008856305137853349323077607162376050, 149365734329369394172748656189363910407, 78389061733746516089086063938023245391, 151835692910895111826584257003250530634, 308265508787450974058248693213732045608, 318032605141190806786544409054030120200, 200032879423310799126308113591912838910, 3647727187686985494968333286508280753, 303028309924180567399001141077766328902, 114468793477594721102555511484499007186, 21762743981099830097286902460715257986, 110689045693543573111434613595948832728, 227705668803751817815187236709071045748, 315139940662899823595435074499587819979, 187725792356006949699359982700919093793, 101174865331232193980925775500410141821, 255889415196571069061161393115504027622, 204288861159432889170428897958384992853, 295485426952637460445657287138461637204, 317165781018921683836367598003271942657, 93090645678470219196719990725339289457, 246220308761030032670938986501246961532, 246810391124386832846626222723265204313, 90365599912986404221950518738793431558, 331990810159369251483266912115206898538, 154146668036929879756071195046219607352, 257447808935735141827025706736709887792, 243356390917859808488201081403601671961, 316344696380988767533655019581362078779, 252333185750193832827657295344401292130, 214198977280408519577631990187417210319, 4335308693839748688586259177033396887, 69659084052039703041997252134331649000, 40568984642734773046088230371413236043, 208737473929197838762385579643896625365, 332752654513223425307753894612654455523, 18616672213465587719184909805489597376, 127819047504934923192453481076810967118, 93391996804844241619402592480350092192, 99286780448957632890987230861610357071, 243339349637084964798914114970632093497, 338730154267755090623975660116027209393, 135897585987477551680298826672582714740, 307450014586372828240179227188613404175, 12184920058952434182648799157797389152, 24546624836099308600077791979823741688, 41226017167381317730686959373987565386, 294959998586803282655635243769706532098, 74549416002848633457213641579894133238, 121496805449205257067112482707274828264, 92183189029129741641055348507797085319, 288002843979247559915018913196344846381, 64942039389069219130938164674758105927, 90215070113670189505542537603510329676, 4286760321979075023980530156207961567, 118257002583339466197967577331259541439, 140054641552764492193181453891002649725, 260062415969213123560691022301108265158, 262348763331399073766243345843471745623, 17528871910395278657736178514348768651, 106791520185795382545570719443109372711, 316325505824426353809950621514607670851, 230260695358067246198677542220795967415, 291731917739679755753341458618545911452, 26686786644752092662765818499567953893, 155047415940304758891921279444408871299, 73288365227190313418887622555061227699, 161937995853861156060339038962402500976, 202628487310088304105774256762892455913, 155643692003224513534726217941052798387, 310658859057583194783143334076292985527, 283789284763701589374352177181152402521, 293288964671184982362207205968968371477, 249169908175016910903795642001242368664, 108111590726239477302465813387100020324, 301684687133691565404203303950743504571, 65285620410639241239132836172348894470, 123360725232416134480997590878082804, 75593746302695423154513289277155211628, 169469765212318287088037144398926940151, 56533755641595652197324106949841079132, 178102811412739455123335388656683106995, 74783401110395039606655287539069192842, 188004113904955567325130867671238467182, 178865374540990078986251381430317772760, 167625212548425833892841630345488475706, 22493491302933709806368770506851443397, 19442577087524641810421251133638439602, 4631692030281350080571700922337980223, 311687096177103313649548187535906482288, 214134889379268683082155364245333884582, 176057463570809363871777168521368334812, 295503884363184440046775812148670100469, 41205769085796515010148657551917892055, 304876395174160309069683482939202982338, 165272932963472167758546169347687691891, 80920570595235180199727456199578590450, 208938496129848530960010370861815395640, 182387755799497664819408946380920040227, 147161058599645461985535645101481283132, 245054352496712748705442632922618519260, 199362623148249711466836686409940794682, 200530087608743441185839984180131756514, 315793504503979207407953045352301797649, 68848982793984191435833331145889458543, 165720783634069708492091995531361358542, 111583250978696109331420157855630747120, 218455140128070859566036310850191261875, 107260704398259011583444487512399867411, 268667721032837101537935186908443455463, 293085184941751230329573243187439770036, 297920226356275502778833950317689098188, 266843767821961411057094493437241751739, 85488829754060718060904961513047978389, 304864853566355150501900210419234166974, 175041435362225718338504034037510336739, 28677579724696661994532046334264003773, 261035183983835795469020899179014965049, 282469633247645143864539599896507129905, 230299479853229345521988500754707146075, 269010137986862940813146770543941798781, 249286100560205619260144859692486499286, 321690975585001439954546235706240264470, 272563499315745946299457915133033026878, 230617931503799576985067080546121816427, 259213112964680352848420956136897127866, 163291152892319724813830511792539404603, 148357119876927345233831541233217196799, 102670432004354849962584183630631940784, 11324635091507511121151784541528750606, 281792305524762039754967678287228229494, 206619336605061112637206310331793207033, 41635076194023043490240102303461181961, 227899001652377219322805631763130693418, 105570029133946689664465992234209904864, 304796490282001797294407595645097060658, 7196920455947515527254563517409164006, 99178099594714967864883810720635398005, 205755371552224476047413669967665705041, 76816493652397178161011538251504583321, 292232385029725899249911765973967834880, 176922261853973440634719378726976950773, 315090027050643251033666799428334173923, 280939524826918863171665044143408308181, 186923166162393825375171183674339421428, 178788451519456824519753116527157042237, 128557729052894363899659045533760343426, 267065712813599581513085580348560815163, 130895812585788523376926512232522846558, 9006986737713280770775391451263281358, 242559031981101084649828026437570459323, 137720176721726200234226705837402680255, 308823628271480558548447040277263401971, 272222464414622104333407119613290283147, 286466814331519240219003034247508188581, 93041609485006086370000997800250942615, 302971959836939664744815493115050253081, 70564098941515865801369454065144818141, 104640951897868066730047558890920545970, 49775543795683019670868560098268090379, 65450246211665620532588943736592692851, 213298910579174132021654513483441402449, 236858245229921715027949745927720481162, 65341525997439388953717366240140441411, 194870177665276742502577712885656569754, 332263636441843335649068979107765045303, 287016819995574259770346382864943190176, 63867629831450551299140800319457925499, 77527594520448548303751282768518068556, 19506197095876255857839863569417136049, 166329937372201105928957765737499154158, 206473593888860018164781134008921299301, 50458153095803849925008184260065079816, 301958896279350457890760540041462056164, 33315841372663565523731125586996380070, 199956504027598610946615741043015223538, 201908897288162035825113860504135100447, 135627764489782203039551480025898523578, 287390390762299702703460180626901794212, 75945046364804256425207885890950149367, 2989117843531607972457248972658340584, 49953716034737386168377760721567916801, 339492300266623674585898312594590811077, 58789215844009110266186399771219782576, 272446527988617177188395921266191710772, 177091341595034993621524921893603159950, 151289710316279130672246650260975295781, 129811880570786646294472466166082530521, 136325800443229079298246030989932695134, 43726322671905801308386204228914792042, 330369210536057657088611430097461879881, 142440640038381088997085855520938441215, 197728950616216149230734711156097436163, 94057729016356776912541199403646008342, 269209163415960682628172473905727951580, 94844035648992153297742101190272261471, 35223714743196827492131035421659070624, 121108757632624046370067397589094741039, 106206451085745905634134510134004426788, 196331992463459503720519604172149983554, 272546203788366525178102382346212343877, 76141885645527137160544749879057579355, 328539590898912226384086137728772201072, 117616586351728156122012849455420680762, 106072540202102174589657646674224546033, 88319430455256537599822897095828875592, 15140431380280465134304960345497469345, 288292990714033081501398007571893929073, 233013991905325195927095646453804866553, 208928316059056844144677469094055581048, 227075056269960187897833606616528252250, 223979697115142784478342785916098277064, 30568938048198429777085454063131951400, 63749236581847811635392572647590144448, 569725108386197411652117181994010377, 90173737589054772633801197649637091603, 332869812233173923760290838873778419816, 266088806875108434232937938478155601903, 182043942932167454510807278838105140519, 63059703444050280579136262352152114875, 298118410325757909894919968588846121358, 251390984881094952533218895037303467662, 153558628264321662103446649460275044876, 63770634837003876788296280237947740364, 165129969803505402908781829642324007680, 313289286844238633167302243978319082393, 21301970553407595753771336313039561672, 284730617555727026176641638744221645111, 333399032134361090980853963353372713771, 135387948808016577597840580190856449533, 92422912751646211784439627426693722548, 191932783527238438937162582989062330070, 304901374047930387969509203006323387567, 23410081197423110074338122324273795502, 197520615776110400899418534661944483136, 259951796843868172267192848572827581979, 299844972910813642810571790825385027449, 100109736671731615424380361232892637239, 316237229535519909833212839167975130784, 308346369675285332586330847017308222150, 75067955336745366200310017450313443470, 98356339878310357944235776913647139820, 270154438499291364024758682355146386204, 265432129058272132696665276656316221721, 268822587150057599175541203248881415904, 99330734312331440736020139192706472662, 149216459320516015346590515251285384289, 302923979933862437897618651309111125039, 50578550076948530635422167243635827268, 174484043177447461776677662138643184071, 329699841407137161078515042091979622445, 17805105963288005907029902734446831999, 158926848566596008252928754583617766002, 58730159766380515745609317198383648042, 205514872975733660952731483032225473382, 47178636720026288323230597145362577172, 87311649308664934193241605701265207827, 170334018395094274393092567948220740558, 164749632953467947507814811751352391657, 323890019964726560541253703678091635043, 61615527844963775006784252188070227809, 187739093343712115739517757904381104171, 235146785922274798312572899950355438497, 136588640664916596678715836030953238893, 203787633365891845275253035137144835168, 274293716864761114019593653889649421250, 108625062971834036940242283502165828669, 216368014276168380572593356699331705432, 162798501034077217423909902156377482760, 162022183768325868911095421944812450018, 115272211285958328447347385306650405133, 127553874287332816794555986799484191516, 178636802732898070525378530533531311951, 7928872160659016044761316399272287793, 164130463893756411856851670144270066984, 280796487039292545424180863140048014678, 72774806220838626222926864844328640303, 250237636961707487639928657257271213712, 26838285952977601753966197300873536293, 37244372926990940289670185383234116844, 279010114188856299301584589098693464603, 137854192043514467608416078732881551382, 17033941787313963056181654599160010020, 2378760860536743810659083463486015565, 116400775105583714906540337053286282178, 3455267059952485763349530006781740811, 323251282405308949266661418997619963070, 169550659594841751578507400632866465143, 164369064240995191291444539826527388342, 84848866839107448892947742365857770956, 190347326101638345897921275797543676714, 321665301589646754457583045240142179945, 40909246296083852742076230262210808732, 43207558440640789397599726216374426020, 248116559832557989598252469074195391262, 72677670305606832159867347527521953088, 62950504107384214081156419500969877169, 89275553873540600075883921493713189322, 121575994152934508593754557318524617156, 5814205616840644509704203898084231890, 11110156858429238710105219070937609519, 33477288971509444714311215103718789590, 80023139455415426762114966810330237069, 243743390840656731821098614468262810172, 42429877321035888644431492991798386449, 79703384454561001459599630528328029920, 79372315209647374438788266520421956, 84322526762890279320929848187161268510, 332005147376577614671361494386690054611, 146405131703094141606211484455880551461, 72209279539644071296942497122701440375, 288832433888975923011997345936226428810, 220785487142709891844709281307050436529, 323866349767134156681978904649533942418, 109940552238833345271260416057324335261, 221659744237343563450311159276769506581, 339450578051222799376749281907998213456, 66048494203639516866209521417308102902, 164175519251288957252938364177255692782, 167927361255571518690153123891594621037, 75794517194712246819665496328197775510, 71107390743457520125194272974663846083, 197980956280219608088158957618255794026, 66924978795201493534775073962497070170, 55608279917030206302688834140731110952, 325453616929904578277727807286184576181, 194240461596696340107920133826072001191, 155085156195211334295073254047620495485, 28081016722488551792881016310768699626, 128108835334240907998593817948412459056, 220053381370702637889984829706419115758, 327717152855772696262465791473975041921, 71731255310914763827597278155892982363, 230195868034418301900296598494100602980, 304692422467534933001726606292458204794, 3383735554066551141775823702047377512, 249778336083900131962772377966969759162, 313396966193500375963788022785941014929, 24758904326375137445729472804042655105, 212379685654979640856848856317986340666, 88428876227175875773837839725651056937, 232934371404156504487562785050947426850, 126938021370817019703818600050530002616, 258958043144005641668937496249665928664, 310021111814633863474465045230080686064, 66912618833960420737497645906614590403, 281137511588004544567024395171414305555, 84815305796588462181151525269090281857, 107483549607812001037720015907511448745, 212237226325824780335961705018024055316, 80508800068087397923252036693761402388, 317002757059060344603989298089957363441, 169819670296154302267636887877518435047, 159895589820491936120604377368780349584, 6231906589835364665758652570001427958, 65184592332273340102385220354497325341, 118777529376014331143918625613009888976, 47626149812014835511744579974129304455, 123993902674647815542097391143904158645, 323258353294333070776131580707501576888, 217093406962068831591534122765046078995, 339222270190132486796048524848840278733, 150016193864726873139727953999852881517, 335358937907282903583628662529088290471, 325670956230679106572609467915605668999, 34597200925136108845339055709961206353, 257497981324460681242258788557092173557, 74806130529955015408124621332586984672, 80591636908453672737439154221415588330, 205684892186611404321771738820977743354, 177510615760053186836795331595511587312, 135086565188852150955271541240409828421, 199636229841987009442959451869896199067, 181384899682842466520873468851130892266, 82860238648623622679935262737761158896, 162702660908162095410226069544186916377, 28992834291284035077996763877678138317, 250065610108489731442589341668125238656, 325768294687348019815257843781928404460, 198486318444199311119885785054736182373, 76863773133105683390209375652528911391, 153448103919519686872348412976216865692, 121385518294480844787403630240081312784, 127665015872821657822378525174933107012, 194635526797614217036272071517721437975, 137794259882372050748676942944556377402, 155585044623469193448490781746881808032, 155761717679448875635925414982964433592, 223853326562387446748190261346820373773, 81481428814380364730268712108768137551, 179474611698547842773057604917821204443, 13139630775529213803699875219775826647, 265210277495287778410697195884645036393, 223818281751631052858722336312456968867, 84701796225658537092333167446118749955, 6657281224634454535473427448566376636, 207161275296308350979571045759971022484, 136290476420162658174408056466336209444, 160432018811948257376462542801123322503, 306381137557540320942971201055986951550, 81399848002585253434905309619965419183, 95328896176202298527742637181393770286, 285043970463778074191517621298878024803, 45378669529415704147018284339607048732, 85247721112305977479823699496923658959, 48149868473050302182075360505091732271, 39943486826532778794640652844160468690, 87568155626899679180850310892317112741, 101068656773852484568815331342010644987, 119432480818130606526482724673994989651, 291358886486346155171036402283660204572, 84148925489558424459027610713473656591, 221817051816778812784530065634333482218, 132882039301353038758920007296794088496, 285933929173794170462037598580901545793, 266051870804283077197541199784438779147, 184891114915709333564472768147685959053, 143434701618542046361532331340002624214, 20437472093439227050560428324714346965, 143808669591771859131968897113249310971, 73584623638303212073259963475383192004, 223095346662814329735077011618931129820, 327229202518845461570559031218121635538, 183884996669989758987178378528145536490, 262804334427902634218815077390681324480, 126393008252769361319660270877723599407, 324627572075299855910468589962904693832, 39946407438537912575862899621179569199, 51799339129700082253523680410804074411, 5929053729459446065369047957980658675, 308873391951555471033325637739214344806, 219985896446424538493488399507456060516, 256789831552111121627200752908529800848, 5643508816457380744211108189086016733, 290158826330639818174283664288697707781, 316006716108323781077937805676713172831, 334550392402808948025327881639505610097, 214398172093968128345640205738058442019, 178190629573991580703628771127871327088, 304573014812342318696198412608408759487, 71049286161599026846053132656073538651, 17023370213641396807243404280259463450, 67228294513274076072372140160642145697, 213262797452069343637860662186215624482, 223709314784905590913720692154218079751, 294384729693327519716540248684362021685, 198495564779659554929292885916531257284, 142550899952479366266401420476067310186, 58980616321313115199606886900290442905, 27527218345466591160045297594869682723, 111394596235877394589756541454383894882, 166496586489533851410342933449351546592, 333410830305751744569264858084563355708, 304817774796755063607304491638487479439, 31947649070689011750226177373989471089, 251391750651224797162044885701345554792, 225130147696239527693823690048522434622, 244798134925895470144725943084860935332, 17938952179029269550324962457803141184, 41797849360607687328271158295335343896, 127355945391285952231400179226843469139, 147023018358574445985706938626411368968, 323278352041924335596470893924771993272, 26998497654760949954103841101921479553, 227283387721625905140619220918102840263, 152076667754517306008384855327735856487, 16468384172658251739520053306681098231, 265213210625377734362631844420833210810, 235862153239113531972694512802057180734, 298032269997811402134924954022668628057, 151817138707242865197826118803880929608, 60203568738763582794597661318519487819, 172683431062960152170237527093460866272, 297228285943246946681192933892993495401, 327826862166415498969278747000230864014, 141257256653893887246538092574451593165, 338997079925165764930187816867488442888, 192977381701794178162318682947484036124, 251484424279351101948164295099153235666, 249590063178122748852888066355081402716, 151736587898937710980134398703826019562, 10170720108340929023554848826389385257, 121053987677946766170400311399413122585, 66571029169715962660856847063799339758, 202352493517914580043567372912039613308, 144813831330501523315298402556305741107, 189874461347730687385935343670104410741, 62716431939778814228284143247763109658, 183035883036614007157392795429336175921, 1152602452755758278142148715759796807, 262971845988130493950682712095016885750, 52230964583330205283440081091488029430, 56868046570959503786841507424020047875, 292509691228510257410084667651511954022, 124423607860408993209543870543543907657, 255617295992585650455404798803059842615, 321203775917280314283136131379662058092, 128504933588243213483782010956018660316, 269189413036577320316753277805924354357, 280129795139590277726319138617352553159, 260319508144089239220471253450944858198, 268797639248952015599061390614062991531, 129553688681580557636538520637535588388, 310555372725781339134893066163189512823, 195381552821447286975288302557128421245, 17450819458195513906619302658839973041, 332008809467461375834672439345349586079, 88290087959076951153088347105680372111, 129031948777184334815660316511740645477, 160285874347199495769294063557097195290, 340127549731071971256573062423736891385, 91149708188573245601182315561286566958, 173210210264573271769920719807440949884, 334318834771387144358475473025439088325, 199690815233561852009080678819559159899, 203679758472244758080722281728597100573, 325696407339469448606828309807958287090, 191408317378260257354996490940304014005, 321592823826682851188687313238996098868, 286773130189876606209136104677747962680, 304652991428082497646193809737113602637, 44123242852667165346720574758758018145, 173416645966231026065815826593395214188, 140295328555056380442535425878994698638, 57008378910786127752140138606345429352, 235172265960472086520436418214118960015, 187630715710107881588364251627406244441, 138746825031229755378956516815818366017, 72874626672216929801537368839460286513, 118728600126224555578468509170322710024, 250977931101260920849835102774236427234, 106605306832869852921997493993282971907, 50709113237009064365580059804573794528, 40916716403366111320354175659989530844, 328148426052315654370025940260955729621, 280834431294294384319980043141649177066, 137509367101983632926518522435521981442, 243886864645461732634183287797701231759, 56148456828389431329364201149352214424, 248483013055589352187430272336902658781, 155939739630491595483255596240698274668, 8937335084978998028391290239492156232, 982167321942799446142183463713014403, 92019725129003776124597704211162914758, 339720603575672212416959170778223070833, 292776561517161371296106283854113369088, 179231676379721622296840072156243262865, 294319910267605352179901556689389123597, 212517551882205865523988322319881753994, 311286986763797884513606343958917332200, 55165258021626128518096924861224478463, 253724216427104449512846455956733423856, 231007492356671100640279376430363047999, 63166783717301344105619052267223175026, 314487237612538463783480949548639857665, 253777564655423586878542797470103105522, 200927436506430197536932981066334842228, 187264771637020589555399049249664242637, 4300375296378014731595487798479914456, 194625533284162631333143519810022776495, 140009968509548863159070094067555338887, 13482967174225346547309038659939153224, 15915241786260939866528394876287763615, 151105683720484265865407648479030114205, 189883057187354041233606664251164478549, 173222908040597537884433679600087571407, 94166850459303132182028224033235938309, 53732779041230231388164762834108357461, 118931100068403610113580297439116902050, 234411881266632515486625506906619054727, 272891430479134987262925686135755545223, 127312329199850406116924232290143280646, 146766972253168500924096762724419282029, 196909878454497686744533706225041671785, 15482587350265303149676592833040289498, 247859519217958898519922858541180193527, 97139797289914885244026776643208016389, 78205882522214350999127366853849990017, 206936015257526323147003224613573648742, 280403299999752652418898344794419101935, 223576252645721571294745586393857597086, 17530547108666406577477122766928063189, 139668647994925427076784650492125851367, 26143984570573414740984165048798457563, 83823179454993817111824452355140696105, 228710399399696555129843389878050232803, 133157830533261786832675225755787592310, 84730805743806249141715844443462864415, 326086130912191180978848524987534799445, 184711803522515293898705122398889450842, 52560255966743927057141536902009042572, 296971427685053371895047788265395961635, 150021500094882314665464126540577407883, 21053866183253740504943091108301458470, 105568106417594996082650982558143215195, 314212534715110230139388957885725513168, 191862073376452918803275470693170459630, 11654746091501382254072328073060089423, 62032132698329428934783321095288785553, 198911677974724872429900069307294330228, 328278814937338003629327960751566212111, 190343363862612295894831176462439161185, 83007500767674349741771612836977036011, 92140123913433641981537468761302460090, 240337640331604425825157246298060317437, 235201800269615924651758237953932262287, 119147427336310896560628771887721811230, 166668603567518997891113646109463632612, 158629455093100950858634330375111120695, 14430823923932337448414372910414356917, 58946543503625273249473758956500781638, 154293804435452098658973669253604916779, 205577086756311553617761934546807008048, 296511201253087224130775632139133576285, 307418565407186767025968906037899026771, 35101166093305830099011209683536316007, 160761645939455999460058224669304388965, 207238238056522715422588740766073152785, 119624308870781948951852400227146264655, 96686926321021297814896910337383824347, 39111238728585141285389233746100648755, 217464632779316781878493004587593636697, 75872822461527324584627900796395119652, 234466170953100400915937455265818000113, 249645245549157504295226249447065895488, 55703070853826529143486819454738362205, 272432918801311407457804842561074976330, 134345911089343627549220837831738582981, 316838099474809352471358526339593378687, 300636735490975432128946546854266565914, 276917662475009147487368647716122631712, 185494979520995120011341671988824500253, 227350661328180968086595344370903327655, 277184956130268564510828467962377646544, 179907582872836089121224084761001116248, 152354246751604065253962625471698740657, 22091563440050939645916391191534112435, 225672081388963172371003942746920181246, 201165093831785495918500105090334884322, 133259180615374736815990981484167130097, 3362402466434092937439638455844685386, 253525155011347181382932640188952555204, 220982931162983560188921037293687677925, 211879825167136575333828300902424564710, 299519793924990115230266349438331862852, 49282971938530620893341193437990304151, 110159837007672549305955836540152254756, 194357072824368520745000499376080745644, 322461503597207883168369562064097502820, 157686519904689020651696350678780737722, 53649159744552682506864136305651240886, 318834720410808222438359321129680886348, 52207766985800368191446976379924882945, 193790902604835533709793563607455183617, 139666499051307450553288969070776080770, 255163535280062411447346790710789411485, 222624453064496684677847479523666387465, 328360251941141916231189945330156962344, 83839898905664175813076788902045679399, 31123409388555062807302649604052977038, 339857531766415019429873694038555112083, 148116246475459174289465271712972151034, 296618027307108283253686757630859540039, 276971863841779317521212327052467506792, 53487215407229208012209208858679301877, 320539153394588322730896835700228130677, 91691948910093157319016942978038804120, 124915799349481676891726443322037452836, 331813413371283862983307709671612647741, 117415349707572763441365065229751187965, 8602583773783526123282720369817001376, 95179676682346508443757740436004231608, 284084655591689564585912842042305962301, 578226049000121012672205844978629197, 75204860405694847496073126666304941753, 208611405996939062303966035948494469458, 86413910608675248957215754087158317839, 174217488683330265328600103832101836464, 329096188724369694618039762282061794937, 65838733485244304751393806381487061240, 86411111759070528629589506197452177981, 270555696080769032059044960750294143791, 43074160046254327099679060025920478824, 219319013099645337209951852574009419222, 252312131000368005794795115759633200685, 290598442996003091451381348710598384824, 181980217477957445524628930799651707213, 234530533708959563661156728429128640730, 53695998361495235492544439434281390077, 241827160065863159131629768772267286507, 184154810986348971083501389135738904034, 330690587033866147260840402389204991137, 288505483039181278059405968985464535892, 286896246391484179169827465835589239444, 22267369154825627625237003404702507310, 48856406934486205003726500049420496039, 77219887013819122029017964296483496081, 49608530609103002622608170406629669942, 31485640081291087222868286110056538915, 30918792542420696418182803671877570514, 83108029295339575425818985553460171254, 61896637378656368309427224921665970815, 11586399962833403759377813618228067742, 339173568420694496374010205420223835378, 99666448954014017117511804932716008868, 103493357767063422199936922570263315625, 94121039373540097937915895669151944702, 265137967759479069443828326659472848870, 1276339221945407649706641285344420902, 253287492270081230099397758848550757602, 160004972442248561770955209302207769395, 165801381115193351325686637923132381083, 25173523490508597334351824693364510072, 246543175856159324104379965760212181026, 291033141855957164260637729532118200360, 961848978974456379230886128586095820, 62407304537484891252364281315957514043, 115102800446787668452267429141635815172, 212915849842308854253261111201773962197, 284637372208249621290290695794572054740, 127196870502795938643908617568998363812, 178499983367517932058161121574028871634, 248888979526414121369042636641950579775, 200240082325817370505809102673074716879, 164091571309033965839955146285661656312, 338316840719027063294655749817397461239, 209309560251192637292176420848207980158, 18112261261931230737852811978893285105, 280473657196795028367956637909591622395, 163427098373533382456523321783038575544, 10677070326130841591076067882059824091, 329251996376470678980798783891344654344, 32033395041925736242405593015813502895, 215041405244583295350869347752122442425, 267843486140016859006118876553023009912, 127134617014845764295181326456221253185, 259994040177718383572667753186838326452, 85332573195026181701844351805795355732, 336239225275529904924566774989442613385, 299098499945512979407761147515432819560, 70104750339282360037466711477499929886, 18656491010928121657930423218556801821, 111797009781906711594251628616450126122, 97517093325742200831777221325287145307, 19962238910603510732005813726822412408, 72923547955216946791435873575430497034, 306176250174244762087296550521384148480, 262133698433634684038431365918520037283, 120776362994880991682559566929804545410, 270789389836284030641388619939050008230, 284925696997909132460271198997090723640, 140518513100132163467923537717612736663, 295680366812549399537264805360031498174, 14775756451882016402448985033914821070, 313024884764125627385278389195576267228, 134606841747178546966116139756769215848, 187108904030064501552871121320270583307, 123558258655754224075233379368718986955, 114768784214523089486678748801191588003, 26936620089024710081939292340686428191, 299949385015073863023075726654009648355, 73437020946141564942422551493574977905, 152131881117344595489347536859885996532, 197417299896684264240094713586016325193, 286895684750132740193531759307770021244, 312091993086211069915959764260552086270, 318535094143517324355824027932537992917, 263665884678121834711595680983954537069, 257302574466241458225885311194640593780, 80230870759147559848750282149053977246, 126881041179784895502104210073487706737, 194006885410367897852071705079769048631, 282156412200049696546907145739908937429, 122582275943151596895175250827978936618, 111020436711666380841097541380202511943, 262021707669183321891517351004579336813, 94738951507740553386144802431255525125, 49277533396223476154092052583712241686, 111888397293320523410353837082287691140, 87672037634503032339894077057531055983, 203470178561333261029804552995968840394, 308126900742070011983218118021525939407, 32888774207466331213536230609705503566, 27864810506224346982498991590644236561, 68069979609231761216762057428851870218, 123057067388420352354076095759680417931, 175298017425095105260270455636652390066, 291377125128380035971462274296918196922, 322008900430958121160734013877325680430, 71203248747007473384659629376599935719, 215891970112089955701570590195017229313, 27771246441958309415909301790800695126, 223328593534720982030590555105888717091, 196275511726845717724561307537359181739, 124240721069399203738894036498313792547, 3658256038043145460085443542984449066, 283299651939147303944141327972774200995, 214232534891503899184807721575911799623, 56777389171308732618571319325120272734, 61403164919315790719296988688110657876, 47344887932374233337279027068684452036, 319697990455863709607907305656351988619, 163458456122889676078761672964594538627, 317253988913321822597381255902899914781, 159467089032585132396551563262337586641, 314448885474310779743815806712818356385, 335973050320654579345267502282293852941, 37247949836865552978702376024678726139, 298381318862577769362605741278966604282, 73658437244195883780304392227510410532, 113749941256991821954008804240815616741, 253041839559600922471467948563401929110, 108418297708132363680629196822470993936, 135417060242742376751712230238325383030, 123173503920180308743984115363689755435, 247775482243782381792380488677582082770, 124752410569174590351823051063050664117, 33681988614205469081321103245150763109, 37728397256214887698579328702996137976, 260101892529045948871929330229770556487, 40014481641543203467377994466148625099, 199705385237354692144955581838317527192, 202986253529222527365532456061841946776, 141696395925177570456592074748211059567, 42472909344519783044532676329753002862, 272536425368031925734047528410917182452, 235007110643670632860835717099346264954, 285620270890331641016065064159518700090, 100084775445937927509190602465269689106, 89805938925345367540654998566092185709, 82127648565178945477924094728252140638, 313845920286902605290616260214377267579, 317460948717794125068987825227330333598, 48919882177181725815841276558158531354, 10784958583495402647967615429327327213, 165600299064901161123678350354862702973, 336420874513370217901417030950604919171, 22133241520090785395648657636761846437, 71239834184738745560744976898443111404, 139314760446491957725171654696596882932, 74784901040453781996693756994239881888, 198229812202248835594431440983791620490, 145444297365920031678904182474188190133, 144980881086707379891725779274358158673, 257791725956985002114193227617419564006, 217218478278971727208862326673503630749, 121383457590834785963855467250131646998, 168855758100332095001213265271187897354, 73154081407907144022622170590794711933, 2602880261674566779671455555186406972, 21783059051812043265612304503628971132, 157013107117530273607721733037396327369, 28023185956686298709650282664636064658, 249111075090699639689072337968707557274, 193285599859762835709329284953583803735, 181539412005211815967446077141643991826, 206275705930686326205689464904144019484, 70014799819769484824670973636552126890, 197585993650323413549401837056708380970, 54033340561650319291483496355646140958, 124947612817388722602994455562085982034, 157788617814975797222627843784272661928, 187729082814528319934188431221901498312, 110086892304920217561040990231695020196, 10959499038463390151104812919442254728, 175338846316362590164943006677152664255, 174285692230902644581785555581623902352, 307778008004327581735559095029231493984, 70954674592866647759376100637788573386, 333588653490504811934849892401883055029, 150944657720541930237486554889188671612, 199726530111835494420937586326240373235, 16278064878700400791487626078256511325, 131779512512743460677827586547212683334, 310853825285508933879642362281314138061, 234200629393688758603383648852066949480, 210817458860466350775652607158377638661, 165643684111518797400529626528676364170, 237805755779146723549308779813630760031, 307830948145489887476304295200292830505, 286436068005729753030850741743855024751, 170070315704216809770148463756950071214, 269875588921157258389186868446302317052, 85032826505739782671980023730707370182, 91423715535650082670975999027740694834, 219192132948831625558238697841633182105, 191368913603193043046007181723093602630, 91027261227000980608197075226927350098, 293351121749829840818318136056163929948, 25474465268936054089198855155693215341, 326419657626426340514340265719074922073, 143812827078662890235241947649358806952, 89319468508485322132960264734694166176, 64331471506623054077837659248990659858, 267661191157607780840531757383287656360, 154381419136897895707702910844783555555, 92316564887150358642848819234217474473, 26484296724855609417726412594361536915, 23899079720236920497157662398941111362, 36946183040658689623933045124447630134, 190598130731266006160158086945819771399, 94665015256161403945261277439266510096, 64780989350134328208884223977428490754, 298240179839147883986094986734099016001, 277032806496181737859675281863300529815, 120123759119924487376220806027628927617, 187743300402532302124451350993373406224, 171635426277509792805696476281968567006, 88352905492855040278859887461454164750, 179312179663119834981148254319011933682, 237396288434777737813982342012476411405, 257341047368380937551430785006225489247, 47229981253888835082247420526017417250, 185004890930266879302518496221765608383, 155338141481478601778418649193676657794, 277500120656379236115270694401067253787, 3043842792926342307047817129655718883, 108965968766636337687872679848544978192, 89985931887140858996951385379923980101, 271548201895902245948444210321566706045, 244242620330606865521624269361896243199, 12995152271351217002793491951516802635, 305092322218684101059341680338180191519, 91726307206519792450441154189387002707, 169812465030908000959605416068400594651, 51259407237711646022016891146748289001, 82268955034236743198019407815772274353, 243064533769607715416812914475503600643, 335355444723171135403775688225710531547, 149542349041594066448890824625110064890, 139372558586017438877369109853636217492, 117177419442798957119254256131304176114, 104158593859079353920985251285198079670, 42976931711683258860266650067774525312, 233353825111456654837786751709870839964, 110354028425034466802378296823036472330, 243312499011754396000491547928011726428, 197043544561943319466028377893763335828, 251544919028066523765520145820283979824, 113467037294261182011397969019100984434, 315055471574433910302168118018596629473, 239825639124120654696335642723542734158, 107666344103583784444020512315846753331, 44768153255379345093624366978041864654, 201004766935856422524135052950810240943, 9860937637500132871357277438372390907, 32924934937808402048037126916718915470, 51908374071737418917851042442408020817, 39952760820484865187687809044547995869, 274890741136082517236597941405242570790, 172161817002600432487398077997851775838, 137525194823460645875943577837693966072, 177962047643932019446960656687651562527, 183247678679422376762195640501322055977, 332955488842641859787929378432103207394, 277352691892738590749985003035478443815, 61590301098750787041872127063039529114, 94429809317101435142171790109391391438, 223252052924876972917626645889774460486, 155189854269392683712136464565568226256, 159667477429933627833929056127066552760, 73627620102797132383757744050621856532, 293711551937218768528535520216925550437, 329172914322349220098432379948162808339, 148039491447621012134455060734026796160, 90929249947739528743787521240679972383, 17269325250135283765944653718792984864, 180780562245361341321507549246458953227, 213281566598840826745380320430919812562, 31452888331050022754474365306018103792, 277949123641940101920550281327627302222, 304277298595772115155209762156219060103, 150357186116668845520154393028876953214, 18445781647262203360302529908833842991, 327367379398662617620286038137401066671, 15755755465978966309986395745113769018, 10031917380911535857937760232218877536, 162844090252581987433426803242285658052, 245930439508891797579443838124044780624, 294020995803762501504496692064252758669, 177623479708054498803424633393982806626, 134846433466755849672721171671158783066, 44328493912244453852480352092267842020, 73605363402001420381230336401319101409, 49970378434344489407986944584195730348, 326421004925703490283408390674530411572, 244636034850766306116588674687351710812, 291110350364398944505460237242122411910, 225049728201859506383061438034878010350, 21202054512637237418693846288182063712, 233848132926696859939392014430309019379, 185295368290519212367000193712369253615, 220745916515303006025845997437225755153, 28286626215579263197513294808198155040, 286072028658480950851531300122213539379, 50379550979980048960052779129274649669, 330925042661843453881986703390956192096, 79954351065674592431232660347146364890, 76435906308204210339067164845741074429, 81737541956042680509325906699908176776, 324033135035970551075290859361577074764, 197210279024082247712958552440662282997, 220307502743398022911707452939498879711, 212872552270647857808940937019593057620, 104804927721448502822974477514027251420, 323753083987050528238539010829781357640, 42599772844654205433602187192073227492, 18423682867430640069660611999103935515, 138477642977248104996970358960962033455, 319285786965566676718769078079852238231, 189891696393630710879978270877930357844, 306820977651621784017071044895175585396, 25134521442454907111442085604430088138, 89523894836821234288800739240503500517, 151559136722637699276724181012290827809, 58915081810921446562700017656872345072, 231446007206743395897946376459970275106, 149940412613541387235649524404281308436, 192138187845662082936757034703465667335, 249739764450881737759531687036194332563, 69059478679085919227487690182895049413, 81929348311409565146501515123423391364, 302283945557876659014520558164785970813, 51854982051154311725315208947742454543, 115301042425991448868128898257159579240, 331763089342416944662777328324290341121, 24126661244400970085808758597064229411, 58644103531681189183666630089831238965, 293103875946505107395347790798374786580, 315379989188100942580114776520315941697, 36312787324425714358678688859895683495, 124561252410069997186372256172866188145, 318455190424638454811196422913349114739, 297974324956146391509880661186678308161, 217915373190550932243874523016165306565, 231356452225524116734832171699156544033, 213805370817062917154107294867905368245, 44468903898371798030029571945494029114, 109973618739048893790709144437584434097, 233146230418589857127981508839807125223, 112075616303801913692266242439020557621, 100639038449710141094928846107204904093, 639135398512474978213462646958387678, 321448742305112015140961805773169199195, 318611086783848780915165744864371366234, 327058198242178216955146908295176183221, 123633457445543710196820204307051441895, 36564607116422669054620131486344772462, 214855895869169608711727626989024762168, 69882487190214141686386799209495835221, 308329437970836042409861517250035600218, 91489312433466823131262735081482848809, 35761399166634063270871952926652978097, 243467231193107272165875954640176944061, 279204296999427219462278528499427087456, 99994938454531425361387035375554185104, 151199385019843072193795694103225659695, 91970230869300911531362199659153660127, 294742501906748301377382496431076728134, 232922815052142321622063604801106301457, 246075190200540052401166216869568208296, 84072827104698140318862975895669366431, 8833763544094271016422945933090662857, 51085798334795203322768340738634299063, 280311620616457638101133026164373385208, 105987429413376279249201780710738060698, 228168634444789272047298803817960410923, 28597330311564344759672537893174740836, 172975153285040613687187670197617710158, 115913364013452996596062696393807043740, 129442396536993748314284268881868324733, 256410570326668089869293780299044770575, 730909424781780200217356948095902017, 100823963570374014173425514864898675945, 264166648347992037451520319923844039223, 76404293901812169489917199571465943323, 260822057495404821877542415954440197691, 84933839674088657606911277534708674278, 66192748659667318308266921763895145944, 152603043690614865878434592634918319308, 290536853100343047165278062212283420468, 95317442108410251313129326136435875466, 328748356862346865894471552641297351617, 45406032481544225623963510337977347261, 86655474882806504472798285456058096663, 105579805745422920958446065729977751712, 77450386704625748370947629451965477625, 220207527939737046083541080894149156241, 89010285979951422729128481138518739486, 54043195866011469707208280584629787389, 163581294194348397734441691812342694401, 259077996938412771262119197908297974387, 56360047841665424226210164439808267877, 16455119969866352582868570987145234283, 158219720262859202410619655195415105241, 155386050068872180910881410396381856567, 64850883245640729364447886710523236349, 139400204237380716153015705960802957403, 218735282118546665394850765782120678511, 254450106587325294833982760758694244347, 61950329054369156172186212318001916488, 59942613418562055544968600286730429384, 41332308648868622788161941476993928425, 13181596254882554599374830258846522478, 87102303697896037769144645053781536032, 233243920063888676330418201150690543979, 325505182576861691790272454810016231707, 159679967910401797934427787086246525416, 115089562317323982929710790097199851794, 253956842287828513334541247240179819455, 150615866861955557651781619650884491666, 122076553450111901970055879778915360461, 193158865363498298856478315034175115751, 72924297168141354696233147452681534710, 171695679498106111742205904744038420828, 286118577204868779148129718013075356178, 173777738562006585885460211860366129710, 160581939079089218831561620960088678558, 255700635874391333170485380052555755368, 205638648289794183224607879682733656844, 26111632400372592339531384269531144187, 149100218933321567214666232970692311405, 14189836238810182060991926150954347603, 107302343883047966935204978569257800171, 228946441687644932760192164684890375961, 214647689544958602934064566691363944772, 318182749929724778617398700120770182435, 78596083297940307204907553916970883862, 217696791696191087585815931012847847789, 276740515272040323274549823892726740911, 49865384679378276284823422616667300241, 48508849137570008360016184877223464459, 136937530231828005667370916567343151237, 34231978748107932264431870800397460000, 64405820160799986042968206577733956123, 329684000603671671563281568183072058769, 200964028500718731881448090923276433596, 103197696047752660793629037200153928600, 165855053874300625731241527775285624993, 177941198392489864194672308317829863168, 325745160318007308846290133163399655987, 197505355444305154573149577214540518535, 323448676083435902347483997252611889998, 169001241656582687797680069327656056802, 175408238574727012305122789196303562416, 283474597706600574362892100747226067798, 268232862784782119447835900038297625169, 273153445124429256237503665623257694844, 13602986446005815700188555596013760706, 275007561621941955268302802626931860711, 96394449871290675236899952956106008023, 256235138807963768249304264104557902608, 218206093482611335752914410860700906010, 56835311150657196890238595784723432886, 3661696405048451081615327079439385608, 73109828007063204385933767458568556322, 24578251913149396854965921763283799389, 130989718325648264936944549446093990629, 284697035615878012672778684433297086824, 315269458520603186681936478166233286414, 205496414949477482898230926672682973174, 338623618117047829705090980105758015442, 246833567031747582903264869720041972230, 142108958166159356309161864823698487777, 241315209221070389020448468333078251899, 326662883491274687639453934232663918566, 220918807853577847262486243587621121226, 293393125654686524506984994380993265686, 32591346517795401187209811323769885964, 55372941056000051149977559098637315101, 195450390140184257110245932301985720184, 234515275832790517203773973326658986624, 70239661920221555988405916385402235059, 212026204141368068480573885773720758350, 261830150420102495813737575922435566425, 295620638007871840343645608551214909690, 12234034446181163284584673321963814303, 255063643138407427724712824216392539506, 240158091491545441834853271756638935893, 260046822432949683134652122852216224325, 33295697720153815343031350978179625669, 120003894693597077524778110979565594042, 215119423667327597410392919801995273263, 156029413957164723280361305566526957533, 336773541525466108635665086789449396597, 229817640155052653114020892042719645660, 259303338732578398241117426336123737513, 156868127493400354008982701744281879374, 212037758284159356895191181633021275488, 212219457325622591045756522256484376182, 141010751789522361775939352184845501938, 197937742232191107951806845256569370322, 287493623719629022364506697855102697715, 1050445574266326553232129079460414310, 219484419915194595996397827810568103560, 331973003769536843345611821046151774001, 317650315928946013215879114393440690332, 266532133930178649833717751843832389704, 319565363780107139831175125095807065395, 307376831663656295938347355965620173845, 54407327670320198773695913659876859430, 256764650601461863392696623531802009863, 321533472929767983602044724687566477540, 158790251284403650645376820988723138972, 211449067788411253792730217576767665908, 90545193281611521802139503942752036008, 274591852468396701793554869741159468987, 87239013700900562474933542744373536115, 1927286118607043415462952773984614287, 123192289585365356389442541707679120126, 25665750203249733479672361503908981663, 44851121801561424412530783949723162410, 58515598494500994836089444750277984420, 107723379345511512530405572251530287102, 327152048846177293881180835393054484850, 115123915221843095885275478788880164886, 118914112427957211077298376868556624421, 272094969364519171429230233281989185352, 259468596839435255959727165881973103743, 82762785243659335009296597422552707768, 2303840486223724504669768437891464277, 219846742909460629137051645745943869879, 9536333850179797743353022464946990177, 142443489297581685410931025292073931315, 240290894444307504939393625634072976274, 51077447727721273366889170140319233685, 288049948949221982992313956288130670691, 261667407453900970738022120793837288441, 272956263627213784508690912564258740772, 312975736773098403024557628524675688095, 324014264788523490451515538615267235761, 112719635021705173864436171761050958471, 19958584780534855457492918694290342497, 237743971114098484193872699065764591809, 27321248955689129189748244935887197504, 223648479220302509021424984913274005917, 299133948386883849826914773011971585003, 289924209277543425823398692701064466039, 173853271498102546816463313709366390508, 191491558821886924036213789778661168586, 326313216014883183047700978659902812460, 226749764624856796721224081646562755771, 270603305728876262471885723441271557472, 175349075191218748156022929939910605772, 92636403481699260656002922589876905240, 63482898612109655162843531955282182626, 69605000541201925103846987766665841833, 204471826236659064196936723154544461735, 133805228041796678005633493409166177833, 1212954048358694108479670425757354399, 73977047159440386331340607678559106456, 26977540232732194171262690117976072125, 299651592204498285097581310075995943473, 206818165575624043775018902453489840183, 174151600960594006341247873004019837658, 48853422821957318097597554989131686469, 113562937614737192722756754742579055752, 24768965700035989124132328073537473701, 275962816882852063278824569638283692090, 41703798889645387246327795234246992193, 44670917301603787920824148893864014742, 198442915016871517563521691328661539260, 258080987077156999485775944276867488200, 99577038465844158863656921957854896472, 138439521347937986487130594084488075109, 51092124259020544877036216294054223759, 129218854445682314066251996155595303962, 230369742695625309688743225965777472573, 217586664378065402314772120860709188291, 60113689971633473307271631177416248905, 153140318860078540041462155590750974446, 272918709184318355565220850528673404481, 283039514834719524438955341911246027624, 165241582125447749359013249959734213152, 254148722594447937539359261315087869046, 183306725771142949907141074886311811972, 275224920932278235714738225243953063401, 195503440470504935790559093000382904004, 237683270011120474676060649808963899373, 337532111659935017399509376684987226342, 26584244903215467505426623220758324880, 283218414905608773863635650292616033319, 1598949408858774990031981777772036455, 257447165446912601705625852355179808200, 49144090794309326862160516611939113502, 27933422786468531807011927378629024511, 202979344820891221186539566140366700549, 123317212373696309554521393714276710157, 290753976658327225639102141856894910850, 297146504746419674333733586042752579170, 89966519289145289286191411455738384266, 80343239950630439859873181219091039929, 219653491390997007951901170598605700791, 325378140374638611833303702091843467158, 100595205206023303958777173185666343567, 282699005790866879887337624857428519153, 203173208096957394485648067195081348432, 219729369837638596480189378999502930824, 114482672933895982053643245777895719514, 187866587355286062398624812793381038283, 338547559096331761279888437507292202984, 81363628733603420031866012801170510165, 280773464036826228142061608684521425949, 306855678004097562969430527650326791854, 182111019142426343830440855307534583037, 65697946067492181065140554611208822217, 122802672290876320997137676136807851530, 87447578768184107634325716680896058464, 125133008244971137483415068547030618176, 334846666646740354879927135554893982884, 277094454794973502860239812107613116179, 310149063595561219360125641347733085509, 100616043798433758175014161765614394558, 255540295995584407590739973563848134447, 228678815849248265277736361004174522409, 289765394525548792173410899603271334274, 151022828574061789788162586129765454739, 67737603791109221448102102729403145572, 249709135617250498927588575588712236852, 119070318019406903977928329281526418306, 140922942002007482385953424027724354343, 58100638891017283992662020426754054429, 23870774950034882052267754342467894582, 183375157202324774586734176118314552719, 133295840874080561230018782443472893969, 114192960635000831919779297761883517117, 309547356182790231635299313095828484405, 171802764408342763828148447149560441312, 181527368589766428925391428420717653169, 296868525838980429367779448866474792247, 228630729457533920157980289846337250288, 16708877077636308277911830762161861584, 127968309193544667761551275246231141670, 305103970053719053799896424251175568835, 240125354578670585709840640767805444541, 304055669947561056155334961020896395596, 192678552122722597638463340234814997931, 280277236943043794078993030892036066382, 144145810159982321274365360548687131035, 171053335277537962670696801272232052910, 122627220640533347175792415492124652793, 313791819638574120738449987367961270960, 159077838943294644446772687124548058168, 245857140448504634073225482216644044187, 253189158712793279310676312461047982025, 198321678313408918125879687318157270369, 338093422771639984226811508236357749358, 279890876816322983134799588949582577334, 320254754516619426137895618204786265613, 254623610123680842957356854442531102983, 330588302775157823858431951581280346056, 319219518041693324294682048436657341856, 237356473860392922770525538089727175362, 185605537789682383617644914834071926702, 306710288551901945999197782895030433643, 150238003100193953694693217394176910747, 338366560466226367658436876715914630628, 99829821568229650964222353069081568756, 81024797670752286275988677491500249926, 73027347399758830865199082421419778506, 228929391100755401110402673282637227031, 159107603263992545732196131325688181752, 161979274068180901537311521469138117201, 312476261710572694833931953587273930328, 62985319054592688629897293662111684878, 240233761455983611311682133003786136748, 308332893642938097314941357689443655458, 3252943609044944760159696807590844477, 157912053025122956587231814064026897824, 133312332999447760777111060489294096360, 273950606141125681068986754423107581460, 22067282171765233975567151041438107314, 244491451185457343646334630001305640366, 128122740082260188961720827398220764779, 330969103344317666140462788449865756480, 85893307055269222009405417817215369346, 92106123072524285417382008452540398029, 143403635905798561703928592933641072181, 275357529529736175608665225542664233285, 224663795581204808049564976746273926819, 106528413973697356550404622277613035229, 109366904952972763347906873449212163364, 73563400223241592262856811986867342820, 297281746373871374806342703144802471005, 263517687149264372335005281536393622579, 169080710899019776785946044482914461015, 5530352590639332701111416134356534052, 7215355280383568533514175570615816129, 335527080225086766253423039059826896152, 99439190727965709185820683644641621215, 68645910673664474829985108333314749620, 72387390287338582684101759995391959824, 305828900129956419762506561839012877407, 75076364824282697541543080779414223376, 148003437272586684232561684781451047896, 224030097756221736905269508438433687214, 184923716913768768041384333994400839338, 140166059699978412006788322385598528538, 97789428445183467536904904556354855467, 32322100900833049531787233097836344011, 234609569311164006433557212718978636295, 131476603667642082687151119242392237601, 252615607435164773658352403499517117958, 302621806261048788894825046421121881286, 151429416246005063056685404788703392681, 146041000060455316291406220303979158371, 32503521504710328406212254847057366338, 99609782663177271326805659875237195208, 122702913992547438663258627041624351780, 335235885109125444959562777949143697277, 272173449163016655961237337858672759370, 246746035704673222706160922210734088373, 200286061726463045305162902038204233785, 53627063166243572510487646973122858752, 116876716696529352003022988411908364388, 80984290923024065359872789392746516793, 220040086107281894305590074758314837454, 208470103997922529729089963469320788362, 1466513272088430501742603781916805768, 21264591273230612215857853139019585230, 246802592775145446924225207794557095717, 88530928853840128581892007771980745698, 216880849180877808537374086577632979196, 67581162300642034865749692880119172350, 327141018476869454130325598593941214292, 236442982142032995476390022693117783560, 233083931025911501853933786335037354739, 164824805977626627144844717903139936401, 121087137429871416076176981805682511341, 11071762232077364618069553231316421803, 285059221900774781978128872878483124329, 76535512717495969554262956084650457658, 64612567042332009406360924006682683025, 315867121534925427305872160818436285580, 170673231867373233096462227281914584348, 288901573748021189380889035053115484380, 114850422966645531686124465560692163762, 63747208549229796511604126276211751281, 164727697219037358904483996236186854462, 21409611021978079694238690203276941012, 320650729642625816176130594680058745334, 6156231665322251327531891398232893496, 276845717591953848499664121634019586174, 48495436067686964403627076352195123541, 240195430589437496918647195482957368749, 70627215654523470031049493406191425893, 309330316178509877613266102354211126235, 47889991752448882602589829724534120688, 223742713741284379407527755913967941272, 212813629701348457211106470726292742745, 209773234016825423537861058501488488477, 283910370398638305489855287426818174054, 278400752813074302909512592927102109917, 22714251046407475133644974081684241904, 283737440480241288832662313401661422331, 299991944299627064879350667198271812701, 194434261243850717152782840089723069599, 301252790811897753313846814732124389046, 298578040400385745890694788768649342406, 332998844095122194946066805149989423058, 11553246458455696021379597379141149389, 141496920725509529832504647892827028960, 309790050186703300824984581459726153907, 110990741232745357351766261163716713130, 218018458696157551733357484939490772186, 57295995497224558252960199608652964257, 294647390810563021538739991238152336184, 199837862034995361636984259793587747870, 262368425340048825574276824723612425362, 236699647684101392642608938041867396874, 61949354953520832102002232111943785018, 232036774642271736169770908226506564314, 264073476310869455302714813330444226835, 295692773649409004755628152093629570601, 222433134138670463409565215692854322280, 272186481963435079048362953909123476494, 323310611885276668641797087825368574525, 10214674641246445712804587432198915182, 249762324266720943168954042265302642834, 185393886317406131196856216966298287817, 332820818870571971880988172579502892459, 213585259950512932300470535931358156348, 55290784807835707354248199427884497974, 95009541750971628666612381523437040647, 337917325511061327507483271303047336809, 15197242098623442682099501817539027418, 186947621473494522805879283252498917408, 80424391620561262654812631506444438913, 339041987516395523866475266125655740234, 61179465091038458853820630381401496484, 66664431848468191021948672000632969240, 325914304949107707329497147580597080371, 79724078091262474297634526070822096751, 81812506873626727331293141691923032075, 196849667265360176712818250765851964191, 52278444107163986059241917731954193915, 331201547760287200102117338162574595514, 16312906065769812219616986286755449494, 120281131383192374856443176612553201924, 263868068310846171404314636484297606683, 186399524851248923368001935479014576623, 248050483245374093060525154770297148927, 172113050659015242590190461991884473813, 158999284010619090627010631652936100795, 117687305490764269956466649225380027525, 312659442215440983514604778688313064582, 340100332970696951664009433555725538186, 80511266118554220260128609877153924900, 179557706224878062920659116965298661502, 259676798434893758311274851835588541005, 130821151050187858901480162852291069150, 252090383759690688787873122000141560524, 269934213163623532286493133497152174014, 265539257399852304555762881673998460221, 54613965596932888797064951296748391047, 248219629036684263487022966013659634478, 29369401959200598481858198124886104073, 251802206945834800468728565548387535691, 102760635262609896161099893041735740573, 334673138645555818754950468168194266290, 319245241764704138132690783240797371321, 9208094898573948776729308375529377290, 327818392750742940262197892932473426270, 48066178952110652594607815429154386034, 165815716022567405023497256854252685645, 193251858095759166172294075436138080094, 10068533174668664623259162883990882089, 66887662834644588310370953158872359413, 296604682788879158500280620778009267208, 329940207870547772506082418171187682967, 207438901425577792799884615423250371102, 222642264874585321072684205887207557291, 244162963011677738396828133192353031656, 282438274394901047670685379331605247523, 252953114044307396926962129052557962976, 20085298076690532593772861277540005986, 238063036567082779881698794422106945354, 294290943362852037037845834349603709517, 54158285827402159496171869675530922205, 15297565226785264545865341864243514949, 316930279959730080390137829000953220579, 206662411764555782804121234350622355218, 93960191246791712198131588099318301149, 82501541528420970895260767624669667888, 251162138720419929389338641216149049903, 118437670976555874816223751034549722827, 274821113863548162130591975906956410412, 322792044979467396280720784624667829737, 257360261012064386618817938135322961316, 313740040231607412659039428072420256821, 292873846294274793799451207968380368211, 303623173443027732310116860656748575451, 320509241657407568348433282013320977955, 91402638478502729700212755034414345581, 207430956563125038435021251003756411376, 263956179054571811694314481688541497497, 103352667085731017522876692121667296492, 292847315131695341666325446271313065583, 97706308068216441238079588474742030388, 67125561447198771871447011094770247778, 203482534337116054064445923545839707354, 220085525405144159728539971209667137301, 205243383441524488469174316441160627713, 284098058095598748120538890815010834171, 255749686552742579938164026981581202485, 128891693962693524964743380080947526838, 187838150451111847069033835507823983348, 191630477930286633466568698301983840226, 106285633060577615364197110589036422978, 159147298367405251364511095709990832660, 169108051185809053540620394739644012767, 228918215245200883835309913472699194324, 56487428102569980379072995319871220672, 11655132811658585330985226281195177533, 153341742021544932379855946833324134185, 37800909265699433125636323514085331900, 318348011074657420136112196743732314680, 68906283542395405624879264501911505316, 129698532971167359304237416934812552866, 19060812349750500856845914862559256168, 169584178697017298921973833469945287918, 46074848097643560824267885892221466203, 966980192351536436031764226047118171, 154210930382660167113095650028692536291, 95467009098254344040142488367292698727, 263488590089837962403635147135588410385, 110388750076739166292530554684590528227, 148841366658149926613805750039696781828, 218122678292316628130219440052670820018, 293018725478220969514673694265430342730, 255380930969715624814442700781818108673, 84395235106569181412487876325867300341, 283961030380353366735158964603546592698, 322862489263711798732695025191979019555, 270911334391298084703190292888903261124, 160631714603655671483475271965359161969, 80424733285711907749525027115797200743, 105774080318688938310001941730311921441, 228720619427336091651392456664589847102, 16951753600058514338784987317351353499, 75111732116517556438406365116453180132, 301669407735754858538996166626878554065, 6549604173481712957541587829243362061, 192281332481626665459188430134235647211, 20912555646619176980728579296651802849, 30005523770449178604309983388401650869, 301006103015476170906910289671281456311, 275303634715949506823527142427287949008, 255907191507320543960295674741396384649, 149064508897133782762817626839729728821, 79846648961347705105814655750868526482, 106198142643858550827339745749309144061, 220396016641912844504679457323829705590, 95650872768490943476442703362218941738, 210118516909715197358298799441915075369, 205654111809107939842127539732527943376, 297074828105084673720184028556068701033, 41355065530096363986136954369908543281, 295971877050504546049853015976186484193, 115773422265947317617668919443189961297, 254853594791195269236008239292249457655, 287491997474207364806635197242291468519, 171760324813931632008469753275788447999, 147630822171385998494245093476396781175, 254773905792245688430924714070955654098, 127630550584749867847714242599341265979, 138080555043436188953017315904419033629, 83500617630804375340576573241696421792, 40992758937338855470750355850901086539, 231754306014348121783427527216726979959, 59085058317581752017747545533181383361, 328211417434116920925557999562193722272, 156355897567423706841467595475012441158, 337789666250317182292493301819607237747, 262219774260071343221999165784978942716, 74531968901913906850130308837646552435, 333307806919844641655766405801085686830, 205503002348739670740516144360815215039, 589339552103357477411283372431078645, 115063082034320442759288993288990909830, 243248215592558236987749221775502186391, 242254078820557693637027963181760020390, 132454648793940510748541092086496189431, 48727677800065947259322910719541707873, 334877922394885338429304330084586283472, 183591781704347164347083485569551693435, 67355504515377367516601633376734823497, 271469442365374323944127209615217749965, 82395215771882792832948054721650930000, 253177542937768602582111153393324126655, 203864066414744109173876439179356377990, 301205030324391315122483264135095024481, 174829485579467524761494353344789627075, 241702397257504894731176463518672760004, 144140400259047532317172443654520793593, 134556464560470290402198394301055697397, 30914575009028792465784167974510967444, 170490438994090490287810730170386459659, 114731194720814385979553142871047004960, 286579398924415541329464565859302638759, 261406193436426376362380886758769215133, 210410061548372290499178629126706379266, 37091026666713800953690340060927969573, 120454758757845911378942937319429957629, 292723813901825405854198341897608960898, 181574101917854337690239047657451987445, 106499880419413549607242880931283247314, 134357213176182552698304063703577952365, 161834161621986191977872106101853698610, 22107656158785128614819319520800991289, 319423029892320221134180140677694994023, 242473694548891905144340075328167401556, 82773763429426558160294492533257927759, 63822049172766631327801879614508371063, 252176350752846729995318022464325373605, 33139840113983624914342887759003015672, 221616335996406356746989044407124479494, 276499485246230105141082591889691047590, 8988953637382211959315877155889735010, 141692957734114952200536635132856699576, 84449549567389091623886824555230629113, 325190915570558721260310261211447256357, 271337626305197601273180455028967537140, 35090554858974361297724117912140680186, 219363176768589025843640189239178628723, 183132387388642024393855683888667854505, 52770158312285378427816260111051505082, 110630585293069811873975843253568177843, 338304770046649904992373637950265036477, 138721481549431167770951626620991805379, 140373454567744713339192324693903818186, 179572077251301836561065446915323454609, 308540938906690316680658004382283844974, 333048105737711493622024910545252945132, 307206621911990730360801077659169485324, 262845344571163908335756123020712550540, 206292163795937706081840252844230963868, 3483951282146713676302495930650939536, 177467688507280188527139518295249404613, 208128990021783723301287502827371668003, 266151931795876809215009576836793700786, 29408720972577334675575430511880014193, 172168464984257472300240013043495158644, 244545985592140072629334965119587019371, 340222985891753002930605355143294389529, 176800081077209612421500171059640263236, 76542083491014627764726527176144366433, 336511856383540354125158506828737567409, 94079110194088998348417115825474613524, 28681504859819225675617938201869229259, 156617772518256194863478211758303842742, 100922334229604792275815852845344658811, 202578073510613528165966380166970102624, 176525186223103983908809557396327359848, 152463639913456143602770497576422145052, 181642034970751260739704582325237753058, 269561515325223840316785109498459328101, 308787895369298928222017292486851028145, 321831741444880893754190877101015770888, 30250816302503830857142205841685805574, 48234056296402439153797430854127827197, 299606722389636883650679306330487928150, 259619095891472832077729607249748916029, 154339661773392672620917364614193842936, 315805365582833401384038872871663430357, 130422034072066175591772443749653920512, 294752264495065221320639900778631148329, 125997089169361320404827091424495475174, 94013653651034563777738360518976072253, 25501064373205384128207590609990395691, 100775735448969388273048600064570229023, 8606965404645889575149430496646908591, 269044096650021059038732584586030860947, 296474155513474722411811169425380963441, 280332220062206110833655530568329132187, 56121715199873982361462352028173749168, 287507606890367450508010892779816989621, 211943981339985647569748024146443477754, 120454056943207735836428791555837524370, 106887642395313648353076852708909005032, 33959917571652901060678453863270778411, 86219610732180569756059208131126644947, 147445340746022055502725458791060984403, 234612413069856969201074871599735547713, 140776873049110161493239454483766618077, 283853381774865980796696479048878513484, 209738252280200362157476769667245934280, 3031662530983721781473496550391606438, 138242630584390793044536829056989115086, 312212947574964009160518891320092105531, 164711683812928145994163409178120127235, 173812890223705221820982262725494796663, 249793654819400925779011450480375425275, 208825106776365018970930074763081776745, 219736136788250787866760782988059676073, 13131955048752668150108855619226781809, 180575278183694331701186844802043105458, 103319650937363939853688143868402899331, 235229502135120101918117337051972385101, 277642968420916224076951819124933527442, 261375457178644215534771879264590613048, 75521788729457878339830757874214021030, 21720704202147450369951018300194927299, 36834894240095196862371428175156373687, 315217841212060333088320076949320543530, 33605124331661713427879917350211603532, 112176901364577336424274567709517286661, 30183861433491254942499921933765901529, 202523729277878563828054022235264182452, 10947082498395758577146809229049477206, 138667512215313323686285478647471201541, 100390529343776251857728870301311150867, 120091039158056615310514046681301699136, 794681425000065832039768767560087139, 58656790330509897535454003947574716972, 126785686063800732563641758217928407309, 326428528652438809648319927683387826233, 261114819865325793552034961842511379428, 11753593164585835700178180683628077921, 36276910373393654906670567006093107121, 34264933769222451123510543581024198346, 272254511675682125032941312156614861212, 162981711733867572397239290871177656588, 273389961482511628276277082881210541365, 48661377828639833966011986899959506270, 143574317687731518828824253001345671098, 36507243609138366572981940938320895422, 181406410339372029444026754525126248738, 190804802599376244557335583267041956758, 297691774835441217377075415398267685304, 289623277892600786642615551397353760640, 106152442842994112852210882356485411873, 181294061222105010138152809630065376488, 257112516504341760765087340260461332998, 176906166036934145756324954382052274514, 119014864951159031286169273716677904599, 221845615133914001311682693467583940387, 46965617290512609745697614129334676465, 103524021390303610417130107316472766717, 288819828778874375844527184407813300220, 183293522177641561209227570686946391905, 316746946200950108788010557857275232077, 250982118068820981026203862786911243333, 185674355627505913872428234185123309131, 263999115936619814804448714028150548395, 235589803789872026450215127865567257424, 162987933163272936341592450473915442649, 38003984509276748919600488191522037348, 250502778228363848656568158261519196356, 85274557348387689691244416330933460451, 16023109249585868704186371707150141492, 272317015114240175045558971166676904289, 190836758533843469810386906106630554968, 225986959874914817100511410188547574773, 277303948947887876089358084927036676860, 68719500436689243356747593315797831579, 236293950231465743423705402648272617944, 110023875011734478207458623341730051856, 214979396225994263657056249350191139085, 308480481968615518227320676547187382303, 83660559795280979664584723452684495126, 189766104228914081033546428376543073790, 139699010188694231436811501342419814690, 135223441948060906259782314339246028106, 18651557084794293345863007402811372546, 27420156551092755912383583040961999685, 52333678337008196488232986022081420812, 106040072338085848839348919523007887711, 54413665919796411084830913353260812831, 56685005812832475728198729185333552829, 61595078367979757596626532597980161671, 276676519553165724411763997630636226761, 186393008701275167290635507242022183963, 45791015646820294962857726693309916625, 142745208374786905321747611073670569987, 159205967952406610095711588186811012767, 320768834106159834668830002975912676809, 101624406977554954085207779916943237632, 258212080363177166209744722720213790481, 144106548055374788621614484904393956138, 232670877184921059534655847682333470468, 305186387423026604922132743856139676774, 247735117378587504155576424568996475066, 218636552063729804082664795531175480919, 18090940224829600092972521812431282779, 71799150560442466971607974899283752502, 192073371403875650548932721454489967251, 126476532519224906251013596367850842759, 297120510937326927576131474755935099404, 254290372420747433838646604340775405884, 254122411681338941805195700326976467613, 80039673670230288088278205294659194721, 173661512514251151825079986437487289028, 179411070779049771658777001958009298962, 90260155959402716554585829172171193618, 207879979387375775072267701856626111691, 88242686378187227094454406011592009064, 335355999052398416773891788521146422905, 101436690229505683747236648988188221835, 45378706472040107164635053339966920043, 295156778889297753866032084784172163801, 74234967577415743793989100395331823461, 302518932522267713266759023910731995792, 269715536295498110065451377741921577364, 47334882066126302348854743563941086208, 7696316243963048152313841542207147784, 26880568210096252438451803264454748269, 89661435019502598612862079005438260191, 242830448260325138790453349171352895654, 294534562306257062164571922503363629525, 313771228232123439568524439441748609299, 217320728258206222010079633656077673152, 32387788812687890110427188620182002046, 1736306657226167048167169794574766994, 120191728934719206998533078222956101369, 298967794561831381166241896893838113017, 286899114470028104766132834971339209795, 69782126807790400530706927889280671555, 122339985452586940933853121858690779445, 269491072230734796110260962136683721907, 64462271259963451976901931724074685678, 205106900673072286759557058177702646774, 263857129105829903660189424731694978427, 331721661945675665695199398923252055874, 135307800464534910502177453096571757202, 33938530517269546348465785239036451984, 326678500101110470603973299170016431979, 323126586809611914157032037946377724389, 318892232619630353492292204982589223534, 289217771686701117582458142073204931259, 278797963022353493006923169282356756004, 127901237913715607473938344777056567214, 253208261290030786301294182671631755606, 327293542411880276087118700433126623786, 95762539354955836998827670366036615867, 39964680888132538662892314446281043543, 69483116274376888625947479542685616582, 30419220296040978999154610901455241728, 213455492122400216615804019957368357499, 47193330675124809224488973745163240675, 125703888574662767278170223088441046833, 258266558848337661722621112680564006799, 51250767770814247155476785162761393260, 211608077861948949553198598865304056088, 324852667546547001285601776820479610424, 72392964706913910071387609500091260591, 305660831559827866043671009534332261521, 323572428747885046727783949540803414281, 302703828741104649317265424508587474563, 197455804572256257280799240374819438646, 171054620780749644907638346654237396805, 40203668516700733138264908746966437423, 66750728862924580788168373942708016590, 235473685473870627079968372670340314433, 199035989685910687586573521833018711584, 113349289645253063177889289958710267013, 12969029249534984580431169102700144087, 237773002387140217075617440483194267392, 196237016992221338234294380953322918291, 85637772744708859507617021180663720876, 250534062046267807946933681804374493994, 192438932477993108322752550266355002320, 332140493130559129128281801347001741277, 240870372370149934913415785849563762017, 189655720219987977103659545895032698843, 196040904522980633002229262134629192454, 221178720932975328989873180515541367159, 102448693076970481992352129629116934439, 265012612071637601241947097981028477150, 131719857214907080736201557737776582711, 42435168613934242960119530256514394916, 258686033092599413400192473390643863105, 163623258583423618054752837666879132123, 331848192334905113639384339622931067360, 303117903207029302215592346227030616215, 313478149998964394952152874587427489455, 261846166538773541393425746114159259299, 176997779253936186960634741404822798899, 157377901663111105640356811246964215569, 197140837170033199207489593307173164595, 8037778224109810292782688321257072246, 21750950567077728159674223115888973650, 51588590600821366232020462487357873027, 69622988773883528665098490241867148944, 126636463436196846192694339022947535915, 79859836092567505137792833720112023978, 55201143566772511739525443664005327726, 250405085257078373305803332322028582844, 293190518447378147674595965124185873625, 318444208946832868920122647164751691616, 319342221579049068355285188610071813267, 255559191103850183945865304163285790505, 43593078543228290538800512846496854147, 162987095865649937148742009729083095307, 51454303132314409389172886191694250637, 156464076666811448844290939035376953265, 30340938965405721340681117269149922892, 102879615815490274939811950510768051956, 177966853366706046406490028121017961758, 154183878710542687047767736066533509284, 167361489826197611493890235809315993816, 121930838822031930402590933276747705654, 164407368436534869536339071055988407105, 172655667648995393386401172902715524245, 278282017138600298260460228786729408329, 127717511765222000858027800163687733982, 28623594984156000981227149013008052524, 1893239162074477223153811731136724595, 6573423460981554948121568838816182653, 98479663672194681029845210085495573588, 125095663029514378672324699066046212636, 307068909089309211306376713137777610131, 264605841595599014046577892999803990478, 220893956251222634020958043162095441561, 52885619618539773466280514547836793772, 221318907783268661126900490560627634066, 183735077224705677984608372509628880120, 131884775624842938109845225854225110537, 182706414788992030680558914004151778025, 7864503904477484543602251999003192593, 96679302987367165253279591023480219016, 141324322950720219835829978480569613493, 116992635362426063602003643117122798300, 248340007333349336640771513432328611358, 227452480358151011511131497626894351309, 319098050623065609005252484758091017679, 171786834150779245276507172399010623302, 21471981428340599288223132933344524695, 14518949674495512116199684299390744690, 329336937684087072968967072997804459813, 286273281313668284448805624612096094561, 256680871093516956621497582113710815222, 324880882050974573001897793325787144623, 140320418376924298870555940726830202, 26655824881940600886811289127510093104, 148957240603779297591910849799536767677, 192125662308061819091022058745298040830, 278348809099514304715641026550905260055, 151989231022850932164456843945920560765, 174363194383972128599920366799393971498, 34374881628410217037693786670084028283, 14687492817184030687067670497751217097, 299454458614644644456131268581977648893, 268999496572826634865221979230525273952, 136715640551096312433751963520833018364, 254157547336947335980033398526876661672, 113458592128139857648586933031555982932, 110712839566637117473346381496351639223, 322542365453193545335250964607672573188, 49922471565227035423011765576632212511, 280793336551754986215139012617726026278, 26507315495198654180819214869251688996, 163443135280004582083986963404037992809, 47050155044655174320317446602835923868, 337599250728288405942142726275206171344, 171201122659480688398599992331203965934, 76111076617783156220165148838406703821, 152890001212505150829842075585733765506, 129696062665692419925994748552865408198, 236057731779707150208246721309769454319, 279700260119444186378003923725123209939, 297054014951234340872616986604850631297, 145225865193662913900189236664852632663, 290365629691832360516288206505007955295, 297453162203707458220333652764520863472, 272690213607185630160268024509237893511, 150818273638252655647326349356594551449, 3890691655306368815944200373616973116, 29365797451827052043226908341902441161, 24866630779101455141946954532975404981, 40816373712912275247833117249586194478, 245515278981483891956667052261636041838, 92769334028218447777456335364221184425, 228588846126666794164247333744395650503, 251974455085420181258664596833551161977, 38859258786864460156324470557183665893, 200348379776893579355359494494149368732, 138909680464812217087369038554398410989, 207715621274833044948567119192962522586, 123081512118568261578668122982861958260, 49824405428655001242218394159952831536, 152847095409828950185258952963664554886, 287044757528890582455716277583182588788, 70945054185277618218802430775794097089, 244632708060860047962158891083169692633, 11126826986708403156202640445521105512, 277617858768926897717413139873093794873, 47300663948484863398854113083064534061, 114363197183243908404955945919054473788, 112343123305694717337681283450167633661, 164625950232633768669535816527312826846, 137252834546917256430583400638487467431, 20914059525145288193170158077953466005, 159898997392641929647956811740342899576, 160813820320968356614633151404286505559, 107920210421141599912407756733932150415, 183964622091335629568363950432982656063, 193740193018847405225664211854974009870, 289445294387078606825684495625920238170, 266785489908410230846229773088289670932, 327938259358872050794808870324547830593, 220751624425576014243056394285122867308, 300746973147089302129386974852380455142, 168310223337545710283294630674196937744, 40361144966264283305435129105403705474, 298659608868463513432544933766562134373, 138814136547693599356926283217716601617, 310657313818243113578364929026032867118, 238634284879604737960119754957901758340, 149778006881101559389898247763006752384, 281995083176534534440397607118582664143, 25467723813783634381320676976510066870, 545595261762061335161287501101657853, 190357097330064836591905006546712086233, 150168724737351025386368406130123719212, 206159768539162298416133093563505225912, 14378627885586171425176043129656755338, 51941389590624685911902498055080316389, 228031506811382733310885299472254620049, 148625597555699935296531320374011792497, 328368436429556964464016707074985208168, 96122520963296450521921410309560524535, 200825368729794461009805391392297074457, 153084866161571534367793151957751699361, 211573522871951450332928823183274659717, 259700601918162973813458233976355943053, 98278742323521260646644153636959774887, 297122600195124764902202362328527817006, 208063833184542673477256993582926410194, 280684982065296615443174332951839961372, 46883584475777175487907183011533964867, 305593912011452380146763577794414242225, 13514331647461564197411565344003155254, 315230473737886949653480046536998773882, 206968096269017414000549811979378396049, 270499606480931326200270630774721628943, 145072220960511516432031703393160611731, 15707061233623801180907256378048870868, 298486204656265154323753577625928875282, 197737591306625759409208988867625541628, 38395238116580052717458710968099845258, 269529984747034797983879152967998476491, 150126335050320582059257976179265950277, 157780625209876947571671785340184017252, 297435494647317389878052085268890111961, 73592387741158395568269717019667385760, 19765404025224979458900602749708585823, 21411723246207843623757422187722718367, 159844369449479792238516426321065459117, 39040279043611831083501859340647633580, 79446917864744229105865244938073878026, 90074752366479224041719199496075314486, 169778590630312215781990136141877702991, 61964717757503014683997307027069650331, 98759647784931549582003192773511427245, 311700359809772669648336207520089187329, 173661520689923073753229508199739185229, 29681003516123377251967913365981531980, 288144749020264334410543210948537681168, 260025034707613383641613576380031131407, 166360122173358419764647423029044829681, 64494083850746646521146778589737040229, 283297748803571550085056966586806594732, 257954231022733413723340300859209514755, 168176177660383592924692808552087801064, 312167334549382137815744216735371266365, 240070998240682877398802597928536332558, 277384749056689333315661926414306602629, 281819650721552860963717081362692710344, 144636640286511189293503874033705203792, 207251938402719724030411131755010687993, 217939774720826569320635550885798660398, 27806998217310933440898622792907825727, 96375601834713793179854050606873537613, 59872867549882897929016035657185174157, 50701702005620882596619173304489277358, 63307421819266874350767054040910468988, 326227783573134925986851345691281322240, 13357906256394680333130566571522905306, 66268554718231231219650292637138220255, 134774759803146154916730393437842911765, 74323605916060741804522302919278175772, 327383737964107689055138084976627259887, 189434571824705882745903636962577521679, 289664602107955170220117176584928446218, 308839376203867740441252771843799936431, 235696735089660050284693822606488348654, 176867879383105051037563653589624814305, 267824720178078632918320022697110392810, 134085147396075413244633565755404821300, 249985791521737454807241220882736416170, 151043799939941880106530795321303471793, 111886482120531444834097967802479162164, 195759787467952276007321814346161873166, 158430874131442793112340325471014981652, 226381383021945140271348185358268969620, 110043042475264317406075458190248700292, 227826221597838745434577526406356115466, 68392283673210474415058165902642763001, 233880291597481294177101792647458921134, 247380428813564417594947734846044406977, 83678210128555211614988131699310432058, 180727681592474031588078301769247269092, 114171749175571989504853841195705055884, 266122812883026471520527658243543147945, 19001921280309545773393320182140097132, 146950802963109930396274534038182516919, 312389196147024561282460020244314070354, 329302852338680558336042238426806765613, 201003263532504691123879380524381453481, 6069348292427904077819045279098899103, 156528538346441690220720120434721222283, 191523296975447610804015301463179853376, 209584837771500644340838878658614466480, 40876173586107762930214449384417999520, 44995104389683664109526092174564869881, 254520137438636561529593520004751838534, 201904433045908294407700037334842354607, 94956040865837651229415070116300118424, 37368550671466318027036195243521505505, 231232784367467529710970725724740864237, 39955380456677995947555914515174599836, 37504693204973085549247559190473164010, 289314225522117614592337706524004679256, 164400472687467274714600871081741231284, 322905269950028654310424368784599155623, 65200473121857726098204721822799336945, 110923292662351806542566434047981073085, 241320632962234661947517900113293331397, 190655963618818705733453636523076048051, 146977571833627556829269678726033750604, 339061699160296183311586159899799831877, 8637699759680893233001408417445211466, 147074598838186806935181113762997883888, 132654074109240561242227384729309765000, 302601911426830208900200096249505009736, 339072934048910568818179995018588484973, 120887225244391348878420226872975948565, 168439083801424478019826116221684907757, 155504535013580478570136515786092692833, 309922125991537927254541506360540756997, 66316769746409238654440647383292874977, 46852765492828998010403752404959841632, 66182939291935112985505173758463476391, 258221094531741495760148550745859610104, 273947685425529377077128914251551864076, 55572102642827656054591121046415520314, 54564709767721865489043599931560469917, 220862403138264663071457976258267515185, 133491907888412712569247060112100659009, 20130112950024026573725797061107819111, 170834370338484427734384539806156167990, 243924246842666634443866220983985767540, 202656134231456654412347171132497835756, 192142124063580990530828314479772187220, 236644458165059782894367914019093132203, 53796081131357694883258776612904787575, 231943077584036058992967589862839557070, 216165233067348210236088494807921781380, 301852750027606229577111078515206779105, 183501382876273856965477283332716269082, 34494195379395788064268155184184503329, 99736336602173517319469440145913655321, 75229606235025460690860583734465920745, 283111468861862246758048718342057601794, 141277189576514763746602348149986199763, 195535618868941045325615998011717312846, 326265476566714933583556111262785467274, 318540384570806378343297691489172910276, 64033727884059069141262258588564212070, 144869759342325804202250683714811603108, 171943866750359216250115546885012713987, 171337743072225319550942440685564903889, 156387866258495501466063390276887622192, 115703647769704627361987212614245429051, 131484952513630654820981052646615151464, 143360728330361445731155555368839576891, 177233096796144102617319357476858873660, 263193405024790017661305903718061924933, 73823495019169372308361872472990366004, 172631553025471538980218498360100091817, 193789495692478746428743269470508399373, 327472426086114494274891465278153141988, 126172014610621124135857205136305387270, 66574535745170045996163145978744517871, 321073792694793292675619776117815300263, 4732759425449395691622489339199002790, 55481007515457056021766450978930976324, 5975611742315861013344120327049477317, 333127678231628943013173782011259564044, 172585350760221972805977100276332050827, 240224467721389601913096841861827108892, 338501507572180408976142890697947762183, 230967387455650468075297388289288725884, 35424008158093203400931671493644907772, 339247404613598459315526219291498181069, 102291626355570292497604219721566465004, 70107412623129470930844634075408910337, 315937300680278147787455670038152613081, 319429726976170032923276516640562142562, 52061041434567610196268819155132918106, 283455261304479309135608835909961773891, 129031019260104563417797620245997252647, 61103173229208978734591537366427890328, 322592462303479225603256032128786003734, 72369034814378991983059050244550225753, 272005848228040142263588607281064936916, 285304344973178411081504515426403495435, 189572560202493333354285162583136345333, 180690826295260415322431342636176650972, 260355596163420216379297105790647549567, 155096418207761288738486435961682827650, 157149738607045242461453049824502855852, 273772251331478689031506557310226158370, 295730443397408861204907126151440928819, 336253097355410089663867048591110767077, 314409101799074439921310293059565740531, 321662599071668278218884800009046415268, 187675636285678684036910009916147187587, 256169279586375909196729583003083811382, 1413146958382977474116736630713297558, 187353634543902829570763969899152121179, 70997032019968697681559422633559151503, 162721471953839682368788491766499915043, 214305513706725518052921476459046542806, 273014638240046642913040653798248856932, 145664885608587524162878355602006494505, 334287520596252352109753585421875481342, 180720296360916927539345708783534401556, 87522702240523948492388173566628736392, 57589148255412626556370376560062275292, 251414083702383253874613488867321948410, 216891562291475477825173190013260934471, 94096948729008378251878753822161148103, 44970256159118926831875100134973833149, 80589275123269623408843171731196145880, 273926617029212149957629062590169979847, 68960787388064089262348175501986042052, 250201792943266419235713899670724775441, 205499724753430475447505618478401080252, 247018314236875340415190430818539051432, 251198408596044162122349326076176792421, 41958337508041015961386530649206213093, 24826665336446420643007020345247242550, 309888263046003015887338256635167320901, 338335307418597093274594090650162655246, 256516904239581614441599623479047673113, 19488161122079566590157357769754206977, 251424439636686440417261146269063105970, 223217366073326861829768653287932921646, 314543938030340349779504665798785679231, 138238330645426256830263078523557388319, 338610781328896325555184231684444190380, 288071033730325224428898692989181669387, 298701223607448070087899208380107731587, 205534548429627692095792727528426148696, 112193064787007480859218750727867900159, 322884972746278143158438703601373294, 313684641108930104551715416727366268365, 96093907582752527144474301222565351719, 330852276435777157771749560188856379531, 263844523348932848010724631738563742619, 247916507710692982122855494785371804006, 152177185363057602623417301073658283271, 302462036625938885439463677377445964391, 327283436500461693656353155735043181999, 173217993817436569758262129990027560426, 249784954160686000159600288645500821017, 221535106573313418913596922696201522259, 187645156259991496718958236630968006515, 208158833048460815933761396876691956100, 206387781037384019098664968096838842508, 175577879081584858830628091331569061584, 227032112211743685737011385313274440268, 132566112822788657922716929491281432108, 165983715322358171548843772628070637345, 205889628118850132432221684556342805407, 145904866612581323018798107647992336819, 298353115219234941720851791364642827159, 300235865299083343859013682727337634664, 161246292051182510752202835494746097145, 250461734847157047704653734340724853842, 236953590256034600237769748756523357325, 298618189720757344541800130997670561453, 109367367199388119263335294086054710648, 186735423526250414175075064232863565097, 278768684143117465984793794287950185728, 7887381302139306904920053615705933519, 33821726309580737623592143896325401678, 323617836188891202608903981314974015631, 214205110663811460320142816226972555884, 74851345266613115674113673523934983353, 200646123002561735046174879632316940436, 213960672185900062960599268766837017428, 143875586401182218457785808801050288136, 67362702853530425548196320051943414038, 112974138472285289809239792840782430464, 122605159647368147592430728870465583277, 54629686183917724314929895972777769174, 141665968255031434383900096493505591375, 333567826940955450033072321448775315019, 50759609701293909773071036449326688632, 47063743658537824515177988205716836085, 333466747721764981949919819350254542800, 298435325854291494307343334849060719465, 146000301246827941395308574036073815424, 184902296786987757953413825496738214219, 22753975440862383536998761050720968068, 41249754610205481293343588344340928198, 319140466944938121899899889640374462176, 41487996466309350399511428993577189652, 231791064273365183391165976495649154698, 56712851665704744700762705067960048388, 136598445924421452621263367806604771737, 107666463944257162747491878654409444857, 181481108512158527057900313129161270563, 104989309386688661363385145817106630020, 89416348929528952601890804728451852729, 42457334454286899551335988667179147390, 73185047812244710239481606193151490669, 5409216418994331219525214702924221326, 76730775994802043614695100172554902515, 139802140321847070922215745052056375420, 139733006617448409490821763442331740362, 268988160753218285116197913164171402375, 97935172950192116264553957208777994999, 230956874407373502035753197360149939884, 94304202598072635401543160959285710864, 312771028491161850705670453967070776396, 99089817033879145960706685107206563109, 125849377088652969967645644443006007676, 95933800488631833415235809541847652754, 94610939333913553123937708475130473234, 13863022944295903521663460743035289147, 266703159133584721706938138074546580211, 170135072504658086370436590520989009955, 103564281926087301830019589467602841864, 169817515527591029503571496822778934663, 285608308747684928963772566968502123220, 76584381313099082963817096260066276449, 320462017059392773885333634705429884726, 317435044528580067934079799268053192705, 228742647632987308855636101713838914664, 107897969833853082550341919294835637836, 47647637989499955578047914011689877020, 146346581121786302788133428386304924340, 75905761106666794984194922252362851979, 112061541261194606597785603275470586261, 129145641160401598881700562823778687008, 230675399421089877751021521314759807857, 322330057385514587170328159086178338888, 164281750606766176495447359182782963253, 206050880288652444441653021637740307536, 33084515801771747640281809138662708278, 55523636067736386614828051090646613216, 309521340449775400525605752953099994914, 274916524498593332776271712938917926286, 60495301316260887468523946728875742071, 111917597960441158749169948689874184269, 107530917491941360586670582685344428731, 117412285005603886501605035177767555397, 315136187005566767781616403860069362861, 97568773961567429666869937354503564442, 306755044331679391045905008796172169939, 281231898575654876172384208668073332348, 255393246085975271596207463035761060054, 36277874688425200557363787111783628105, 169161536426207463003310669758788518986, 317222387173402834991152070711301815314, 213037874108907530025287749550715710075, 287392331176614213430276531900122559665, 49230646436504268262727799759359063081, 96014838246339450452340997215108215931, 199663268668823479739810693538571501430, 120656717002541069151832633603269195517, 258453203283489215161916385274028457303, 66165205754337828553125873797119541417, 237905491129797388737129306638013923730, 95424044717522889676534671376411594549, 257134833131637227727432392582108384943, 249474775805742034381984036505570574245, 196557831072626783231016580932595921279, 306488570981811639644571681379741366440, 159669899535822664757815315039461024108, 263611451174151552151080712193977399849, 334494142384560096711352748865889562623, 141396778397811649402144600366495035458, 327250773239530122065519811894341008220, 143905467112950187069484010312468132297, 237564613358028113740943750649006322362, 74437345692846402800661958240326075550, 257840521721799423292681121044936288472, 122989657925644507655961133918795117362, 203777537281521433803493847337465250958, 126318018966392271231492621582000072711, 166971285377302203897921010234625431920, 138840682225025373143320323582470521670, 80515002665827609065061156989624545220, 193011559434021372033113624982951525270, 250245388875215859174028796713897748645, 260833220726033049035455311389138329747, 24918407679473624356179851100078168077, 284849071983352128562046818106305783529, 190628578756592660531039584312533438458, 287317469891455477831172479755863056457, 179010534174472872604378777464260378393, 216696229672383586107532330193275001031, 286217338725915176588543934048148925868, 218362084559406791592717845346204930644, 152943395860087385083000726010722415258, 52432162192308279261545670870202033642, 144379355516415029010920377745067717572, 72921668535208464508147038182912228373, 282068086166667115827030383764032089534, 263877982338520443419840008273896112532, 197474602232747429004895256640094587468, 275267265972549838124940440116472765880, 9688892983938295659064232153324505331, 150329496791113258532825054581782490764, 240374601169805526152501356430442951244, 178054850820220988251851566655130711098, 43193023680924631627357181871457232915, 78146542627902334087104655396610699308, 95583181321625954681741756124236634261, 171215990978193558507643474432935552455, 135171882514166557697831342483455981698, 27042329901418600267023859710873920452, 306699509904891245009969547326388145525, 45114836235878723354698866117827455095, 105347372900924344159476876738083531851, 99865661629145818484132265534227394632, 167588920567853201845978537635129519152, 153129998513017858741361619501064352619, 238526064440049206795460604894512737122, 210621397631939396512753646299192613898, 320330721147978998303395297636042855948, 106067651225345196855717670555949997935, 313610237329143897203049071983271234880, 122352125015221069723169672130647925823, 311776914084958371379049395566428187045, 72079834738186354070920357669601328264, 30288949956406336803366948654429782054, 111113429200811896372045877688856955470, 321451198816266296343921938779572688695, 77833541077759696120937385124467705091, 118012354248797933987429078453058787176, 188686547400570662025373108672552137972, 217482612784810721826913683536698666111, 314570264993126703481689240803794931859, 264538881508517543504894523496966672345, 330274385025632862797415166146390393337, 258286179376531833780633807187492021355, 162694025938194756112580767051136974707, 324058467028810898283726039398779420204, 275117073661601146573744614460904635215, 283964612631168348159244527755368091606, 129758187048723292636743033261310840654, 174129119880538448530535618997622816608, 117051229627344092433879484209878894069, 22084957701160636086965517795192345784, 136451894403741973174914819249610957056, 141390587828351546879040167733764019533, 90463709667752625827198817989727536583, 207271831775291614735098898660122884094, 222362844907692519875794888366606017702, 63825885815629571194720677429169831336, 170414715652801032707049268070429760715, 277030829332364881186891147741270268321, 45648553610180823466827889908756476870, 193451899395175623510701701438651363041, 179876150024774433099406292000367980201, 198661035627953207837369885618737295358, 75231886062943896946207406950195265454, 99247383160216132886718362974938865955, 234924965333312977914241081755882281596, 239152533770268481250602619637728432261, 26875245966913134260896748251749047958, 34938134161872846863485183918696216890, 231582409603089739478881774433037868949, 76327684120900612874270679908830933403, 305191764116896337883108226102206753333, 259772010915775127340157550617569450001, 18770839625973844544282795925851075345, 74315311499834608161297549656998711342, 114754168150991050367925065522248268335, 16850767691642816408051556819421831149, 134998441868140773756450874652074356506, 325069273970308413393432425305638464073, 167593934198032523918676633380084559388, 26623164238854384153052532096882717778, 100596252346438427960507999163227098196, 265019794947666374762630711131826318202, 332472643616246185676175535198406820139, 232644314487038167565830059149093365904, 192468988750608312808600425409621824130, 320443851558419199832541712087772581027, 17090359516839829729004109684840043065, 51504167262901061088909102498568776211, 303320912927899629694922907363176775957, 5337056513524637804750301465573042978, 53348619337841612995048189923026511516, 59578233080197300284853667817466051333, 55348034542850173960813663258701333993, 305137248356235208308728700044521758533, 330750383257122721033731341052440462892, 139106689064067476856814334328323927342, 131085438475346875461149010758124748766, 237465490694075157241618015483126294400, 192705332223146041493450657656894447705, 161523670823056618033184442563286159468, 329743830338064226785868816035850050819, 297757849388655091645977192867650733908, 307459336494228889088077366875021918184, 332281330480330772723454072018268491055, 245417133656929562785141784218373462048, 331069858270397887003933533719974978349, 318617468647851763387114145144808439472, 260264742182840955233802123713621479867, 156227012965891836323974129681041027781, 27074560381802678408012857151158728756, 261008908185972580925840263844105960762, 205465886264626925270339971475724045004, 112888791671626214714470745691583209415, 220105783890964888328581664584662334654, 261391835369678089002225778679703559406, 275025459369821851275472670114431794284, 43123937300861397767870218290798373419, 156028595809288022224456671384214460776, 232732781339015027846799599977849015403, 265256642568733555632617501957552313483, 216374843208262937897839374341664176739, 213285540594637938324452558201853780769, 200703685408461573540757715325976666536, 253328763949860111497635034809424596036, 267935583114076211544046984112527650416, 204460884332187602402386459672476073952, 30336405147802738310509711308707255065, 221165560382880832734231539424200385008, 329691357051195274880479370598540828785, 196671124785148897060242752343026350327, 291875828139795369853151153402258012743, 223279938426672088694872140254732140036, 130077390298507381645519692431025195956, 323459005639400496798109189923869105655, 95751343313935771428602093743298060093, 310733953983794732700650534722398541527, 262831643376953226602966227569983672371, 141791054769213940686244931400717440784, 166901275228377988824062782244332900400, 176912558108723189712506853069643732061, 87225357843427097400412774101913213159, 136670789195222406942686606975350524663, 219644772866364747449536185712271516871, 35803629807146778881692678060554166797, 221592555234017953729991638038166229444, 248534500036582603125013204210491253695, 37321639551547007735407544759999004030, 183431365588514997071677210775210791364, 249650831295557365706738121335272432906, 73883943737271809153584411020982205644, 290703116489639019405941451249404790765, 93002825697533347862087635455783358337, 188505216732873149387924584902547971555, 301308483675011176055105540715614334984, 144105382174594890575246501557779187681, 35532935174716214147520805675836917024, 1522657214378656012466634943910185541, 12199988588323157695603187011504590187, 195878427134159635727249382330591639778, 99550538796918586269159366929105616866, 101995969069573955695927477287859796404, 242543656825546619238564364864232419741, 224745579950651445376740960302903000642, 277190212416578671083584569473677853887, 44519226539423120631206423523935303261, 6341663151348481319091124414403862291, 152167833307955557191504396966122663162, 34018758832531798464270879164329010871, 227375266701480225417429168341859886859, 211301798017396913696060950700619322141, 330201248950928484242910294772756031030, 219523751681753209728230386732552127123, 103198273199432591174970450831199005707, 275600692106204496484264735358238453595, 89155468847852393208855313538176514750, 182126783155017528016163443244328756209, 278470486554564687107461651710211983599, 41786813295572428567929167360430012988, 244620905281994418219240496342714169178, 184031333189937837081633267779330855310, 282233566005651588876532282157579193298, 196054709244811170026857953225328210560, 27457407613509473261579875416664337262, 194509880197748968395589621563887459533, 52559750610879461457580321734536733285, 120949856601809933120768980439093006834, 59963215777989571135871457202358352971, 139172569617182097092012474763080177547, 300521677949870706141099514794929745493, 40356423025952151279659758637019471627, 108754299315434605157700934806677231848, 5556405969702027295990969553489982312, 167333059447680531309242354236631087707, 245712664428175416603830421522357694728, 207752271270896871814916581336999990702, 148011488501098646807496145195341020659, 179765224654517152913298086767869542866, 274814252915357168165760717529258015897, 238130389323616044405654333759080853202, 35900205400873209642053819105124969122, 57262193927308350406815557821412798953, 256324394700288219444768687204694523240, 128601334295502809148359004642223927199, 210140976892459241685212253717661087407, 28326971639595778365210117368449166295, 327659837209835497581567526432143808261, 63298412134306932762502316772287220304, 113505391300711949317230451847353339973, 190194935739868780632893619611458095498, 122298580626256243194376395050507393780, 102050140038549148877714698413018208750, 181870901817329875423596601836065803020, 143552377174283825208482087314148591966, 75729577033519176947187784936890755865, 67145482923431324455015738976297044750, 50397295873209757444508650401123236737, 20884859554340645976880145118181430455, 55216278663730759902278543704862741307, 167714990805728945509388499983461581790, 230264612776983666120950560294913887320, 18427923969316143821582912980076194578, 313469708758950931338212955577708639918, 94730996240950499248624518312281299694, 305811364285658955828130075409230241121, 41924023628172505542681600361417638147, 326602766962301311241702329158440441714, 221556648776640471879612310039267786977, 223352832450651484797654654092816310053, 242368639898897324987567280805834049937, 120605887443498862070515729127453165214, 67931006411032933823533809615493416312, 25650924477258775927091246079980062515, 320365147016906098145898232459651623676, 161890979915310617388641527155298418909, 302491369597606215135631371801564696395, 235104191583789853168673952730546091496, 25971534066658993064046482175036853011, 193551681696145889752647648813736702128, 169331258213594570500735043462821863995, 187195964739445047208405996958914161607, 253253596717840415847068436893352961867, 329910850720119409394652701421455167486, 157701387590155048570583187192896541186, 184665044878117759156545413045526326627, 295369492311222010726230594515867430407, 183547344373167192336011768119435764108, 23312384772732581746232529506373841813, 158245801086606658659114426594486142971, 217370817269017033920135246802449556654, 216966917437569591229099414910813295733, 72760379569255691924885273818848973717, 65665646309901073208057154238165664457, 231498456360815274943474178870919425917, 285577807830573384055790699668911233648, 62735553239691891837836295873372017258, 197269886960966121010862602834713930542, 63035491251331122519288872034179012715, 69714323132001501583110653462359855752, 125677243269924653778135422516389117444, 274860771348099442247523189789252606466, 339798114389349531029975626578071442013, 62470298731501642793596220439892844408, 190851311480095765620949592865517364286, 64766886904035312989512689257085209959, 155216567823305282387173980978529309414, 38410084505179265936111748096088447894, 525275513967718746004013234721819351, 31806380838837750459178408007644919516, 235941050154831154053414617655276356831, 117496547070495573101264573070074315124, 182632450248423347637888768867743594008, 293343324794147604805408179517055338851, 187414125417699161157695864817112828731, 155175326517418410123463524503078440593, 4411802959529744341270043252434652889, 333828524160932804325459977070164048523, 268915597665234603014003778858049674684, 16514723359286827623574766299191647573, 320195494516609832821419541797179734426, 29899912285727793686711357801648241588, 91173507385361273429413641211671115407, 112922762882966488599988733148988359738, 178770752392253695880754035922042681301, 127196164644469194321102150158794767067, 190462808888243431422203268452863072302, 8429969287628326448328333482942742872, 91678111329763067252996348092402086132, 165816228504246796204615085440564544323, 278020609451932547451174576721657554961, 132527554179831099549182487439531937645, 229744988229309674066650583117580991949, 55349318358218887632745998381417301020, 105003283431548564907984972360280417567, 127508165786725280719333453079284046856, 86939564347665413222846220928938330840, 139939639695963627007133892763373357053, 208925102624873430248745764795765157101, 20691278019708498689726498409993574744, 289135267168059005794780588353611945252, 97671069363092187999721714439208627588, 237581732648231856539475049817330679547, 75508920177182466404979340721605012123, 249089409139585412507571762853823336747, 166536665021868037654333003918406489188, 8390747943140995880391832971507255782, 196879818890288462284773062231200829662, 268098617939095752615272033831815781763, 129224703460663976291429085299522493278, 301840754400052695412963601978180045051, 64748386020962528346061620238706929498, 166569263388836542116006914828915978500, 11184455180196914575851732472863134092, 326188752001092827233589513525930261997, 246188343381079438986595355066662958367, 206881592671127320615940293527576091363, 52753726386306095316677395475148569809, 200821281959430984584237966187574092472, 80739501194343221304356759152448219964, 254708842971926846356554272924991429008, 235405259005546960177276038022239732176, 40748833225382138097169742707400402828, 40767599412258756517379705289867812187, 187272029256241683221001718136924551803, 18963463307229796721707475659600845871, 62828361186151186711179449148388064714, 233923457205333816923945466318280215666, 77846342987284008460902937971122810381, 286071934830846299883148154083232086384, 1518995614163533240456961758962982142, 36783246518870360455670524883839787544, 293056213446592889007328431564502933252, 235673665207968479344785950383971772721, 256978154654244343827228687159743150135, 47818844790058646852294456547061855251, 317741222537562470679937938385494582566, 158777180848161462295256266609484099022, 247982933337436193715649627257628842711, 50116591156356834451546136960408216973, 60237791471449582227431568043067353231, 241109565116115796190319369511499351485, 143499096600301530314316749033538164448, 100227181519057835571345609216425343597, 214902224399991297396977062647892203034, 139473994287073118291913934529604710990, 74151796698584941108514203117798327328, 317173617957457337205424954584169270067, 91081619623289926422869649021749034584, 285134302049576583972236331277557325227, 50139263524374756089054211169917812630, 102365907588485311009463820166039251518, 319109494762352948859169524049012318734, 286110728365999575454975600854941467360, 166602297142300727911748254837630731342, 201258380481124627375271661906893693818, 238458976823204605442234129377402627382, 38751659590151559051509684870888858186, 284626264742237960759530191289116410373, 283347769418477681994527270893459002096, 108885645966241330493424277350582392046, 120781210024297249212092411686459426823, 325121849134941414753108577230803947084, 162907982800083767802818937202267134251, 128493535131430421020011871845247467449, 235161301467480109559546251088299039691, 64416098825232329285944625212673016280, 69401573396045870578654313984741342877, 161027382334829739565595308785828420479, 234468528972035097035994597476706117113, 169501315981096996063249651347945840901, 103686396373338096407677780807086424931, 318862414119913126913867585352698760201, 317457335940621195904029501228066515335, 58944393557255434153258661730818556485, 240875773026884346879655039509694041614, 228884546366083320525412373119093888206, 242905745724392069775527429949187755505, 314577285197861929468830556785331917421, 29668720788170020358279092758361235188, 317022981219135550115643418212890863585, 226061481502413012232359865222529167708, 339509479237530835278399415373755516178, 233840213905900990325833480375114287658, 218340120182276109300271978779104416774, 96143374654636633822531466268836264331, 114727931722783913328606569272415124686, 106933647180221606767797009756628035782, 291423694757616711319469244076080272034, 223764050619442266336074615469922557238, 184527428926702959789577336679306117072, 37119543323151414562899457318078561793, 306608675962224120735229884432963456435, 98662023539961987398256831682949652877, 93628964020189902051926607993609429877, 287385590996067314562206247256458776369, 231724640218282092593510302008973972546, 198149682751236514000947845916137572260, 301043588251954694721339337187574902084, 2877157629346854151396518782772045680, 203561640296429287779882146022036205557, 169398817295085881186617640680174487431, 322171896149612090563927123488514683329, 140625092799650139631791908000970011240, 12086574279129724996285527801200940060, 71861132744480548904248975139109932844, 41385394549980944533865230713826787027, 98119159234384966607619136993988203097, 187278175805011780199469693120930582825, 249191293344932746429878245967099679712, 19779423542073621166506308775971685219, 293118252703219247859898313930168990066, 278006387444470613541885219992117358760, 212284775431130243942647741870706029539, 76274544022386690102045079151673201850, 311671634837954846211738106254866730634, 195630579145981139448947650133461567337, 23281617188916615146763775279534548112, 216131665921278056583414912204726118873, 180243039497684610731126808581392374058, 336113017911654720270366811137937217800, 103172343650817161669624125944833157901, 65934033696428602728948103506262826277, 124034379005314289446558668178404321836, 70353934492115684192743415464611010612, 52906391921961378578032773020545787614, 232646237416362027366659125016893174384, 108258579332123351092443126859528923428, 274919076855310813884444596032565642911, 265795232930921749088336718218822967173, 164129357893814278348448602652021588406, 263775828174174687226770685756279606520, 280807530274433929669751195230097083645, 221760006001987255942921206416165276065, 82957983592750238191034046834393663740, 334566271283115401722162528053185554737, 176749974880490587630970290307687745187, 82643919650309521009635501471685590688, 87920481713311758865706752061126561084, 99624266828822106311239082439542213228, 87562265803510052560474941545327287992, 100850740230685366268970255079052568861, 283867433696548602750967003225381133293, 112170686957767382001841307329978461748, 161598586638771559163779448557097663028, 15816774416515995829289582600301747503, 265222731911249262369680604543575166823, 318052788365299820674449012664357098526, 204973395730921837621517262272300959640, 33480248844240870832535842806168903214, 86159492734696669433236868641804531142, 263170947818708018590042665427747737561, 105626173474431589399730777401819970553, 58290316018508587329885727309343586776, 134071502640262423036538595494045084244, 114403345008253042464316527268315666155, 280340583969115365227665414379560784531, 290134930420790397030725706032519329662, 97767545755774434389896640223792291596, 304593656081693291241179720523730872724, 8563516552205364057507475803647268861, 270784776664211226462754376021435805162, 168923326766384415721506216468383539043, 177575837073213194543257916394868422194, 710876510106140607903265789267118484, 253053230421036643722986742394023026255, 257758570766179849550812123638471682428, 254112248055164358664072704323305125036, 93583004747933773355084439213179924909, 321627015266351787498406427688520777335, 115897423840552687099969040602977047894, 165697210112321198125198966636963607049, 273485298891760439014519106845651986574, 71223893811306678604499381422139887930, 63600540093710459204849733239556709287, 227000605174587880564853344921760167900, 266333783986306128067981261028037539568, 147657915488583528255029828908776852144, 297767575362542761542893002123560192279, 202081173539332650849011379608537908584, 30851618506841677225298277049210898626, 6180884500467560084398771182723369770, 197029510972241138065678802879716741293, 298672219122434330340491190513356084484, 61741077225642656344288286456388296148, 282716488456422107745984709563361844306, 191366126288019583864611435125912326532, 50691329436056458417674107032953524540, 181444458012970479807968387123906956128, 337334200585899254011713597444833462760, 145517886831032425368164047661644930668, 194021877690873739303519456284086371406, 308713136011254804417488987178769088318, 331732890438370183855083166579088259632, 246972734764544499318890410098634655006, 298610958276917700826729582776536215320, 47262675618003932879770514486737756294, 314655669704668987571225509400891906756, 258684545853252830191424411515974287147, 325417588220414977256851266739236468162, 93944616321426337885910940423976321638, 255834460639691089707759822359305765116, 129192167509716158539942754525661951055, 195696802880063811339432149712993620132, 323604575060015235461708308788114178515, 65741105245743397849332635892202863935, 118863816209235616054220912885277325738, 51139841044739775235946407939152442270, 225732962987344339864640484381350014106, 284489205319637040451457825077371982313, 1370772448919703518590875533685090291, 219248271291560707344551255811828015703, 41330804680658006383957125562378835559, 114173234638331770357930206628861617039, 272921025380449563705445396808344156739, 45432006312062201251809463683738767243, 172470163024808611815247482215048330039, 173108564832734987925937759963122503960, 244324953314386232542595487919314164312, 220902379533080228044362496001758036169, 177958286235715180521671181110157828917, 34952900220765664608537659876335202137, 180987627012669206354958000632733630169, 90095237987107511430920664404151226346, 162513746922407644822427828421670616874, 19961814626882769564908326686797204392, 202497773389188139681512430345549917784, 298902657656635831183563672818087631880, 86753343082112380086695547920991684143, 185388748610985288976250959344253854212, 215264273379841881225321747785870450473, 210354710141564482041020862169858559919, 204970401066086566330683722116385846861, 45193964184215750662629390783832504269, 271029069566369677985826062860285245537, 139374154846427099491013301970103462056, 68007709885686413164077402922142063168, 151883062938251221495643583834970181981, 156617971970377699902271942451848982430, 310700262419621499393031913717681748458, 304421787802591289158572435470422902845, 162305250018428449458637624410604241923, 70380005370539978779291877976589580215, 222662045229078035413057060374630145590, 78480207610007704735521548183958365015, 330421655066522774052265608905277395536, 146625219656587326849043266514885505758, 275409675470040644737451662205104305505, 9067635979527927783214194966712561787, 100873243072761098324339604703329787081, 120188935466680517293051337172494726154, 206321969864090301018806070294800736271, 192476677450735872143446744410388482081, 169253696651001086044849109469351625810, 152654248780182001244878161090795206862, 86651892438800771728569294043065187020, 98122851510900509910226480289409494331, 311770812626818565585930066717607899295, 148506663320204927825506728507979900522, 183153669441299636994168208049758290099, 36369260971969540951578640440246771360, 327731766128384185926607341508545157526, 7146456353588936813778667860798451333, 332214124494179219944195598955863910004, 308802466638697217847592848144135141409, 82170172034106701529365928365284385293, 211881306840745283693698515528727469349, 103141836774973746793859696144633940703, 286989985669847198679406391949118079214, 151332920825301056003665141248522667246, 290782398452662053566184466697080574162, 390713034710445869320348252038897232, 252035301395077892808445235430139123017, 114019381655491737711178984826345569719, 226989482538126477172590646188650379545, 330526662776182253149630140809933730722, 269885271686904583824230510266837880568, 47066414172175023556818908557545398753, 6705710636950019286067315604034580968, 104965603464871642326455266749192415279, 119867467288429007598868947467733102470, 34968018775149990506515512481930215472, 213587747575019970002275341188560785593, 264345142710446259968898353315128113688, 191920541374757986946891072818541403999, 153820596387847163769921952683190417129, 43009229535575746052721101644297097430, 276754387367392922922536285922853880176, 39934764926743421230110914161183161139, 165554888675460192733371493470033912602, 163326324144990478521430781241114285109, 208125102404361315301188569760947883868, 332687873099904102086446308376019607918, 298605753729226611583592669687294223107, 165524123984692762117574374685780267271, 65367848165292181178650420306056341658, 19661789610075543536066806287452566811, 274848270340068037937082166200279376535, 271118500442655841711892818437965270189, 127880030456647735310583130403172823945, 237289520522042502151836540794838759443, 333222497243861055033911003516966853277, 302272214245750016476024134685050040842, 133561987048869982977299274207798011205, 229640643785388395682639484852848619660, 225751396148631258746732550469313103720, 102057879983014224314176789190794967441, 321183994802563310237824669694344103348, 62206354814556494566030684137322627675, 322417912767161396094106279420955488357, 180622343330326391274727605672040793411, 333158793570862676106935754692914623647, 241874208683332365695762680050487747008, 294056008846953885553850788785431133419, 253616329691966384029946239421421593477, 309812121925423501008502942738367933158, 97515719982517571270831289577297077773, 114175769850063547567601293358583056004, 164391950798164608368406802311838633663, 108008505682743022402422305555879357973, 22424867485069410018574639396630337098, 132544123667976219228619152832759986830, 45050427895915627395991149395525865166, 67575094843200485135106529800140406493, 20293111262664486051201431112535071949, 277589275604504196192909809743959915782, 70386464549199189795548867225207515621, 225047787295763505833877293308623035401, 278048996730866510277740795656675999623, 287245933635039145050606610787700171921, 158435478267876965342092076027509899025, 257120356920713150382110404215286543843, 220522733236763169554903188284409886171, 162394969441937246827719124915804575583, 132033039154417304414389946118532694916, 227501604093050949382718647542648601285, 69393756569215937562146644274238525670, 149154826628287963069695968991068679297, 206038986467725146421011808689268385701, 172350038594019421128621202506045971232, 145165178726703162784083284614627500684, 271334396432419490362430012226836168941, 195783436320531985618414898717443726339, 241636517162587255470595621482216109931, 245787040698914238057144859696628868661, 12451780947482917737743994068563358891, 18842224735715067309751483412027699554, 326863133946840974862668221149626024382, 87497237165459635806002813261855473428, 130674912248111869522666820307929602041, 324071080134422819890410633832447659648, 189708834248976318802656232490883650604, 157912905068859905545952734643928348637, 267292375555623197946186737297652248463, 199722073298125762239981931598975189403, 160314115280076259394849350228699556106, 279961926233176979731758076785179633744, 302448560633893770084394446947703576567, 179464313135923641081466486984662601057, 76569267407612289105416312386271877559, 135602793628184916040635472798547385612, 91594589107092903637293441362203840650, 243288845814649104669925739550882994305, 97466115607858220714546880244168679315, 195014486886727206024731587003240666926, 246611448382814950559142813825460230518, 4557592775366650646462833892085429126, 95859994628472296794714097094138864041, 24699667004486012510038957873575254119, 298855278815296145164413322704702059993, 337271527625445510695382709027743354076, 84078966752691426742392669198313829743, 291263302372306522961913104037877149283, 279887734617092034666494382322308727380, 276044999151927893813382834607583060024, 225729764175800385596486215631829658063, 153957284360852943651845846141685559291, 170409024283033397322204238703545421757, 154657333885130649752902314677714237863, 6143029354641375966363193737936744284, 216895308023646102009778631475120074098, 322829030268190938775930056952551163813, 287287889810307059203270880000188192461, 127704858635769839017838341803054790476, 293776581047879471067123463319878052458, 197142533386437702839085769365725970613, 226094784823540440463160873447246380607, 273216779791707453824327573386780190880, 72906481155773063015542749230725227295, 315857301643273508394588880080963036352, 325812241998059808772744466520140381475, 42452400932710819552305789339112337339, 98624480190836681032980412841130879026, 152753775336408707911314004030856229307, 217370647330900159867444487918745866767, 218543969186178828837605466947110874805, 322080701174672019946635811070963758280, 199417023858788958946899278523800854096, 247976333170576896311677402356939513564, 59622198443477709976468379874248392903, 222007881439268735434055595769261593212, 54373261951561581840665672982246836028, 15710503193001126674785456117284771318, 31199499805810320038316486564335230783, 79892339595380208606410931136833076696, 109025560503240862929863106200688445357, 331712511463305638962156347617738633575, 124326281857286368539928787574533607161, 22396800792828057726525593567234465898, 18485732257178374182446483572679095081, 107771304553265802954287312894951869573, 299093750796283481038538050451439409363, 262949698005726236906201859338429842130, 301565754313562557809042431878090399749, 308050258817660446624712586173088003204, 326616410501292163151575108922715010897, 144829634188115707314866235469279415749, 78178499115464893962198572081800953814, 8144565224099685311994953335508169694, 308695321125506818525680838608066899055, 232375462158486268275634784709088998904, 334534425116375471453608480718362632962, 83880055533099459969942810840513297147, 337892914504020403840458898801917773849, 104835510095848730717959640610679878159, 88126382096569497574923721760346005913, 73433170660394328071371219585549481072, 224302241877886589454033111369896130398, 337880901933039152273149183302088300727, 265633170855537745394481892069927409499, 42480402313959339958674568599702846577, 220364704712442924897557976874812121897, 270092394962516694255345886327138705053, 271459615957360870614223525325213296740, 264435594969078247766675091688502232902, 236438450345285182993731614657253402057, 329226296315802716643215001522418650318, 251319197610637362503586720017353148553, 264791775271914619418999619763968584165, 228644657761195360519983545401284016592, 147641582430054451131320118493101536827, 64191186886173888048976992834113621374, 181569170944828295869656765597847600594, 302771072631305821665495544186909025623, 220987353417014980163944798515819660452, 138462717029452705136543450598258563914, 129228688997233926169935565965817007128, 323187291494788649808618325235997102472, 102424620764531047406387208448557595024, 200834014325471536744191463852570965830, 335177403273031256423863671134884819705, 119820935210209877742664698824553252351, 279713395434205023622224065058132758329, 271420435297044959218949982921961358218, 174483385067464344877840474135205341077, 226514354893577278449645764144588367113, 304782171681333127564513266861261770146, 77387971592762153093483426687545252434, 160525302901543792252887816292256852040, 199657393854074092469022605307290413381, 333828001683787229701007222235699228302, 326152294821322011222735626271606122022, 143450040631878380456274100559884501610, 322684248785179863324189481138338336613, 6595883673268135445848277527288598142, 314066769814638941828874578769147613493, 139102428018744886401555181164000918392, 235230415364375301959399815548636823010, 84139180576040721649487260134286109979, 115730957071482580663904068215611271980, 292535450910821778850018574247181403579, 72463957549249538367129528600708066924, 301550401431314917740656523913521741064, 253455355461980659690323093652629318280, 165877773846483857095103257292016320921, 197996087853649893837235773309834801077, 235783949339110439750136072599344466940, 200615811619833895799527349345035107021, 333127102573085884984379367213936083619, 10777483734426328733928868994445111893, 336357786087923749466550298647731292389, 265696036593468747483908779755683636832, 25757631808566942095745992762581713206, 279356015228543245102983661809226209496, 112719211571041530639296870054797755763, 137891739430538070307740842243877505197, 17900333700647191963644650174610039046, 125558621441706787705229663905048729239, 21437074592357387872686849480416184413, 232364056752827891634682095070037954691, 50123888926731367544002886455304576699, 207145785936373426494565326720340401134, 24431002887431464854346895971211638545, 282069007448408306028784742884825687251, 251209644269218368557505246920168747043, 173147684665721119657571451314595382840, 118087293574007413668736357250163377971, 75042870412294126025340424559216781168, 36872699086779792595258148608893826525, 92054419623420170269901273778181218643, 38747006666910939296650539715512664138, 217749942745198245979856389342994237631, 162823126396578310018215976446809529009, 169735146801235988072661749872794491965, 113080717138274475304202787782701245352, 111019254595222740633312823600604404218, 133739835236664072792659104143697573563, 9202370607211164116465316647583591781, 285964673241879640106363263746381446396, 139007027940281492178024924249903662189, 267093831103266704272653649130077377278, 211768453622043734699607628097636937855, 50946809334695559065399131765915222365, 18828070189768848836353970731213993942, 196136372881209132577320425042793719978, 57025660541811237435482836729415379152, 180453161090641805175930352900935391067, 120117381664245076356776810895124381000, 25326425881100495270057775457806863573, 327833800807845339801116021961358797304, 218823829221568346607826359226391872543, 168605698528246744504978643717509474915, 228094217192396822193840774294192208764, 273073121483220631772042714708749138344, 236139816467234468627827580502739978537, 201415829315464676671838616707105277140, 230925711687252220687327981400265506485, 212170838944830436449930627796627295943, 268087406992103763430233269676475739935, 313787081593745611184883306161669978699, 205059573463895294682437129721107071458, 273553169757252532183738104432890907846, 56330813202802956367961569512884931267, 220770812614866998103380267389389342649, 107396981658911662997354170798536933531, 154025770574938589407038221481660739166, 52616141213578929835926162681717619065, 248568691862367907506924287669088207599, 38462813346498247404032900242394408007, 113530792933882320673806771078525621483, 165028016523136012970580987322905628871, 130511481641160838394610380105131038170, 302581974388051570775850488733111027389, 93307900456419029105059082125769517126, 189300050628779401157925007038214226620, 53670371627268492992701805255932654718, 72609967098188415746451103681093661740, 121137380809319502249813307544655173665, 84277614721389488937083174497624282783, 95054073205315588564087583680553833207, 41959773530357200051074375299466498544, 34836429377462264545877782237262835761, 17097005619771122091009065417573018211, 80274429771926993168446582776068088278, 108890552233951209353066683389420350398, 35090691107462637285261648168585235312, 231182701885296114803919863124828079272, 256136156912399819608144725483117218930, 197645425956761755533277889654685705002, 210332618826288356292838639265353472105, 70965262820855711530067569125668701101, 329931412010674967861527216601912742630, 216720294626147589086113914528370367277, 58817205312204391539600411588348758445, 76319027189977520687047133165308091379, 287869438440949194697209858456962524839, 302924482277847244946667513607958719894, 29372918499150177897003447892751928556, 257183627800898104367480377935770443069, 221665885760676798165761714541748732013, 109606536774663571374057143603179653048, 187400826886299429801428534866015030558, 302117246283532298638740336698939633009, 220444024013905714450623177936627381681, 16077500824678444800148507790158116102, 328214715605544141764787379621240555690, 322610501927961248101948602578328680969, 308957257113472241969518654284207190571, 317349964947671924590939574106429923805, 244153072312857923291730088336859911238, 256286705057698365901228097177401541656, 193978816882707543768658279766854093123, 293427841112691351588330607633992198605, 26042280871159330343683091342391386740, 266747311155464620507412283115946214655, 4800620921368325050597047433469997809, 35310286038099320933218229088863310797, 254983347834662463280201665736998910902, 131893917079356616601307160548663732484, 12365405808347655607970997951801401069, 11746675498715849598802893241378225863, 29411193766713188090097451134778784869, 335614276718358006373321947685524477554, 308940393555671888489359686086964892476, 126139550143827937392083618978012073129, 11538448879017318937619774166129090773, 245862896217529575880018683615822041042, 134050716391304638638191716773022313548, 190354220006699591926892149205543754250, 336742940997928171371984756384844226425, 126556715891779068763082519574202734968, 43081815520570800203294184465872220986, 97704452557089299596184159343982777633, 194860667271591463701928127238376205010, 196426460843836916258615857230066917022, 209586526143420326873815580355589070389, 296825709389387694021862448723978895612, 115139032661024407814550902177565337089, 214013727307837619562563075509344040192, 268051732108613259368556739468962232031, 330229668961747991389215062865761956325, 105309147478466243346814864281615994127, 56522029027263774455331743791484039174, 145394159557541857203306641883727874969, 187919134186040218421230432782126331116, 115730006732856150806877317813927068599, 298292610179249220235049563447193486303, 36218632594746920982886019555176605741, 236053608121537846199817178199729038188, 126751363242631768067356562642823786118, 200577408756724491479197675067938494470, 58940877233289571999286254409726038263, 141405616494463988976871930921731589123, 285088493252627965216905244126453805333, 234368201005260881538337787217353720896, 1637228717083421077627072282230756725, 14743453556478367769262812236705075511, 314296930357791604178970491443311046377, 85579224253801068117818537892372115522, 314362995945147877251182701559990907050, 268729781660879105733575422091153903952, 140557607569422088850050204386895131833, 291832678875738163834242904405916240245, 242676884806747304055078276980956523416, 62124224920572367927649017366359464045, 68368467733331249467130409140387805024, 51640015110439543687540052561244101284, 335098906600000335495526728734764374398, 78809452981397342040502364089517878806, 58060681018025610310318510499047184935, 115345233850560040191076937707674325280, 317693410675625326693252618350055615064, 171732891393743261883296565648521016668, 285759908971019247863876141908463846546, 163192564485263173538605551450590085227, 189073555631044609417397313026082188552, 49759869582813954570594802265888772938, 83453586374588074605659917850954344804, 156066907143392964296032191389775783712, 126842500207996785023268235221933557056, 47566622808621660161066175692116346944, 41718128806390322188579844443854447782, 98146427186276848773408195753965976314, 100805477355859878161012125586619682062, 272269147954251045965652840105949440365, 6172459482442893465784354616362155359, 71916503475460973017486239958635732728, 303007681429108010670835305518266508453, 154661835462183641801457043651966365081, 183913991742195754004324130845140540612, 33766626159787206573437546001021636934, 104236399668304122644466195237370661393, 334858512222911109573198989904207793759, 83103900483611375053098171122276774832, 322488906320644279130504666811411579440, 303055187547904246235783154654675072097, 55719330976237276810478378803760805410, 278315194245751678966323117827513251731, 32610712853579043908863472521144448751, 6723930194116986913959141176917916727, 213936447780779447902064830893043430409, 70746527560214367758596384649273286751, 121310944545058193625237481549501481196, 110154241369756672361730587519789649305, 251723525098214888683563823305377935921, 64057377647920049075416844233935676594, 235961813933943156105972838235953857185, 331386756211724179614493148921871548022, 154189481667325645808434013734609413027, 12553794138361367227985090091318644436, 63424112916295684557048361986850717161, 194299612569956944789632944945926541728, 81772734234711024201566769023161387088, 334326033993415601318215130182642302961, 284936043717034906146517152147064659279, 325668912679132271583897353256140613103, 96054788191391895588118510414053595843, 259186150090754961071530080164545240127, 148987084042407770017841587039941708448, 130683751695774090951079938719314854679, 200973846572532157494570045010155617402, 298006963514415862849245966558958934665, 76484510800653575335957260851810748598, 137443724276042162064844488412716969783, 182693843987622026088503301303552749646, 115120888476043591856576268113850618492, 79002463618789637823196974846189077928, 311975712859099951848537344818513154758, 95462087726461639272854403268749793290, 71841788501317423023668776872500887271, 37642383650834028883666866259542376520, 79321950657278312812317265665359326380, 155966435125564851779552934904440191456, 52562229217424455709324694909843058757, 203977130564961147456206834690835136268, 126821688327873627245668015172048384562, 223408243847789639195370391372729360030, 164081015690667136288560323963347426374, 116839185874505687528419853381751718476, 333942912884084285591921647885164791811, 173293158053887399959352529737146655311, 125246386871010198602500705984418607854, 237636767556053836392072333449578903508, 248138333203460229704710672705302257158, 205049786552079693259381649886390934898, 187657355576946891420054956225650016387, 10552805804737481662893512825831489680, 126639289919327025701274639323668583543, 126404965830499858890017983759431423785, 35708506401637773317622013247437775141, 19952223052779512895697037159610983396, 319746762868691990072794053202586136826, 189743869446551845616559792518104304513, 317802445360459818686662116838021651201, 130524623052285013617934180510164505379, 11365893138540763041226455617229967755, 265807248115126806156243493081411623654, 212038639582043654165738962809721680331, 104673935297716313304634900027591883711, 158623261226508176407960068962768103883, 210928381222359838594258900353126581615, 123543159975620602408940371467580973478, 131870363351339011340862114530292838650, 64910476306325090038078306155920894983, 9884089935589403991273143418888009576, 86848367966869031948622523456860846667, 88725250206309025836400914932669935991, 56823111566564227568078982283160246875, 282746706915258025073951122078877870841, 64467077529118119617563414477038971986, 71252939641929055820707449797591928064, 47725392766019899062143328834979484813, 167331723130407362732850696509951473187, 140937277304619404079666501573886884690, 225798049247993573003755778225049112778, 158909716002342629479348773260893483806, 148356355683857303755133161100423714155, 240937604880303173722938389662462500965, 85867643257642772271863189013249428122, 291040162940441928007459380701833322732, 288988752174215392351803444169449643919, 235601473354463773667783908731230519064, 326204185963809573723061644780850292503, 231388675136707670969948101902838053883, 220029188542973454930664483993963931920, 285444147784734718771858204383983488632, 278547565509087678449346612061304163989, 200684591582044745527762631377278852955, 161600333849003212795450230978259009100, 333140691935105520715968449622110739080, 268897776562420153937205549818415943219, 277425752900196208702874547350826133602, 91231731667044154037667786520679489436, 226588182531957216610123976031918995396, 142749436217500087591187715042267522279, 44733869913731353930494009015979791620, 241313697493322784551228815295117515454, 315925498287446936854769991680536913068, 175556865172618958751723130189536716443, 74396417066512775931873228679795025835, 81576148318048019386790868366693782372, 138101958128690628390019925850675984970, 242445937955894253143847349297245548678, 173263692164865955775380928950499636824, 10474720390057486572900552242852731620, 34906237912817604940054798945454681453, 136234109546055860349297084695927814233, 60493671218031215982185312467546618911, 290721197563025976874262707176010674132, 22460637070779515894911889843338604909, 233663115181423823773465398433062654089, 60445007217044377525060048699161061707, 177864235678032874039598725510676148340, 704266058275694253186653970056420128, 274877533784437834330291122448578032612, 55615005715166051940582396095795408353, 9643864235051386951958897569045285372, 257516758818559606037585439265175677690, 324799178916121386669346732980213734638, 13800552369172904554448390590160940022, 179395969744647402504860344108655895599, 116116741144776544314616828301663387917, 129759204493067283163852895878497968838, 308825861531813599778168816819026835485, 142070892309452781915251134205177199570, 218058802782294810982237927391533144461, 125671456296089788247999759123706509876, 162496123889156943206918834457370399880, 253893166908071493383193546685976606405, 177987559384012824130135160561195174236, 283480612619904952005749866144291369544, 81858833655221145772954124443962447372, 244354351167227917722183745798551477732, 333650153832874369103155009755206413731, 208452051623976415609799047369959175010, 305065453439295056772541113071043945875, 263591342210667902395131678471319021851, 183789064245084830403301642038671821381, 33217367057089340961547861195972619653, 254883049675735361019594510375943447698, 124536732259420972803194717357442845537, 186466183081056310101362527193114407634, 37504656554599362063215534977472138513, 292149652089131675060773257418471619670, 232262024474681719985912353512326592830, 319438042230713240392417381325500357461, 192916330719146893292073136485904450997, 218608766034664899508712013817246319854, 310973657277058095891258248864828309707, 301986685185906176626926011038009374370, 173173601564811347410406820043838720426, 242765086634679903226525073244075593109, 204290838661251357104619196179388327163, 88504839597379575615916996982659538593, 23282608345745427657290075889375986350, 250562044495034358166612634467892359593, 256458617061170371100958315125509182148, 190105437077640998994741915005377201480, 182711902733886495350588578900880093124, 16987001135569380962248090691670020937, 146192998105086810747675004577556783412, 201362104189716763974711394720870995358, 28735476347851953791810074670067723589, 263486800734229575139338810289570206845, 34381708398974158729076682093130940336, 147670491223289155637945471601817486566, 10780591192840810010998451546504311812, 178519563742880841472438728548337313916, 312199011618786094858879527096920790478, 59298972975417327041764975054108114894, 2268070458691668468814332841179782689, 35345928721608942153673542999607094444, 40161349416857694809252509838677170584, 59342620926296480572857545633928708376, 84270930515788682049971935674681468180, 186813905347279065762405032430731324189, 293228303264388655078818384571262436813, 191795319289755427267157776963022495672, 260573595192247244381921340538705315094, 154490330897741174620710131123346973415, 326322644659540712456436589265004117077, 281052738490909767902390570816336573881, 89166886553773359947184490530258307528, 340219098082577177786931488218230751621, 211436519689794424186737486580174826366, 86319998335585493007305858166571730460, 244438669042335696001253105014686699487, 96782598795134523109849039619705197547, 317620603712730925770478604166171077631, 19559726222978310753916637988581302302, 160759362170246955703736418096337170719, 30527260788050616289681543652932686809, 91814933602960193599539597768191592006, 296014066764688575910574977654899957640, 13823097456335697539317732263125197841, 196496885852943556376816542521149378594, 232388028996280658221030161121329714093, 6272055794128370507743012656274715121, 329807067612742847597681876628542773288, 90833078401924279765474670399099159576, 27180461625754100723025850050348320175, 121502181423505770825135455896347586519, 58008690498925979692886585196853790495, 295808505703966504644602662389350344948, 181497257579692668344961313551759380696, 17000019839385133501976730160066944521, 323071128052529765596734684898026687381, 258026299949419286022729743740326473851, 81675300599156010898623791604393656932, 255185240569254368256525021670017825194, 72427103507152981183677418583036440457, 218358640532806672590808581581515146896, 338356817431339542760450762172646214363, 330265021318683798317907070030350153170, 186529392242074022087763418410188204647, 239854862339049068510629232429154508046, 2006223558212057829611124474811014869, 83198417545791317907693505891403723701, 153401258045208846789409462091678646280, 340280195927524059175118886757296136396, 267254928957276701049660672850959177052, 323232237858431385250155624891741169740, 1559445062460324569191779934385323512, 325955416636339200038003197985939325758, 150509338281836889057411872479616372264, 243663014460916050671218021038652183759, 275912287879220318588504472474556762395, 178123532540340317099110051776736282975, 253288115973418798125317169190963573921, 120801126811473603589451740817718091309, 303299293937100171437396218635489198583, 169405425459489602383236114613297029629, 294720374229119821700336677306607891589, 51027099202029968754195151614313866532, 305444103539692593580510298678562866038, 161730994344622553598277487240714321324, 138636926311751772010100240738541685447, 277971950541388060735971477146583363763, 123440839376162715443489543756360493262, 9277535206906238930854791001040477345, 259972968235424734915152915769264640192, 129733227809786865900024784663583549516, 337781767132350317316125207152176631414, 269304026566630418279158189191846787702, 100861194886985895829022843354165039630, 315600560682114092680593328772906629645, 333056735730825040949613128348323578648, 288908081015642136406197124516762215035, 338522505921968327588766141023450901522, 167740699890111432771690606813187748567, 154567130102807645728119218155630681985, 8846475429543672553174743088156117365, 91676458111329854778593970706160439323, 292359930678536804310326309468164385175, 46809168830615230038664172254690866369, 270973378102022116920629973571702420725, 138363685673571716886060691882928805919, 155955738666044676259354378537231689862, 310860982860161050793559369284998874301, 262546194247968394976201609216336492257, 291558781372546274211891295124985513609, 163956706291767291730444345239110446945, 140140068264355121290046287741167382838, 22635766294973647604488824122744935848, 90393171095093683565590776139692947508, 138004669129480385725108443557007933936, 117347440449380778129846192014906813449, 171805734956661907927535520426543376282, 73231493353167629832808633720288518162, 95132133842534946104811496702708766721, 164597561631069737565567990106155394499, 187144933627167756346160123376140690245, 72457538448889268086153619767597367802, 46870902581257894239285701402207669491, 91252433672944554178407086535522723014, 158628747023095715283799587792760939473, 19571748871319989148527274423634761894, 163587956306987439641319856250083660848, 102292096767435106735617790970400407970, 24425049852623606329559956811459930243, 52266489913644635153352970746025634627, 289462542575457119994697956756897655092, 10964740150651172664041687054458370462, 271115075448446492090462636090686721257, 65196938762074486036604904048883613516, 299201692886084240999576303783689700916, 121508749692792447660023181154909693081, 178747090641407938222777659283156982292, 337680847380353273507544344413210609014, 283235805343390092620127197046602613020, 259842968849226021059501776899642181507, 136131358281924754287734625457527619053, 228846777508825415882582761838514465735, 196916659989785615142520262640111313071, 114400134203225459613294274360398367334, 50271330103079637562764770539675913265, 269169635583479684848134517806208450142, 8687538190193750733758038945083357199, 78760107538372589899746748004639945853, 301887170695627360196301713351482551393, 188129729958703362599186260386562812206, 75606281185778280411470479203409574474, 327250215213878757361888941645003484255, 212937271000314951194838670679267198909, 29274707877781807255974096978841617767, 21881351375214990626303506616314745046, 48830770904694703780155765396200914151, 12527704646832103637166689838464323791, 291001589623879058984489231141814236771, 115448383046842786711835221391881576068, 7277784907841227139995128583201248795, 113903496524495498620667958236176227543, 286935876984844969424662949143631654147, 148210649836848098921504953203216653709, 275099750716240535766982045863533126852, 111411309660096413173083934045083525214, 98714064141617140511552048171326165411, 146881014158774507384440148425593714328, 73747041304738289923340351020124804969, 126366665289920261385275104348354087529, 310671997472220073520483091242863901178, 29487911688750690326381874085673936599, 299063025956473753063285480685564423730, 93111955874325789911316692175162691305, 327120422621273651279773577568780178252, 63247839845667020577156498151827596056, 103763061221553504575516055067267704014, 308048535530937349469660454767919820058, 156413090012770319019604945041166693806, 141734212681513307205209783544982725256, 136566773048399297345444009698574697296, 306442837940417590447156851850691251160, 55784245464877220211148828410524532738, 61616235041643374266491051877477439029, 68929197962914268784061885718982532836, 239120017583809500964985029960394971058, 95310592646641821402851541185171694671, 188991088792980889166597390832175884441, 200546549358807010587013109526328456886, 100600432991351901612749947422338877956, 99198873675398030183641111023261411585, 33513302268826380277066451987950717276, 158917563628389680705100080357568285717, 170313740183174955549572938381604721506, 260949953824680945281900534271740753708, 209214856067437540897565912123024306188, 73206104345415816783669989526040522171, 115195619231439822401494662055471364517, 232454530925752071860866511323733953311, 251077376604552258550225906659668316885, 331012915039553090974317568075726183998, 337500683378518387117929051689708697515, 289662640539480045862724968421680488536, 182311014786366823654860279138614474509, 36985992834564161821921673701629998456, 126225096997995260468417751135390552255, 305470094480160391235849855390773967046, 118279073723176806246451167843397719254, 211315665962016502675768038821992318401, 40749680791486152787981032899405351257, 179167135797210274320847441529202738278, 42161651962864202579594437587586411777, 8701468807892382831665713904834308570, 46353012815213197736364258773602547393, 100618367687317464016871228048062151438, 252111544637166066503964613225247573843, 340109486361112254208752306341897134042, 211907798327649084313556548311884522222, 19146884977924489598738254610493862747, 185727778036475148055828911161151830991, 176471208471231258043322710465469455679, 63897105571044469743164526049508481456, 149519758885289065271674366782060923596, 207489688473571324998053629722155823846, 36647760563104041905529501730911791015, 88782627210718894084080353392573881988, 188181562018531431653628085968185023143, 314971558554096274916653311332463424607, 110369422119786185396641046965057468040, 186943169384215852305175172690903509784, 22008345009059298923527738203408563415, 233018245137750332697017866265133587906, 278383055000790140512319988898303024454, 133751195164022319748397439112740413607, 296200523227432111381008134227585792609, 246784460546212670100651747459297698587, 25944229544899014375413344859358978637, 303183644669981833985692799048875018509, 306388758130799498851733169511451253233, 100008490067978672861020350961577185721, 13684230781064206512078739225854272526, 329258873717908040616525068019967846502, 260839051436107813809005748089578701514, 189650656110838362130998820342826620972, 71480128124649454938603352059143586035, 52205618441710008360824727039126819518, 95929136980608046907557149533697828474, 66558248393629524441215700676422640223, 236589980486172605645502221088382707797, 284751496085516509178069939033251306524, 332928824739803349923593640684316591665, 18837729644175623863037620273659612556, 88234361685323800941839636116786504278, 331779651902411431986274363046889900103, 62887599865093082259883544452987810932, 164188602918247427332575007585039884197, 12852540531060993593482859451690972366, 277043655713989426923231176000729148984, 82121780044000398302677163053696648992, 49524567241237069870149614269341631845, 89237825146207643973518436550445125472, 154511648384188808011805740186795899916, 218036589624740453005171455210192607039, 213685808410315060557596714241586509744, 35894105276044908282619786849051657126, 153307281458187865663423210650463555392, 13467760842839567324476518697872927738, 123381189911024360735949323993643542661, 215475789051296921989104535841525111782, 239198010534211986920274069391930855207, 168361210666595651777949077930503133840, 26330570321071037653333024430760230101, 25811424710925294309900780667965206798, 298030182618183906499335594643612944589, 309572951985116651051646978998134953746, 307875798596362828055848664342306370407, 130313781459890992004904927717329510650, 59469564030833793771744069715600648036, 296720576114567278847617194242338164440, 178253752491732658871794096963535436736, 253814472325505712431264915031954184240, 15806383686833167477758652444496001544, 21083083211483825851587687873902397902, 294132917959695991267082338799707674909, 228496313881032008857805815508759879719, 181454467508788201014709222213044556672, 243552333298687023773894935799628899276, 186519445740617665822681956643713784017, 67774815617691610468558790674595782394, 4416391201888808847193766630611409068, 98591472733399326652043216178824160208, 263945554146670832589598661538317243323, 226054247602837964097123485826391271424, 145421631616468085876455150934546868339, 104310195376362064993494656994362269766, 305215134475337101349329043026163742003, 310878298320068341513531591524605304880, 145853864864908932959175805392410416325, 45366214428966627822323375955297860, 215960968520066855209572689010472253911, 304354399941139932014167542971853369833, 174444927308244625446176561440204272782, 275194499935704415267760929901801296604, 159703534702816877056044107259105600332, 51354662404743819928033164964975503492, 54683000689861641253794928282755996263, 135759353589201102650539485581848561384, 290118389754467390617097116257273101635, 235808435803313109860050039453582403480, 145983103682887300004389085503100317804, 42927878540148257372398681390075411107, 331821970405576431665171316312526867828, 182940269447157005755518182530113742861, 150700229999202881767790166672535072644, 116323374445528063059263424409708388171, 338894335664284176892285733340604094867, 302484620692152525531198792479569712689, 324528199312424767297430568901308645993, 3562519930061745050030103279132744562, 210746173347362654095001084558897037758, 203984961537386938790401148523278078600, 202158033098441751454135522648811340981, 297558351090977497483035467816947701356, 101716197070983854888837714804111406342, 256572709622215607805117775048761638683, 53306404959082726517397295397065936718, 162543942111028691241561431073962693662, 99266029476653980527484716333191070321, 79147145387267168879559253267683425078, 337713531030269538429165349145434661429, 65371510245978012232700279053236904729, 214241827227734030215473900502160852950, 111422277921378092703846493552015622375, 223630588190073733769858977394101790426, 70574103585337884746729728208699582128, 4175991793031620392999990850617520548, 181761325241293190308588955642234929879, 28865942364976759979448762982080158080, 322319341098136937809193878241174730624, 84174290737716364161665511924675212782, 59790497787711219396093210965504867098, 40310232641765775146845021925148153457, 297972098752522370875950293327193922732, 70964721852090266517426763254067588825, 185227276618768731867662803903311553346, 56778430046643427051723180970297679832, 204839455713202772133264308743970467306, 71279938868227873524056706864926544245, 274449913865366179774231917942959922381, 115522678449116908843080974636442026402, 275234078738608409420769859767387805710, 316210712601303848588370508335697956897, 260823865306913232356598149820254243671, 259163472651322241178011654747614249424, 21953246145616287249631958256422329948, 117550983433541856815419517128559005215, 333965343194484941730298919496491684499, 285560273028796512533585550305742468723, 184134185950099968380040544905812597682, 76276713367960873822619011600667166501, 159782228135211391945936191977115169014, 22235468433601023856226163821047775295, 70676716165971000960240868235636377073, 276268244808099888186240236036277544656, 143825550428815162805937450329808279612, 313227974917865881377334223267631836623, 82888399689795789437860407517889435744, 77383237806324954017781534470850393075, 192361873498645011289969940792610034942, 202163596706017021948389108280112654184, 227736870550197739173451447338132389160, 139064868858547008567087502821071505265, 128913695326854681750816557168102875650, 26513157156180298722045993835978001685, 43082731463431159484192804072593452515, 142636581646350811338406989221749377440, 247410661780998725393581628852314017398, 153690240641071135664437894074380391270, 324424382125286123376858400828147802884, 81271564513981460668258187023637054574, 303915347069608502080523775064780449233, 235796724913086667118279585024651319410, 159389168760087179274521576033204766993, 143402847547425578541474186057270967044, 188737958033106972378498297778981101950, 92625296654449028285126214145032848242, 248360284792810911395762839991651626211, 24503351060180079886303998469219733841, 88351110880820289707124532786089939702, 105099684562064367201383518818893225617, 9537991787692573762156663161332180954, 304100209388354624087564629080465874007, 336772818368207811332142652980016113081, 241558860107610189338626006921972332346, 243403310844457982791423838042550927515, 27215624655424174177303915232536999406, 63336934177979102239255783426449269510, 98751555688884668368097369241799979868, 276836713197389705114740317630979090270, 126797725872028019371529385256818983426, 214272256496586854512912366781119457334, 269348920653315248158583198272567967028, 55001134450414127287988415611000883677, 40956917318984070867060588979747813107, 275867060122658547186851733476506589702, 322505349285798964612636829918660267642, 88161654855140459191184860405428363998, 154618652956859502445260807434353388416, 74499383500063374984937781038362827223, 143473791187570848690655004826879139272, 90856674109649173647396456649942916918, 110969121521547873868181843707505607184, 332127234033714581422211616575916387840, 74056369518036196463543947047570464839, 14213347874255851123522373195607561558, 335509443985250059104991198862465976763, 146130218428030709942257488031688836328, 310715032711313876543556719045064927320, 35962182076480281425782681319636443537, 56564238786950799619887825422090659572, 282304699936999941977251819342508551722, 231454218875819555899233493045309726540, 234752714147295138020864615091190873532, 12634051187880544702969677540284240640, 174975314608100989932521826227795468957, 190660777960329828770070937619128001786, 319413646810058952022152108076893046038, 62777094381456836464322907195768978283, 61616669280403669799575861530813871506, 61355884442708319555461005725407000398, 311182814906892192848681979850064193805, 164539330323603894718393611218402880154, 291153359862739125679910886146923021629, 271389932005932397685383202341317515915, 261638514216887955617744128637970799940, 64938986905650317223086482455738408194, 298422675315274376613140210497016005339, 334818547143712483858648294058298320478, 297839993263358837194047072343529475825, 66479720946410116335806490128418309966, 330075416525854117571365807815468187927, 54468484843263713819755611529561953800, 228103455647566200028211022949996943762, 142560984821129849997534605424108595542, 74880128977413189148540723115830935121, 140081399940459907712617600463633618076, 40218185293638411887333455490378981224, 318409478806970496391100148896817902596, 90596655286931005489747753254031119658, 245562733850654718151937750889949371303, 312632180496833340917680259156802699563, 56756415880020897514617205673544694525, 88860099503391790344916317078586484297, 43378138895872070556516444691278198641, 125397363581549281370102787595619275381, 308041615577133679919175060673690873942, 11939296285254446739095839945830385962, 65476865376476738275058191968519582402, 121776293938924651950338844156780524956, 213331320974324718572263539935249787341, 196524084597832280405549796270693674824, 8486810177456167334324599434515185250, 139202998678217466420647707593205062816, 250541444589901175876167419816989223456, 330897474110112300597840868627255219903, 317003806711420939613706343957866525332, 219001689196621173719206190717927822756, 29267627630051859574834122119479939450, 25334890601261153327323137831875575052, 23567661182258598591446005383938572543, 19337735952371079952781002465654580991, 282438003745316465265987548525022919819, 261476924838078832100787167309917851735, 202682545518659779030332712128631518236, 308693865811226943566201632965005624808, 207550291263634426464978651802347938882, 112524689495245769141998309901707821306, 245125312028307208345716105515965557838, 65731274207584890743154120703091883909, 143421307332865740040132128508402922633, 120450091088099551868361583986037279310, 313014972112226424450165601612798166013, 70245862700719942561351994186701673921, 159442608116397790984960182153755174824, 174648397512512688471113408237827886977, 48557915091204358547967738601005503778, 296704006246533376208001402596526227522, 270849720810430773862805912119512022915, 235389911765138660096788331526901831140, 293059188713733843764910436969021199146, 116780055364147444759831557942737629138, 86362996747355340196479815431710082471, 171864863731791310352119549514905376764, 56970052235282656488736692099860491387, 114089483506423710540840315237953668921, 138617901754026869575388129034549714434, 152737121418670415598384336742223436721, 73709166504876504054326589394648365194, 19275726723792100012039497932415146348, 36538110458411305881803150553930064257, 258181489586395722220495158668111494681, 197162179855329299887748711857382520043, 73922615960036481303689428290853428125, 16141871345682732228770066891965157680, 6115375148503558091878774733972711787, 124764486900260179192618172586834656819, 86453173382692282033591401896952176843, 81841942061086895113459762702701038836, 258043412079170640328511040156939717573, 135773163646817640499956228783564408420, 284716310952836927671696730177895806802, 166292060036605985193450139872184488570, 9353622685141763652514090341167029751, 179061625966446742842085733230058530834, 104734991985152193940317249896029535088, 69892574431599081578318819333997568208, 14008566967574808040865985319614311782, 225451965358957156976156518478680353026, 60936947377493993796456874219221497455, 256671593515817404139211980495937007508, 311922324244613184744915373339755587008, 189660932518477966382691856265433813780, 55569457449305340915079396615380920348, 134519365904292479887201510808595686835, 229322279614441740769462810108446156303, 183480059938089176428967618061161993236, 244662371790650317790092921047365826470, 176043530721517226910964642226911527290, 184669735978333841326775644404855805051, 165982717363800804888090038431144089147, 306092044422605642395156206700678488892, 77013781817073329689975029461165194726, 155133844362206741678599131978593975867, 307503838732634510149154486104619899793, 35293470663434575452641384134852452252, 243016076552221966535068135869524582439, 40977952141181875812729802233685507289, 325104054855435385137584733202228281929, 160740392180676474346151106797873050155, 315149502604840668945736158537360064529, 92938737025160716420820550897796369269, 253407800466537137503942856887659547664, 95800850836449079484279323145987136208, 28858404875440211709548936196037032656, 161235302812873741838815672263118090130, 2620316707456967294913105233184579305, 83186820605293358535746769396934246900, 192269146343085212066059994663364129029, 62515302717343205479672923339204186840, 289068506323397586152612146560136123633, 154801702818059363288651234940506973360, 40460190116093198401635489306599996694, 1971562600207196241238644818610628647, 156825086638247170286433320707754678004, 48690172332885255349284443065276717307, 172736124297281450954204165526487218291, 251401309066609344536940179309977565725, 122910623086048951109804777400852715931, 329806384269724746229272893047749159151, 159168799713560363866578002383489306670, 48180008223972657923561700565803069180, 226076371794981412652527133614137926538, 245142363209843810826014633612759724428, 36876410051317573484881896552149357256, 153447945159598286228864969266531636779, 185010643025902990284448078012387699749, 206231011131502631777571891668608496669, 185675293601213120509001559604305406632, 27208923176821497869244134535669053517, 211588169372991161369770504200854494141, 208116500794420433989695693089107859101, 47098245428885713145164906415868442251, 217790476949970578125345483348159398658, 159915509398761997659440171587545749032, 114494262195716937834652952729337424675, 10073459565828422306142063842588623647, 172952484903152543436788704726934771137, 299597197005851785286789661004869539614, 171767084044015480587181687417244678072, 204328047814651669519483887213067735906, 172553909465812629644297457803087942354, 84331256691608754935740972400994865440, 185173539387529165191438548750061365558, 287851123211557347123796810565665929354, 183978702646109518907957737905750470583, 233404528796923529062730317373230149819, 241703648768214261171014917465390513339, 211916443298182531415548524154608152641, 330434072886741994418004046398943871614, 66012631525968980219549824257308288013, 37347173867591133178278902747545555683, 16868230853579364630216214165403334818, 201381605954524106091715771475695669689, 193988528371643484765627437747345219832, 145988487658772185227752639777336549574, 125729151223441134322111307389556463792, 69062879130084900532362816454169982043, 20085033036884711800564240635442955649, 338531373997010809374470306750265350887, 207518759153513973556104236014200605147, 24650908198193896685377355339233367770, 29397540536054951167132920953707494213, 68793969547470724555213866086333373192, 111433763965346513233485044841933882837, 13642181176028918699145466750986870942, 234319140362528363540473023745470287160, 167820846476307210363165184011890491185, 106630217178341156795522279999964975636, 297589857733481074920372791275918859011, 271739473979290266823392049349048321696, 161472241957528993978349771557683113250, 242295851570177207209672475491793602842, 219908512228060711215212977998527451911, 100833175357723897896166486442305587227, 135160884727277876846116206438641551860, 319755492575166115184934640790728172690, 338686348739735068567654427582743636748, 237285654421446264193902254335008393040, 174871712106783996276402830010384809101, 277679272281100013021956548323598558501, 309720638416922037458235424468164754190, 116172743297957665569016282106763151524, 176512751542707106361573773709977837193, 192515458529078803686474125064680770180, 153733689363088421129178586257278097714, 295018011190409334688596037759338338455, 201225460431028185013277069015406227018, 101817690884247773698040019441161586065, 215861966384397301613413676156685050688, 79928751425276203597116957971237652518, 81145430724810017929319449814759327252, 285894485870501988212366410443216661796, 70777614835693282775790934791212081696, 174822184913525332369945147866745287552, 167222050044418436462317949201762616262, 127508093665283313586368409697631425074, 152925094977528712334332827420232177698, 301802538700448063641688217288968838232, 318132432334217225464629913182063141495, 232971979410957334540699092506027439173, 131046315626080349764753231696277630404, 51349206605591125001104641743689821414, 5919621148370076280707400825890229611, 274377828649736288701691198132626396337, 246826096966278091555148438880075554068, 136490660597567617797223397447396753758, 997694382832857662172102330771820674, 173125560399182497513126288794002558081, 120751840250433630915707034621039640573, 75785389912067055745438981767994999142, 181725797734775508481270971297029059713, 126889596866381318122317988938594917404, 144754958247828340911335853560423775056, 284950496899465249402325612449059065607, 252083317279628408786273104844722227758, 55223443470393317559956188672684036017, 323549049692560859496958910381716046819, 274021862129808835613097575445081454863, 140634943958852668088586828370102157646, 200972995271472291188670016257929628990, 206725671493826174507483313332724440771, 36117165149433113284453818453115319069, 302424916005547770722104886043253530015, 32436928497325130967460502747375004574, 189534309375944240494838896881539598277, 71043479221844873488850332584203641645, 127773340314404643807998063866349491282, 82474450278128414888611292502177345405, 42611457954024267646380329882432833585, 173652113566024766230594784845493449148, 324321264778696087436430922854613940353, 230461889837418577262784990446287879947, 153428665560416593437532922987064690325, 219036088548830253200456028130475065469, 274191860925844193816013448967731722520, 338434982518544494228423640292432097010, 297686571365906788197453748419659128365, 248672696652815415525328859424521691418, 273252529002858022360384775914929062115, 85702967889048622375453313216457960221, 20669374735931792864595538940172958264, 133552580939466454184955599592304370795, 21598703189448122604821131009468198183, 222039670839116458304023309048692787076, 309637570639680393079322923769636777079, 32413962320083513182425537360683380325, 230389048243786850663988717469335116778, 114130807813720118537939275817847714915, 300693765871179162572531363475882683828, 272811392403965592712520096106992974742, 5645086578812727494939253387473833075, 165556606594938310395939708866015663966, 48488322470544391134591976069700171280, 174718110307245729812870369723614410518, 231519062214979981775589458478060139767, 309001649487517561808359961209728919506, 218039832791108138251731969798060635416, 228984108966136895140475033555551217243, 285218649046825897930504387122482077787, 297467601262722903324411111695356420228, 166032988727147892081343356183933871178, 64808575107005167466521538489591104725, 104550918955431789074056393348555875825, 324642325480228734062722244683084740802, 319083077694353639236536141676027677310, 168919955944622895400630392100259680774, 206071866385676440236201640391551634761, 337071118678942719796015596825668459741, 187381584835528622765909555769021075307, 325022438883591280518614079794802264925, 221467662669734378256888082259506765273, 66084195662633650219971773094427707341, 104946981591014594212934271540201867909, 106718539889782872670499959622113813580, 51693931095000011532587944359990105247, 331541061530605868712190178707329497279, 116634872928551104576874816952029034150, 130794120355306827066124994767381823540, 32202892750428980039274254169737652537, 50529299381019379972816807651649561817, 8810964763898706498896426907145127348, 100422584510054145609534311722058737008, 11769905411974282086502514679684503653, 274349434025505616292234335486466082211, 89891402290216643822125319071397226163, 305593797326380609068045116204590558510, 338177271970996777547972560774347866244, 303106237070978474391090031886253316075, 184356742064536660742446633397403905681, 278434336033456615326426716616222071032, 128751322586096936331356868002438525377, 332233015099240170102961691160661653376, 176181906589613347765449365719612455123, 272490053425791988658724699485338135711, 263281548924179596342270185929496604054, 134157093349688182948195497896272112282, 291561180008806342899111124301189294018, 202663335666570005600127661651802450307, 158853478027904204257687158622032773107, 82794511320195153334707041603878622002, 3158758610503413362191033273499657640, 104995540922331221028641971967339650191, 300524206845395344784381867790177158460, 304388443285443686067189222058981947769, 564943058335191087345631094931185012, 247555327427964625720861757727884060022, 37606020616586452279040083314287764349, 270790503742423022140784826447528670385, 258999375054196759537484782848049448943, 292932935591653386788983357021272603877, 39514197034716194139383675698029342037, 177322815111356677401385260888073498026, 117055605453847456456286796045522524812, 313169330851742736155024841111575412452, 308331435101469375310131392927120737242, 80277193321378858159226954244210086691, 32212642468532193353547347247451324281, 9174950450135874774084475428063973848, 110247535716960085672122915078494802319, 329785577784113520008415007842186006858, 12149322133226226512798009540679016870, 278465350398918730853576113767077687524, 165774461945229226750554774107860353795, 320362243673161569650891317623467044780, 328545982873562948143456018329666513840, 267112469199549520318394752227709538420, 150015121442671483334684247255342873775, 298974806535820584588096688277298443302, 29447383995295327581210391399753488063, 161189161704699278078316756447096831322, 210502403350478606814166130926374084956, 292901932631579770417363309539507647991, 204777072461535956051536988292359117346, 225574670437699738522500093548056745708, 327659280757922842250429104363722277575, 159434419305822358473884327979230659609, 173871845325985924824712399857477450196, 12277592226356704623842133324604414159, 6746481189066546088826781672171595693, 38677722941900253828881999672057757872, 79593576955557717821027346268151410492, 192267267995292281930861377162606046517, 183366593687495365761507501393565311023, 118916428845532158179936908868255983437, 234709222764884900429000790031554257106, 66982390281932139908855681874477543536, 333323189021266175448984673770805033150, 189170105577736500773756913712420773461, 293202865631086741888678357282306514164, 83016489251432062783138603163421659985, 238141572956574473173977926497420487051, 38890089634162135235686556044500973491, 333528984313350632515678691225519010706, 147508934483259288267914973063018568484, 301197051881225373750089571812540766120, 105583245644421487343586043200959742804, 249777656736019320875647661839497138133, 235865380086861082124433297267504802900, 11237473540696104784531734731327573815, 169511563092916597572130303231589644624, 127928325259900390328651514190567001408, 40366283278926321973445553903210526937, 53039401135547856419395346162439247804, 227935081386572030073587684878193338776, 7056974316516372447947494247832637917, 39174821271987568431106629377605859677, 288271491570090813509364593567457627367, 180780563623414996468559636662846571551, 55177921388553868063383724152673150898, 323614684129379833984019464291482502174, 32377842774169415660750562482213907346, 207249685353811612917371858248061084264, 301679167372196794319677718146550523981, 195423734640716668890150563726104507271, 101606172575359412305953361100790057536, 111392155187207034080870921322041728893, 300935410940452322326469677308533320260, 12521779410327195036192902171503886104, 326664197159974526985088176871528075141, 201888115052638110412604487580355133720, 189403867515259654419417407616551040556, 25900934574277324342603240284835008785, 33119634783604117058015117646109710325, 59739263518821790985216167938896678489, 219015766927816802041941431011786972594, 332754665143309738134828368513701386873, 118831967197745759894294773596586765586, 83972857581269528628882413936405502815, 297621352269372288280226601567630628480, 247862081631900479808919735786169077932, 211383626618529870577932565782029803169, 315724619257165113041235359587476376280, 108644622317411687781447199040937081235, 243594757618087451839045934078192479595, 61505074988907868076775924954880426429, 153979923918258285727279649073358213828, 214218583684775096863428498409897194419, 329705139966625841551148439093981806013, 327092704001013284436254027465287412927, 97459913070021904523862360877646221150, 105650387857417201511488670083250820339, 270821822839208039051840379488032682881, 238322165824244252579667979687802419013, 252210804324233014533640002085761385792, 273686725195636327076711500891528936163, 233594531485614700969475836346449812059, 147564164698235568789909099694714694808, 164329653633637287898045085334131557400, 240382199751258488141588225477565538272, 29978018516044167335503009990923910162, 105638659525020337055433338972948287643, 337037940379646398018541583173851975620, 197801971639760501228493682898328437597, 264838600079045302763520493137964852685, 317820860500041199313203817279523290275, 39757264203276991463918007761531402141, 212190808823359729549921076197863886522, 8144129934993781724154786661637590051, 172477255745089745786216045169628804071, 246859636222318325350209941209850883034, 34576100329695140189207509512465365517, 236738324459826862042190961231481605287, 178874874297431424220052925166123797733, 324724224594608486561375934640945925412, 73665640943116863666793524217927246588, 100660689429852336161860114641711850114, 19902852474154482059076455725715726938, 328976890119005793687098990576963436964, 188890321013451529847164127712083277384, 323339722280429740618162786850402866737, 3391612590185891583552737931375230181, 11661633744839773077766937157584971704, 290440690810135203252778310421532417338, 266325252419505707354076714145473755858, 170902517086386795585015465303162553687, 118542512842828537783406563670124476845, 244333725617138718817255019618848295631, 144900665668429208672460766456835560908, 56117977038884001594193638738182300602, 277192804912519644879162990437014765825, 56925132506674969709087996208038642783, 203545514049482114010613119362326675073, 127495916692797763022443709892396971092, 207995806865143706210723823108825135282, 300565274021234494815183241413035968044, 102384490788091432004107270018295378008, 66492852269079955664790577864320670984, 232561422507091700321210237533590316501, 283611378572367849080841359300063283349, 104347775970295066253140583983579222880, 122178505053897130183853632505773409423, 57751082577078271620830836464739012532, 298558491819740162794231608457102808146, 220321383875666207457517501616216660360, 171129350753236170751279980351093849169, 337932983664411638052267324117838137597, 26731587193467864741571169731934371533, 100329230502771711289346989222335612339, 252247335297551296470120421431049223595, 70740732980409085586318703839708933043, 289932418167386008085812126518442071513, 140287029712679332127802601330959904125, 94830532072203256165898479039783724865, 309694711283756338471612244372298118137, 38165403568102365434292972849158161615, 277763066820119009213714178622517081880, 284385394735670414388617677256035361276, 25401527531020164447578502564275236613, 85889555008530126676151418381737969032, 24895629604995962298060102493547264712, 317145422794709109959694486225430691619, 217906004694745832416122250072551965520, 256072389032288059834034938084109028679, 179008233264081870047918159709381172777, 236969172615921543800211498672809622157, 162597333693272418830706713667161181388, 86870681681263303642710571271535827060, 331721863752549213964633580414445105230, 329427073904189396188035940834857537306, 100608565820077342230920120194047271677, 25304076850541302756587520054592493246, 32645939513111436140923737352346198378, 315324543127919117968961611865098384057, 284094752211376166702730071168039510716, 101560284694803018360607669271858571891, 96190264580841173479420443554569526350, 254598182765787907427880614237723748913, 234088326633401779096061339771866474217, 63609309689395946423690070927371634723, 139570262848591720468789112074995538350, 154141484246269700655655518132897904801, 118599759804973025717609925885779800428, 3989101827759195586643414539969950902, 164099193983700329732967740500756720306, 315146110204814986121864895563824365614, 290903771440036629932985449064014477418, 48810834190890066096360307374652296581, 199754786698696408218947922939429162245, 93567916485937204824733448463261698555, 10107884004647366307587599178461754258, 325875513147425352859718115630441798860, 180325959792987895831693948204718443962, 53764615342656554911748863169835448918, 190861606346515343637365675787486183496, 319247373614678154817315964945541950196, 294558960843633758912237037662144842696, 66464999886217295406452506617988161248, 117482591431653797881371915433960611274, 145334947579652973968602954800704015764, 10278588682739523061123355738271810504, 291109989368325353142364717711701952976, 168835252885164183524481712861512865519, 192275074691728790520588261868379635359, 141239662959373378280435727343707858197, 139299258168967715290646219024718688458, 2228125925015603953557012423008267504, 280736854057919872456547214935685519698, 41746461467478292082765007424728486127, 159278810062515433836668412689122281464, 124674301256758752468619445914412882671, 139228800567170025939251749459360754868, 211756867527138269358800019314691592557, 69541972860169861029889709558166265115, 152227790727458325012817446623231410349, 220865816648829273100404045118602167903, 13094765413489007912325567375490928322, 297671166996577317059503412720858988670, 316743192094130116021691919525131874817, 28400676034300798115559686141964211812, 166036202354492669928032738911917603360, 266520528142698792941787716159750743526, 221323503621950854715537316523818642531, 251025347263845213375908197471424132392, 22316645705462355735246089821710856408, 259528853547245995270952562135088855752, 263777565208563071297697710002734724486, 204545254308135576045132507431049521733, 288376035781186047276990227258468235255, 169611773716204575721340089206253357878, 100531951720384897144375449749501411527, 307621251510477614348137332944768565370, 63296356442332814108415146526536557220, 60212063671819685370246386130604798846, 113263866192481790536125649977388824393, 242678285792540854063905097021713886878, 17984988382065766031651120552299045623, 66682763028538414161759232510390772660, 22188922225020279731684682193059601000, 113895638267202608265807654359094957794, 228458346887410211530925128707305504608, 233083084504282504890188140662077382402, 255735954152858106172844543659693869905, 156316065243056233458635829896104285859, 89342445464436577982326764274796227799, 173219917281127305874993845273605517916, 53437810036185751444251145426364307573, 282740392878824364604751436357337649639, 150245275985478592484363192666595322333, 167544601306840212488663201598628567830, 260204182771798382019994118856582829986, 215927896525124229603933178418776807516, 63874073720726379936319195879428008555, 204635133119976350719298606351199656640, 217567427571513542403177701106569971497, 38645987380559160805228121256135975542, 187332210544020253654991253748073784880, 295951682363349435836010718802868619052, 256192110489662793081210512829615152331, 121883125288138776231411104246682018735, 311882459069936193650020063456594752146, 64451605575182989167842784332702339239, 69030286328590619824068203313263697626, 117140660290961467136590296996735406677, 121122606779000588972479514388894654580, 102389496883905714422555217260939322159, 78925766277480329383939277053102806098, 338454532896627619046152410973082703754, 131964263489864161560089197686726929587, 177289885084330414106598082932531817726, 149152306156437368564945304980880046556, 71514885673139164553759681776612474413, 131584400263632922965137610447778412288, 107061428250635938070170723257402079445, 107493336871183569428159866077188255247, 290342393006046649364228559171726402143, 213501624123306875281242354883273507273, 327400704263008068807084141742980138852, 313633880107121035872149498425783196958, 118513686957880397759012049746950423288, 286092808101335731608552257349973650886, 159693819735212160365493495221037699993, 14730597469672453448253753663053176242, 68625518530336865573226589659305581513, 98085936330878967732383199726050766259, 329101713338729709923565504905640501303, 304577642780858475327165645357408365782, 16731231910612646581725769931385440157, 138298755434822540216154131674869535239, 89904856465199463228918128609160114136, 146053585684807439518460384888951154285, 226903535466917675124249604976471251973, 319025848321356942421669115372141793464, 295454681228894157455429166048404962822, 131401672412586723454164410047802983863, 260445977589377051835706093071028635434, 82985483387865758083898356826507430247, 149819263165834670712146015504892929495, 103305628211900068191824860723485734865, 111709055374259773384046582210988767538, 169305055753411209191009287817035164453, 10406350948074872681595859585740158536, 291328995688123753164659038167510511895, 100608939458868752699874522391022289250, 248762571367123106089071332737526037666, 238121778044595419862404291345732485706, 278452350992960725260831471984140416669, 135357724929283907835785203396228503059, 175037585793795927076551498896292545322, 43973804200033359962113086970387372226, 255012875080301281444869505204154199952, 71196675314504955239946722219603504567, 270760286529465156597046896775599680999, 302895721465168551085247738028696877358, 140492560629013500349415012095603136359, 198605666467550206878755232146719983583, 256796962452260948787399765824342819811, 180721379155243618762694661629651584992, 87816152652128831520028843266541537698, 326222274003188822924643088573636776893, 237250079580176844353317127355871398138, 256644539025952327566019602634864329051, 265352116119588061306224832186380035974, 208009067302987877787049775786887786645, 78451604654075709780166468442536418423, 168114993427535683457327761794116797497, 77112814815111955876327027605723417406, 207160162941613010386423266318366966025, 91005677271061832735436705615780649702, 75716668352067659603006322271369814737, 143646850433119323881964577761599676857, 19101078104747711368930182772285070285, 277946315067731030076892187681651823275, 228983383614531944144158121487968372147, 50570208492102206693196674592644862112, 245022976994272695826080070808282177093, 112835555366271619988499074084708670323, 307159431579882987743329157943261695396, 73955633362140055510048009750156480331, 234933733204318690810310186675083924066, 4336672216937775729843097299081499399, 248505611350605443369411985027745378394, 100544535519598454555481841237141321959, 267362754096526237198196468238249924616, 173215055904954465218774008550391894733, 53330606270534418803415680384644637465, 295359829096520812642133012445808135814, 213242404916663831447534429677326126549, 318441920193809074143222839434965441758, 189002485296142975401594363253458972899, 65334224871316452346631646424117341590, 9647915874974204692283561743088564675, 299889793231742418450669634310514363686, 207625482205222350072574315155215397849, 166876948347560791472356376016687198779, 239375458894204975715522819569742479733, 50699949168843452920072905820431963597, 270230219901404968663331020893364528384, 335986345734583561122920666884576716952, 1522840645267196668630517286695182049, 183051627768129658758287211211006234172, 49790592916641426364803782850341914096, 12109890589101444045185363616608257424, 291553722929558523403289252622140571766, 79422283783736070159007851662793393861, 338120641882115016247065891069890956361, 155621520743042837844209066527254480249, 188847240672382246645572939116795568088, 308802957755790136463566992866264844099, 239287694127000288490557691071136361354, 97712310367836159500082263339115922894, 236341343376267651309683884639471492981, 269521564289163527644716904303327021554, 104408591861160720130051239288848989744, 58349581582525238504108777234570557198, 41066721516489889919275148258723507986, 203783229673239066107431322368659348198, 297648302382156375544840577589497167868, 122454776896005222895788682915619928277, 13481320723616371525006948404735899033, 146878415293673531473470537737273665503, 156319424304566048061788767291453217291, 257939502667738558810877124219643420538, 191369371217335170721789884682912240899, 55691649691398835995118583393566034394, 270473638134969782964862495908742532299, 30354774395825328435892224835301601753, 54648094189353000944993654899995911035, 206027305465801720985404023884234423422, 900231811166663909430427644458294544, 103238033492883502162399274659183055420, 305719379614442548418638538876805873491, 47807096445654808442200960747061205353, 49428874475855772145474018591671418895, 157423794336311499080836262907989156314, 164621328648004552155292688295883078451, 278910619730780755653497068324072070625, 154383706033132059941845471206545495833, 339245463311769057631791224173253364498, 142380957824325058248034693152931877686, 110218639925596543942172305758554239327, 115618289006927890643705182695723084615, 135202103178209920052010930943016406503, 270250906391761922773589896019345451936, 219686073143767186734523768718620214697, 167540429846042079550962878545368897767, 333417379202905915396080514023566449504, 99944687983781790223529529095608375857, 12915827378215083408846977996086677004, 117104865710141198306415765512284323246, 53977927633596618804335231601756009454, 48190633368711177443324229128897566460, 105359839493096525629443026318440842328, 139748291927022508692848093127041173617, 186105052317892857052945616329036143031, 44782497866874966480900735969103053149, 229707205730053392909502243344514552846, 255428542725363860942367608161046333882, 23965953443239477759416980638542701351, 280362481909712718595387684503300333182, 195733121817396714932845644019582810276, 50636789765115603518273704205683169677, 75348495749489025685577425306160395936, 166536161737312076418832288141504168703, 208548538204880331859280452471299536338, 43978805992272649132548573922178003549, 143537603065388609883282480397641203429, 54717758392102359360342684887696434788, 107047758112452548323502188793924782928, 157460688158290877409855529402489357238, 234099728865513542441753362056614118844, 301816212670489283945552556103533733472, 195995366714522497282233294296266473858, 144091348759303557222006892365531877184, 110022866215077287551344975164798836205, 215032052504511690749653688871462838117, 24308226706425690737112831477827048160, 154906982352221607701501020540836005105, 234716531677091415806946956650573132074, 58195023815956993881988478601844777766, 9465073238299734542946900804610216145, 292601788721559248583636890483331970972, 121419005665267754479915774796951766725, 216201228436704979904811472154241003746, 339032062727570754258379003407006039241, 111250090279245276678206667920475043347, 274100338603192461461512971236803944010, 270017146016236368667965368567569359456, 50243523361568793164463610024227599021, 212802557556628759341799375581362749870, 178270218397683018938595372084352258238, 320853438055503106024123669403365719247, 158965665980794265885792662761521576268, 52756755285936895612086683107157565412, 159760950284312429932717942190958773743, 42842074986628589984596777768640144664, 171358084524083271971023352520946038167, 108734119132667898856288721076642639789, 259960546493944419526032351576517372066, 216357943948453951972728215977448270748, 44718578139834130684964262134772392068, 309471808331933813511899140254722933383, 85171207851073180744098892954248204989, 5147612517532654939966062877825933852, 258860668080435972545643831926129152024, 224171586125615354346931507651325452157, 193293078353982345932238046952887939925, 131999010273128436608709569604392342406, 313370617684111103804418051247388816626, 172720932532319210830774815459313281979, 94880797308489861948818377146743447711, 126944716464001164350926251752066300752, 61399791262000651618417258344014927275, 16632131808316336344412379821416731111, 100732717408331789772998788392660907641, 123462704402513584199334470510393295481, 111700590653618886285292185745389747002, 103106964075393305745058244881588599279, 151928979973781833952295459895234935485, 87188534579354947729081420093120907174, 7258694098894930549922756386225253063, 115702987342903252734512721924855668253, 252072295575303476355525711177218872735, 250549051936172255377536109474855420745, 39467077975205301498912587224398789912, 298412660427175109949955226852004494357, 201130506269597838909823607774824800211, 145462954718178244988379205304769583733, 282554933788413532218501508493464368717, 315488969797418599271289949395434346389, 139397206484216346606630495407869786603, 206444319590506196310025244162548603038, 189723281470843697383936341005042698042, 72926880547405566179780691614349535326, 19270949824724108328197613936865081398, 107614655373607047785136097621637465938, 209487976253790550617134911459512614121, 64397397494124074707827465552695970255, 268497307669034583415454542558195850118, 195907357077685643514487757708733522044, 230322585645196853577881707961689717887, 12220946843345253005500903104408276618, 125835865864371780163243884051220295221, 29436253500665634813191570291301828472, 283372457875972579354276029776960574887, 54085146346660868216077210471293257904, 278985401270370375779121678469524957209, 316972315240780202101096574796642463405, 142592966399160781723463178767190740026, 257460674454482112445874929431986371190, 307079175253844905706411933205781998369, 201596713031857185356599837375793494699, 218678355831319723150736113110646868432, 39293247477171832404155422641049982304, 152883908896171556236287204327915035691, 67149171613105928822165367406622493615, 256381024162549282334933027555537560279, 239856035804265748430636179520408635246, 339744462276823243447232468042827347089, 225887579278085694209620180701743493030, 237185624664659827402251376703935173714, 140642423804168989722145435055351219497, 417607027431505933222375777992218298, 209038265672648833547534903411859099425, 72025040270835149948395718026318513401, 66127572793021187354859466689839708461, 134612426578444164382715929426324631659, 7971167846844515397304356595929911269, 190175676116876177270294639870868976048, 231200790398602105181087797721844493159, 293787831919683077537289988698370527859, 119293930359962301327914354551618510717, 148684309765317323926436217027321993565, 209824331632070934656343120541308455787, 318175140481745369422476891911784536528, 129935427150460367104789952384135309036, 145433252357310289648487002539075789654, 215723637987543002291547172797882963384, 129007597338285703120089277326376253474, 242482179953989577256330054027683907951, 233329040014183238346421749527563073269, 10751448356234796474994942719819761665, 311266539023066764760803801201067477157, 140384106741997493709829783178310998837, 237753309702989450908296335201812583283, 156158046424754567881408659392640201319, 27487305321946415503193687304539240299, 43082425864807199052277239417459234405, 50804265623798478204658824494838645268, 181242075750831202051788628759976631238, 17786338880650633562503152680961445324, 37362214253732534409374659193363044491, 190329252802547970510092542919333616275, 49989727682067687292206056614828013131, 50820021092605074518562182708558309593, 317828448365812265134406985267629181818, 298070957174227741967014816266891131624, 294626949535574705212788459178898046554, 148449289838663079942726666494491938051, 291119185882304315263422208427625887255, 86669037005722584097005863299946089435, 159926777215582527881713914537696506586, 52396277242163032148723959007883412635, 94836339315866123549149781095282864965, 99011086577443076662188885701862455450, 58123257765922263055851852874996107007, 51491407507217188543365787917770404502, 64957232087118721992842040750916536537, 280700148044025246087649500553242996230, 228680367089177658932929306630532273234, 251102707596358780275975139270326380915, 237704118421126940118852368600349629014, 132343273972291736012493952678057857576, 267398728065944180682436308836016587665, 231348455614719927613357907776264330876, 325867843437736354995301601769190157141, 5759810452716652408176726531955209548, 44216664075739044338799878120071083563, 198018697657008143112736082089461004911, 27362992299749477801041210443089599406, 228177623096557301538492141943467048320, 132133127009330883087187734692628745149, 274885714732805379223235654853592066215, 58974421308239718536938790956802403566, 301007204355409169459102631922366443902, 301635680519104390409134782612395293653, 14586581952097271856933716444050517911, 53610889985644471521910278506956584694, 264902052840905974769565945629856253734, 318110569921943706001324756662368571696, 84051836655765399625709136814928903636, 75596157212044384172748176497189880192, 24393243307998418363962417015745917201, 182126071242531355156753393622423605355, 216923824550712647278657225554593879910, 47201632175308986305760038845338185738, 296646520306060713619774187220434519936, 42509029833693385798101571373068714947, 133740351782808456579601300172587052392, 152919460295649728112985080589487583800, 117530645540402191298508465474918716750, 96841642228594199514018057982080155299, 331240644373562303957104667475734126187, 284653380931438329939575587308098978975, 150161267300674515669709709878802605795, 47593873899601257711922894282184865774, 292264077794392028988712765283229445234, 224742033237118911304750833203737388693, 221298691240147825893611740517453472092, 13549330246254067175180549178755953360, 277807674442134066639086249388702666962, 147721183100936131315009253995757035338, 17808708360894251450615542684849286537, 208618232726078565959508477838089927876, 311850714040518183975806159398043995280, 296040826583831820561943072541207538133, 209237207205113618224643515615695381908, 214026327733419201848935763279712826340, 295452880117245628706285567898488599832, 14212433090531240112624305559285997636, 161245398353032526070412372374579437194, 56428291482515776049755061604768839381, 203281980219625965520902560812209651719, 224256924812501360520467102807889521435, 303313234057082903342554215875390567149, 305999470842730397862113892432841372387, 156553379898713815907576841666420046271, 143129714997472261350866155072153123957, 129908998260283727812934900519488706513, 331845336054997718110333918446042960848, 256545867722952598985527946874964032430, 132954816036009351382748179130711157614, 168145162799099450647466355826768734902, 293225185224952521830021893547610671638, 25998009485080510719839617265680445278, 119241099127723905448197686469118458374, 107847245598888978842603170514433820825, 258603812984705032142036200804613944971, 75422102146697887691270037546466966775, 192011802612676857542685433733784791396, 238141038950997441354280999046428092613, 37196949433327233652858576223883248285, 140412821658087312255053925572837517402, 130156948752396341845068859386410971489, 5780836398347248418364470652682135052, 7452916057175758746164386878713175447, 28891312391793157537377634546493428884, 136076882972813124811285100873722603611, 136624824810140275151567238370672076331, 295095909328888882554214332559800671702, 235934719119936614775722280959594727928, 53901678715459601869110933528633320975, 109533391615179487388848486828619249117, 14089362552811781611243031657080278370, 210184106005139145639238218218715488135, 176431559058754403762602735789842477278, 321035712878644834753031919573942380946, 16069632153125749749271534584998380052, 206860579440749799832664307563089286185, 57705812217607101147016191515614918538, 53425064830402766513151386889562368462, 273016843834127726703687434180680591953, 25958770487161290338100200055517433370, 175806018237761054087037853557127557059, 138131469273831615683223736687213808750, 48789370425811036719202270777506831097, 222242483592093508450973381808191966046, 225503536483040006410041655240904457797, 312828143629294750085756992590404437030, 86784702712583607824576223754887953212, 18070787472601596954832676199276073203, 299880004461860155624551661860602126155, 322492988593194706025929866596119857278, 192722772924750460821002127074628561029, 185438390681768397540257943710428677614, 274567359094866541842447680020259319479, 111374875557378306560175830515484669943, 142264132677201072297097060500265241113, 121353695447122553337403974650552121947, 99509533426357696622994504656646253541, 1057661331658801037180260309145577080, 339645802476239893323553110660104385260, 84105105880977219964023871921028458761, 35571196369815151699079315806696582083, 95371713452447862212430934962321346243, 271027856801996297161835962667078291105, 335223319296948283316416075599779057485, 56892696940619359405687565110906594989, 305047524523298697690269970483451103735, 236842432795045162963058784262334840901, 275235709749415538422794006492615000977, 35484608803810563051745713170817477817, 82375851233241524942234051125479153261, 119364355576039593954834223977611389323, 249123503708704170138841060050837189822, 52324294898306700612309458055563253814, 245163520785162219953777806147303585413, 40537808921049330931522324081046071667, 230255005536534816671118997338532035312, 247150768327116464588894919507642409680, 252773812951416177966501915926097173609, 24241170838611806089439639324068031694, 84594755703359097450599736896320216893, 247091826833736775594002853547750236192, 143867214891603375338289814677193635197, 254573225406899172688984440979614533287, 326383179803944546553942223883738693560, 117093280989784497300018374159167634575, 131641497934834855151664434252724304411, 108797068030344822390748573674339113284, 312923112883776542637001033012767568519, 278094838757896818056290185289911390218, 180235402733434982100678409924095139447, 212118534789922746033950501093132971836, 154563916561268905396671096704904058924, 97407806262957009271904455372151516129, 103461417023328127995207661384471815293, 22924110824600872804632837175090416356, 99075669036571809157039198023397908448, 148373720551793991742420559482048724910, 175875724184660611496408069176088447724, 277382920112606916442177583363477029005, 340121305340794786046384353509922217769, 62285358751001874549513441275759086139, 61923781954762876092417537474029072935, 257741236486476715226511899620558236955, 324227649748664259390138131926430538629, 234351847502750221725988989824273918455, 206831970890317511890876831196925445359, 236266813067959232511487004430891016254, 65957025663076099171697336817601626887, 20492768389278689321126384620927802813, 171160810498043815604654616084865302894, 224849756632663920127308703487262898776, 178432024805475052637824194751397692288, 331855192605017574510503284097581157262, 12232599338229764639822023101871833633, 134648632061045796291942309952361092395, 259172377597183688492808656248148308134, 33343211587352053418428504463575840165, 104744338774812390213140595805714327155, 326640575347063128320272031823793877889, 135701101190794033899374272705861235976, 239087395688344274143080982512638711300, 50972531176866111105229134808031240959, 260297212026611813530582141145692509277, 127077842988197673429798258918168359805, 15013340923966792552129017888499522310, 234016032673460863595894391692514659057, 176430719873656284623395199091447009838, 313025663178452461249704319434359191648, 39008076513199329686059683319846680645, 49938628034651442270153551521026154074, 15679876579084869843105012937478591766, 326581473920688295304856834445582523642, 325221611173762993771627541406027789938, 41483736353530626637942531239410901564, 18832606424813299330228321075886108584, 63183454198712490130393467897567876379, 116873746788664546420081469612069753594, 48808770441459540020136319587519775178, 146209351392417185457751211291922625118, 146291774643119428986844408297698431151, 64562024263801137877379386101691658689, 66775918251776133671460755572579527813, 213830666908313325917429071689612303904, 52652585740051333048596060958860638722, 60039172082931289180945994703473096711, 230879650782993632085423264120044672776, 168377316070539553588424161016974150458, 326602998723832869597476601805748658341, 83861684786475183315113589602407515363, 266868204840477580681871913450938053576, 89446338582560789616388078663740301237, 273359558853584923812812916669178776712, 77005297454954048510560329440648089013, 131515768237375042877198512554439710832, 15659733151506334493183742957538347731, 139904942201538432559366603264710326182, 23217049095228247729875280082844608638, 83066700519492067912046498879322798592, 77537399039000218193854631236408914608, 173784905727563900466513400161674483286, 271945342214168305125232928032592033984, 87621102750413623972641536039475460662, 285314869195205417374679923675454560934, 231955633293447650832909966478568540757, 183160755528652935364342035142138337265, 259248330274544500450953592462888208973, 148323953047314869672947742605259505246, 324064343410806793645606349545828203648, 41718977462701209773631114541521440317, 274555006953093451210414497199422650083, 274091634276036171942671236471335352281, 237353075066042852820275095163187715467, 41416968013151257551178150696903944458, 287414186939670523528020828421557937722, 209746210177211992287856571628177598299, 297477083783168709472024759084981817681, 315925798442466262682752954675685025715, 247677858115113517535602184721734755862, 152603860812146706005367532069586655694, 149551273183998698201672097756228855946, 66644592832749433392061150723008147453, 301794788180682053076858284393207118083, 38060337669325442152903824858454033786, 200238772456569414787573819634174164799, 47157603734921605279070476462646689073, 72665738420575648013457681479176199668, 49212623909429236964277670414236877913, 242375945962450388682303929660071267396, 56265320454154794644891242857328405808, 26182208482942285696220913903962660044, 34374821218200914072943348654398456235, 212269998289506947112425169596969872356, 50787157579416912185421271900816549981, 333087535410221575537326818205914398515, 251897496626962513022944523735394371102, 122647687216170479604565450123133153723, 277083958183117066747782249701357591221, 324249509109538246064152517426516177020, 46438930665934946639074617638588631158, 139889275915590257325717177022471530619, 56861437985555046545006497119417941886, 172027226814111718189727002764905782836, 267349469223851157020910374357122101393, 193614877713731831562303140087059645633, 240418274019820232509563859706971970098, 116188064059851921749172541408943800218, 278725282468341769871931701365681375608, 201051417223948035470762113230291043110, 161861152550378404268462826448460482044, 143352303295888862882019631573467069967, 338606447855751662685495330773826970195, 280992016798793000732727289189513485197, 2298239556273196828402967741737870772, 50880703908380000377527162866982020876, 27917214154765952505127510358753225340, 199046401954451058162003982572499946064, 115643355735386257343469763608960945111, 336478049980890266342030957332757108550, 271163279292691675189161612951491294585, 304664141637802145410959738862645182157, 256424036677748340622830558334544096765, 276047933365617403850132602021919695461, 328522435733133908468349318641799807091, 57166869015806602569525903432791158809, 128105933016343204355759965670489128308, 2707844679713403211627672595338769640, 49452372782180868741909897690135573887, 131610026892407906166017366358421901322, 73267271583191794613965202311296700665, 26870464183434075058997914523188766725, 280323854203406849185611878182736127553, 66638977298364723145909567647790815247, 127332495183154950617843343441334294605, 51011434282773289680026113250099150137, 288543723507805392961037349818197966101, 322484837315498003236831060039955553163, 210786649587582541535802740638707974589, 116375278062919232089966923295109921550, 233042181837143453643338963455846144892, 20633878944644087407461490582168433106, 115404590956915393702290579952887928600, 250948246342609732279527343600476623234, 176204797897993486585592091923942961680, 46518688902221729581069393852671944425, 99908917895502228306904010876204627430, 158528035413068778246248517772713502407, 89170198203933200709398278285040965053, 293313371276078771811948362995718006631, 171344656118746019573066663217279186905, 138606537929168368706981050719250637907, 116295559490044617944827882007502328235, 115235296662797639361231045794557274072, 69425213141252521057947400420520200782, 53703624585727097907708110555048064987, 28244858916546746744007354538045405532, 42844692725709646478525616555464453309, 199525375188661347269625528155540668923, 173895886870063018137781419767221533708, 202629077280425434266784369287432128232, 110593477716903744698619753098418515259, 126818189726812185002924649099262421723, 225575386447030778411266453697939640277, 7990231788010320752320401507703799405, 270850853716102384774181960221238254060, 22662151974392350833152881925388222271, 315243971891473064846854660349371362396, 19044851008989289070532971906663712393, 126293548276649939244287817080302076215, 235316784812563994472750785367089080333, 328418987335294431510258390233797040915, 72440496525057138000397865219473473022, 110035195181569950566807492065871124693, 172817650914817608410470867628570717447, 190566451254375532853555655069191422612, 205750646187194434122375627368988317439, 32832333783882453719843101691597320387, 60845376781762055571375285239139324463, 186900434638630994275969402699346003780, 15672782440923348294164222202720281118, 214131851569736902678370514216355368730, 131168533531318657736553942028288007652, 288323305666162189340665701515543653178, 98523912045068079818610077432432863332, 8547807908506946217995364096820339479, 100604697245395213216081299054404958586, 21406777256330019269262018246476963219, 37214512352130019661382750208270650633, 254571786318587010933693975462639067978, 134428578064146114654383558371521911332, 164811462159927910639487052409037542862, 175619511485910841444160808556857087294, 230031401743579225583854677167006463914, 2983525089272205128423377383027273975, 331421301902604757258915860226209322956, 28141858994944219426175882935603364937, 292384317083788555981892512833414085767, 137155874731244178695908707894219008220, 150732078528510038391229099361155025089, 145610300738387911199883792665637070025, 262637484698546668598336390232881129887, 292556793882014984577476886600298019878, 289680671711684070873304284571575339566, 32325365520332266930424550379095701136, 171251829701151082906928555984444474251, 323160491198046816121227550744911288490, 177887130715364931739738703167994102549, 101014201992610394873918900894247116644, 275372527138716232248424837624487299788, 242320653108313620981801883823255261830, 122886556824890280866736894905666808715, 283343635054323716174079151516270769384, 231726662796990986808953768939237417714, 289782131712332933809197928485071986243, 331002164504401614498970910439422850623, 262053446348398072310590535983106560959, 338000694493326365592927682480142003949, 325693867934165043496339041883228984664, 139554963152873247653530650472849317275, 302442497881428267056876975904899037322, 197762186528911268146006108984147059207, 258483309620929458051061835822314626007, 259726652619350810501418929341747768405, 104519302103975070017174061941220099831, 31227561557443125899531793572643626348, 39340989602229279869164893290859296050, 217361892577695000674204833206012094228, 245622769861714454700667897844972589679, 21315338400428402616908183400168336714, 242607967733128233146310320318779260918, 50452925410992010323541472929624494699, 283989164659216683101008806962224072466, 74759817766664852616506189139369970614, 234668921579918932047885109562646036863, 98842360767455933980423408728906866603, 201885524535598169088207194134248861625, 188994854890990364315330199273596860717, 187303902144839763450157496300554686713, 298251306840432886846410636819297662858, 233619795845465482308109236820009496776, 141966057259084405108083837578240042597, 338307216935844320252275741471521839011, 217861836935032649404875560859904537599, 139603916915492426304687683425059607772, 5721816887902339164555774288137400206, 84254332666472251510639548308424525013, 92149884385456231273465591483726440850, 24678835434158972355693887852468633715, 288245661277639765893741171940225447071, 252341765020013199378613161299043129984, 78702518770149338625872629227728236902, 259832278941499101358613757422894470876, 202378774193126458693010663767643213929, 247199517672013243401364298394049755316, 339816182123423919844454544152461322442, 228969105105183467114792769396016717083, 4830361999945731098502287363779944617, 161409323503962554445029516773819142796, 331275669000230792705059246783085840094, 194858982240279789416691103304924575623, 309141802374549352372905461304426064602, 108763230235975950730922118731267600677, 304327006548728162152940900332215474616, 64145198691198873293205978131647389392, 70463147614906352606380691010999844024, 230704260862680884409667351558628135334, 287499916461775328547080945152394669363, 333187419762047540010709686464057945520, 306312034409389420690194923630624130234, 17107735021383560420831922058725989917, 328258148374832337312622818419241093949, 267348998636194810248421964221133397785, 121922681540196029727357100346824646640, 83297376273124650371857655467817553142, 262787458300658972101881023082517888490, 301940092711131199061426099660622457940, 213754986173539917764780120857419456180, 33158911145056270597648031767675892323, 34250117702160841333656253428493516920, 136583119405045826173373621545667561801, 82609749640931425743856535590036329658, 281160524648229230748783858668833531795, 95637792776933074939369786368951256427, 55418578713925860177082709870322609088, 306430439546080610140474249381518037927, 208516292871182444896423677376424116784, 48891397767782517868260406808259568079, 114502795724300216001701241473690760074, 153563328708832527248499285336873166936, 198808904144419018946996606333299054601, 133469610595732499822709338800074338590, 130038296022936226634766722345473706047, 80878505154256100943439530803094015076, 337953825193528598384759589777790328848, 147264247136665432940200341381648929118, 22041740161766369418269514349948923399, 15075088652419430234568758837932379476, 293680022586670286524957673366737595859, 78904746085840871386274804668675380847, 70158920829408704853642889859219406517, 11155061786476402059993745614858037249, 219644450326458031236648961312511814423, 331500450408903907589610281261828504074, 160593618717518328653688513535660122063, 180481287253247224692027065682006022768, 64478246630366225544660370311814998324, 237443256753197261929443871203868625051, 151706799674709897567958673259422842045, 107947211459230590823439403814836919382, 11251110099214026078589093498451098893, 133953210546903465461754272037014487188, 67234003043864974820565540885757610401, 41380886717315876303877364243591718454, 320860347410877190160355792999889031785, 161431836428981786943661899732713109939, 326366692678093370208930246149647189609, 130372272408454264026792402478399409326, 81454406540039996027043326670504343572, 52682347506828640856512225837330067666, 154822830610648407400674750461151529237, 294703176577689212170979831720273545889, 153076895833677436904743099183077975824, 274456516203630307287899072807104609951, 119011146360375124842863101195824332414, 162681880028906127908416837312426541797, 26963742488793821067061845805923482457, 153068645325622812881304141594844500936, 206676415662639841523375723059624714045, 202916717763837761536125289387123585218, 144297001323960322779585879651920568664, 225464711341869063073260385511555802838, 236225973570047665728825488552687683192, 126341172244245054424236110041535201082, 305753043403393358050775175664211214727, 47730179263010610132350449529740391547, 137585412052570822624283090724368845231, 102576843378265619227667067613573381423, 34039614656222906524284593810091997553, 166355446491134919634885004152129532695, 113775731557973747739920082527980221373, 221787766575385162801971341508022140092, 298666843664160686120360951947518876796, 149535982043186535273420766492674403632, 176563596612197805231595235218597897701, 60588082750732579336034790310228282928, 153518427661421004407784694565471551492, 37209572173458439777325121637379581549, 70197142823629822042962397999013261487, 326486676080330337547481983113752667152, 172917483582526157530357871537083343756, 306916393888774716621937404040286538214, 246247025531300558333639006415167231358, 91174722313271305143998669457340366940, 151004075943149565132970428949166535689, 262596863778869493360837125130738123187, 168149370098493270446208056924293121066, 27340678676694932786466603623516610426, 193444771139831016285971348227292399931, 73611866288499924808571664512083529530, 292016798131835262051539265604110423578, 35182265830379469751863927728853528992, 73716444915105510934462362041642628071, 40151907661178148986477108535245537567, 173003670077070145617059371562603613837, 267350105080467892573354087528245774738, 170076016581325763455880210740701214236, 320613418989055654229303477018973587650, 229796887721190191827377883295742467058, 338322940015560316273408630913176154500, 294963388456734162870999825542563300994, 131986137914120757591113468369748903445, 168746651076370588111887452416947850861, 295661527389751850661091689682994673187, 48791424593162481210248407893266969288, 240438321497796544628325245020874443427, 115461262741427885372495409070365131684, 150360432679842282900898452010970421217, 39138122535219172181400372804150402723, 268882115696362079512433329627707848895, 197131686759006523967860271500971732643, 180898701786951767922279175171164266462, 197120586132284175608851035992724975265, 274361780620634273951822323902826070678, 74358476432518034936439675025112294919, 226733841037660557339139423590044829852, 262247971943326620251801278923774651540, 275037595760104588550406510648190383581, 339690099478992784443837834012128814659, 131547692122409541009576227987451062608, 256556142676371892778197675395619790860, 165907742678675172478770716095996011517, 94883099092610248337312347421754809347, 40610131019960309328600975740794400379, 194368982745996603108340887454348685755, 6767047609222913877054436983639002580, 216914096962017303621544663431008818667, 61451507217248395252221989862677722457, 329611390996411803805117584175504367029, 193666920661914114810984681328206541508, 317524364204801053171319859020019701360, 78891353796976710860177822646667523371, 274398091107189127893665167935022600083, 193409593101471389847488368722181068104, 21280880621153656536637835237164988330, 50778432057037207391690282490742835100, 269080016489452501892055007083912067216, 120720542138450673978634306380114765789, 262900403398944196770205491119044388935, 281387842894645754824147036524804503670, 81148970739739507945596911214014785264, 131810250925298129605729352626274035662, 215600340862862989440825881426026510425, 149584321831566715817918262370697774162, 25090775406154166913276424320421498966, 90181712689795571761875480093518003086, 208953458130534944060895829948787272316, 194990231761149867536004224655000998347, 278218735932307307683683273458444926865, 129467895802384759384609713942561786292, 103858597418692087385345137134059546776, 159847976424875576458681916110002193780, 182986829930832271783369800500807901966, 80314900896361644534841124279411699322, 172801613992397173615834210145287333786, 269157958102510067295744476747583394074, 175346779720493560502635966039341227166, 134107855602182425750406488137819698464, 209679765821068427374799470489192889928, 295231394187886204499471993390397713590, 111290485843327835327751585738305308200, 144017986521122111694703148904506398398, 273937256348868846161868895557041474850, 81676550056947529883189770924577305006, 99029969457194441823330438694970289321, 243103876776928657140510160968333610562, 117001162229975399806478010193974958428, 144307995387223617687209294050607017852, 119148071021366107922985241045518022230, 173187042603632836027220621553166000903, 337526931942734507243878924057107585936, 171634030776511366837451744941872437827, 140076589891190596991952872926828208894, 90475438178146088279582160904641617374, 240264090856247977166060020709585365987, 131506308130429809799512716316659074396, 242103176689664004374413913101642876932, 131528660732938791345137997806424261321, 37134289898319894866230454701048542262, 69950805898797796830840656841735251986, 261302822669856566926577497686400569959, 147730918720509979846491684681722190465, 35956964235343253557170740638077949395, 85431972348408997638164849146926550596, 242230778039340216487235416350115545154, 51323817404692068630899970511461603542, 140972464969355978533025852409921744348, 56575078332165210839325495328481020726, 307679645688434334264908491673272321046, 131222597307794421176232544395318383767, 108708533140618966467674879308082639773, 198913745605183533341536908143717842591, 99729503488900784752254304353873002163, 238378661045169779401240473735027860806, 65636695175129754142828207043294182576, 248082779715165686860541112165256996223, 37080125138156474834541862126071903447, 273700814494412649424966246941683560919, 157439796668799615991196759987257272236, 200632039265734643047492341961293583255, 129918925107665359826216811792720141990, 124565036737448568185643394609030289150, 305757854631243167846462816647588595605, 103086937530991193212835050869392607496, 97715131928370450082168890021585504736, 8067744185855665621878426302608423954, 110247331739840457783272879069007975041, 327254752596899102903385437879662979963, 5352573880392832609448407240222258683, 85965492448430061767922482939311662307, 118736841472968312800906061342610844816, 234545951986217598736446731753543829813, 227302464090954765726801113242897359245, 259535471670253401771818668033118608831, 27681968308358389632491933057427556813, 38446322791435700463401144059814134892, 326150192774851701470734069370232636585, 9084809126085405888344947909396953401, 247716624118896895986292948954108064105, 59250489497741450198145181952237078500, 276624648412716303632568801269116198790, 20584777830408432295838239248594302434, 102324356312077690053441494178697188786, 154715933648242384954013749781459607578, 36119263935052465015317317947552612948, 304999895869000713963864929079227480661, 123059779590471421495625650336115073059, 238606905842914531710137116236592311856, 332313460880274566875681251817977395538, 310588608698635639406586888391887674653, 51616664475603246483588994243486031327, 195781974554118412403998254673531435932, 299860170484413285779843522087455176116, 67263936350212249008228321134433222506, 223312484670572026472176945733204076035, 224828620462205871998559543100858283234, 140077567410228050218203638273586410217, 30474707669843623402345568834796803754, 21188011104525120415901067043426743175, 41814042581092017475038669049733862155, 300920894220857488656876588931722986625, 288758608705334621824665315755355381303, 284897796118907732765342579862849351, 314189796235779166591433695125638578734, 52007269273586990712691168087935423695, 237142535847235190947128047280900987669, 49419624498041494531692786097039394302, 295043560184561156280200465367119069466, 261500045893052209103024242807817107313, 67775726578171190040456214995237228274, 140258015355438663922741364390548485553, 309685414886212805382381254746882553053, 166330559886987939987397870911647803756, 334490618389200975597289777608009841630, 47316342007156078520747747118755602959, 179736476051956834281986099894644339511, 210156375518187417843076214494930662354, 321892672944041053810202700055616700372, 320764218348513727240418530877634866131, 154116403048164764878201760031420824158, 49214981430676884492040183805618726536, 98384120766353172935403481442331544674, 244949608042994219486116324783381204448, 254037750881859213377651393019348494722, 336442626787390849146787069442199309944, 174490489413458768635268144864792022313, 286402514524953880977331432257329743295, 309371760028153349352003964190298310885, 149259872732198109974706601880866091576, 85214044004171428498944737470518004424, 199203276957802291736477639903987532864, 283854095499051454845169223291577103437, 47383503287638959046945564710992186265, 158666942544749917530227233790177811555, 206570811603542826616137800343849158398, 237769282830040082145903156465512658113, 105514322161508342714951161470612370431, 45692311602195352061487580789497940082, 23796588723119747401926498711318037900, 56217864349711022674935027518459632197, 249109396719780211948519697573699872627, 135801068418079003535037308756519370457, 116052786632663927739738707986033609964, 159633622312610749645718038319826621546, 208975179078532186764858207905890794286, 208404951135542199483794861791342136390, 194593013081077073748777966680097898656, 299015028321100065361103039009201573730, 243566277919265564741584700469334644912, 129574204825845035046288489190489444473, 53229510677882738547028637115924553574, 61282708443900694926891030417399754381, 277328205691467953004373275018203814437, 136066745298436224034383989908443870365, 306050816671040736393188998956903208225, 244477544617569926000695155058852441038, 161280459623570537314249242396352596599, 311935159710538582033727067264991918318, 106639861060683974008664829870224179447, 50130338971530236735219521069543185608, 85198616223242057557886922827832068870, 29080937807747790296570256491108387288, 244688201375315208617121841586379942335, 186423569570273933727626156616769322268, 7492942034586191117098312940420424935, 1255074003446112727696215507188204103, 136484323176847331151862281529677725884, 18049414549426168457322599752757826266, 267946597190642735413233183813978945652, 124247459550652224929277652414134302850, 238949965447802268538530538033791108097, 62699550664802764209479338201011132742, 27316188164703674057725508869684688752, 40839215084165125972536125282800733167, 145157959213619160545385809266463320443, 160947088555582544490424392956612492755, 174568940351420152003444491377321816406, 247657265039141562286023542768661965795, 55445806341915856969299724853113879843, 134666747186878437090320835676139775091, 204737634251104467974431948293932211685, 65563477383307582529392940952516635234, 259269642681909694698295579078102274189, 179209165520047549351561429020444805173, 201105431552937364527231321307560513224, 152086295562162925428866356482818876498, 100160068015576433639223228376870274791, 258079495305840549438884232958742166847, 236421340754012132746906289045712682033, 136176139269695285454293776952907939581, 278889836648196043070033281924657442108, 297745673436061575383924034637472153053, 77591196940016746075231257220871689953, 27497808452242878848025468601641755115, 224709905154530161052606266851646053929, 74712902101431179116663165580032515888, 6769770032680933005383481436966200449, 161590634779427639528415390907780784394, 201060261992648542718814740281732837498, 324891982320697822675843519036101730888, 269565840403609789986175295449062684978, 152685692879439648063148394088813754746, 235044969890457793530057658182179353749, 242517816833462328344415309414224272562, 70663980650300405604370567215275411782, 288235845379874939887542810442647682749, 218968248517332257746811154180235276134, 8564647254754123001260351597771852634, 260826102148457584869399141873272237430, 92657718834186311291857542741114599026, 135969793366629097344979938987675385459, 228723277371299717245918350805621181162, 162160300197943848120342281366439914212, 234588439687194005723240860597642893076, 57456392029260839639880642275539271791, 49560826548552529495048119354901804905, 136260066809640410347011290411430514650, 249637798684429274385000156731188709739, 78705977313547566418024142595937557408, 240013633607984029046845705554301119149, 86745527144123372356851324646155747073, 268980265062186362530813003044226553381, 322860375975003363880095800424356763525, 300022437335757445043364431550083116987, 224734800477040043958601139627904379423, 321801062622793108211094201346925671128, 72276513039362875434485602438064892992, 311574680912589122591816368301281996070, 287566875349107111653739973702287079082, 97815305768137913656457592775823855442, 152338851844980134819730648064569635827, 170328593134473438228816086522466012641, 269146716481926712056788843668472421371, 18436105316674943546518850029023490594, 61814746741422920888547727230040209493, 170301879162327993818061017704155448414, 144638693627257384974238642232231559427, 16908325549965028895285592886619690256, 61741040387128261465634316095607860307, 196006384834428696016013116974756646069, 296225004018970405564237293200396558217, 133722074037253131122609285227231684079, 210203942824678622946883019689945655084, 242457738097820411893290255066151431193, 78394654518358212121065641044303020116, 69216178811357759924570801236021088230, 246661073184221349508038944332748554854, 11697054814034383069937238686224794413, 142034066711482989512732925062794164901, 258737777369995288387609445913719688023, 39795095475025501191396703587572427852, 281459373142557991891787449811882438001, 152076894056395492566354746878929436515, 161978021005639555883106899090825970345, 115682894601359954262591149059873223075, 134887194738376883158850284692272553044, 91155813392614199734294249778777244048, 230049628202396827294955192235618619338, 98787891026377424493070484170653642334, 270373974011966983667619248294365725138, 4656707366840562650656473944631508301, 308080028088566274709433283620009991049, 97765286615907420053507796047277485260, 270474728540132250289608876550775433487, 53595757619566596372844443365228147895, 47840481033341443543062813527239124362, 22357689967593618376375496581457660904, 289447071994449097345365937293586474236, 129855993449131760912899530087772830226, 244360288756040624607938250969436671548, 143210760061704092871360464946124129456, 306183257282207753757089742097390377562, 92984492236640384835561454878347475491, 149231845880240312082175246053353447170, 218871558768041967638632826704021147994, 19212432028434027842764617296796828193, 76151417369759704189685520210691418624, 209037376705722191018396366074441409963, 165864427530739775369871667103664907045, 199242145011290649184747848507049830343, 98326684660101778138583058248430830838, 160730427325361614238558163266360907059, 219437941200853517498407760065211381121, 237405608673639143633028724870666929819, 99031631033578767993454266747191702548, 50401048615642292307707950169375572021, 124845908995666006399478174942662905017, 225965427521799521681527742327074695615, 55225555011636405619351974050703825716, 132314763673888104164622122812073064292, 64148606326695314348246254359966721915, 295773781873474330480152699017551755884, 139531178580798459229851314665911284559, 79642826359225674769138635827027898922, 92631702513315491227764303370582806302, 335053139354926110281654676689593964563, 201714134729849872304383208697766912109, 297869140346106557121692504958072919395, 188958076636048657690803784303364875857, 27123827753183841927110905854846793308, 91732138862585215258963096683334415001, 48880694876591281971338031227813468352, 153035731642626079150753889285125489433, 288758647893317163857478811097398443147, 239158378935444382702225304697747440465, 211026019903864479482207479373045161480, 318534585424021067919705277954865259341, 222399787763346337704927131499250396067, 167244903304304825894703809606178004409, 288737987582126083905056314225198307854, 227031719471201282301778359309378672032, 156718092742874975662217443288547683364, 126091397717669626106124811505223005529, 267526604291777350362048654710901399795, 334653338248025187131323024388992554083, 302107946068960144863701284335140253232, 197702781017160598590370594808542950089, 47047891814118676252673738857351115294, 278835919772888561930209704406118100146, 42133461882111528745397024646465573863, 157470785980837188742755149813695377551, 274756154919722086513859193432247984563, 241801291414278306036935889607938977176, 301147626497549934291410881131672028326, 227822817711086363746801267354107218180, 235440982595851323454261719636168292006, 94694075270562633560970711357940009461, 42116195246567293553126311142273540243, 309508963479398472434624481110755275061, 220384350350450540679415210111334958183, 134708931808123206589459253713658229376, 150372501460114376522033950443719787819, 97409542439239045309528215283543115454, 164586137360005904037228821753914088147, 230824402724797503658445054116112410557, 187762847727649344768044774442153246345, 296814381175783890682402819512810736905, 230054135462904311006904721293914403283, 276077315199742893559975522102216985085, 13682471965678436742303891228164598865, 132866529721836641227753302156877362473, 267439273700533931818205441741667004683, 246635957888288505534192567073693959863, 16316689649318521196772456953160921963, 183932335339387720548803321392223161781, 43533551978548176410952651752073912721, 177920295846801797634548957672664816854, 244389869345250142962158671736511433092, 11883064071585982159128684303197673729, 263575874629921362226975043616328540154, 101247162679971392227476532008968479616, 120330635936016248611123239629457774882, 336005543801249640032592190339048518550, 5314789320538268532723857303988212092, 67631596243071460423053539121971909405, 43831289105767672589161532937420293923, 30942739467201683026148656925071411018, 102389122502271203066941619644310624455, 172288122486224019837135704331694683006, 293327858277276041638015212243519342369, 180821691727269024004213897245641604012, 146460604175803796963267568156120749, 283572616314417135621648206306660322135, 3710847506164893392111072856840767108, 262844360023271608597402967129928600525, 177333818460053561272024025078294033750, 196565062290925170207721465390543526661, 152998416894177878070455397820618822652, 311889769031783163457805982669188678577, 41944910221282937558673178156278462858, 299458809405279607074165289111068714490, 60420427321647958200392430615085098855, 293427905542110188138025856417375364855, 190693479408497013971556731669958233113, 82799989808811062049080880168996692149, 145677952750484300293713243661019434023, 328044664177740734095142329098327927749, 294337769847697465735189486825865302060, 227425909881375070060630744115340425617, 178489586526938627026178398802578294999, 320082036757954091296215048652337288600, 130598584877954640708184167691797546466, 14243287892407863348550609395628009146, 166241942679951232103281900891064904950, 78542287200963424006470652455556839999, 26241723879554498969613853760013716388, 194984239214446113018689118925591654815, 19806469149878388854314111213871081781, 188889417763138235099383497703799832340, 209871220560894080803449301529720823414, 103035066845293513214981411256058444073, 139907070536582987400973518752436032131, 199264459651573679266117151294710142749, 317474717828682306380990249080569077983, 128325014300710583395686335322297525940, 186498808415019150643960186488430428600, 166989276879044754365656043951890943808, 35681168108514331538491649144281363270, 152027251976340302373012548666759848369, 330188396641057907359356111154307699193, 210919559526298852748372736506285174743, 112646656190323353810081511333766068003, 313712194518115438996061536128486022644, 154095725899890430763972491708828185285, 20116824079495224983820634169060664880, 148555985746188117958100271228891261169, 17325435372272919727220117117758763047, 246491519334557039877124018009965772638, 88310036562556507748820027912204104463, 90715194241264303730315184313942488843, 239306478601489541695410554686330445926, 135692474520893106099831197048385928579, 63084749625199034475216672600531036499, 62773437557737198261823835535785393930, 306415428539534441020373449821700302174, 77569670866925680425554406254820363761, 323420560483008155335383223269301148186, 233235968271631508097065221947207192574, 293575918453559792488349743222233277070, 336127096511787183053251995067543344074, 140815334939032938329543869536997915911, 107725423182330956316186641666112287452, 57310262324600864833925015597425271461, 319031834880415357003510059496724653131, 141264529921260961705648050677623330499, 190032421358154502110264665176754727905, 326742033483856719542685174617255799345, 235235452783412628182924741783521227058, 21921561790624618710097322067046566485, 91214083701971388874708279881154128786, 334483033710830538776708696627386378269, 122405306113240900422753960100169208156, 338197282675317914249219829971429507247, 20270817776600265013708784363351188543, 79927734481704954941553819017077696686, 137991806200782625810617229260377778652, 183286541997977848807942864258246695403, 30421066831505479006904569201946163709, 4581183850380359131048305113458170538, 91966667142316526965053254226844309016, 148041046455429214517854020297605984461, 283883587080765087962185443825401548540, 78418497479432255979249071287527451484, 75861554833756592377171757044655008003, 35382803892642196715721542374715723447, 113056486676331571311173330403769283428, 144367109726164547493222222406108540537, 126155022291959741815737532733362281708, 141973365026692482837047537379093979069, 165342717564210420224152623154006748158, 166232326642026376829076312052658593205, 266406021444560650065632903800730727517, 267549005805453549966847615194766930674, 285102297173622057067161900073622654440, 110297065362645390394654456138571884180, 75038759153192940728382419857959310552, 7374296200247918807311090424059690954, 198955190232883411256447946327587299657, 217776991711240442196791717796294236135, 257287504938408150162721566208635657788, 168817433493658519047756721299535212718, 322956548760065429150567378941651481153, 30876859794357849037016841566860884063, 15176956969894981965442281824542034022, 172620681610898360085573364284888444564, 20125476446175369867502042659511335477, 338187564833446062299337306910575073430, 1001153654252845856027970159727316846, 122708151032979354947881172659501740917, 105626000754834829831078371569107872476, 205521582147248402446873579877586295448, 306061387789501145574370450750457683824, 227259157008896834445175517768665821127, 194987161030410861625565345622871442019, 32963234762234752442922114070293764327, 115186479727989100412481992522982043198, 272229110928452345621202089685902957154, 319513935674074681453455399363747921241, 279072185081812197171221709660209917508, 196031193407599734706330980545349855037, 166090975012279752453214212972130904787, 322677720603140139725396098777006353467, 66505327131669981024809131966865664269, 108277879590012857111694649589697261072, 7536391861146996029736383041472638342, 283130358514673889569802703905575996609, 127749098270623926904196020257820534470, 251445713152692482039598966046863021384, 72307222021692695900691152460061628958, 279005531534942813690377812090641776663, 318363492902174154804705520953632996103, 136888579470328008421049316163434206850, 213847743648084376803880165977250074435, 161605201242897940374675655033158525692, 255993265207225873897210456212429310268, 223236080071315726301474729102532221684, 268241137412922542101447530559086146816, 108409290781963358571101708218310796298, 245652549854830101670488181318268994331, 302901399288764967753804498022835585799, 323993482787857219638957943879842933465, 309661713891672736623685631779613860127, 338507282683308595821716281457138390986, 98437077285745007926674623692175935562, 100739967061938849813742042783320678476, 321147659619155636899750089087548916569, 118807412963352619308734230390499485541, 264529598469900358671378337791799631225, 28422486339252184989990417415913189975, 30972313588087023442998500015156274512, 310923913055361270567902210726527680356, 314002075597457512717343617238990727464, 95905604382893316056272096405752438724, 149019315841204561264496316801367102187, 10062008035856512600488944746841668762, 264526397170054402578380707368525426960, 58101837579470169103128800195070645276, 127725251155485587924413593604680432290, 73588867613544509047400126368220849874, 228895178632409185647080789242038106527, 25848752439759645194686072262644136476, 238044889513291669105500515660885609021, 138010013515202517633748309682883253993, 156385000511331933849343599970207083082, 103819016609364103831693642760992329685, 24050642242318232807672039388944660401, 26913748994896417053409384562510532921, 143330422713184915746258231681125771284, 105298198857951701043580459553168576188, 254386232320898174366334005861974446651, 308263354571434217875899163148345559387, 289966364218342433903765418551146330438, 299284287874397955667942837653331752929, 263896874611086812823109985292787631038, 195394974108481256144391386884603322843, 272971852026403341082082067224802709569, 70129507237929303106801387804085085272, 23154630672240556161633426446840020418, 259557731790327789073209917431639476800, 39935927960832925588136890405343971344, 66867209825075190373038749783945749055, 284463313140162564221283654013792002508, 266266594594863314712837094663772280150, 40067345301255117707769514254444799214, 154064940175455767692443966593747338326, 142101654216344328097131235648880684085, 220862331689994114441178229673329440525, 28173744404699539069084689318796387745, 108865267531973780365813099269955577322, 79000988781430138575152018282227107702, 300754254352616027362758561420497520919, 33638033823331273377601692269942577152, 298731016862817928418164000922936016858, 299651162578442216917289752509323062127, 3688914375979664182385719188014905498, 95032360785407906257307828946016086130, 20978350640701290331965845468270030582, 135641377093476915253281247209552805098, 292003021515049263286930365290319714667, 250095197767850459941916301192673598216, 288346926181708157367983799449735428441, 226106748153307785457657807069897084804, 65189177311106427560918324532309945656, 18947829186588505738605406278385167278, 28858482366417187723344781714733347117, 253012662655841049229433568336425461643, 211289584985945432299512161512339537947, 225083040642835719125168963555645071702, 316874700822155376613944787218967049463, 153134277990002233584189360358668816787, 9114831882533396237058841249881028397, 214182035497699524557749010760269115906, 62240069668425784245108677185339244772, 159155895651854468532217167085721325730, 199608148356194502909445905994358456129, 268003465091796828779808175506726149552, 49027498738673724893617961360016511561, 292116395173963281222159288448839850789, 255184900122616668711603531314516097574, 310983576651546951397820447694889735151, 218815944647582965044621919210964695780, 62924806228532458079044977558219960072, 76883335380628506871414283249964717716, 218973044233234094997460299461938247011, 208224360362017467873493442721003966430, 70900772868380533775467628858004969073, 335517239930785542047090074369801677275, 147876447569950912471809891312189086249, 103255401177898428735433280925637493310, 101570128523833817892696040107915343938, 75656648313207529184178962789389366757, 41493805148599988595631990125698969379, 324906478549893920865061672502449521727, 82231456818808177318667205258248145697, 336929643381005068861697510329266304731, 302503304571808702850483140185000107194, 116110216125631583722239039900191738126, 25902484803481293804057266542813611260, 289096486418586139942063720249391398547, 215042825431445945382452057805733560297, 272506102254285777612398717552198045388, 71221591665784248483333747418255478167, 218996144603160459594470425418903774410, 105121891648080163885723387890075548240, 171122245718743493581682054258904929522, 32894264658901607432267543253270801624, 298301117222485113555608124557741696856, 154056319640624387857753044489020539285, 250936907459638861062988331620225674158, 225440189903897811658966929830562605769, 40385832507359449686217460159060428809, 155679626281438575768250421126093299389, 237513527788529648803689757629535443015, 33697735253343742391135870423691005207, 42770987323687068582129992449505271320, 206764567730011299508869362312579259843, 56140863756285338223806016205169692291, 85031323341722606781751351287289633403, 335183652171472074566155526264850069062, 197963805778592887250834771985306237106, 268737080702497255942302465496600085399, 291971884481313423524382425124953039653, 299258905348964575257570951428760982692, 214390109923847708182269150375063528250, 324006103730997957767357998628484912159, 101405388850583268810452792994072818030, 322904523887765177266695106140046394666, 248206390126467318493956932661100583013, 53695726038079659250910297494804553456, 301304117459343240671558255797757543173, 259924613946494650262377909658721249173, 261017479042319449964581196738849445417, 15202858217646005712964664159376735996, 43317634295280710047696839363402873133, 127591965433047193811578213837153375019, 136818533641436695434309704950641987300, 166607173687382770871536635353992248088, 223348853370591921050226280312019878863, 311382859868230549237090406195934432042, 254791650427996124503104361711469463742, 296324982323464387823308486108052138795, 19299557576193563168240860448576797281, 184702244803413660686422759124833052828, 172938649510933230770946057598744300691, 33854553677357340957382172780801502343, 147065796614932395625199996578293337910, 48403778109263279166111829821301836903, 176944316233011494846974514544490998821, 165887745045997746040939533895938222220, 135127141471997583030612250132880512528, 232204251702632476708688395885365232490, 187839711063316017626913015694855270159, 172165286521877916344286380985471899548, 29900300863929580461215977146843571693, 30817614896619033039272735291809678962, 124716694807990963857971677893966042410, 261780490552473429100816176072030390275, 135641654364261006484028445598528857623, 124795390802880935653150675699460415993, 120029103562852986161595437854537522524, 296957707558631656836425236319326896551, 141891311999373248638336983374857271716, 284230673860286175105162546449659068436, 7038888110575418241817957019713586656, 23337093695458027629125197383411330137, 178442616857148806539557202337832523934, 231887762888243200433152100462187215641, 320166799406345023338726446668422186912, 149783422087717934004657323537453269572, 256510811491099371696962223684674946644, 275081776527011850078670201683886206829, 136658715519517810614352049803091933138, 91236904651532321955612535414981712377, 21196156520366190954773283407178319918, 140197569504017333782372283223282550005, 309256418324640208037825637458229218532, 293078434354755686727516602390888929701, 253506502492529333683819328483458710821, 279965628346738920162155181708097176446, 9918536275299453592461088656947292108, 258784108553907717273601073486562869056, 326962295650875999248729101286251205666, 237169089433562311511192865846601105306, 125693073718259411773731505915115090294, 321822149504926832527483127308009220118, 187585665487263358610731495585756468004, 84185703027516250843478819205530266359, 179677011031518094603125111433447852106, 106889344547620045097723778290721792104, 30410274141438332335544163598443448920, 324886995049032737165450173060638683612, 181209279170328060520511522812759077769, 275672902660053805825246546029638697655, 178815551971171231927897095232067304390, 325479922978061206058111326901947415510, 183932084552862886827070097660660611589, 132038673400368555135403010553347488101, 69140246700198958754275228065794045254, 152770104066844859063365710422993919178, 190865604822286523969667192863598462, 144552392446377396071093381266467493521, 272740066907294184916983621467134345009, 212731107234210860935522323201160291778, 163088539908698746542628233778135008495, 321391534694940574879525357654386453967, 17793293180941000696531426292626042527, 201119475502592211909587304421866055326, 22537414053572987385794921209507684204, 250369903128924776698485406582024157132, 289057244287578473421848126021429214082, 221300436889044597777420784143202049748, 2017324221452265849671866168923226013, 255326685646518632359014212175707676489, 85938743362512907796052809745591496003, 162326923882595692569433871250731135571, 176487299261760759861112557015561569100, 172792492782255242369808211273512836180, 210641826755503740829380135661931819182, 315000994545490330431632543242033026957, 334095613166716591580831179793633250174, 125056413339063989936290517649942876514, 22335164473125879782958335316527043491, 323344604546899783940862287624293283197, 215084338604509923295858783186002990432, 137135899358249130190696904951205393204, 310183220002660918796272843212075556030, 245533881585730913993265473774934018520, 138431214158211525042772294899842997860, 64059562531966338870591531301785216951, 201014750127539019614058700557784392846, 24820793514358856567612428262378522117, 310324246089659429411061807762508296046, 240008861151725435946056708619374158996, 16791917285044791392004848731105060718, 99444710770997056951984747364865840083, 171050372920949759128866486563325493772, 168836129408506149928514381646784771599, 85401171876578339367355915605449161136, 74544607160086449647664779574477044756, 123451511007936409796703406891587632946, 128343431125236415737712651338955714635, 25522891720758066378456823685647387450, 283556336475402736472978787520955955253, 71763600135215500043127153205249560947, 67737254493947403088916626154997332208, 183300116567890417153303790285018749475, 286529443222005623193094537126647547759, 113032124099026981609674694864297005570, 71560243121806753382130449462137902682, 136625928169279186127075208055081555263, 189056346148439978178581703338334402275, 288733943152759889756882270935216010888, 234102932293875161816480652699082741181, 312266508851470615452691335821270149517, 215192388976057470963042621390878161112, 312865948991458199906577360580888670383, 249114056363611567215938308381745533291, 209168525664218972757294121022300309650, 103286864631774632205907190694675372361, 208545483794086526024449949180568447833, 138700302757930698085188360649869112148, 45644195983630373913480815133405312375, 85410942604355486926988076575205365670, 105882007841477509099367195943114100634, 93962411156286171470227447730627658179, 62383365104040622118643003970319679624, 16003365601485861121782827644291216926, 315336402916413172709611599216358571576, 244733593377381089341060725073357120942, 55427743800796591833280070397776681398, 136696433032529050990610082053651428990, 272314665493529377289401591157672016642, 245215527134153169254421227741416762123, 205783817488916338074496529496270978668, 87147233162208307311821511260373438232, 66614787857928403150404740360252451932, 178416664908412736014271167021193905354, 145919406954634710693123726772115840806, 260452661820523083746227552687989415416, 264322296254832654508067689731035050689, 135130094089125336697788355391176136537, 215999153658671424393948392475167714304, 95312370917824399062419169443518642872, 278890047135724472846296714964285913406, 166034353359012740721266419141165315299, 58814882904524812490445301475234670268, 92248311379448155916045419369272356815, 210208871156548886524918643806983425534, 166689322113790612816862317722941284051, 15968233399246463557911106461787675583, 281684102381562079123717730418743765852, 244265364026721006857669233457018230253, 136251885273319118928241618028317863571, 332865149523876314338624650237427467484, 41516733559057893342819898781071895116, 153338053221370702823876264128461166740, 245049510104950862295095342123252890251, 53751713287171734986575158956862467689, 238866712041914501016073547281250881540, 180456576175243571704486842768384230394, 198830626245576147237818869870987621095, 14325373047940679213261429797610891505, 305718954836416194330683709242188140864, 98671963252887255461702192519883809356, 100864061419070207478228614155842558022, 116659726622752954579684872627929503611, 83038966344592425545358068336067945832, 241460506468393320461140929350382455294, 67652202270778866171614969629756117840, 241039642686113729165030157579533812703, 32122177039793829554193719766212207355, 260277290110156729124820173062192531007, 59603457019496186121031266846605201367, 319416620740340566891417062342339556140, 231055121758852445937913852530149339630, 141963601069596582585647447997277868740, 161236340544659935054676623132448481313, 173028282444356148357710096398299186400, 226219857546915544151159395844547224122, 186905835621687710725018893211606055333, 35885295997969097587003276142691786815, 307701763630058047576968549859920545776, 261775165335692769833191668934368053058, 238785409031124094114983494278677309521, 310216922535444494681431996233181342957, 153158133680903056153078721786496855970, 330777537310970854934960277550339491662, 261557677621297810814736239106170003029, 186248562179172911558727497381392404495, 330139698902122015326962739634454762392, 147493528118468884071561048063791505797, 153329186779953680769041837145962657709, 261776887654641281245663490546783658535, 57314154370428558790722441206402803229, 59316415917430191120885905810505640642, 30456262411712426722717125513773592384, 169545263568271821491444594565987339637, 280017052534232679308648134638125294449, 94651152080311167501970693128871901447, 218409703599736128171031571026109766512, 213224840854203690716980564508420228947, 542689631955839349061637701243347101, 12142405091412449920993190634465466915, 165052084154445267194315705796378448093, 267530197123025319964349336523895860995, 104030340538733059789543459821559126868, 252320562517801479219960874014143584244, 126883589459207132541643557960554758728, 80818076030052565813871379011320540181, 140651431838875911038089164408047254422, 134496258791241927210366338224100051675, 82272877766347555335707050702192633965, 256476399693498790718475792875950640692, 202614213814074407241279874763219842038, 206962441355197462745775479837857829860, 143746829402694803460039845004402687023, 148962879372327527205199429620216696172, 28567929396625735209888624460125538635, 322453630725694161836019651119282591410, 238716871020673201214358739917459492151, 12766261958453502660440309303590470801, 22824862138992885489609997616368891400, 317243705774460859658974780921373393479, 193026658415558977682178405033184999818, 39313049830051511678257400567283238727, 265264093450904840894093158994149900477, 284416275719422254486976458430097604466, 221807636752003445994847844453870893055, 101699495668098168547600821548674750335, 155939458105711748730979222769730035617, 81915010295229100647533958369873190417, 228041295374159574503684856451811791828, 35608670427037974926919156430223696018, 312030840769117221690776832403836048438, 113818122923389986510296996883021169506, 283391966193678855985924724116195503843, 6135059173145368269256203872766331936, 204119546860995682057223332195433759419, 96563914808707942954851655171327343456, 24872923980102816535191648339294289476, 171168784604594771480492985800433945071, 19571327092466490644331105025627749686, 91972440700602582596020287303814349368, 242435311912636903081040873042374149840, 34527360219033811983591702106139543809, 83457197102071644431176211321877175229, 235695242227421748573900680180937054390, 83973454564270334608475250573081706068, 202775899147324983019031669924951160194, 115944406157879819635639152475529306354, 322968338345426571717629486987600010662, 3450824818099688897378544179236014829, 304372925014312585892614170454486001179, 42473458999651739568846660411639204982, 37723716607513691840915429417470345507, 11134029599448829625346997755382510977, 153145821578307214500465657329804842126, 337610844167509708263896361814253757046, 231612829810184004364732151889317857027, 137375426833439588092029411712118095438, 68230999409517396445133510451282091521, 200169960999796823057347549327738397631, 259230414905117786434236292046406537232, 3704527313911213714427816052917040404, 291802355836589645966465948247063076966, 127910191208965382403979206388077147748, 110414637962342413620394823456492445261, 208517849763175993380402172769781420151, 131128199793488016885954020179794644287, 44257390677807847621749285033904288801, 235606165161769056049294988708587701651, 198314684037743313363816624073786941015, 241878488905627698231763333179016767520, 331985781601250793069343891829955036788, 150247202959524918502752901784131259770, 204594469880546787508887971304191018340, 330968957684029266414143114755779399965, 269744878318083615566504826834812870771, 254180559068615044406147070742959713681, 78786201590637846142162231418839738384, 69123401327333486060292457053331573255, 271292779203786911686985793549777733274, 145577673403510858482851288949155082037, 286234543767643835058996890060736711069, 52647106343517974353517981957869251658, 125209109649725970036329460685907139922, 331432378015135567435698789150821678184, 127675839576148529174446046627063705042, 160011781034893159424287320956158067494, 272040937986116803553097961715327491232, 37885629878165083084603299815897775540, 294150953731715056569573988496462773756, 81669822062643146540993402971144371753, 137502076754504198287516419032699179420, 68903800467708340877462443519859656322, 176512166911075726870880572544170060284, 35886977311060942016945365577511978728, 339189907383733011063066742916442664879, 193538099457846306340150322561069156639, 307607219486283001086392089313933008834, 324288608530457983269063524430662838061, 141774980919123673981342047407152797296, 328646611261909523019049365599689502161, 279656642606349929505158669984703060832, 299913502836256146742256955363828416887, 116238165690643867677331027898336614347, 47719006241909455731835615692346388763, 249628437817851958215245315264049207673, 65332256016756207152343680266949735433, 104036446082113453765264169537455316844, 86640767734499691985549436970461885421, 92561696132407720740210533945487414466, 15568092556686654184783877682441833588, 27899406496753628445341738929745050563, 287389453628119628683755122662345333271, 22996929216803015696236818051391699917, 275045944124470747590073027376869955553, 106508290651387732227528706149667530373, 123005338897536376846699109416142630203, 160886601884987757617863323024341507676, 189064022201785128955195520659251987560, 205611031302813989530309087628630506762, 62754872631573506378310795835630677756, 65954900489799898990887041163094057166, 192442380321518389733195972976680993523, 285821600413725588187599806455797398317, 39436325707893883431293466386353368717, 117312198592361279025108476904142427422, 246003571934099739767342019775432589487, 103282448974357137203147415820744226505, 24460322986262618548715237805595170064, 173071319195352278493282702402000991374, 256826210295092280904275564550099879843, 217620343816252422846794724085416123572, 54687029573449533920411165089375110997, 235291611371153479970281885576291716092, 287229195762372011252200790435503872605, 260063697962733409339132583188253800463, 134696720264369599086104109638045975069, 22743101567546557530561419436456521669, 308203661870688303577735456369470470222, 180556565906768332890997005829969247051, 251779364252797570093966784327576381419, 32955701952867360427005195202342189248, 2446227327397319032048413317643768096, 62865884981675913498745360683307731450, 194878620029108008388495125409111720592, 61781183204718408686138290823139280602, 149225880802078857123006529247136072855, 290167963781835092712445071393536302581, 79828886479113405125984488934562046073, 261597712542974613296844900520524249572, 99159522029149907521073405843515579135, 259812427921422379793659711022603251321, 290224795239947296438523972676983106710, 77938659462045021142710510689952874049, 263563647790077949158680378014879113970, 161708180950012171168452672295539260687, 230829207092013669574532024777050894928, 203737252559238400643731966486007681679, 106173918974724552847541539396377756511, 20776404085342778020114936602213734514, 21236019380006084090339271480561988152, 93598879609911553225284872801898922701, 190142950520853050398992844912356811293, 204138103695698905493071409061292523281, 258342518554516361405890317955682507076, 322233775251781697144256365092060072775, 248775460633281548827907511547777427485, 162057870031758346620779941904383268313, 23190131627464740510962516124582249767, 44049931739165435109827470804604458121, 1583674722124535442378281792707253759, 329375103051377090581560676272183765038, 220108016540815186528343652737429351119, 235777671171340703106901429408108090358, 320675983437590234728003425363407093650, 39508857381833336869881454502068790832, 161261859762345088253110612208310729338, 119517567087580548539908949663752073754, 3196444678741302705116116582697490992, 227301877400815863130798001989217775640, 289462762205341995639106750234954974868, 241689084569133018115832385928364433280, 307454717470724840962898609538735641429, 54365407653819339595918353922119725411, 293965247737943854092014669302490568716, 84062750975765094785292663951072646708, 335586842705520123467165726587134879750, 267765940676564078439482114203950449477, 22681454799197058621886226595495914254, 209289749112225973475075940745458693896, 217675916729061619463139748949935121771, 129200869835427749887631548057606483202, 52357403573844215218516873737513321403, 4209135513770727790938478641375388971, 142280497340993399268565674533129422755, 277248518834616430962012226183163558037, 121824158091535488402082139118012076662, 100289227035133262623865656995463497809, 21262742761802159896489392035579887228, 90547209332995217257907124114092803864, 202539970561689866566402940139580176501, 62536368997523731518930921449543514960, 166852454118407266594428833159328955282, 9243652005144401556386622452219664634, 7362796932488991855416947212951757470, 232165515200389995128240099446742274760, 312771817828618613150814233320709127952, 61146791466303737978951424017654122895, 285809077009775006357620659353094381197, 70548269951679737886100438351675939736, 14899229470076460290995655445558823682, 182121652799161694123081180104326707317, 36600685998954560419235105200409738007, 300638569224439488180058663837689980702, 127875934497678273908479047370034729623, 66926084128356821633483899445148098785, 21907712435562710609839705273664341601, 71936436892384430519780748856364605696, 74632473345824475290347133556302291821, 170969326735895503480516992228223910028, 122255748295749565051395879207392059798, 122048845561300299002634280768371997146, 274159024719492654356473636350543179302, 23996092135573055234114844361123325334, 151928643008665399601943070279457965497, 216323499993142650385710679834044750061, 80020456486768014745659870348859297753, 229120164362078022375546486146304363733, 165135075614970411604122743109849334138, 249678437710985773039659175306321460272, 191431405664115969283355485905874001095, 116560765116570820158463246009295588266, 273012451482875999393310071676314368398, 54367085610696425076899185647895428064, 337519062889258816366963014945352030857, 95371096235584274068921380882547127375, 68414747169403032517848539031023588365, 258811548694148220264890991732045963163, 170390940299804372027329151838636930246, 282934129851074700362359401162791262588, 23013768435278996033218395556258951589, 112001992055202831413722839063659060921, 175048502503289181472202517044419226378, 166820715456211073181625774312256844445, 54337463833771865926888452811443210377, 92666581895391564367883520615468717086, 267571454343624760191874383041511345821, 30650405678820257713817514931955339909, 22432132301319668726888562239461277746, 235440617003033526972066460553148600177, 223148173981217822625654944943765523822, 117848542708511775394161709431174975023, 261712952048711610607599498761727114615, 35849286734533567839840187322601309712, 52338325571958991327142902397847667444, 242675285445775928011131683767130348279, 93511875317552535915716153097070978175, 112891866910065134118454161921562511650, 67238275891414321396389558424353414550, 67119310118712721305097379855219271458, 31973828236991504846706335588922441110, 303798618305259283611990562896231340833, 113380412734812673136690547121307233290, 209389368507644687300655023323514069930, 181462290387774978601521158621314775865, 267223041654511946859204077273380309202, 269196647277669748122878356713525838535, 49284604168950491968648868438590440494, 5690314447107627323182594225864389578, 166633444268744787054277341944393389415, 46797534560450327265638569529759248168, 105436141231593803791048887090899064256, 39697219656294523811808912694989711355, 108780582334818135209695409960010105749, 106933142990371521271734796417888461245, 294530962362160136875833246899964973373, 106661012318521217183367407792792140767, 25168540949548052974261374862069671289, 268985657088735325767676637106159712122, 110078058022877096578297717428349359793, 308003266378277839852976071582643162295, 190395893039771040725206124655130554853, 69658510354057211138389265361051603688, 182622577802228481638119311517467988479, 55112594975317535302391204674123665959, 173085850556139656900988213477314684845, 200277946656622424345921951092459086791, 258111950632834855002511054910927757457, 259104569245978978564030166621295181527, 110425177505024880364289091018992560244, 211183166655453628602963481323111075863, 313906630883696210955639909334476495210, 225842252652046031999873500055930127746, 251822108272049299201820128891053984621, 287152708830736863812310883731270854066, 298085848546937317423629677912499111366, 153768483042659308788059037290883181756, 113424627438331946731772360807978993423, 41043016061714110167539066587213387580, 76297574291332937201704587920195784405, 97695595026432680975046686427764140497, 182369400088051257758643584412346257756, 300534994889563667312934813897980582379, 65040454964039367025689770029083833043, 34696173919287724593845007122912644838, 185203309624781787400489600719357373877, 320620909393225252200012710759469183721, 108786640322718569569811754691822324399, 104001329391418058430994398466561241325, 25194377604496596479878434366299895054, 265137395545980038520382104059708283320, 173354754480981270028395676546448025884, 286340689310327152154460349629060854858, 27667178369652744617632274637819259528, 52874665843403399357949857794708165098, 63179991418888672151412227085993834206, 15089813539222107372192981374651833066, 248345066844312332611965671380108167747, 325589067307705134196634371699486243681, 244527712647416105554576039213941137110, 15536626608633334469186366702750510590, 119283588091419526324904908341652872975, 26688848037797881177412944087657601896, 282429940562509288413440786680731404188, 294499399968618110472517975932935174590, 312469473742435989473268715798748228433, 148510207325472013761491424229464071111, 117505462363741213037349621911041299074, 20417326708075298083724823055577151620, 82114914888982509678568984748307317093, 328784526050717765827803961104750200903, 103707763254626400746627144784983972804, 2152530511259122574579819286431069998, 192312243216555644058321613474736283447, 283519050952666017962155774382939557267, 88904592036100613092977635912774007635, 153983187932405442057368028693010633761, 43841097330059256595441886387339772052, 175316543132104579328445459472860953172, 149589669234211909146360100129058074338, 144336911217408348456659330367722104800, 251495761870481677979136367889277154047, 172530689266635223749139768427727824069, 159860756767676305130329954605088056967, 315730082174008506413813437234514592689, 316790951134100883310164070958829622185, 260254790970621280481286531797891041263, 47957730706574395178921422964767742044, 189191830494526285374543350299027118555, 293514380624598352764698072750746705521, 143005572541123358665091595155810398239, 237823641948602138017120851313382567455, 314545558490176878811922194271924202623, 98744501379910391821632430148188865080, 80583852528215803173806980733629759802, 276363231158122404024770944427982724817, 202043482371302401811563802189441367129, 43144609608261746662931478780632982986, 70975682192679531391996265355978961887, 171482588210177485771105225234857326231, 62639876424469721354893620969461672014, 100172975716529972062082249002830999307, 118935257731164398332291780360140030746, 218096025661481234481509983975434075942, 118158107194247030495296446202739949672, 249943725085463502428100758185920780896, 296001510547690947176567296831114060031, 330039724837515783998833519325875999145, 145512172144912028488831530927324688889, 190496628781322792756470264043896266950, 61793570019189616257969776063651318783, 339930250622863537048029081178990202097, 260758015842495531056187867647287669873, 113663765623351230634964250953575390069, 82089537822651344561335255842879641320, 317108978665721819503448883994592653748, 309711031097411146252619889343384980281, 192734698620380730356418223301955193694, 209873095695326718166345968094431238153, 174631605211320676507860089796543315483, 110653896640032190395828119387504136515, 129587961704759955477506310091420653906, 127462500683128154043849581804385859871, 283995316233926731493048392312947785508, 295130040402888523838845720226631545296, 19684472081473212438016122590297977777, 14130451147966731773858684029636002191, 73680510587630218588444881414679787173, 161451681791197581494083142159590661001, 93907805756415690218221240536062814719, 291993709554490811948980084737163286332, 180470982723165332859612372270506884626, 308571746662257550249370127407167883454, 287240159861103497107090213644499172617, 328668939416291669291106141457625955061, 299941762626381726087816554301367975051, 277769334966922478987073559273581657640, 172901805834157915766330732753362964513, 201085697638970284376195593441115436915, 86727985684680979053491051939772026993, 54856184126986217355489836990667954828, 87533914434091715553980688885367409061, 45721163037456739973506528603291781958, 290274669985675671370078850256421521915, 228604640629788877201954307388926847192, 299582930812378143687782374235049364729, 162155193900203993659392332014340393870, 294841816911157643165370629428798075278, 281939371872131169759555689179892966420, 99501564870027967053145813383301029300, 12755274249938171840330880655818514191, 27948444417291289390963259517464856493, 53556810556351982963507589758401084054, 48095159285784931723794173678401815229, 135308631508020880042563226669985948450, 245042360750180274948913913076230543705, 67544051455889237980598807302436693447, 53681154070208174780955153349410076409, 323878480283705253698163623300058743698, 289872560525704175952089602064785649399, 322896051933931141212206283267149401759, 17781825430883871281605901924782212600, 243717262611407798393044821671505489003, 246712101045566208940297746233986516504, 293950586935598446342492093866295891219, 323740782135630873364709315618931750481, 83171137539048827710262672975067715934, 120930084146774904049722821695930021419, 173113376160288211330625192969357940103, 215672598462948906287985124834853922493, 117012933856784986044433344868429879548, 115728292434200058544664183537392782776, 190013135304237139948294155546514835475, 16668855028853192229022619706081439, 261749836749156227704068640761269391319, 298082922506826843244882198292702189607, 199780276738161798160607582343839535246, 61070908536091815747591706133289925940, 20112941593171572464344465339040448912, 181443983102524592232316802995259208666, 29256884155875584772253058785531297778, 228182447601675138005141014655534691844, 197421286851303917006394807885319068695, 57850522492793180436894704492060689773, 90465536246817350798197321824853067425, 106419234050830866539578387932946181847, 265093283147259284028363806541652040137, 160559832808182243170364546386804247056, 126419559075650504263124909218056705504, 103532305108256425229200294373957709814, 245379765052173510434544081600772611031, 168694486801282775498139621937315594785, 61855634274291281610117277507129641356, 127622235958089154816116047029398739286, 143807058236558682884241676396857709076, 150249797097801251400818578452101557571, 72308862401625006908402399685383689997, 257216028713623372722753622548098523485, 301228613519325972013375788460597343877, 234437577001983631701733503765339017563, 62007105522000564328813624166892869114, 152518377726773264763454156894006115851, 129743617940166235621600990720144966967, 312743457807655725966058710301257237948, 226329031186852832438380411522610290354, 223710028720678304629328339930321985572, 29390288433506211399314292838874320489, 142466958783588288517627141330095850140, 294402462289614534989440543665253080740, 149293546655559212188710826214980464105, 260520124630688977889781663228100695394, 315605882849926956014801846525515335046, 154228831814573715975697461326335709486, 262114449153505733713294738421929801006, 103799201101373261664807206122713040323, 202921549778720401412458474339411058142, 95116999348193972313425834851734321151, 2396226967685516952816323180829184083, 32043738584556278228938479450237748244, 253415731067297707290876901612145273, 185593852456107790455037241771040515357, 93090105585334854643752840316606913806, 88095020762615157575184500902652264993, 203965636499735303749730874902389871254, 161332744722154989465358762829335784019, 135621147932909432903478755436574520363, 331247709733423577147123138346351118636, 124195065493226311880487671383749501873, 285027350978142853463805900565331057373, 334923633635819210377116577878971183824, 97671996600965997387200942730499185556, 267541997855322527179475837633716944923, 191307110149055013685469612210831095556, 50578868473025761108111185633910162712, 219692163909211276445557219140267319037, 276805751938668066599948630497473086891, 66944551423004375355526758221230463585, 175043087152637502381669439526665533464, 182268727536565280712316305270253925350, 28111925272242473520344085441858060493, 153919250743308149553237364407436602624, 141796381524155506176536567969472880004, 239656441086971845131493620204649230330, 302812094910697373735860312935388219474, 301370739368832276641284095520336989064, 225261457741813322480373074769988256664, 91465952457504454093328728293373457277, 75332019934916781096120579898164474652, 290561066054383354824492439340949145096, 215797442345377757437333620963336095160, 272093248183484511632501672219970949605, 8814399379789524350586367795880823141, 175064679709991545874686582880128583455, 290367435727971851431235841556564009639, 281966003055307961334364757456442186530, 253195595702302824488129543079637496979, 256391399316662305859665483035302254848, 83059527418779190410490149319410240956, 293206771481080667339756429725366971539, 303568476549680053484083392527428677992, 323097023576570529775509052531086730806, 85183350270413687820227902845484032646, 264064146390892460907975094233162548593, 252354604802317974797761526061571772702, 185469809897903295899305804300501726642, 229070897886908006604454438972448513678, 139988402066324961912949795100246373336, 90325321103601798105442627474726081221, 253403513890324771507581331507607590675, 29446275501445533594466440856146911140, 265146298286326604811737403678859312304, 149922023566544220305453479428555817375, 169118362997764854366733893749073229767, 20281486419167935135787555124261382893, 161945548845888314847481275799994044402, 30450230306371416144768992185414518602, 35564098177499116194261896917199273213, 250676589940123899938756794419463454481, 65580134769498065330608983116089630005, 101486994985503223253254904495492149107, 272506374911704053547926604764311602333, 327684701139153726596255183021383595856, 335244697052874046885792420986690694862, 222216505717135152753191527770898911281, 71275206296476400856692410477514169890, 143876945428324503071629830702402954082, 317503762966788708880685112639292319502, 44269171637060354041067358320655433460, 15687912799404805482794651060850549521, 300920153622159336102486002947434084320, 266867783022241276458013687718640752004, 113897935733497890450613271469277271490, 296504734236239792606824612163055653179, 281776140662248752525574379532620975346, 177109406258781077072769102696621502680, 111354500247971205525527361241376474455, 99769423322885974791747132500976678045, 73935433278777806095573650544447865423, 197650602089323820223066780949391559292, 268020640159507523665490543940119753373, 86908487702114090926988826260148060644, 98120831121264859517162721875736772976, 97347993568775127372549959090084707946, 80556168793180676332958759836610792773, 236347086472985305593003783777973486830, 63729904347337289540453053837424296789, 53702001059724129642126410330077041662, 48301072605400931935868900232254027236, 317629720159471707798483523885702887225, 272228300508608196535911015010280234291, 308840634626975252657571332301143727937, 316994714343594428488190465669713311061, 305357310827563001884393065669935073854, 70481541875337549277600808127404749788, 18600663892971458192758802686902474304, 302971006830288603518936928954929910560, 98282209691442151846139999456541406226, 262549021147660435645382765044928806124, 189023795345179183523703535541800669015, 224242527991334988497886260212789295656, 77838605745257566925510470125188158938, 49002677270774830345458996271158403762, 167656706263629720739933677295412754785, 233608771672587655470672457316558529029, 92616060090533841143196668781747033451, 18622844603204687267121311907194975969, 280208482821023735211672634689022726896, 295534019783319786179897080470472943301, 48036284938027359267830030270329451332, 204698454773951169475205139094063409537, 247874120287938206591011492010424016344, 285372365205046638878460030769866970560, 235552017518083892161294969802886988383, 302961841965833025163349052945999742916, 123070952013817176640005137734439419276, 225653759815994867392395948483385469769, 78698508933433616855909405535127796161, 235331066425918972032909109307269208046, 315639954194035818027094494489667146909, 167745224556225518978045715882477678163, 133764112616433545266962761137905024905, 139906278546045891641040652130237620172, 330078485731849790667476719190362913985, 157812133986793476450852086194137810030, 298533166956822700677894739635685149901, 188852778301488461284415671616849374764, 167034092202407785460750748882251626863, 162182965609655515004139802866930672119, 250593738000501011074833679575749281611, 58633669991051926938535789099719960750, 99487921294402442550803834279025836821, 72713732095911294098082002403666270186, 161149924532238320883846134551495943806, 245235622395476110283778645747125832346, 169343686186563905748028965493843294700, 144830174295325408322677250057692840418, 199250355461682692334062603836564360717, 29170811331827050027326017636321184684, 218322922865295785584632953662104176271, 115029178991175123613711116497506894233, 236115927929760128785515649282957179013, 89886965328884326814964316286778905401, 328367405935869611570500574146517447465, 114223753098346799517808822522250175137, 37375268670608228941305266732280671933, 257243085509447948461514626641447352235, 14481877795734090980394806993469565069, 300261495012614696826450691488164780994, 158009284382142513273487393539883489132, 175055384153864771729244987354352167512, 92572936836205007190664468825592710124, 272911884640750844420596619634973528964, 159931634442617731475423750595709677215, 201559083970471193187840024144592115116, 276085383526330222463654949089310280848, 66881466403458594965317705763877528820, 291693155697195659235281577193200692799, 11406872649364174181366765218056347730, 306468631578387628897792924046790193147, 146697769438167485741043903280225135684, 273325500990768287880492635612305155682, 83829380760559602658931297513331850311, 87746291067731145423626596735010031533, 270630207419985307283207788546252660634, 107882809414568142265503843233121741939, 61926860773972065740124245517518961480, 154546089204275667438651098353624773037, 92729500668972249835207990661259951031, 238911471187579660100500121487170661446, 139315620253564371701738139249607167356, 95623477256778405171416901465330329380, 281123645013798793126585593282568349503, 172535950044735847839956869529418332684, 125583249250654327069045291806766147155, 308959803902378918587268946792255370464, 121110902802589970200773791885837949720, 233135019395459097567792422536105337938, 235878632866481079071407940343003205478, 120418390448794060651203552611694874673, 97450841188233133739106494072119642312, 112909374716156379639597077237304026258, 320967821363150598479047401153347323359, 183651320599631009924897946889724628581, 234773158666900756728797048191736984223, 182696027135885391570608291749169953662, 111540057337164388004198222133871382021, 101022683783431540640855034537948746579, 119397566219088439638798708899317282117, 190155045734571669846675464107758272698, 209141420724007460470859867325016717901, 57509690903929925647795392668255237100, 196825624836197252236684030869959323855, 108770253480927763604867170455643196158, 13545643531851391322177718684939879085, 217686100089561892592760497182338366979, 155778332453994545173510869900114591332, 320174518150163667125421502806560844269, 99956833537596772791464443751916753671, 314585202044856870029154241570195254499, 134370452482015791576815627841613149162, 100653795140062872205289931656448475492, 257156570390199112533867132852252851612, 92877644616742610666150541427769585063, 100556761074737438437144224027351687789, 237601868444030560686006115745406321871, 132755912946097315677587792989834129064, 164277542804339123659869916378973285430, 53653764213712627970475964680621171510, 52018562399358576753718536193968831744, 29295926194860909020577321805411975745, 90826946435074221705573850725665914917, 102011168452453931467096753710468656419, 17572863960376277521100507657069277194, 165947828748464904764409694911484392303, 215397325547661449104358789481329320340, 209771466466167852441557822983267994890, 85660152277363985440503865525584114591, 46291651737154507814200355060278398721, 129974723334612044788209554167178784167, 126413033139942838878914899791132216967, 116274735837736995070338585002669935027, 111262424745571082325543104683320494793, 174461403644807032961931371901010893083, 95066723577092728504016190176854751742, 226664747475827589218559028283972732432, 6267593266118481386084829039008642493, 259262418296129586497557897373220037815, 85052625012838212043008856388331987226, 15305634164043303894043799518872704712, 132299245089713467350928269967065749459, 62959601471547511042254887900688514624, 46429679531562575130545985748423416486, 173484237354034300070145119922344951408, 129093949883336364827265222789148560614, 16956842198778654588525505652468123145, 326799183133630662735298714840354191028, 150680787246324204915846091357872788066, 29180176248893126498508591190671664011, 42215952544479792870751386087098735312, 264178714467942263279212883303961932147, 150188773706086207447037902299937371227, 296217771728228923422183288494106896013, 45429418697260146432742645742114946262, 290059756214147244343109041055922457942, 119751541975826724969256825079679588645, 303387147024283839909812817890001195644, 184976878199563405284714225375560387959, 327606258072362770158510083315314160596, 99614438930537758083991006369308431058, 212651161127688914662031271537527884338, 249196587364408982983400843319877322858, 16563239353441661128976077756392388093, 124922974654454364021663761877850036765, 211831065953417593279425772183934027525, 84540480544276061463716201215367477481, 124930319247592730643026955634216049887, 105465312985047291783774258800377376698, 220588825456663183933751586758847342624, 291757897339409934577566123949523868705, 50320041295261210145451149907279506429, 143531137277322665116954550632906307801, 253558491195727818270598418627608316121, 71772645014814529940308928611713996841, 329372796623930521259911707991158694201, 129469533106351818687426503210440224770, 114951857892089129571514407800679661093, 183126859398827182494530290055666272983, 52502899930005711410501326712196497105, 321790163220620527547735461996018223535, 310465635122127586456939438797800381493, 94908694874032823592095850618516637876, 328620242727572474009954257669147798053, 308138219376753830543493046251322465156, 44588077935772363896653785825072634878, 26154024086496594015508294230472302896, 141815271837831824183790150581712165226, 77983364302419775707555301893710000437, 153160397665286222253341128705203502730, 261430040621389635664409833311256863868, 243544665649034915771478256115342757877, 241999628780839731468472013415942959176, 239401443982652801587886585839896682247, 111997161468216629778479205149672308289, 33893372279539782890992229083374915072, 127445057243922351141418430599868605750, 50747272302750124975423154878775075862, 5693652383956663802563998482113073835, 318853944298010884561437674553881950516, 210315196078601967826436998990225189265, 222148437098442561076447630733507533143, 54892612090300633113668462978479699634, 44674401943809166085584944148182803831, 93766724340207166648358738871828135186, 196121125691056571897963602209042080688, 151121583108554910845517929194582003970, 148583211147302982327017651400406602818, 128287959809885552304225023009842420699, 281516855304559949736173151451747697992, 336309682072141448428997189211417777013, 161754489586306335529148389088742426463, 248720350637431168374677172929311180943, 219296024327477982931759915812833800171, 293102260129496807669966253352298319229, 70897519988694639061750507632339183876, 335129298237368268882563250620635308129, 94842719664988495825777750598386148325, 121854308081166925025662577087441276551, 179348512419644278010372349454961361516, 244106827005560298294169558925981195319, 193148669649200654476005535784721687323, 329347149555744489993836405675317333873, 268017678121750972452873282137068799571, 241225103726465391575236646649425956660, 177248492720743600105418178722957036259, 134524044981019777011784846693424398438, 227799555713876516175341189999185476872, 109757632268026970472342771489811619219, 245598658173456462543869130471202324684, 224808635853871483642720787011488276818, 110758872998898404386900803369669508273, 281486586159044175965445705133226336713, 109395325982681131654335530077419595151, 236294786284687592317978644414243895313, 260591286837596764382258516158969752918, 193816210830934189667343999021356570170, 166592362009064987986215067965609921065, 132775685135762461428931542943017162952, 68840450601631032529347616817284422050, 171855971748384270265315264991273958365, 237383074476423685279729698942666391555, 79859272322204263712989964585588727131, 37313133845764028853671182422332442768, 207348590960612523790121676385377429796, 202090355919996160261834047463773698809, 299018674601062166553374630160692763332, 87311009170524039902479023803110603019, 299558117521117580648552287289555693895, 340000819858209672343046627608024293281, 33719469794812407534754515926338466866, 197487339904392431036376666903715295879, 315924218523414658769918644853841348857, 160717957894783266890177556386907294437, 98893926846392265677016179854696158038, 185015884963567867329251683923131621546, 229413650792682864933187086311861013485, 157018846141080264692666374734052260023, 236789155527530278681009692773674486200, 220418625640409848042451444849447836986, 187967490985579929671525571729517283918, 297578957075297043045111982457437013338, 113835037401417362942654313481682407462, 14079363641383287587054092170621575538, 293353290462695486087829914917675647355, 150495151556834176665871729516207801555, 134276103662524197348533619093650238108, 296857560565162517860702711434833859713, 120205009606943447518436060629033589512, 307868659594777647780325879738852694227, 219876347346986897342495001790521441158, 73571351983457101546944743037530212186, 158350642186231820925337617004050133504, 300277263726553783811671706064023502355, 66174345021204841845966889215376442908, 116917422189509453107110151888958572266, 308009215223054867038123415163013303119, 254777566463139708082925100098017966014, 131120847496679700861144154010377774438, 323291229679735327227423795220396617979, 223925535507330126336626275427729563801, 253394434215248161234333990444506573139, 50006384203872052779956925498567339690, 155324664402185658333058944599619289717, 340179954161250397173144774904685736362, 136336239953286311787716281143435476609, 31690755561021843213347189032881937120, 283326893855997673566949607690608549043, 30341654588991059793398861443730392361, 275951547395393010493629877540926810046, 194781070314756029285227653417924936640, 214098674501282316851005925570552035410, 11625350463882476160565595709542766537, 108494082237005640499853363063655665496, 1945424910606298054056173717160566946, 50320979719027764513804816931728757118, 575897275181904329489371700593694747, 317395245184409400684044800023657625825, 313876595482528954986248048050455568825, 328217052714956142657124691840586965410, 40146699053492600434045029161412168107, 138036392071243945752245282115227639322, 73602232043388451948698170511511212077, 210208459212695898233222693544334411473, 142022029627985342088725010458949416552, 327527799754179575473197059191763379909, 32571263412237649124968157045018368177, 104886253150077199346054781695336051716, 237635504303199033526517816223488429548, 297197993384884662678129467608522517874, 282901150068591370368694863295764507553, 12143184177187844963825591282350752827, 138349498043014398989915644103221332096, 124695653210571554840225262624760881367, 68087955041402518571276253499125291928, 264284700935443617778223380230878107641, 327521874048221341747554249555602215864, 38992235593058746223121813625235921636, 123598664568355190819456935582457621364, 71984454818343546578909616396432037689, 218271134461443610184305672086933461993, 55923295193101528983701688305759211244, 234018647686708567792960539255225311123, 77914617067399454500358487878776535383, 77757139575258000697530600963978300624, 262979548819947320281610150440732788822, 16182671375673154791285979769242263532, 29300334450017287496397662621939487119, 151843773339066366043698241049544539800, 137414024252008426541936414120060043515, 86584838736306761516488316946045627618, 309101280536917116822647975444865649662, 286290202381106077954068327840823335368, 14866574435113185053045081725637896957, 290282250203356410035822638419458734951, 300568734722259949522527098450785513045, 24441006817802010420149198866117600709, 157676923062846841838664360720824488383, 204738290270100578689763735347944094435, 306788058103038493339172169165337764110, 287523650400224155407217514638364976219, 171905906519306256877534816532401476020, 218760108557599725971043342209976534570, 311516994474067290725449325898423573227, 102778033205650392614158046732422883015, 24561519644796968788090786934874564456, 107956091067005041657380614894709437056, 29261027489991698807900296993848060340, 10489021719717777819818488158905243553, 41531789314370264042684139255360076355, 298297357568653865830907772913255501434, 160390822906255501509865016992663793627, 235119259546400951989115735026127698454, 109990561305914582898475185698858012203, 47452244032049435868345388277202308112, 26190364906315140774494917832554256186, 305039602079060989857547645910888116105, 334885477755854549046314135500046818960, 187873402643681877713752759915114030883, 118341736804931536159056801135050262714, 313853423464679396793788859763227135284, 327309681711557711246772343648490842298, 231245768901297845231881023807930022203, 287862046679042033283516006213393540018, 154389029525211664902949691612615128390, 51849644299031516889459485848534151342, 156183004112483975622258013231038172009, 70921634593088361189369133236894638769, 280473808918380812517655912116659287071, 155393297941863491138936536565482035631, 119734940481564216111728565208670541178, 230824662717080861317795156590973314504, 63410956875382512039871883294908149515, 289513374996748328070798986133192289039, 334796606996301834310743486960684703558, 245006048772939765297018308039362742313, 311881777466443910210570832854418899693, 241463002141215525921406181615003597346, 40961146841364920176545174208533920899, 322599131120604527508245241481040640027, 42232683768965119779631427825407557588, 123874032447331725083419725467987126202, 202408967965756313903703391626585250409, 90172744389133531527002494807240187784, 48034106795623786664728809443602031286, 141326637024931210708268329334115527598, 92885456560226506700500341454763790882, 5392151396610196553283061304445639255, 265028640851418431178756193984699132761, 279987968445931485292772377428372546388, 35269921005522237126266074005763001592, 196946487036630502523869919702276641882, 44995082886417478554770949284515863207, 281376301782881201513735308567913150712, 23489522353448327663562297467177045412, 124182342872041830382928877347455386248, 262733219215710012380122450914661853851, 27028236421346239738434885178170132960, 310462834689419980991092728188098999644, 145004967886936697866331569412063615104, 215574510925821740767095344106702886007, 71221094977834479026842888689636607009, 82139564840665377637657489660736590652, 77299312659330610642578410598703188155, 12096189709628174630476594377265661839, 128223598340591562982016489090217134352, 86469760374833093103176843159457861492, 43679174734022109107772216437681711379, 43802838306432703873834214010545482767, 30906934044833116789344737746806091464, 145391014871618923673232247827834278953, 259136738263704987108417000321601834846, 233634314326773898038538141070066765403, 208448405426759934972024081224978691037, 296670004652663779686136154693087449783, 216827911300129737012764719875445762735, 65007061631700843485427483698592689732, 167987993227193782543797012585132570578, 254745236182310635409757773410411376836, 254640356025687335500977973246261129788, 176154475146378919977479123476036540929, 69237911927824046070909172295575039899, 148285404876973266700354124084978737237, 265318682853487209557044877588975128593, 33818270063223244580808990946985924082, 233787878095449466634659891317414584192, 263986269966246942192668061139343371020, 289014197085977169795574639561367196360, 304858233137471429875694919983366148890, 30729997869874895560926678535156542550, 192557139377300173219443724679143198281, 67219420389663053358183569678071338138, 120587243125190486699130512698736750069, 241453606158765219963624048158777492702, 37953209122987060600374349141414287030, 25847582472524030340124896438168513647, 30223213034633902629598708472417026592, 175482718129937350340336163261274928293, 330797883822285656171018828546827663711, 192651930693818532681520933462668397073, 152556540032647490055812775339118276008, 143196255625269523867589592204105373392, 297071734605765359784321858517311321178, 56592506015996937355241941175603780080, 53059549299728470990725618989156199497, 226685442364703366460584968925506560328, 236998028497057202435960216236016738660, 249499588579465752028546240272058072528, 29657867611271498008098339180970698673, 188181677941400384912998112345550361079, 11656703837828976960335391346911790839, 33417278571839551672129262497868851312, 81662562576784653339956108707594804076, 217191520887243663571449385581605063766, 104910939551109997543538410294655964293, 43218271074461267004397697413304276598, 335635597493316904306546807766822376870, 221296531896230016101190100554329378017, 60287382280512457834065062527547550140, 203551633254565179006114362261779848363, 250245828756150622968260540305653567531, 90386203964830556013756232052564814545, 279679316165880486924858757443507801994, 287271001679925711988645514893919581361, 92695763946198137874525371841249187254, 92420833805198305773585109555887657565, 6224650074548991641644023145086410129, 257996402520296336064057809500430703652, 174760909119167508114355164921981576705, 297564881629166924454103715081398019073, 239318524974288678067042539967272464964, 92222423714007223315503994552333976050, 305992580809053278431968088478125801039, 290422303489628609189216515727274235641, 29911593466241174075501619790541175044, 47546614877829695736876693222923058521, 166079497946486731843914470870409726251, 110863351128465341658873629282320979471, 98460754523854479235116545094463707855, 277108864456518722946530386930164230216, 135329042665166576011326709744589876869, 30723800210478861926438657716699435863, 281774177700259145333166968339474765576, 255835613909384945518462680228553802361, 2907095655557936984990792138103529350, 58528610396568035054922885263758205287, 155812028231643467096196152759207728723, 47747380160028104075054406670710236077, 200610856100943388591490973818813966324, 239462550484763563772345723492622884986, 257733778432809052507273952319625289609, 98092714721670733212180487556643892187, 123915090672213810599731538489227827814, 331489520381921827349595243004446159381, 253751807179064813790206242204116836050, 240677737143748961147181985540827437990, 92236867025332684821474409450837051166, 340176766571996959580690551882223851393, 329148336657503792650597011801154846917, 82868905106600426157856205835499694602, 224081699326250262875098559686615595969, 98786996354936351045562747168604104872, 226491082520172008866714230603015926280, 265027497233793386183465707857477480040, 110144776440113024369193794507343226418, 37997165306840986690337244972113227156, 3241805712570025530136092275914053416, 170455110867573163455085616871811259581, 151006109142001058361924908717315107900, 26054137324386401629769519330105450267, 279726506117461192519280460732138884736, 43210973828055416749186677630164946641, 109386460261047627689518501766631572648, 328332344349468141603587934104973124411, 96976606653265099620395555382321425864, 85983654717448773605437495673443025613, 295246497786200251952784295177746216207, 191913966507711969758651646353651330300, 28008277349008334226131066095687798644, 211249692098166316779233619162623097884, 208712753811285076646584406956664463440, 240548605625454345087856949572351092499, 326427183367658369206972966308868960581, 39482499268244099270196626662746197306, 131079760203107585462824162593382434584, 54048441613813071106715623531990671468, 213524059151180297587080630156918242112, 255124432072161344380474253722990002383, 126859751211216752559285937666989849226, 239186470890849911520232349293673050454, 253557773720917303139418687050640208645, 114966751299420436274190037066835408094, 321216662770066034710636539553376124883, 108461979466198003598968164873215322451, 55226177374591760238342921704234501418, 283527967810555989041998042679839012734, 246846447344498026930944757861396217315, 225289219355691924398174648119000508891, 236991798132504734793301027184608965893, 9902900829723654546069796689493727394, 176354020470532182504222166693083898427, 230878969429840517422157386068131960482, 4420530861996623400084972114105323007, 306367476765478096546660540619184780650, 271805329263649563310278724459706556808, 33489828821779277890932557622073516315, 180489472971488884872718558263680271158, 154795216885301984276210392601853837415, 228867113531347627379339233916675753283, 159879425567541279831770109863738542045, 217953001092742594539127963028537389113, 332216809616699866505316283324894069840, 33775074729944591624729236091500462626, 310014189628596370693266703817177826872, 199624439002218743997641487142893729424, 104139674780292577926150957836794903650, 236708467853866637877699704887019114914, 66760991060946802352182372619830798475, 140671782404704647622080915531313703721, 162510697455469261841698658927711550343, 305491400168260237859397700106221325378, 314854900228442925524548419820536011858, 191861398973467607199154125750270345131, 245492547643456892897614274783488892928, 9255046830221982337884242593176063514, 38168189212772721634536980715719158655, 281077400349187276366440243811616659702, 325390694773392025988694743553014117357, 127885781991549655605566953666027919227, 313027414505362148673227039909875798993, 210472267753259618530926102533944924081, 92798732448199844262539870507911877145, 42164361681002052794492522365685583827, 331227400750848457403257601839437551031, 199250115285480594106550227544094083656, 91371082152368507632664350203942090970, 190355418542701328954960829050025316767, 153994769216217144206660490795507835269, 25447303813503543486241147111969714909, 38042323371858333804022781005991525348, 124246114707539262451398543933239415799, 102371958924686084239399451491015906316, 216836148484096036833346879026755448335, 119682131277833932650950137270753931238, 189612921561418127379882548431818248910, 173572385158100746954566440702699103605, 216217460227824291502903583326756505954, 146814631116546147188792500332662192309, 136332114335065752345096249223707379308, 53625126894170745006670849079355237150, 272238300409950160940336097191888007954, 237859299871244327028984677810569979126, 144683865106589710231627103852711834802, 123021345866965763825146283160956577743, 50874775857193268310933325606995838287, 272347913213644718027947814292035402726, 110270109664374886046832015075864900262, 240592048059446309825801946888817737188, 273477892771420593037294569331139284497, 302691674560029965207537513692377798541, 189969972709326446761670865351303078393, 73000155168025031950104705906044755851, 220698219216232371643886781647966211959, 325080870260560842332910444942426484814, 27959160686612526761060257517471283770, 38986253873017588574352616317753545554, 37365055008608796747133564300177071311, 32772209378101875465098108442271274112, 266397502165419522828839334047181949657, 44533795354268239081845026073623072917, 113002714213506713437900508093272881728, 339929901595770128064016045618523612694, 36363315273930523184929369324304340427, 14430561881018163978989386171021999679, 332770202602082750363276080436833849060, 95586556394517595606128341959065460971, 139013423764558327532182089153870551712, 238662260187492451863651046551615323629, 311768166189739489717279428740359096356, 254298492805403932692654243009273839245, 183927523652586088376535865286482675374, 282440535977153515537168668546196344257, 162411591448844581302292050254630330491, 40844033495652384978165179202612427848, 292128601566786402462406159497949538019, 148464137648858300990417512829785626530, 159649502967206535831704773120149092686, 226356249988757135191868231472514092742, 25580139126631956422582532220763006796, 135266216435735076428287128979407181724, 28351772047825284843172495163734824694, 88878103355329669363789157857743149400, 50446021035210205098882375612135124507, 8328386847237255079246682330735850749, 281835018834764559679513524684180353858, 55643860156273077149350993594370446160, 320378631544492131292618405676403624286, 78276288933489551197652931725086199220, 264242375360543845498589049254331979435, 80989799896583938255283195136956363058, 15399203256401686638045012799963908214, 223121550554217804148693179745211425709, 170013789152997535147774030549406496541, 30437134642480117305686826113552701600, 155264993998728946482511189210304277048, 107379217090589474504044240533474932615, 152480608421963994304930298757645657646, 192898646048293628850440665365814362891, 299978321026120952340015088157724392275, 100765747261192910924159026247694141818, 8067354721659140074980656640485271135, 29836136281480029151206921712250681725, 201072929667316483330188454754214093453, 122452598521345066866829006346867802272, 145994694765782783230585949784658316157, 107618702976629402995454550172930851455, 245727305680019479932804739020084275769, 179236245715368431333806357362867756721, 29753829216989004453229635330831154955, 24301954305405713782853113732656484456, 138263678609622072126988146822389723705, 339353236942377606743339320476035469337, 245339056717828739533760538482303776163, 5458106555341864008278783449231383251, 31230164711424924862289536918615720339, 31785026752845516685448842995862236635, 11923151228222066258255388910971639188, 173679409210972784485367187133122130823, 301993586834027329574169052204396221228, 230283632709929987561726419980489083066, 256368925247211492543077228906284143072, 99737929865402274413268853056855892942, 182082852272919454189570568293333091937, 325572970167015072051135759009875634929, 197390228052568909104524775705686823967, 98532461983729812737320184874251884806, 77283745666688720593730431988956431327, 213773554282632870444316292831156116591, 229902012332564416840539418852045156224, 196589054952547253983695860755305110385, 97384917609708527167669582092955531704, 48232700729674353969405979812655744700, 160123280493228500460601809318137220275, 32908163020410843631755108619271050456, 226721545956673736814525767640445153604, 161701399667982716500582347996286509108, 117848888692232246697861549491150232131, 45098824057960348208966212609168451532, 230020472227131778117150600279356791195, 60287839524201556799025434226566394069, 139618000042978451935066461630550869709, 17086878239159143918854008577114367369, 58045831759503371413187438117826767857, 132181916187100412564185042793648393548, 189648858842581462910678373404206527177, 134675929230399126930383311881293113289, 298518563498757939357154174743315614136, 124907656809954117984796658137057177075, 205666782804504553428394470520849967748, 83112018924456740744939632931502973470, 132856604708940290564774809703130302782, 248094628893314188935446294619991909069, 336104327057630929881604161428909991551, 204661686851732932355860779702283407295, 44038532275133643036866102031169990977, 33264090438124679113988341508040587821, 284294906229771254552873690164913364048, 320761910951713080829979201347246125345, 328198090966905270327253158291492212563, 22599895363693485744626006070038373259, 184043378910398237951953830814175765193, 88259485493808185720231170062881385043, 294248168140538620462235113937188786262, 279778925704244779169424845644285067639, 293990423100291470112892566873216667580, 89092254332310337641770326605155153516, 240217133338816694207829623594379443874, 338848448947411818839473519501184829191, 279280004942046114651090347291462819614, 267667304979212039874675747628999908428, 242693097441615642030884021306344670886, 109573640778819217326414973989012064961, 252809262286362352245989856768388625279, 74895969403511825514467608661507891770, 34243828430604437011020320046387993241, 336133617772666457168179304765507605128, 322416944646386437346171901848174711564, 23330120389508529932615560688496697840, 332922989351074894827341141950140143287, 147585321789394233996633438937919374297, 38659141446712435854348622289590175016, 36533107933485930698273262159621045098, 279842086477297321751014066645767023263, 110067027804675470516279592335161357216, 156046957188161811937022659311708937316, 1158737534823907769015785801781649668, 240281773347014721955412585434660921077, 45175412553013060511790078259892870888, 211694333427628855873480206248515168915, 213714553171105243057963181639997670260, 272832785937369815950959313305644177870, 202932815217614710147251927396713435767, 241155605180239859065676625788889432454, 130501331416309108923290460495618199911, 89919676566749462968612959985323880478, 223893935879567616694749519924790165587, 198712900212194872442588942287043777306, 310785170866926182682183872943465363838, 101075284643209874170319604918196946914, 279671701967253086411148612374093661317, 85723044519120462663904703638332182877, 273325227790172290795770375768755320846, 168348992888229719100834858281779413368, 65712513432143561327364109932336735099, 180238550788640537633122100259471807612, 181707114988123025500645573625245471865, 131584090290894617263086706031553936132, 126295774559448047269843034332320207439, 280247894384203417806383933047261453887, 237391296105939214219804270466896118525, 241692426321279073723774987416046356595, 295024241400462792449294254844055017247, 136202459460203241045366328008439202447, 154764914638526833990153846902803734294, 94029221119961526343837666346554139133, 156822068653552175949068582492326053199, 181723070660944698681372999404310422410, 333918879571671352520367215920175193214, 262043238088029439685907401765829300443, 120536389159308832725061977600803294880, 14834657438493216721726928823618541573, 103414187906631781201922314016225864472, 333769645095625160676830349699565482325, 313409707964207005144026869641951850689, 323172349592228245116484401953237933976, 176684831875139119980569057279384246529, 4098131799957993317608852975282417902, 75864359174090661124582391890676703568, 38974181849564279348472457265017153097, 127556517929916349634542721721355964493, 219445890071076794686141950895602540912, 221402944361405720186232336496678615243, 18324550205960016928077053608761564390, 179137146455723909479848227200170995360, 236166196284453841328680896074965811099, 158201853112960892834352067799619522518, 329443973002219432966930570468540320311, 260488373237618466057011499296553357488, 291731655028718042680742850681539711502, 215796844542539379697908531411816695526, 177626196493322714391454025265924165764, 50935577170671907200231627930036243707, 166292946579673519533586093497765657371, 196834850145411351202037693376182944604, 191477463948303897245629719135662613231, 47165174927083896211299654825186074711, 150486730096417065807444816118195269531, 92745420029410218072575314123674349663, 316408504700292212276540892411503089578, 306721672378223151688416396595610853807, 280731467515940102883147010774416130281, 239630400303518959982288694987822367355, 207924550254044925090958733417193177427, 130527020092867839332613337130827977502, 82014478660242520332020456592789554032, 74677057608594532693940265157989790824, 57452867457598798613717111880631240408, 312049630369532040128221020974869585592, 24915303907253813323576588862919719522, 339700914493064403962662634151413701900, 70212963986366972316161840975409354490, 230451552221099324782052280701820265184, 296420541826595229040536327259715580533, 272135348690586996226339859512034393725, 111175618563090195205213999884198657874, 213911012525053434186029230327484728838, 147120745057579843744705114125815813854, 229386240775531106228890652176918646072, 245826810447470849212226921320907373684, 333450533680387684233880483313024632692, 294622683280901435159659912219298025905, 336540378583110691171113045212901593546, 135803436393713864910964795311499332522, 131461839245474089041220962894299048857, 280821072017996788043936049241101492903, 234129809899983248470575017296380139795, 257574025795756237422464024233681157953, 308038054790408525905979435345334806450, 304332351565110013052882377641828299706, 148530607483850175095656375765363515071, 183477071637626005714119616343290209543, 224780482944448943291764038714454804222, 209857577914470949550796572408943937124, 58215710491290411850085738082824766797, 185895376278826378308039246655292825295, 3673639424127854353657039889169428594, 67648962762699579023227258333232760130, 192175809823040434165280109659963254006, 119599491443507438889617281878639755367, 190982549407244656259007306154519349724, 65798941040840459203535176834780137458, 290258364055133185505603208813912317310, 254834979968318420662822683907452793134, 139059168114491055173721765663682929902, 315601664506719321779584430218106799377, 46600295996005048561519924681876889333, 38052755261159209471149240301334198712, 109546820797487431479933850720252055380, 91068200348807257744908174030191021499, 13785012867626953541857271385457903257, 206113830688184132376871360618726750482, 114173632475480541809273124760625525184, 63821012670891038592782035174385978642, 180678473798908703979624669525348639224, 276212447394113704736754861897203596262, 316914180322659015658294640252258041592, 227958185610548793152359475168292533166, 313328373828133201053579140474431782840, 256778035611156804956700350981008716380, 132359149197120083789657732316668068965, 195251406451260912967623766029054113022, 139193850409109066462127657814387955435, 277535211866387462660870747640575104215, 138774502460173280872348731077393658317, 326631467299043604078071885541831750597, 22218887266119677228591048463948458894, 166233723936444651568221411848053656594, 133693982671932751628783299236611913333, 12971594056494288609419812610256044615, 226707762753610364649903635071172575745, 145135515523700047519323270179866691778, 84132078731019784180358606498209856860, 310589608969996517606327188315431340898, 4485036894523206696620395177019892722, 190895899073475571426852436321146184124, 15892495637413440439486696648856372722, 50696897758165188841202943658013453836, 181056977690102041302134961666258603102, 143505850499318646494804739989300264824, 17962801821953762786126915871752496387, 115843052287392222571242525134368975916, 87707145931640061769150432399309393197, 330083606413230067316718267536301542704, 227041968091429265653094416757723071266, 153551869820801513680104297401831190769, 189134399433956118678774937421685599309, 114059450832684989506769579118646860690, 280293243028619564141412718760507568869, 245188512579021464637462907865052963059, 167932302913315469375382097999328080960, 139987206864171176678112678052163666645, 61330900259861451902169044371954606437, 161857341954682493005038859493979354851, 244576265107623408922664663183530144861, 110676945418548353896756755749768990852, 120711188069888298009689089030398568344, 225059630575113432382717381081404939287, 10196465709105799920430810028625807851, 10315669404024994814192872834945635244, 228621216802633019384978723868521918038, 38747714043494532647318812345808808410, 153607295129958008611119783603719351256, 42553319631294697666631799205250351120, 39443388660112116436048022873033558720, 231642474795168334206980356710838411392, 328054578992079904254551252778876120443, 329807259907778825322843338046151125392, 307209730944729309057422922031230671612, 240663969863085001798058623152947903461, 214467853031127824347423675208225557454, 45060922648547972396764420816425146954, 46294245931532709127079126039416312632, 29031642870944474543373793595635499832, 195665856090909244784594893355232452067, 45568264583748064877350515248511232863, 26387204962611976493598440124129509735, 321208574548491825715026344115375624333, 62216261224297728616409874158195667953, 229337255942520211178210597183303753585, 282969526754683022506676607479712233798, 246458380357636013605998609746170816476, 114222987200186037968605853249567106640, 156317444704556172794060338881249496379, 247829878045918850079071526641358527367, 251879795909223949356008993651933504800, 180266000028268539477956897006944473837, 294002312567476335925615426073931586293, 40062325904134912197489755830242973713, 42581673574558096407305637964147700659, 330562446891120196600474507675582136638, 62579237974689285848678753036245351953, 14949175603808328593610465813251025461, 139270764961631749512175934337352832831, 26645720112199890695156103807562061716, 86599573664567670052196600597013605615, 174551733464784261448288098361470289840, 293645523234573115601998911521141670243, 102272124296043617171284541772530345154, 212039225240475069245259829053407841958, 48521375489571333785910368508971659097, 19653021220462771533486896409911237258, 46700453184609161660939855660725356454, 325771207007164501414436150509022879688, 314591628714627774048428917029782287676, 80896220231286981859452902405229449228, 270968337354995838293792698051634879971, 237549684793431967628662779943811754282, 103346255104381049889788104786002938363, 237270983051060618593550882814242186491, 124233355639474873819316898526304589951, 101305599030586811930386710130148630111, 132332924173153957983423102295924550526, 63615529556929550790345814530510232912, 149180187934175844057954856131535258719, 21327905255004945238565620621083397630, 15731814528413286397977224782435816270, 175288557195169949629036380438583902692, 40911230849131389871372456195407455315, 331677441577437490715698355794375430303, 222356253666378295443757767042109732246, 145684243161812659180216888441657286945, 152111682798446519239292124059215549960, 132517860737726657006035819651708248665, 129649160253137086181399529345240778186, 85980730159679016882656042377094879911, 182822562513790266479322019311079817742, 159029180305604323852368125844688668531, 253524076685769154307521511438892222325, 181779764082115662366808550481102130279, 278768941751884481848303036072943542068, 290614163050101282069048815915104179411, 309340305073591037085994408701321573903, 305097950763683744078748379755419516078, 96923879907034038793875588846748352794, 23168510211231862666773667740694560443, 337785871061298047541888664315305414802, 88272447149139213156646706500113598216, 136124329299823224723136974748233011662, 285488174035017380331490091867075045422, 194327539828263513524009278880334312556, 26694393653564883754285139370107099969, 141444611883034329233351350225773134000, 106177281642219968703169802172182091790, 7029691600187226198318551221570757954, 3956915952912851891259799033524344877, 73344632937839709404823793796017192343, 58389168377750621747939175828008846779, 193823670568675873939510864280497968547, 147919930712113302354970107955738624452, 307825006052417477564222473387927991782, 288865113569340568385652328523043459057, 270427159167756416091837440180377568631, 140569125841303818976973971362275103551, 118615558867644487827981756345809916437, 62423561138834087026489456846943407361, 160636894146238097072053029737172274115, 183480775154625759845345306348639855575, 339135150575621402791827817514168280529, 108347679100191171222209401101334836658, 276269879906820390354389598635823987026, 309838770883862541913121358052260570617, 190957812278361430194255029447638096925, 315156660303900506470087593695869886435, 133094102538741503991677119080455195222, 96513993752347678065462977600973210221, 21993562239021304423280559873928164557, 293725882861979786882229869545438006975, 172472816987890728046091972524940059088, 164859191809694459727101833670242426982, 97906391925119592596765790340572304391, 159919704070649914932799382952498515125, 338745329885539337584678066550422717437, 202016120794478839633132743052000840496, 12069118279168742922210891380546569887, 65252108821536350070848866462654221893, 97384339642013627808799517956674399974, 4865007590216985806208659305988187792, 75433357104403139671908997035097818393, 48249325913784436351419045059635340138, 198117754468103548231515701749860547181, 206928513361574179726533927033445733443, 305195331309446138479739966940949305614, 51831707240163172026963023223595888120, 31847585570439822309842953923193465756, 196563915854796973613872380710773030944, 104163217570719977520090262202818728606, 63324009528027932965649872636339913567, 178722004020654064884976245623625168107, 3584159481862907542557092068475191519, 250358292409179553816586607215211549504, 324365513499211977023831613692746857289, 25576508717739014726317700948096974229, 175371764475329208970243161566313788488, 296182801411851195111729008735420407318, 95331964269667783531772889948961890395, 189802256830518576913091600041117321095, 71140237034482420602275191572220080269, 117344854962902578202385502707765619029, 48138140313486943530704149887274059508, 160885458811597901248828769749597954169, 107380732251617390517172843724234534027, 140552061141151221419409653492942577689, 23050458071737479361098389387404403612, 195631637245522000676855748015264896435, 269251534086382229786256052169316594013, 85616343609679803628542179795110834081, 45933443737802262638538966026874119703, 79500315971447293210808611290369339496, 259798859902620236603436912907252910894, 145544515551301229302593085729704447201, 293313604946158634815347353479665824321, 136616352110770206984699154509859844540, 337892745492582239993999800466850878413, 135316847533656286606651767287371674374, 322189790191924146629092239328717674556, 286592338207717090370031562613362808821, 152864950636604117275884867616590020151, 108661548627799462434893600262398856673, 83820945961535005504602337301816642406, 291168801880166378800688124549132635975, 303993845310614550867483937244520634235, 15784562474094566967239024356784692757, 77177516503340708526513008519365659394, 67603394530017975482884869156797543488, 196030045331267517927553736106788960913, 330697931042854209807090283902800749556, 118276114788749958558730523332977817760, 319661719547842401081136351985981259047, 234928660550251003059704514279393585964, 194692343018472525198237565277822343174, 169109976620868982163284756816557356515, 332937523291164346072157745670608623790, 3094105169413083592306075513937866235, 286905428344640659471675871426669858719, 307613510275108135447244341592985664892, 58997786165893928514732535850639043592, 269890086689940455157838555967955816000, 94550644343081272314789014597860430007, 72301797702143106634470599703796843394, 69409839953055324554749053683570604554, 284240235411421636447669060344046213318, 200754391018992865776689445761036059104, 73525305515079864109059944090878190528, 279961823436044189057057981185883970555, 21807933268900147867107100774629216555, 32069889965699676512007105249062421587, 249996248684973087810668273238660474438, 2056504455955606282659314942475584097, 281496636002914242620396272433518057624, 246018651890735150041910621056163370022, 27674410523597306313157673294367815336, 214205358902857892682383897575862081778, 188069276990429214429291633325469160435, 323230857891806209414770647352850919441, 116236538595586250148124314030012129230, 260793725132372906356598070098662408588, 318960662909674944207645029153551002424, 293967349370184982243813743488887834470, 114080351749014978372543772602903248968, 252132589024163489997523892660876037223, 278349575518326033942727301175754182132, 91986119311443805019870484986508805561, 192259366432629088638099691222759256672, 111066101067925193647482498151153915840, 178112899494554941579378931381917112412, 118419182747786507611070027065271586578, 306757449864836161611949639261170062363, 131193313792527481747749125896777583148, 200534077824264876038343217542525475626, 213665382757825249659325088312054142388, 14238519912202274333815973908408981449, 284779920038302816649406655095859205145, 163644823022105523562034248750385215807, 316491985003671842486693808114743664556, 121507463701696987399150068756424200555, 206906853847960233081770669303762165738, 337290269052036705937093495891401704415, 287830336827385681215573890385945298881, 300378757530211071993743438043502354102, 67809128249584088990637820185234858265, 197785070875601156481875056735736643671, 106058180875057111188249950106966589452, 99381014967951910400224681267987059810, 18483472667423441484287599288083868858, 241331574554855382307470003597633217484, 214709154074673162886568042085413610382, 154503290982912448215328936105748385669, 128866511350751548060560423606166781547, 307729368552562747333285152633643925577, 226980506973457215837872220548555569086, 230759710450761814378423291322824522101, 36563095001654273059105052051895701133, 99940560578771321102461551566823655532, 260748181169669004397509853965617500740, 65744816082081391478193579260217184216, 275461799735195640547734685819662482751, 258837728588758014653655264239877887186, 316505506777171616714397644174369594259, 139358057688268878017815469051982028734, 68484278711838839949248086449944542329, 181630807769825985207996669596053162299, 185183669060312794372553004045701967154, 283554058640526327031428008911266527182, 175819911815091276058394748681006335629, 204848062867724510685312026292356496182, 186686652964828233347357239222541268365, 324545996407181104512195585376076598292, 87602233164368179263437182150040850203, 230398162279113256231485165531798638037, 64764496767878371030307222055419268180, 255169840147731688163317941505712700684, 294070381847989503485314829262776392993, 148797056565371215775580903507101678912, 315598798758223935711909370196324917145, 318283405532519079125598462835559724202, 155468302581479609556586453785177737438, 8077788960769045944820816263833288079, 170781014638473293182148271819307612153, 336215097965916669890712442669019999616, 209710589753384158999583977728711050454, 124883699906925092442516091570638142830, 112189728353437095757950118235189131021, 260816670775687114701588579784573780012, 212794451110589380578681511844756659517, 97216650105674116892067539661014099629, 248139744748771825635538753182093599247, 36059560736638368730951428346571430067, 145728681881444368313816976412463119387, 84068428380129151969482828078587767540, 301838548298600042340731814710184692776, 221978661895624712428938855321846069482, 159000915059844624779401882817049638811, 196110037740134589328189939661316924075, 251576803558546046880139433804779691211, 182048915146534835913111053349855977259, 78319287903997127402626926241172520698, 227521264256435120632061956397827475552, 82115544067087366051665885252393022682, 202954583540063726245795662108808731263, 316880271831093833940403089672751863914, 102366368577347193080893124122390990557, 310257417329754002246008190759759191454, 90861991300208419862982049285343340717, 129589985736213404426263621853045053519, 242168061345920073552930954909525257412, 104670426467710081944355734888922752677, 332720353522973939783250608236131996846, 48357689628901189037676807988634387137, 135249761875549149993427137383681000866, 325969949078430542899216545000231443086, 52451956777830629063839579587123571374, 224372725708323040044765246136878245664, 47418968480077304508487345904888418337, 28692899423977544626334125480235426927, 96025398800479998259303794516094952811, 45502041065216415573590040970525679234, 225795125301183038629899942767455854566, 41653610796569983435214897702379384156, 190271465015041827351708868723417950830, 335974357470756143858876724305486197177, 93417447460441728333463579640170370686, 324126356460025756754617436784689247617, 62963558107585127041732695810298457750, 270390379738471697796740840800828547691, 59363717755024126343412694184040283220, 95364592251438913380519780705363869305, 187772663475216074831062326862491261719, 110646623212980307292807650412939734222, 323842421968841872821056354079203747372, 89360098512673728488963087025872174647, 282086563939975290944299390567416017514, 123791881467811724722799281052062480209, 260366557092975947412736262370115759976, 314861519562623491769613283047465729386, 72347732545999187128250550067940796915, 201749387760970561428117141107012121455, 108339353925574291726999920409240248448, 247562483799764287844524304723528260383, 330736763408134258022512402617620082699, 315135330735501138322165477020611846895, 36580285207881281105517263003274496807, 38685417059622093716431719082178698429, 82732909771919151693831073701667374330, 316594814542949572503224736314036248244, 185851588807904705713809970170158369681, 110274370431324194233648860727256828755, 308168681439424264208584741634849841501, 10993140833333987376826590600691164627, 321264714916385861692567059536349775733, 57283995786813522030143636072372974253, 141901315825811838784258220045235167970, 113428557182328022533084547912072262414, 75626517193093719203181489613873841749, 12279598503825669273924882965928016869, 183332564898256388750314518800267972523, 336060066143698899140439324141684272945, 68103578080054081140729584626883466027, 210893649494278035076049537125452607652, 110670123708955643389712650125921780585, 96429063037304353183713880224921078825, 41894087745600550070236781511823007834, 307510370377446390207615753659409361493, 252608124445799986491337127232516096419, 297001332015817949113509391615900548740, 178141524207967852845210244697058226509, 31901919694596294702592530840153547954, 125079490528568109674250242169723742957, 89453849378586885329600599628051865072, 173040539350813612860888449303808771443, 194104141319380154694817661227162571214, 124119365410480480852369359426252009183, 159986142293013559686558012446847883810, 301793754522324584049677668623364224530, 329171080108674552237128679802508926838, 130453672294280145199767051558295440285, 60973615098206741661160430150472246711, 286481248435048082785712602161044767620, 307676992240345281774905773924945970540, 233975385581434650908631839591694551235, 294395371951163681281111985575881343923, 230048546076984909374850879853326872288, 80698608389983401544049787374491050637, 151854738036760434930247609757086177960, 192411389570088416213877224797163643781, 337823654832716545054006060332412123963, 255106974699923735101869294922167187662, 320694957289317842965971557858152018976, 36021152629104709676684115365405245202, 307498227451474993469326939275365607502, 125100935836912106428739800559515250313, 73725548800109337144697269615933289516, 37925729383511851367607542089856715483, 111977289621499210578530366108916628244, 127341276486159606741077179769149125093, 128658113657870438418248690579120847194, 41812647173129682162050444392676040974, 5002988319831880697826342981875597254, 44182050715463045062715763729270747159, 50222630966455821716459163415087799228, 123900370512186272545934419422081521155, 66435875683795190901076177486447897662, 263624137298599739971109639812771210254, 156341708827819735646956255743987685189, 103926078671368916378371667105733981438, 154341496010935264495730967573303044370, 155924966350091354811145319744306566235, 124770453048493771266458551196554698005, 21017002066459212488542464947294804919, 308798389909725848875318178172349565144, 113906249575079297858107707831603293371, 151387911768226410242296670088416279264, 135312525447727081946153593200567425816, 253562113758803192107936260251428436249, 329124633484294395392569483299852745033, 305486174833374477127983083083710149982, 2632888450325090272243652041990574139, 335666640664781497966726212586539776305, 95758544368532572406529016689263797655, 82860958816128371805593220952723018984, 34039593306578176543112298807612305567, 97386228325762268659943953887603373209, 178644014164340224650258876434606697237, 92977034799185732312714156463909311929, 37380846706954578661680897430141696314, 45407195300746380670437355811325949954, 254953543197273162056198470475821587888, 289964947654309282138077287646268295587, 133928741448642651857011966338065982126, 99035778198938205598817135780542621854, 46465710169961272537137501868303823950, 247186152302890644142267142139568937830, 148656443617694141559334986096613224698, 334273809532944886078872999541278577918, 87149881037436363595101465180520654345, 54134644120258944162954689846740245173, 323901867444236132905729688910159443416, 311926878316219783099932690320115492386, 62337970276791886700716380178778571883, 76265970075131661063682958145627355391, 217844149592714981709415028765987341364, 151462829110330627297711172951252614993, 269641658991519118885263362903875287148, 184410693865618897932588395292392028966, 181597857914044060552192176150522853605, 285160028265978733411957104965155953946, 157307973277347627218185994926999333484, 59368581981655648495141267741694650577, 329918899179811964738796073499607273138, 116892013273815213195250927005927196430, 15548165870373601047999780538162098790, 108141375839060289474026801118601167877, 47868552803809858671491593411853666935, 24905498098779339853235270002197881671, 75533254253132881815679839374069780696, 328635837838985708140021218826227929701, 189754106783157718279493754187351265622, 70347321327958880335704808338994066195, 99303770434740634394814405389980924197, 307503441157333773094022518067979016372, 201646143105823690616282411429103864265, 6463221437421876945765616324194666351, 299821311303245114595463266832602088118, 45802339527056954045335412922353182239, 141800810449932820010986188244818676537, 321440374795612984712483390755330030490, 144630456011784080167839286644598243441, 143902807241523454149333398236462983454, 170778687435385806843398496155185518467, 46581297996650699364178247391900121137, 23778313723348668045505570799066154094, 273122650784432899459345419205655027428, 324956745932198103047665902695698346905, 270565356592587581063828853051951704726, 317447653288391206725780460078351174400, 247973421718705233508485254699911566090, 135116797307944310729133623230522339219, 91285233741875576486765237743187137910, 317758900845318348659480259378425598476, 228449295451902066564850822345876678583, 293593684188143889416944881058809552150, 155541005940750584212255368195233321811, 136583149878836540775723807200364075590, 334558868483501157756517322898018321048, 164487009989719307023837061363240677283, 239685829563728779913091771126490207195, 57700625619598065225745681381338744796, 199294483071094476763337109760810403950, 179156278913107353183868285927932701032, 318152162226152871165563516245575128732, 333148100904141352476947791975954524610, 311414425796363424396580310589346823857, 153006601773592155890448622772207068854, 124859678033073793695585477881196899440, 131685591959649699912691061605950295738, 303102574875729019426022936889227382318, 267952734587557356736456149665134039012, 224812377171654992614762486938129259404, 185265988091766057638451121004746802830, 207896121150011140098066437390030053349, 258300399544512677263301318613041244806, 305486766823155467511908716093381761040, 18630174519306556648799355260996757053, 216550946149737163009926913436923845200, 228863602252744499818166265007088356306, 14223940662348704821729574847151771193, 58400339329459387371625798267694175479, 56961694066443988599706605867554359048, 110190855468693527701532829874284893064, 73240235403544785678423361032754407817, 267505325717454150269344243313995868007, 5853371986432754240815714850917556033, 220845860945358879129805786624098313257, 293061180979706902858833768151380522003, 339232281544859180703315133132200817237, 51106162000619751443618894893916911848, 13160505234385851746598945398507586602, 314688027552592317242090261787649001835, 275125099976233817526315622142662048475, 203524042838604709086455254885403415117, 101511348529218630978747790282218942189, 31021148051527660976124154854117677262, 78254359041479311337319702177163693300, 29293884287280093598032879685690134794, 178485769419384594212666630439595578716, 256877183195451688284601610682783217238, 289223256890548991482085494885137087252, 117892391667649731340922424928898903429, 136538389960220072616228813557807999291, 276811253068974228808069134258061640954, 67733526171148447955952274067452743646, 315093302413948631188305413065823959487, 161598112191184695276872092655613686103, 40176052383370641694066390204670704733, 282078521049934139718970682390423389973, 10369514414368590981278455399593015005, 49944571026129303661940924375303302372, 49626228163913646273540833954312199770, 263175583701721778467191743730803638147, 29857916173849750680657086175737007532, 109103862151899883432469859404435179675, 326696225221128011373521278429121442532, 102269647576945836262439805107511112935, 72443701816470039561568833726198256359, 170670921458307636844074385407268844477, 315754743727280683976609379383201241355, 289061984712065246743310131033444193561, 149750315044238402330271721554090627713, 217719698036842889891428215113335296469, 176759531572194101577125922272053964944, 329897256015113126302884896173840436373, 151919146065235164037790557237911404181, 309921337470638689664976675382585340192, 319408614561823195604095490939776252530, 22619148470958567353930810508499757995, 175031004263928251022172211770079685694, 43185916287483279141561395512431042878, 270048092980581688780450220193454970396, 145104198501293288208139967002261108366, 317632560461982220069707615920631758830, 216255529886998709187053074322805228592, 253439026902167380203045034656469099430, 283654302784603180363487034389956106226, 103788747549178013091782824093795701954, 335658360902531197782517510228620485233, 193585241523342301626450457459264664930, 212094443290796852148408769429596724467, 97139073374876825280169647766747450926, 193664596568469446166293229727857833446, 52174115069408781112529154699384889081, 244454504135929610539469596878245252733, 175598691585754500838369658942682280706, 336254607401300965305694082696860610002, 86131591837158552682576517681908167644, 199997636607012571113408497988305972672, 55923797024600301060097428673595254140, 114779546156816585428858008872614000857, 193381168353906682885885567391618170911, 323901323142783038416529321029301161226, 266514063090025910069673624424977353705, 77418229165414022659000852536663010538, 72373750333916255307940109023653571870, 263899247564435618506962731944108856337, 331327416355702551904426118812323567093, 237483293229602783536155410326766249357, 139022533205652072304055251962955143485, 9473391933275744844124209605529529958, 217955646295991411911998757498051816148, 58194528857552239792576294554451800576, 90749105356910971943006496969911102988, 298007514171427329879778833351015465900, 246396755031382467816825702421229796719, 84941828212713175102362181518441617717, 335242502867268038407460222014427578835, 271964398987854183099573791089753055053, 310278160768506997024906640558246889541, 231717513717651019026987257049501620050, 196303133658326352276314123651548818862, 212632951699673644375460131436983327446, 125875641608732553221749723634047514974, 309646168631490551419689145997953409023, 101114879520975037528554134636436600853, 143323720337597840300514874177723879755, 331266701359689275197089201957609601595, 182143803930268096996850697964526240511, 334770355641343825842380769076431330469, 273844017308444749802324196767988874746, 139777299805631758256443206620317717788, 593724066104814471670174784796120475, 273283323103708614133254802772275558170, 77157357142564562311043504037961130693, 86751291062989175649698406578322996677, 312767680239862508547122244841658785975, 119675677535077830624935217290706556054, 113214838174766095260344952545089092622, 277268154893935802470946000171771199544, 149639177121017627813165140773603627878, 52962744296077213452587858307240507810, 127112109064658760446700760793969829665, 136121852870131970995975092180597322563, 75225518497216285303005951873047401481, 321579051720572450063665895028090485494, 266090324856557354357816097844002277993, 229023983936590143766930428591825288008, 174965805909953328789515163432716293009, 210233766672728649722196378331058114472, 320385422215483058788815309723186451268, 233187226898268161326678493478744878239, 293719388512050072767978409823295199949, 302488156339377329774224447609767434368, 191063948948031931694993250097415383698, 7479080118874986968484179593016309271, 322850354961902518235715892560155722438, 312746166943550992181568754436767157491, 142986830431521517426952523082549129186, 140752118440837183873857446908121648020, 243029769255106239154296221058771491217, 223539101132134470607252954664184766548, 52272885808252825827125369425714318708, 2092254919397726905827889403708724528, 295998865799842004816813300688069179178, 243730768128507915285338968555756988861, 297107544612361935097935452708910292637, 104667909206485887769535921017468375723, 28291653012625492411058223274137548814, 322068652457124877799151719754060873488, 197435840145004841328606615153449315703, 234963821624099500295185849037103787492, 173132684021453388206841369779421934332, 231980165019042756242352727510399226331, 288364617808400591191501677477496964890, 77544967716718452271915650845849373980, 96805648246810218547760469456078634276, 20298324131130521427135236836022855745, 32722978184269577910934065330363692031, 327289860592394679533489175845805341966, 42017121678003704804481521326024539909, 201076797044772790928416690501732137556, 54700987664788276230667026359083561568, 98713944558050078093435183747614860863, 38613980582359860339719966312931317356, 92929859755452211700341117079682239127, 5905961796488544184602299339511234514, 210904373256972168091349869360772170066, 52975025779262664460398745151321467159, 37600060582472851795779504476287091713, 49069846815034617608292868753539311717, 241183638310864844018849668022048742633, 135947675116438722323505856566461837352, 294526241765546151638601441319394378848, 38210241092090641450451276706413668357, 37129467327714104333764891177087809913, 192692721688515718435104621735762706046, 77676995465621659793143603541304743326, 101787172778533650837119287029124046190, 139779318329995734130602151715431079565, 15171983166641055494683717049121694458, 39043748365013930747067869295092773067, 147899683750427356934633858032903529063, 62035843173849052549762822149314266732, 115703105454499514714439047775257839717, 117883350280150152128408028306868405873, 158934506908650928317189414401784262167, 79487104766867651376851449531492300360, 66077950122973844180103809205462505793, 89266191291215152939206905364499224601, 242004740531979861781521273916671659634, 247521893454440294383358389517317810648, 235077518899670193393458487381398515380, 40116055287470617514076800625765342283, 58995637047690219771969951734839647012, 316137692381853655619360937101877658687, 216114714098624174154121880999923387719, 325811590813218255130265761896010095334, 318321558434434443362440675924241515834, 136894943090533368808087866334318909310, 295728740209659094878260834850345205433, 70213479197581753131137353908527586939, 289609797175486226754165260319536984456, 331747369608642202473107489805200973800, 100028324707465876993729878782582642855, 101364706921880515132355674721342108271, 232739797381929635092867171966681237517, 138412799179272780971926287509491262756, 143404390997404094895027068072072839899, 175680604787585778797306460092907980802, 114982429593109579248140778807387395001, 134549294347889609203147277338129438341, 21571865917344818825667341715258099966, 271833180863354640092199656699734912599, 260009147758530135049147579063639212567, 172248164963780776644647487731339763149, 196250454829103993512019771691997608530, 38219442920652115896346504497899691566, 277667688565307697216377596322421948592, 2570725759871753777411054537098893855, 197899383945028319234842078467704045507, 78984904433797185299718465424959858384, 108999888977394090777213048640022282468, 88187627323037077524800427232958671589, 242199830000468106419387112681307081148, 251284906205582748365270164611053815779, 168833378830440807949491411507274446370, 31258158063925040142184869852602213247, 272402311945688150891142415123841509840, 289313628462046111677462408500595266468, 275744117530022962845377227647944708649, 256588708014093944142906310249263634589, 43357565924959402876235801897444313445, 211356308540504091727820855883354537387, 229720594775122915131260286248467749538, 74621908792300560194383510576005731723, 148596829578899569602617148646370912355, 87129890346119319298303867479276730497, 325216442765125131497973860859080798000, 251121842871263869941667560340097085793, 72334460615189614275525763940116013795, 113977256272930225711223181655513217915, 1876866660607543847011604584774914052, 136448200730146947541898464245858682672, 330977320186535446200128077812133517324, 222050108630207385150902197104011863340, 37016769414294538927293517484810108889, 135137186895344454827294040131417645361, 243963448125996970290113067670645267154, 258631305895348091269517158805496481912, 17047095874448360521503345494945449964, 200570418427800936594459448103524562790, 145863112772779845992227287005749002633, 13897743537718843606084508570308370639, 12171010443526949025979094682976437804, 1400469904391233868477949809832604491, 111622785789859389268412887005231347606, 42940300166133588086700429940205973299, 179367460892740812694312124882502698978, 154194638305055423782571976112369576192, 242637913427277310627611846765738876327, 128585179340212503649196087082053452249, 105194735633230960836580902853694530133, 34281262210173446909591525049348935435, 133080618591512366803407999456014902329, 294094368808108120020686299749726869539, 46023647585859163168332030593886669537, 96348684748589428993240991611104867214, 63941116415139116345287452512032891707, 44964794185776527968234567878035714401, 127108375706030152957583816778877702528, 313531995850005639861162704453419260470, 221593460419772910570625520013316575996, 51875785528885857101125151866500376498, 114737635902798494650622671169738568628, 24716720386159742289927742671652715211, 238962547465246018088793589270594333362, 145883671754058948415968366558328010593, 286559065980270607330167477472012632547, 153519777424324090165409194883371036329, 17752013108339400496032246371820332342, 108498668569106227531706128745418908836, 237249021286376103815592027028287432723, 3673962531675844545730234925791081409, 276250700114547338657060976579350935805, 172869494634593118961284985791791288426, 94875946122737598166969412647724869157, 302621650964827665187599991633728585373, 127962555839165678169938174988163848169, 193328013048300761600735625454947657206, 47985821954958476627859753968949482438, 114318844864440515869134063995006380237, 165107219157527095580468333217917865744, 37745442632532624978142554854857050523, 265707602834235123735076029074153291300, 267314174721262809955012761258922062035, 217301266249743762224437070231579942915, 86605895646025780897360438163996616330, 196748278253608612161433085559904639510, 51644558955388761439517505138173229178, 240084356911625278162850111283699098056, 274541589229839733735993359739975486841, 86303760165457306644479093583992929823, 19373330913379017764772364519690114149, 233504478727644709663309297119749816603, 273104329060505407549874267036209836407, 299137758111430416323560716198100710166, 86587753391581213819474102832885425800, 245727210972997870967785801245131183560, 31222737848729629045507725043917760792, 259598062863635131006471151886148207475, 226660451765442115977997206041506319334, 225866093714998453887216285383676572708, 147540140590383977857026626856257386524, 326138356762576694789350894235704648983, 103675157589686006283502435501242671348, 154215771937303930767036701382537895534, 265448158494873479766573523637207167525, 104638017127917063953012732660096898294, 69678735936336423873438547303582625660, 288692448068324415216038000124500367970, 151219350591257979123600731823988300291, 340005379857589072574080302689129449551, 36249145386280614417387628805424468342, 305077606924959549272634384263579928023, 255201976477089965242768108891027143899, 233152247538011514061865430480270630881, 24458899279296573321919103747433750099, 137462393916840740854476678255925319758, 294574456960933821472136962742224743952, 119903938930682288560123596509319664107, 187313267796892147182344183177884831391, 196902517798570530891810895816031251558, 10578140798748947661963048815594993448, 277165853110794399306471972207343752622, 49187438217429027458284219551212657764, 321833263164131228782837398796869135380, 239276259741971746588614436165908422580, 273037672654142426097204000550994666836, 207415525615941273563076218028529566812, 330481661007037921818495464786135873584, 175483224579141478131252208132964819562, 270914883717840735870545685290869987946, 42805553154548269278871688063026070816, 314277846728812092889284133502734952261, 43624445192216973347501889762158313406, 305602570695288468618098499136243355088, 79460419333470345775838206562403997452, 4894108888817847381042485398972864251, 103177047947704592134645410326420584648, 221044099776118439188850391516439765334, 226656457308532869131764223444837899137, 96801646317713032948457983835765103103, 70925329095514798522941736360053819370, 151144043737371409351634585100864745620, 324792075838550795553135670167377095467, 96516020116126359226899728429008481798, 294317813034038291497950454108390873314, 333067527627409349479832368607190986409, 149648929189798235078315908872119722887, 222444031971037970686740752420783529326, 71901450066072948083826119630705642035, 182332987542111954395558683068350190464, 130230621630303139911924436419855045877, 5901318077112273969433350346917083415, 23044103333788534698219461418602400760, 262027416838197663193655951696540261661, 145434526866144029649265933823134780747, 47850338676396397993074085608151515913, 294829854815629400691122957675283682490, 267986003333599313380703723564106445524, 211889551267789208925256321770508556507, 210930527365134718779983310497331728226, 259415217971412832765556129489584273734, 212413969585302163101281426336187775089, 283387885549711098013516810671472770288, 316556442127279986872415867986153077887, 39019475668299721585161768165623724393, 99478396230270245636195060338430003978, 181603317740697303142303979592185353787, 181482170784727452407051772428172812198, 43319725488422199575693096856133759009, 86736718031783685565926741972679858905, 20958889815204435041761885424404497795, 170384533691943475286881618248004216636, 275787228078771091302696869951871365598, 260229981488920293966308674748866442317, 163374777805150394235093854673766310822, 17765286354622279069283258135962683761, 91938039324775491224987388309833588861, 265464405592306445382397820052066504585, 157293702920330357572277651044147524353, 38423881948504057685771063585444496475, 171479913403224784095119728279185090768, 36557273222966161195719938982000922926, 249092127327199990719724728879956611068, 308227160245203458708022423760944593479, 228594338664733590042353919627058451994, 227642646579932259945752795536696082745, 311985068390376052556377476348906258404, 337276174909292128798133145255333314820, 150127962811736636191433832877431200825, 188850944020291427989620563530450677427, 79298316862105679832239554757372748925, 96389385150644488386477675732186756725, 174884580895265799510815430002131119301, 257370424284197886128329134480934938398, 118691856990373730147705142137358411873, 238756914298543975602200630821963324367, 89303755457200125647905069320613547106, 76247244868908608814286002176227791147, 287573583763923098821172144397396547942, 157340907904517932652636494922073359974, 186269385392566258758719875326884950428, 45899849069547086357595041333120655424, 28720115912606924300125690505045640306, 109106580236292814914659449631856775522, 155561773828886272256709071086410003938, 13630333719093412618715853631471871170, 267486856702938819047918099905050857263, 281822327538230539251456954115081237188, 75305918895179176889838509982183700080, 265832603512471888431896098714290738824, 138467200083514953707846631437056244458, 96106543875102354317592919321609102605, 48200658195386437823775029967609445619, 254022252461366266262084230410404348648, 203752146892701195002837302125780171470, 60520538520528880795506951870695782153, 96526474482558291028168837021149628796, 328962622467433350001370211844072795668, 61004381673438889078998448341048620805, 265460545433098211312242101983005574639, 287853848968793969418426559458904363759, 127981691468769220357896971742480312539, 260126838208466272044291754328654876460, 28497416180245540248466116828227144913, 63184181597495754739415958339958485273, 41600984042604042666020295198485863426, 235608568034707966603701538957865891783, 90699854753652779462598719248816268009, 183523737953607388726822768521068224757, 49701737627209539146110979321033814840, 261766700973974962710404326658730235062, 260245209841821311888915775998331241492, 212543254050315049034927348459539038604, 104530758557187746379016684358537920003, 90833149655546329669403924720178816376, 90850440028930870635918487224175234122, 250883475522460686640399555853094020384, 54231204783851746887775177924129174698, 134872256032413171450507884575922127201, 64006007783690506070718585187068651556, 167622670172522861092455473982206076343, 316286654984262326093070599771102984726, 45703161725214585438738451183709100589, 122577033775374590129658253539734916547, 263299824226396035960227708611094126305, 313035207896057314727953549702363304397, 242165868980781494196598362943146079271, 296244139662276138827826342228203134434, 83359643231643345582454115497040785555, 220760590231510096689418135310234325394, 84925858956011753800675180752817876962, 217298845885835228634022333493269903373, 46409378816092384912481903747133010267, 159874096631026263512857698034794031417, 296426674438501221036514377812765072261, 107224498515193970997321232708365407145, 326319720738815937104636527767787048196, 149797331184686300769938005058086730394, 204466120594066798721096135735330890343, 103702800502723331746345849342259935514, 81758036579046272516695620611989463149, 18579717315398277111165468929268392187, 165577392292341652028928963084900443458, 246232199370471927233434768013847048976, 77664842667095047619000857665901413853, 104698284525307741661030334607942814104, 305059627166146592741191578653636690775, 40907279997699992023846019694938558808, 234029571330199635148840526203355302475, 21246039182707904403222690015506738586, 330772765789788657361282091136201969538, 5696965673332240748851375214196412350, 10265212461501650931910767987270311014, 22510396246564416567786655271234108243, 118685056833020169885461133526375446254, 61709402538943974765555243450505195308, 301788739170727359255540233205276589956, 155555835375207620746716560058259454740, 333259587715074855972399251276483886848, 202195566080745702784087322577148408500, 11713781637659387650392350605563537963, 130032734116016329657926592001623810529, 91169496965673367539454607073630036336, 66168245605459435621535275174051922745, 40062760487091677238069388534738328305, 230131222683206602746757834427857643317, 93695038399167090399113324926101935637, 206120858762745274038256014483785023729, 89922792958043473564485711636049931257, 208419008390392569535247203548033914244, 339976587106002794501298436005913455812, 273398604096196529180111753508013450501, 209845093913168458294405841274815076143, 324056318784536192721402683308646279077, 152658112398954953967319245242338580911, 193466470542845251866800111065029753320, 318640297744704072226733246415976386597, 237446280127939589456756965758610894602, 244022631359559358970854831920936160821, 53113549868554700432453719202075549615, 288114922242978204412025171052187305705, 292592827968700563722127022504028509579, 256696242744496687152378213914945470394, 186025573557165354120434607960802693893, 142551240202777473990373289369795900846, 51350768777930093736413119623663849312, 8910134575010264795254262230720478041, 320754322630628815041885503202929797825, 59061379031962867871977568285607067023, 179601051362920058611949604451354988288, 58368159764832292050303032625661248758, 41972115917882318020155789529696314989, 296708626113129122646225284639928551964, 293786638284099655684459255493354986090, 221441727273750734193231263024118712040, 262465046436205027256536280766818401993, 165680132515386420899218732868177802727, 79659902135338525808096422697305497999, 127912851913615531750683821848490083869, 233355294488943185371429507307207406806, 221000020680157246673514240925357303666, 291945606253099355120957516784653959811, 90922317581769618405296721847553026158, 263898112415449121246653469831861579811, 275332181371077742087207757105772461890, 164816846658684470523709326769783835516, 53270780570859916193565764528427522443, 239801178330654407732519138778384173865, 15064712339634326010888388636093719171, 55929839930110831887882182056771722734, 1775019456838140109597910094259482134, 284144868923850900290222276349885725218, 253447521446926739334207174157353695480, 40973582799175139410466904603091541199, 287857927298573579035708014534430941034, 177653373255123157886084021101965095478, 328314789936857848572554913956833094213, 59656271104913058093918377678276448784, 236707191465667506793761157844103120920, 224382364255722789392753447711672829439, 36333382267375725434842188545674622080, 233836506549970440433420639890325980051, 190309584037677536878122856999414601440, 331273095427068732842748835574047905767, 268867701374861215916763570315774120350, 58441949180508294411630631622485894988, 32832360871771003791154010559413223998, 165799674683000415010046441028901007598, 105297219692343780084712707506813241111, 299102408000741193285870563274649026296, 323627684342561229193233890560342916159, 192423292369021690024206611184560144797, 10678335309702032362041479122427256854, 39586621907751473610818375463156459184], [0, 1, 3], [0, 1, 3], [0, 1, 3], [0, 1, 3], [0.0, 1.0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [0, 1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='20', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='20')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 6, 7, 8, 9])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['int_num', 'float_num', 'float_num', 'float_num', 'float_num'], missing_values_reference_list=[nan])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[nan])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[1, 2, 3, 4, 5, 10, 11, 12, 13, 0, 6, 7, 8, 9])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    #
    # composing steps for cognito Pipeline
    #
    cognito__input_metadata = None
    cognito_steps = []
    cognito_steps.append(('0', autoai_libs.cognito.transforms.transform_utils.TA1(fun=numpy.sqrt, name='sqrt', datatypes=['numeric'], feat_constraints=[autoai_libs.utils.fc_methods.is_non_negative, autoai_libs.utils.fc_methods.is_not_categorical], tgraph=None, apply_all=True, col_names=['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32')], col_as_json_objects=None)))
    cognito_steps.append(('1', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 14), additional_col_count_to_keep=12, ptype='classification')))
    cognito_steps.append(('2', autoai_libs.cognito.transforms.transform_utils.TA1(fun=numpy.tan, name='tan', datatypes=['float'], feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical], tgraph=None, apply_all=True, col_names=['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS', 'sqrt(MATRICULA)', 'sqrt(NOME)', 'sqrt(NOTA_DE)', 'sqrt(NOTA_EM)', 'sqrt(NOTA_MF)', 'sqrt(NOTA_GO)', 'sqrt(H_AULA_PRES)'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32')], col_as_json_objects=None)))
    cognito_steps.append(('3', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 14), additional_col_count_to_keep=12, ptype='classification')))
    # assembling cognito_ Pipeline
    cognito_pipeline = sklearn.pipeline.Pipeline(steps=cognito_steps)
    steps.append(('cognito', cognito_pipeline))
    steps.append(('estimator', sklearn.ensemble.gradient_boosting.GradientBoostingClassifier(criterion='friedman_mse', init=None, learning_rate=0.1, loss='deviance', max_depth=3, max_features=0.9957706943773863, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.010327510404560968, min_samples_split=0.4132170801949588, min_weight_fraction_leaf=0.0, n_estimators=60, n_iter_no_change=None, presort='auto', random_state=33, subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


### 3. Extract needed parameter values from AutoAI run metadata

In [ ]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_0 = {
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token/',
    'APIKEY': 'yfpCrjQjN_OvPQqv_IWOH3mHa3QGYXNJgWqVONHrE48J',
    'BUCKET': 'desafio2ma-donotdelete-pr-ivx50g8udcanbk',
    'FILE': 'dataset_desafio_2.csv',
    'SERVICE_NAME': 's3',
    'ASSET_ID': '1',
    }


In [ ]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                client = ibm_boto3.client(service_name=credentials['SERVICE_NAME'],
                                    ibm_api_key_id=credentials['APIKEY'],
                                    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                    config=Config(signature_version='oauth'),
                                    endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                readable = client.get_object(Bucket=credentials['BUCKET'],Key=credentials['FILE'])['Body']
                # add missing __iter__ method, so pandas accepts readable as file-like object
                if not hasattr(readable, "__iter__"): readable.__iter__ = types.MethodType( __iter__, readable )
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied')
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                    df = pd.read_excel(credentials['FILE'])
                else:
                    df = pd.read_csv(credentials['FILE'], sep = _input_metadata['separator'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename != 'None' and project_filename != '1':
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                buffer = io.BytesIO(readable.read())
                buffer.seek(0)
                df = pd.read_excel(buffer, encoding=encoding,sheet_name=_input_metadata['excel_sheet'])
            else:
                df = pd.read_csv(readable, encoding = encoding, sep = _input_metadata['separator'])
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_dataread:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_dataread))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')
    if _input_metadata.get('subsampling') is not None:
        df = df.sample(frac=_input_metadata['subsampling'], random_state=_input_metadata['random_state']) if _input_metadata['subsampling'] <= 1.0 else df.sample(n=_input_metadata['subsampling'])
else:
    print('Data cannot be loaded with credentials supplied, please provide DataFrame with training data.')

### 5. Preprocess Data

In [ ]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


In [ ]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


In [ ]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [ ]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [ ]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


#### 7. Generate features via Feature Engineering pipeline

In [ ]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [ ]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [ ]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

In [ ]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

In [ ]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if learning_type is "classification" and (pos_label is None or str(pos_label)==''):
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

In [ ]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

In [ ]:
cv_results